# Many-to-many matching under preference using SAT solving with z3
Sander Vonk, 11868627 UvA <br>
Supervisior: Ronald de Haan

In [1]:
# Note:
###############################################################################################################
# Values between such hashes can and should be changed depending on the desired variable setup
###############################################################################################################

### Reading csv file 

In [20]:
from z3 import *
import pandas as pd
import numpy as np
import timeit

In [21]:
# Reads csv file, sorts it for easier handling and returns it, along with a file to split on submissions
def read_file(file):
    file = pd.read_csv(file)
    file = file.sort_values(by=['Reviewer id', 'Submission number'])
    file_sub = file.sort_values(by=['Submission number', 'Reviewer id'])
    return file, file_sub 

# choose cv file to read
# file, file_sub = read_file("test.csv")
file, file_sub = read_file("data2.2.csv")

file
# file_sub

,Reviewer id,Submission number,Bid
272,0,0,yes
285,0,1,no
286,0,2,no
287,0,4,no
288,0,5,no
...,...,...,...
365,23,47,no
331,23,48,maybe
366,23,49,no
367,23,50,no


In [22]:
# Checks the number of reviewers and submissions within a file and returns them
def revsub_n(file):
    rev_n = len(file['Reviewer id'].unique())
    sub_n = len(file['Submission number'].unique())
    return rev_n, sub_n

rev_n = revsub_n(file)[0]
sub_n = revsub_n(file)[1]
print(" Number of reviewers: {0} \n Number of submissions: {1}".format(rev_n, sub_n))

 Number of reviewers: 24 
 Number of submissions: 52


In [24]:
# Number of yes bids
yes_bids = 0
for i, row in file.iterrows():
    if row.iloc[2] == "yes":
        yes_bids += 1
print("Number of yes bids:", yes_bids)

Number of yes bids: 205


### Setting up variables for the file

In [5]:
# reviewers as list
reviewers = []
for r in file['Reviewer id']:
    reviewers.append(r)

rev = []
for i in reviewers:
    rev.append('r' + str(i))
# print(rev)

# reviewers as list for submission splitting
reviewers_s = []
for r in file_sub['Reviewer id']:
    reviewers_s.append(r)
rev_s = []
for i in reviewers_s:
    rev_s.append('r' + str(i))
# print(rev_s)

In [6]:
# submissions as list
submissions = []
for s in file['Submission number']:
    submissions.append(s)
sub = []
for i in submissions:
    sub.append('s' + str(i))
# print(sub)

# submissions as list for submission splitting
submissions_s = []
for s in file_sub['Submission number']:
    submissions_s.append(s)
sub_s = []
for i in submissions_s:
    sub_s.append('s' + str(i))
# print(sub_s)

In [7]:
# list of combinations of reviewers and submissions
var = []
for (r, s) in zip(rev, sub):
    var.append(str(r)+str(s))
# print(var)

# list of combinations of reviewers and submissions for submission splitting
var_s = []
for (r, s) in zip(rev_s, sub_s):
    var_s.append(str(r)+str(s))
# print(var_s)

In [8]:
# variabes per reviewer
rev_vars = []
for s in range(sub_n):
    rev_vars.append(var[s])
# print(rev_vars)

In [9]:
# splits lists of reviewers in lists per reviewer 
rev_split = [var[i:i + sub_n] for i in range(0, len(var), sub_n)]
# print(rev_split)

# adding ,1 after each variable for reviewer rules for Pb
pb_rev_split = []
for i in rev_split:
    pb_rev_split.append(['(' + j + ',1)' for j in i])
# print(pb_rev_split)

# splits lists of submissons in lists per submission
sub_split = [var_s[i:i + rev_n] for i in range(0, len(var_s), rev_n)]
# print(sub_split)

# adding ,1 after each variable for reviewer rules for Pb
pb_sub_split = []
for i in sub_split:
    pb_sub_split.append(['('+ j + ',1)' for j in i])
# print(pb_rev_split)

### Setting up rules for SAT solving 

#### General rules (extra: 
- r1s1, r1s2, etc. = z3.Bools(var)
- scores = [z3.If(r1s1, 2, 0), z3.If(r1s2, 2, 0), etc.]
- rules = [z3.PbLe(([(r1s1,1),(r1s2,1), etc.), etc.))

In [10]:
# Running SAT

s = z3.Solver()
o = z3.Optimize()
score = z3.Int('score')
m_set = z3.Int('m_set')
goal = z3.Int('goal')
# Amount of reviewers per submission and submissions per reviewers

# o.add(rules)
# s.add(rules)

#### Bools(var) rules

In [11]:
# setting up variables for defining z3 rules
# "'list' of all variables: print(*var, sep=",")" = z3.Bools(var)"
# print(*var, sep=",") = z3.Bools(var)
# print(*var, sep=",")

In [12]:
# small test file
# r1s1,r1s2,r1s3,r1s4,r2s1,r2s2,r2s3,r2s4,r3s1,r3s2,r3s3,r3s4,r4s1,r4s2,r4s3,r4s4 = z3.Bools(var)
# data1 vars
r0s0,r0s1,r0s2,r0s4,r0s5,r0s7,r0s8,r0s9,r0s10,r0s12,r0s13,r0s14,r0s15,r0s16,r0s18,r0s19,r0s20,r0s21,r0s22,r0s23,r0s24,r0s25,r0s26,r0s27,r0s28,r0s29,r0s30,r0s31,r0s32,r0s34,r0s35,r0s36,r0s37,r0s38,r0s39,r0s40,r0s41,r0s42,r0s43,r0s44,r0s45,r0s46,r0s47,r0s48,r0s49,r0s50,r0s51,r1s0,r1s1,r1s2,r1s3,r1s4,r1s5,r1s6,r1s7,r1s8,r1s9,r1s10,r1s11,r1s12,r1s13,r1s14,r1s15,r1s16,r1s17,r1s18,r1s19,r1s20,r1s21,r1s22,r1s23,r1s24,r1s25,r1s26,r1s27,r1s28,r1s29,r1s30,r1s31,r1s32,r1s33,r1s35,r1s36,r1s37,r1s38,r1s39,r1s40,r1s41,r1s42,r1s43,r1s44,r1s45,r1s46,r1s47,r1s48,r1s49,r1s50,r1s51,r2s0,r2s1,r2s2,r2s3,r2s4,r2s5,r2s6,r2s7,r2s8,r2s9,r2s10,r2s11,r2s12,r2s14,r2s15,r2s18,r2s19,r2s20,r2s22,r2s23,r2s24,r2s25,r2s26,r2s27,r2s28,r2s29,r2s30,r2s31,r2s33,r2s34,r2s35,r2s36,r2s37,r2s38,r2s39,r2s40,r2s41,r2s42,r2s43,r2s44,r2s45,r2s46,r2s47,r2s48,r2s49,r2s50,r2s51,r3s0,r3s2,r3s3,r3s4,r3s5,r3s6,r3s7,r3s8,r3s10,r3s11,r3s12,r3s15,r3s17,r3s18,r3s19,r3s20,r3s21,r3s22,r3s23,r3s25,r3s31,r3s32,r3s33,r3s34,r3s35,r3s36,r3s37,r3s38,r3s39,r3s40,r3s41,r3s42,r3s43,r3s44,r3s45,r3s46,r3s47,r3s48,r3s49,r3s50,r3s51,r4s0,r4s1,r4s2,r4s3,r4s4,r4s5,r4s6,r4s7,r4s8,r4s9,r4s10,r4s11,r4s12,r4s13,r4s14,r4s15,r4s16,r4s17,r4s18,r4s19,r4s20,r4s21,r4s22,r4s23,r4s24,r4s25,r4s26,r4s27,r4s28,r4s29,r4s30,r4s31,r4s32,r4s33,r4s34,r4s35,r4s36,r4s37,r4s38,r4s39,r4s40,r4s41,r4s42,r4s43,r4s44,r4s45,r4s46,r4s47,r4s48,r4s49,r4s50,r4s51,r5s0,r5s1,r5s2,r5s3,r5s4,r5s5,r5s6,r5s7,r5s8,r5s9,r5s10,r5s11,r5s12,r5s13,r5s14,r5s15,r5s16,r5s17,r5s18,r5s19,r5s20,r5s21,r5s22,r5s23,r5s24,r5s25,r5s26,r5s27,r5s28,r5s29,r5s30,r5s31,r5s32,r5s33,r5s34,r5s35,r5s36,r5s37,r5s38,r5s39,r5s40,r5s41,r5s42,r5s43,r5s44,r5s45,r5s46,r5s47,r5s48,r5s49,r5s50,r5s51,r6s0,r6s1,r6s2,r6s3,r6s4,r6s5,r6s7,r6s8,r6s9,r6s11,r6s12,r6s13,r6s14,r6s15,r6s16,r6s17,r6s18,r6s19,r6s20,r6s21,r6s22,r6s23,r6s24,r6s25,r6s26,r6s27,r6s28,r6s29,r6s30,r6s31,r6s32,r6s33,r6s34,r6s36,r6s39,r6s40,r6s41,r6s42,r6s44,r6s45,r6s47,r6s48,r6s49,r6s50,r6s51,r7s0,r7s1,r7s2,r7s3,r7s4,r7s5,r7s6,r7s7,r7s10,r7s12,r7s13,r7s14,r7s15,r7s16,r7s17,r7s18,r7s19,r7s20,r7s22,r7s23,r7s24,r7s25,r7s26,r7s27,r7s28,r7s29,r7s30,r7s31,r7s32,r7s33,r7s34,r7s35,r7s36,r7s37,r7s38,r7s39,r7s40,r7s41,r7s42,r7s43,r7s44,r7s45,r7s46,r7s47,r7s49,r7s50,r7s51,r8s0,r8s1,r8s2,r8s3,r8s4,r8s5,r8s6,r8s7,r8s8,r8s9,r8s10,r8s11,r8s12,r8s14,r8s15,r8s18,r8s19,r8s20,r8s21,r8s22,r8s23,r8s24,r8s25,r8s26,r8s27,r8s28,r8s29,r8s30,r8s31,r8s33,r8s34,r8s35,r8s36,r8s37,r8s38,r8s39,r8s40,r8s41,r8s42,r8s43,r8s44,r8s45,r8s46,r8s47,r8s48,r8s49,r8s50,r8s51,r9s0,r9s1,r9s2,r9s3,r9s4,r9s5,r9s6,r9s7,r9s8,r9s9,r9s10,r9s11,r9s12,r9s13,r9s14,r9s15,r9s16,r9s17,r9s18,r9s19,r9s20,r9s21,r9s22,r9s23,r9s24,r9s25,r9s26,r9s27,r9s28,r9s29,r9s30,r9s31,r9s32,r9s33,r9s34,r9s35,r9s36,r9s37,r9s38,r9s39,r9s40,r9s41,r9s42,r9s43,r9s44,r9s45,r9s46,r9s47,r9s48,r9s49,r9s50,r9s51,r10s0,r10s1,r10s2,r10s3,r10s4,r10s5,r10s6,r10s7,r10s9,r10s10,r10s12,r10s13,r10s14,r10s15,r10s16,r10s17,r10s18,r10s19,r10s20,r10s21,r10s22,r10s23,r10s24,r10s25,r10s26,r10s27,r10s28,r10s29,r10s30,r10s31,r10s32,r10s33,r10s34,r10s35,r10s36,r10s37,r10s38,r10s39,r10s40,r10s41,r10s42,r10s43,r10s44,r10s45,r10s46,r10s47,r10s49,r10s50,r10s51,r11s0,r11s1,r11s2,r11s3,r11s4,r11s5,r11s6,r11s7,r11s8,r11s9,r11s10,r11s11,r11s12,r11s13,r11s14,r11s15,r11s16,r11s17,r11s18,r11s19,r11s20,r11s21,r11s22,r11s23,r11s24,r11s25,r11s26,r11s27,r11s28,r11s29,r11s30,r11s31,r11s32,r11s33,r11s34,r11s35,r11s36,r11s37,r11s38,r11s39,r11s40,r11s41,r11s43,r11s44,r11s45,r11s46,r11s48,r11s50,r11s51,r12s1,r12s2,r12s3,r12s4,r12s5,r12s6,r12s7,r12s8,r12s9,r12s10,r12s12,r12s13,r12s14,r12s15,r12s16,r12s19,r12s20,r12s21,r12s22,r12s23,r12s24,r12s25,r12s26,r12s27,r12s28,r12s29,r12s30,r12s31,r12s32,r12s33,r12s34,r12s35,r12s36,r12s37,r12s38,r12s39,r12s40,r12s41,r12s42,r12s43,r12s44,r12s45,r12s46,r12s48,r12s49,r12s50,r12s51,r13s0,r13s2,r13s3,r13s4,r13s5,r13s6,r13s7,r13s10,r13s12,r13s13,r13s14,r13s15,r13s16,r13s18,r13s19,r13s20,r13s22,r13s23,r13s25,r13s26,r13s27,r13s28,r13s29,r13s30,r13s31,r13s32,r13s33,r13s34,r13s35,r13s37,r13s38,r13s39,r13s40,r13s41,r13s42,r13s43,r13s44,r13s45,r13s46,r13s47,r13s49,r13s50,r13s51,r14s0,r14s1,r14s2,r14s4,r14s5,r14s7,r14s8,r14s9,r14s10,r14s11,r14s12,r14s13,r14s14,r14s15,r14s17,r14s18,r14s19,r14s20,r14s21,r14s22,r14s23,r14s24,r14s25,r14s26,r14s27,r14s28,r14s29,r14s30,r14s34,r14s36,r14s38,r14s39,r14s40,r14s41,r14s42,r14s43,r14s44,r14s45,r14s46,r14s47,r14s48,r14s49,r14s50,r14s51,r15s1,r15s2,r15s3,r15s4,r15s5,r15s6,r15s7,r15s8,r15s9,r15s10,r15s11,r15s12,r15s13,r15s14,r15s15,r15s16,r15s17,r15s19,r15s20,r15s21,r15s22,r15s23,r15s24,r15s25,r15s26,r15s27,r15s28,r15s29,r15s30,r15s31,r15s32,r15s33,r15s34,r15s35,r15s36,r15s37,r15s38,r15s39,r15s40,r15s41,r15s42,r15s43,r15s44,r15s45,r15s46,r15s48,r15s49,r15s50,r15s51,r16s1,r16s2,r16s3,r16s4,r16s5,r16s7,r16s8,r16s9,r16s11,r16s12,r16s13,r16s14,r16s15,r16s16,r16s17,r16s19,r16s20,r16s21,r16s22,r16s23,r16s24,r16s25,r16s26,r16s27,r16s28,r16s29,r16s30,r16s31,r16s32,r16s33,r16s34,r16s35,r16s36,r16s38,r16s39,r16s40,r16s41,r16s42,r16s43,r16s44,r16s45,r16s46,r16s48,r16s49,r16s50,r16s51,r17s0,r17s1,r17s2,r17s3,r17s4,r17s5,r17s7,r17s8,r17s11,r17s12,r17s13,r17s14,r17s15,r17s16,r17s17,r17s18,r17s19,r17s20,r17s21,r17s22,r17s23,r17s25,r17s26,r17s27,r17s28,r17s29,r17s30,r17s31,r17s32,r17s33,r17s34,r17s35,r17s36,r17s37,r17s38,r17s39,r17s40,r17s41,r17s42,r17s43,r17s44,r17s45,r17s46,r17s47,r17s48,r17s49,r17s50,r17s51,r18s0,r18s1,r18s2,r18s3,r18s4,r18s5,r18s6,r18s7,r18s8,r18s10,r18s11,r18s12,r18s13,r18s14,r18s15,r18s16,r18s17,r18s18,r18s19,r18s20,r18s21,r18s22,r18s23,r18s25,r18s26,r18s27,r18s28,r18s29,r18s30,r18s31,r18s32,r18s33,r18s34,r18s35,r18s36,r18s37,r18s38,r18s39,r18s40,r18s41,r18s42,r18s43,r18s44,r18s45,r18s46,r18s47,r18s48,r18s49,r18s50,r18s51,r19s0,r19s1,r19s3,r19s4,r19s5,r19s6,r19s7,r19s8,r19s9,r19s10,r19s11,r19s12,r19s13,r19s14,r19s15,r19s16,r19s17,r19s18,r19s19,r19s20,r19s21,r19s22,r19s23,r19s24,r19s25,r19s26,r19s27,r19s28,r19s29,r19s30,r19s31,r19s32,r19s33,r19s34,r19s35,r19s36,r19s37,r19s38,r19s39,r19s40,r19s41,r19s42,r19s43,r19s44,r19s45,r19s46,r19s47,r19s48,r19s49,r19s50,r19s51,r20s0,r20s1,r20s2,r20s3,r20s4,r20s5,r20s6,r20s7,r20s8,r20s9,r20s10,r20s11,r20s12,r20s13,r20s15,r20s17,r20s18,r20s19,r20s20,r20s21,r20s22,r20s23,r20s24,r20s25,r20s26,r20s30,r20s31,r20s32,r20s33,r20s34,r20s35,r20s36,r20s37,r20s38,r20s39,r20s40,r20s41,r20s42,r20s43,r20s44,r20s45,r20s46,r20s47,r20s48,r20s49,r20s50,r20s51,r21s1,r21s2,r21s3,r21s4,r21s5,r21s6,r21s7,r21s8,r21s10,r21s12,r21s14,r21s15,r21s18,r21s19,r21s20,r21s22,r21s23,r21s24,r21s25,r21s26,r21s27,r21s28,r21s29,r21s30,r21s31,r21s33,r21s34,r21s35,r21s37,r21s38,r21s39,r21s40,r21s41,r21s42,r21s43,r21s44,r21s45,r21s46,r21s47,r21s48,r21s49,r21s50,r21s51,r22s0,r22s1,r22s2,r22s3,r22s4,r22s5,r22s6,r22s7,r22s8,r22s9,r22s10,r22s11,r22s12,r22s13,r22s14,r22s15,r22s16,r22s17,r22s18,r22s19,r22s20,r22s21,r22s22,r22s23,r22s24,r22s25,r22s26,r22s27,r22s28,r22s29,r22s30,r22s31,r22s32,r22s33,r22s34,r22s35,r22s36,r22s37,r22s38,r22s39,r22s40,r22s41,r22s42,r22s43,r22s44,r22s45,r22s46,r22s47,r22s48,r22s49,r22s50,r22s51,r23s0,r23s1,r23s2,r23s3,r23s4,r23s5,r23s6,r23s7,r23s8,r23s10,r23s11,r23s12,r23s13,r23s14,r23s15,r23s16,r23s17,r23s18,r23s19,r23s20,r23s21,r23s22,r23s23,r23s25,r23s26,r23s27,r23s28,r23s29,r23s30,r23s31,r23s32,r23s33,r23s34,r23s35,r23s36,r23s37,r23s38,r23s39,r23s40,r23s41,r23s42,r23s43,r23s44,r23s45,r23s46,r23s47,r23s48,r23s49,r23s50,r23s51 = z3.Bools(var)

#### Scores (if())

In [13]:
# # Code for printing the rules corresponding to the bid scores, can be used to create the scores that are set below

# these values decide how many points should be awarded to different bids, so different preferences for a match
###############################################################################################################
yes = 100
maybe = 0
no = 0
###############################################################################################################

# list of all bids, indicated by a nuber below 100 to indicate the preference of a possible match
bids = []
for i, row in file.iterrows():
    if row.iloc[2] == "no":
        bids.append(no)
    if row.iloc[2] == "maybe":
        bids.append(maybe)
    if row.iloc[2] == "yes":
        bids.append(yes)
# print(bids)

# # dictionary of combinations of reviewers and submissions with the preference of the reviewer for the corresponding submission
rs_score = dict(zip(var, bids))
# print(rs_score)

In [25]:
# # scores = list with all vars: [z3.If(r?s?, bid, 0), etc.]
# ifs = list(zip(var, bids, [0] * len(bids)))
# scores = []
# for i in ifs:
#     scores.append("z3.If{0}".format(i))
# for i in scores:
#     print(i.replace("'", ""), ',', end=' ')

In [15]:
# These are the default used scores, uncomment them to use them, THEY SHOULD MATCH the above values for yes, maybe and no
# others can be created by changing the yes, maybe and no to the desired values
# then the above code should be uncommmented, ran and used by pasting them into the list with the name scores (scores = [])
###############################################################################################################
# 90/10
# scores = [z3.If(r0s0, 90, 0) , z3.If(r0s1, 0, 0) , z3.If(r0s2, 0, 0) , z3.If(r0s4, 0, 0) , z3.If(r0s5, 0, 0) , z3.If(r0s7, 0, 0) , z3.If(r0s8, 0, 0) , z3.If(r0s9, 0, 0) , z3.If(r0s10, 0, 0) , z3.If(r0s12, 0, 0) , z3.If(r0s13, 0, 0) , z3.If(r0s14, 10, 0) , z3.If(r0s15, 0, 0) , z3.If(r0s16, 0, 0) , z3.If(r0s18, 10, 0) , z3.If(r0s19, 0, 0) , z3.If(r0s20, 0, 0) , z3.If(r0s21, 0, 0) , z3.If(r0s22, 0, 0) , z3.If(r0s23, 90, 0) , z3.If(r0s24, 90, 0) , z3.If(r0s25, 0, 0) , z3.If(r0s26, 10, 0) , z3.If(r0s27, 0, 0) , z3.If(r0s28, 90, 0) , z3.If(r0s29, 90, 0) , z3.If(r0s30, 90, 0) , z3.If(r0s31, 0, 0) , z3.If(r0s32, 90, 0) , z3.If(r0s34, 0, 0) , z3.If(r0s35, 0, 0) , z3.If(r0s36, 0, 0) , z3.If(r0s37, 90, 0) , z3.If(r0s38, 0, 0) , z3.If(r0s39, 0, 0) , z3.If(r0s40, 10, 0) , z3.If(r0s41, 0, 0) , z3.If(r0s42, 0, 0) , z3.If(r0s43, 0, 0) , z3.If(r0s44, 90, 0) , z3.If(r0s45, 0, 0) , z3.If(r0s46, 0, 0) , z3.If(r0s47, 0, 0) , z3.If(r0s48, 0, 0) , z3.If(r0s49, 0, 0) , z3.If(r0s50, 0, 0) , z3.If(r0s51, 0, 0) , z3.If(r1s0, 90, 0) , z3.If(r1s1, 0, 0) , z3.If(r1s2, 10, 0) , z3.If(r1s3, 90, 0) , z3.If(r1s4, 0, 0) , z3.If(r1s5, 0, 0) , z3.If(r1s6, 0, 0) , z3.If(r1s7, 0, 0) , z3.If(r1s8, 0, 0) , z3.If(r1s9, 90, 0) , z3.If(r1s10, 10, 0) , z3.If(r1s11, 90, 0) , z3.If(r1s12, 0, 0) , z3.If(r1s13, 0, 0) , z3.If(r1s14, 0, 0) , z3.If(r1s15, 0, 0) , z3.If(r1s16, 0, 0) , z3.If(r1s17, 10, 0) , z3.If(r1s18, 90, 0) , z3.If(r1s19, 0, 0) , z3.If(r1s20, 0, 0) , z3.If(r1s21, 0, 0) , z3.If(r1s22, 90, 0) , z3.If(r1s23, 0, 0) , z3.If(r1s24, 0, 0) , z3.If(r1s25, 0, 0) , z3.If(r1s26, 0, 0) , z3.If(r1s27, 0, 0) , z3.If(r1s28, 90, 0) , z3.If(r1s29, 10, 0) , z3.If(r1s30, 0, 0) , z3.If(r1s31, 0, 0) , z3.If(r1s32, 0, 0) , z3.If(r1s33, 0, 0) , z3.If(r1s35, 0, 0) , z3.If(r1s36, 10, 0) , z3.If(r1s37, 0, 0) , z3.If(r1s38, 0, 0) , z3.If(r1s39, 0, 0) , z3.If(r1s40, 90, 0) , z3.If(r1s41, 0, 0) , z3.If(r1s42, 0, 0) , z3.If(r1s43, 0, 0) , z3.If(r1s44, 0, 0) , z3.If(r1s45, 0, 0) , z3.If(r1s46, 0, 0) , z3.If(r1s47, 0, 0) , z3.If(r1s48, 0, 0) , z3.If(r1s49, 0, 0) , z3.If(r1s50, 0, 0) , z3.If(r1s51, 0, 0) , z3.If(r2s0, 0, 0) , z3.If(r2s1, 0, 0) , z3.If(r2s2, 90, 0) , z3.If(r2s3, 0, 0) , z3.If(r2s4, 0, 0) , z3.If(r2s5, 0, 0) , z3.If(r2s6, 0, 0) , z3.If(r2s7, 10, 0) , z3.If(r2s8, 0, 0) , z3.If(r2s9, 0, 0) , z3.If(r2s10, 0, 0) , z3.If(r2s11, 0, 0) , z3.If(r2s12, 90, 0) , z3.If(r2s14, 10, 0) , z3.If(r2s15, 90, 0) , z3.If(r2s18, 0, 0) , z3.If(r2s19, 0, 0) , z3.If(r2s20, 0, 0) , z3.If(r2s22, 0, 0) , z3.If(r2s23, 0, 0) , z3.If(r2s24, 0, 0) , z3.If(r2s25, 0, 0) , z3.If(r2s26, 0, 0) , z3.If(r2s27, 0, 0) , z3.If(r2s28, 0, 0) , z3.If(r2s29, 0, 0) , z3.If(r2s30, 0, 0) , z3.If(r2s31, 0, 0) , z3.If(r2s33, 0, 0) , z3.If(r2s34, 0, 0) , z3.If(r2s35, 0, 0) , z3.If(r2s36, 0, 0) , z3.If(r2s37, 10, 0) , z3.If(r2s38, 0, 0) , z3.If(r2s39, 0, 0) , z3.If(r2s40, 0, 0) , z3.If(r2s41, 0, 0) , z3.If(r2s42, 0, 0) , z3.If(r2s43, 0, 0) , z3.If(r2s44, 0, 0) , z3.If(r2s45, 0, 0) , z3.If(r2s46, 10, 0) , z3.If(r2s47, 0, 0) , z3.If(r2s48, 0, 0) , z3.If(r2s49, 10, 0) , z3.If(r2s50, 0, 0) , z3.If(r2s51, 10, 0) , z3.If(r3s0, 0, 0) , z3.If(r3s2, 90, 0) , z3.If(r3s3, 90, 0) , z3.If(r3s4, 0, 0) , z3.If(r3s5, 0, 0) , z3.If(r3s6, 90, 0) , z3.If(r3s7, 0, 0) , z3.If(r3s8, 10, 0) , z3.If(r3s10, 0, 0) , z3.If(r3s11, 10, 0) , z3.If(r3s12, 0, 0) , z3.If(r3s15, 0, 0) , z3.If(r3s17, 10, 0) , z3.If(r3s18, 10, 0) , z3.If(r3s19, 0, 0) , z3.If(r3s20, 0, 0) , z3.If(r3s21, 0, 0) , z3.If(r3s22, 0, 0) , z3.If(r3s23, 0, 0) , z3.If(r3s25, 90, 0) , z3.If(r3s31, 10, 0) , z3.If(r3s32, 90, 0) , z3.If(r3s33, 0, 0) , z3.If(r3s34, 0, 0) , z3.If(r3s35, 90, 0) , z3.If(r3s36, 90, 0) , z3.If(r3s37, 90, 0) , z3.If(r3s38, 90, 0) , z3.If(r3s39, 0, 0) , z3.If(r3s40, 90, 0) , z3.If(r3s41, 10, 0) , z3.If(r3s42, 90, 0) , z3.If(r3s43, 10, 0) , z3.If(r3s44, 0, 0) , z3.If(r3s45, 0, 0) , z3.If(r3s46, 10, 0) , z3.If(r3s47, 90, 0) , z3.If(r3s48, 0, 0) , z3.If(r3s49, 10, 0) , z3.If(r3s50, 90, 0) , z3.If(r3s51, 10, 0) , z3.If(r4s0, 10, 0) , z3.If(r4s1, 0, 0) , z3.If(r4s2, 10, 0) , z3.If(r4s3, 0, 0) , z3.If(r4s4, 90, 0) , z3.If(r4s5, 90, 0) , z3.If(r4s6, 0, 0) , z3.If(r4s7, 0, 0) , z3.If(r4s8, 0, 0) , z3.If(r4s9, 0, 0) , z3.If(r4s10, 90, 0) , z3.If(r4s11, 0, 0) , z3.If(r4s12, 0, 0) , z3.If(r4s13, 0, 0) , z3.If(r4s14, 0, 0) , z3.If(r4s15, 0, 0) , z3.If(r4s16, 10, 0) , z3.If(r4s17, 0, 0) , z3.If(r4s18, 90, 0) , z3.If(r4s19, 0, 0) , z3.If(r4s20, 0, 0) , z3.If(r4s21, 0, 0) , z3.If(r4s22, 10, 0) , z3.If(r4s23, 0, 0) , z3.If(r4s24, 0, 0) , z3.If(r4s25, 0, 0) , z3.If(r4s26, 0, 0) , z3.If(r4s27, 0, 0) , z3.If(r4s28, 0, 0) , z3.If(r4s29, 0, 0) , z3.If(r4s30, 0, 0) , z3.If(r4s31, 0, 0) , z3.If(r4s32, 10, 0) , z3.If(r4s33, 0, 0) , z3.If(r4s34, 0, 0) , z3.If(r4s35, 0, 0) , z3.If(r4s36, 0, 0) , z3.If(r4s37, 0, 0) , z3.If(r4s38, 0, 0) , z3.If(r4s39, 0, 0) , z3.If(r4s40, 0, 0) , z3.If(r4s41, 0, 0) , z3.If(r4s42, 0, 0) , z3.If(r4s43, 0, 0) , z3.If(r4s44, 0, 0) , z3.If(r4s45, 0, 0) , z3.If(r4s46, 0, 0) , z3.If(r4s47, 10, 0) , z3.If(r4s48, 0, 0) , z3.If(r4s49, 0, 0) , z3.If(r4s50, 0, 0) , z3.If(r4s51, 90, 0) , z3.If(r5s0, 90, 0) , z3.If(r5s1, 0, 0) , z3.If(r5s2, 10, 0) , z3.If(r5s3, 10, 0) , z3.If(r5s4, 10, 0) , z3.If(r5s5, 0, 0) , z3.If(r5s6, 0, 0) , z3.If(r5s7, 0, 0) , z3.If(r5s8, 90, 0) , z3.If(r5s9, 90, 0) , z3.If(r5s10, 0, 0) , z3.If(r5s11, 0, 0) , z3.If(r5s12, 0, 0) , z3.If(r5s13, 0, 0) , z3.If(r5s14, 0, 0) , z3.If(r5s15, 0, 0) , z3.If(r5s16, 0, 0) , z3.If(r5s17, 90, 0) , z3.If(r5s18, 0, 0) , z3.If(r5s19, 10, 0) , z3.If(r5s20, 0, 0) , z3.If(r5s21, 0, 0) , z3.If(r5s22, 10, 0) , z3.If(r5s23, 0, 0) , z3.If(r5s24, 90, 0) , z3.If(r5s25, 0, 0) , z3.If(r5s26, 0, 0) , z3.If(r5s27, 0, 0) , z3.If(r5s28, 10, 0) , z3.If(r5s29, 0, 0) , z3.If(r5s30, 0, 0) , z3.If(r5s31, 0, 0) , z3.If(r5s32, 0, 0) , z3.If(r5s33, 0, 0) , z3.If(r5s34, 10, 0) , z3.If(r5s35, 0, 0) , z3.If(r5s36, 0, 0) , z3.If(r5s37, 90, 0) , z3.If(r5s38, 0, 0) , z3.If(r5s39, 10, 0) , z3.If(r5s40, 0, 0) , z3.If(r5s41, 0, 0) , z3.If(r5s42, 0, 0) , z3.If(r5s43, 0, 0) , z3.If(r5s44, 0, 0) , z3.If(r5s45, 0, 0) , z3.If(r5s46, 0, 0) , z3.If(r5s47, 0, 0) , z3.If(r5s48, 90, 0) , z3.If(r5s49, 10, 0) , z3.If(r5s50, 0, 0) , z3.If(r5s51, 0, 0) , z3.If(r6s0, 10, 0) , z3.If(r6s1, 0, 0) , z3.If(r6s2, 10, 0) , z3.If(r6s3, 10, 0) , z3.If(r6s4, 10, 0) , z3.If(r6s5, 0, 0) , z3.If(r6s7, 0, 0) , z3.If(r6s8, 0, 0) , z3.If(r6s9, 10, 0) , z3.If(r6s11, 0, 0) , z3.If(r6s12, 0, 0) , z3.If(r6s13, 0, 0) , z3.If(r6s14, 0, 0) , z3.If(r6s15, 0, 0) , z3.If(r6s16, 0, 0) , z3.If(r6s17, 10, 0) , z3.If(r6s18, 90, 0) , z3.If(r6s19, 90, 0) , z3.If(r6s20, 0, 0) , z3.If(r6s21, 10, 0) , z3.If(r6s22, 0, 0) , z3.If(r6s23, 0, 0) , z3.If(r6s24, 10, 0) , z3.If(r6s25, 0, 0) , z3.If(r6s26, 0, 0) , z3.If(r6s27, 0, 0) , z3.If(r6s28, 90, 0) , z3.If(r6s29, 10, 0) , z3.If(r6s30, 90, 0) , z3.If(r6s31, 10, 0) , z3.If(r6s32, 0, 0) , z3.If(r6s33, 0, 0) , z3.If(r6s34, 0, 0) , z3.If(r6s36, 0, 0) , z3.If(r6s39, 0, 0) , z3.If(r6s40, 0, 0) , z3.If(r6s41, 0, 0) , z3.If(r6s42, 10, 0) , z3.If(r6s44, 0, 0) , z3.If(r6s45, 0, 0) , z3.If(r6s47, 90, 0) , z3.If(r6s48, 0, 0) , z3.If(r6s49, 90, 0) , z3.If(r6s50, 0, 0) , z3.If(r6s51, 10, 0) , z3.If(r7s0, 90, 0) , z3.If(r7s1, 0, 0) , z3.If(r7s2, 0, 0) , z3.If(r7s3, 10, 0) , z3.If(r7s4, 0, 0) , z3.If(r7s5, 0, 0) , z3.If(r7s6, 0, 0) , z3.If(r7s7, 10, 0) , z3.If(r7s10, 0, 0) , z3.If(r7s12, 0, 0) , z3.If(r7s13, 90, 0) , z3.If(r7s14, 0, 0) , z3.If(r7s15, 0, 0) , z3.If(r7s16, 0, 0) , z3.If(r7s17, 10, 0) , z3.If(r7s18, 10, 0) , z3.If(r7s19, 0, 0) , z3.If(r7s20, 0, 0) , z3.If(r7s22, 90, 0) , z3.If(r7s23, 90, 0) , z3.If(r7s24, 10, 0) , z3.If(r7s25, 0, 0) , z3.If(r7s26, 90, 0) , z3.If(r7s27, 0, 0) , z3.If(r7s28, 90, 0) , z3.If(r7s29, 90, 0) , z3.If(r7s30, 0, 0) , z3.If(r7s31, 90, 0) , z3.If(r7s32, 0, 0) , z3.If(r7s33, 0, 0) , z3.If(r7s34, 0, 0) , z3.If(r7s35, 0, 0) , z3.If(r7s36, 10, 0) , z3.If(r7s37, 90, 0) , z3.If(r7s38, 90, 0) , z3.If(r7s39, 0, 0) , z3.If(r7s40, 90, 0) , z3.If(r7s41, 0, 0) , z3.If(r7s42, 90, 0) , z3.If(r7s43, 90, 0) , z3.If(r7s44, 10, 0) , z3.If(r7s45, 0, 0) , z3.If(r7s46, 0, 0) , z3.If(r7s47, 90, 0) , z3.If(r7s49, 90, 0) , z3.If(r7s50, 0, 0) , z3.If(r7s51, 0, 0) , z3.If(r8s0, 0, 0) , z3.If(r8s1, 0, 0) , z3.If(r8s2, 0, 0) , z3.If(r8s3, 0, 0) , z3.If(r8s4, 0, 0) , z3.If(r8s5, 0, 0) , z3.If(r8s6, 0, 0) , z3.If(r8s7, 0, 0) , z3.If(r8s8, 0, 0) , z3.If(r8s9, 90, 0) , z3.If(r8s10, 0, 0) , z3.If(r8s11, 0, 0) , z3.If(r8s12, 0, 0) , z3.If(r8s14, 0, 0) , z3.If(r8s15, 0, 0) , z3.If(r8s18, 0, 0) , z3.If(r8s19, 0, 0) , z3.If(r8s20, 0, 0) , z3.If(r8s21, 0, 0) , z3.If(r8s22, 10, 0) , z3.If(r8s23, 10, 0) , z3.If(r8s24, 10, 0) , z3.If(r8s25, 10, 0) , z3.If(r8s26, 0, 0) , z3.If(r8s27, 0, 0) , z3.If(r8s28, 0, 0) , z3.If(r8s29, 0, 0) , z3.If(r8s30, 0, 0) , z3.If(r8s31, 10, 0) , z3.If(r8s33, 90, 0) , z3.If(r8s34, 90, 0) , z3.If(r8s35, 10, 0) , z3.If(r8s36, 0, 0) , z3.If(r8s37, 90, 0) , z3.If(r8s38, 90, 0) , z3.If(r8s39, 0, 0) , z3.If(r8s40, 10, 0) , z3.If(r8s41, 0, 0) , z3.If(r8s42, 10, 0) , z3.If(r8s43, 90, 0) , z3.If(r8s44, 0, 0) , z3.If(r8s45, 0, 0) , z3.If(r8s46, 0, 0) , z3.If(r8s47, 90, 0) , z3.If(r8s48, 0, 0) , z3.If(r8s49, 10, 0) , z3.If(r8s50, 10, 0) , z3.If(r8s51, 10, 0) , z3.If(r9s0, 90, 0) , z3.If(r9s1, 0, 0) , z3.If(r9s2, 0, 0) , z3.If(r9s3, 0, 0) , z3.If(r9s4, 0, 0) , z3.If(r9s5, 0, 0) , z3.If(r9s6, 0, 0) , z3.If(r9s7, 0, 0) , z3.If(r9s8, 0, 0) , z3.If(r9s9, 0, 0) , z3.If(r9s10, 0, 0) , z3.If(r9s11, 0, 0) , z3.If(r9s12, 0, 0) , z3.If(r9s13, 90, 0) , z3.If(r9s14, 90, 0) , z3.If(r9s15, 0, 0) , z3.If(r9s16, 0, 0) , z3.If(r9s17, 90, 0) , z3.If(r9s18, 0, 0) , z3.If(r9s19, 0, 0) , z3.If(r9s20, 0, 0) , z3.If(r9s21, 0, 0) , z3.If(r9s22, 0, 0) , z3.If(r9s23, 0, 0) , z3.If(r9s24, 90, 0) , z3.If(r9s25, 0, 0) , z3.If(r9s26, 10, 0) , z3.If(r9s27, 0, 0) , z3.If(r9s28, 90, 0) , z3.If(r9s29, 90, 0) , z3.If(r9s30, 0, 0) , z3.If(r9s31, 0, 0) , z3.If(r9s32, 90, 0) , z3.If(r9s33, 10, 0) , z3.If(r9s34, 0, 0) , z3.If(r9s35, 0, 0) , z3.If(r9s36, 90, 0) , z3.If(r9s37, 90, 0) , z3.If(r9s38, 0, 0) , z3.If(r9s39, 0, 0) , z3.If(r9s40, 90, 0) , z3.If(r9s41, 0, 0) , z3.If(r9s42, 0, 0) , z3.If(r9s43, 0, 0) , z3.If(r9s44, 90, 0) , z3.If(r9s45, 0, 0) , z3.If(r9s46, 90, 0) , z3.If(r9s47, 0, 0) , z3.If(r9s48, 0, 0) , z3.If(r9s49, 90, 0) , z3.If(r9s50, 0, 0) , z3.If(r9s51, 0, 0) , z3.If(r10s0, 0, 0) , z3.If(r10s1, 0, 0) , z3.If(r10s2, 0, 0) , z3.If(r10s3, 0, 0) , z3.If(r10s4, 0, 0) , z3.If(r10s5, 0, 0) , z3.If(r10s6, 0, 0) , z3.If(r10s7, 0, 0) , z3.If(r10s9, 90, 0) , z3.If(r10s10, 0, 0) , z3.If(r10s12, 90, 0) , z3.If(r10s13, 90, 0) , z3.If(r10s14, 0, 0) , z3.If(r10s15, 0, 0) , z3.If(r10s16, 0, 0) , z3.If(r10s17, 0, 0) , z3.If(r10s18, 90, 0) , z3.If(r10s19, 0, 0) , z3.If(r10s20, 0, 0) , z3.If(r10s21, 90, 0) , z3.If(r10s22, 0, 0) , z3.If(r10s23, 0, 0) , z3.If(r10s24, 0, 0) , z3.If(r10s25, 0, 0) , z3.If(r10s26, 0, 0) , z3.If(r10s27, 0, 0) , z3.If(r10s28, 90, 0) , z3.If(r10s29, 0, 0) , z3.If(r10s30, 90, 0) , z3.If(r10s31, 90, 0) , z3.If(r10s32, 90, 0) , z3.If(r10s33, 0, 0) , z3.If(r10s34, 0, 0) , z3.If(r10s35, 90, 0) , z3.If(r10s36, 0, 0) , z3.If(r10s37, 90, 0) , z3.If(r10s38, 0, 0) , z3.If(r10s39, 0, 0) , z3.If(r10s40, 90, 0) , z3.If(r10s41, 0, 0) , z3.If(r10s42, 0, 0) , z3.If(r10s43, 0, 0) , z3.If(r10s44, 0, 0) , z3.If(r10s45, 0, 0) , z3.If(r10s46, 0, 0) , z3.If(r10s47, 0, 0) , z3.If(r10s49, 0, 0) , z3.If(r10s50, 0, 0) , z3.If(r10s51, 0, 0) , z3.If(r11s0, 0, 0) , z3.If(r11s1, 0, 0) , z3.If(r11s2, 0, 0) , z3.If(r11s3, 90, 0) , z3.If(r11s4, 90, 0) , z3.If(r11s5, 10, 0) , z3.If(r11s6, 0, 0) , z3.If(r11s7, 0, 0) , z3.If(r11s8, 0, 0) , z3.If(r11s9, 90, 0) , z3.If(r11s10, 0, 0) , z3.If(r11s11, 90, 0) , z3.If(r11s12, 0, 0) , z3.If(r11s13, 90, 0) , z3.If(r11s14, 0, 0) , z3.If(r11s15, 0, 0) , z3.If(r11s16, 0, 0) , z3.If(r11s17, 0, 0) , z3.If(r11s18, 0, 0) , z3.If(r11s19, 0, 0) , z3.If(r11s20, 0, 0) , z3.If(r11s21, 0, 0) , z3.If(r11s22, 0, 0) , z3.If(r11s23, 0, 0) , z3.If(r11s24, 0, 0) , z3.If(r11s25, 90, 0) , z3.If(r11s26, 0, 0) , z3.If(r11s27, 0, 0) , z3.If(r11s28, 0, 0) , z3.If(r11s29, 0, 0) , z3.If(r11s30, 0, 0) , z3.If(r11s31, 0, 0) , z3.If(r11s32, 0, 0) , z3.If(r11s33, 0, 0) , z3.If(r11s34, 10, 0) , z3.If(r11s35, 0, 0) , z3.If(r11s36, 0, 0) , z3.If(r11s37, 0, 0) , z3.If(r11s38, 0, 0) , z3.If(r11s39, 0, 0) , z3.If(r11s40, 0, 0) , z3.If(r11s41, 0, 0) , z3.If(r11s43, 0, 0) , z3.If(r11s44, 0, 0) , z3.If(r11s45, 0, 0) , z3.If(r11s46, 0, 0) , z3.If(r11s48, 0, 0) , z3.If(r11s50, 0, 0) , z3.If(r11s51, 0, 0) , z3.If(r12s1, 0, 0) , z3.If(r12s2, 0, 0) , z3.If(r12s3, 0, 0) , z3.If(r12s4, 90, 0) , z3.If(r12s5, 90, 0) , z3.If(r12s6, 0, 0) , z3.If(r12s7, 0, 0) , z3.If(r12s8, 10, 0) , z3.If(r12s9, 0, 0) , z3.If(r12s10, 0, 0) , z3.If(r12s12, 0, 0) , z3.If(r12s13, 10, 0) , z3.If(r12s14, 0, 0) , z3.If(r12s15, 0, 0) , z3.If(r12s16, 10, 0) , z3.If(r12s19, 0, 0) , z3.If(r12s20, 0, 0) , z3.If(r12s21, 0, 0) , z3.If(r12s22, 0, 0) , z3.If(r12s23, 0, 0) , z3.If(r12s24, 90, 0) , z3.If(r12s25, 0, 0) , z3.If(r12s26, 0, 0) , z3.If(r12s27, 10, 0) , z3.If(r12s28, 0, 0) , z3.If(r12s29, 0, 0) , z3.If(r12s30, 0, 0) , z3.If(r12s31, 90, 0) , z3.If(r12s32, 90, 0) , z3.If(r12s33, 0, 0) , z3.If(r12s34, 90, 0) , z3.If(r12s35, 0, 0) , z3.If(r12s36, 0, 0) , z3.If(r12s37, 0, 0) , z3.If(r12s38, 90, 0) , z3.If(r12s39, 0, 0) , z3.If(r12s40, 10, 0) , z3.If(r12s41, 90, 0) , z3.If(r12s42, 0, 0) , z3.If(r12s43, 10, 0) , z3.If(r12s44, 10, 0) , z3.If(r12s45, 10, 0) , z3.If(r12s46, 0, 0) , z3.If(r12s48, 0, 0) , z3.If(r12s49, 0, 0) , z3.If(r12s50, 0, 0) , z3.If(r12s51, 0, 0) , z3.If(r13s0, 90, 0) , z3.If(r13s2, 10, 0) , z3.If(r13s3, 0, 0) , z3.If(r13s4, 0, 0) , z3.If(r13s5, 0, 0) , z3.If(r13s6, 0, 0) , z3.If(r13s7, 0, 0) , z3.If(r13s10, 0, 0) , z3.If(r13s12, 0, 0) , z3.If(r13s13, 10, 0) , z3.If(r13s14, 90, 0) , z3.If(r13s15, 0, 0) , z3.If(r13s16, 0, 0) , z3.If(r13s18, 0, 0) , z3.If(r13s19, 90, 0) , z3.If(r13s20, 0, 0) , z3.If(r13s22, 0, 0) , z3.If(r13s23, 0, 0) , z3.If(r13s25, 0, 0) , z3.If(r13s26, 0, 0) , z3.If(r13s27, 0, 0) , z3.If(r13s28, 0, 0) , z3.If(r13s29, 0, 0) , z3.If(r13s30, 0, 0) , z3.If(r13s31, 0, 0) , z3.If(r13s32, 0, 0) , z3.If(r13s33, 0, 0) , z3.If(r13s34, 0, 0) , z3.If(r13s35, 10, 0) , z3.If(r13s37, 10, 0) , z3.If(r13s38, 90, 0) , z3.If(r13s39, 0, 0) , z3.If(r13s40, 10, 0) , z3.If(r13s41, 0, 0) , z3.If(r13s42, 0, 0) , z3.If(r13s43, 90, 0) , z3.If(r13s44, 0, 0) , z3.If(r13s45, 0, 0) , z3.If(r13s46, 10, 0) , z3.If(r13s47, 10, 0) , z3.If(r13s49, 10, 0) , z3.If(r13s50, 0, 0) , z3.If(r13s51, 10, 0) , z3.If(r14s0, 0, 0) , z3.If(r14s1, 0, 0) , z3.If(r14s2, 0, 0) , z3.If(r14s4, 0, 0) , z3.If(r14s5, 0, 0) , z3.If(r14s7, 90, 0) , z3.If(r14s8, 90, 0) , z3.If(r14s9, 0, 0) , z3.If(r14s10, 0, 0) , z3.If(r14s11, 0, 0) , z3.If(r14s12, 0, 0) , z3.If(r14s13, 90, 0) , z3.If(r14s14, 0, 0) , z3.If(r14s15, 0, 0) , z3.If(r14s17, 10, 0) , z3.If(r14s18, 0, 0) , z3.If(r14s19, 10, 0) , z3.If(r14s20, 0, 0) , z3.If(r14s21, 0, 0) , z3.If(r14s22, 0, 0) , z3.If(r14s23, 0, 0) , z3.If(r14s24, 90, 0) , z3.If(r14s25, 10, 0) , z3.If(r14s26, 0, 0) , z3.If(r14s27, 0, 0) , z3.If(r14s28, 0, 0) , z3.If(r14s29, 0, 0) , z3.If(r14s30, 0, 0) , z3.If(r14s34, 90, 0) , z3.If(r14s36, 0, 0) , z3.If(r14s38, 0, 0) , z3.If(r14s39, 0, 0) , z3.If(r14s40, 0, 0) , z3.If(r14s41, 0, 0) , z3.If(r14s42, 90, 0) , z3.If(r14s43, 90, 0) , z3.If(r14s44, 90, 0) , z3.If(r14s45, 0, 0) , z3.If(r14s46, 0, 0) , z3.If(r14s47, 10, 0) , z3.If(r14s48, 90, 0) , z3.If(r14s49, 90, 0) , z3.If(r14s50, 0, 0) , z3.If(r14s51, 0, 0) , z3.If(r15s1, 0, 0) , z3.If(r15s2, 0, 0) , z3.If(r15s3, 0, 0) , z3.If(r15s4, 0, 0) , z3.If(r15s5, 0, 0) , z3.If(r15s6, 0, 0) , z3.If(r15s7, 0, 0) , z3.If(r15s8, 0, 0) , z3.If(r15s9, 90, 0) , z3.If(r15s10, 0, 0) , z3.If(r15s11, 0, 0) , z3.If(r15s12, 90, 0) , z3.If(r15s13, 90, 0) , z3.If(r15s14, 0, 0) , z3.If(r15s15, 0, 0) , z3.If(r15s16, 0, 0) , z3.If(r15s17, 0, 0) , z3.If(r15s19, 0, 0) , z3.If(r15s20, 0, 0) , z3.If(r15s21, 0, 0) , z3.If(r15s22, 90, 0) , z3.If(r15s23, 90, 0) , z3.If(r15s24, 90, 0) , z3.If(r15s25, 0, 0) , z3.If(r15s26, 0, 0) , z3.If(r15s27, 0, 0) , z3.If(r15s28, 0, 0) , z3.If(r15s29, 0, 0) , z3.If(r15s30, 0, 0) , z3.If(r15s31, 0, 0) , z3.If(r15s32, 0, 0) , z3.If(r15s33, 0, 0) , z3.If(r15s34, 90, 0) , z3.If(r15s35, 0, 0) , z3.If(r15s36, 0, 0) , z3.If(r15s37, 90, 0) , z3.If(r15s38, 0, 0) , z3.If(r15s39, 0, 0) , z3.If(r15s40, 0, 0) , z3.If(r15s41, 0, 0) , z3.If(r15s42, 0, 0) , z3.If(r15s43, 0, 0) , z3.If(r15s44, 0, 0) , z3.If(r15s45, 0, 0) , z3.If(r15s46, 0, 0) , z3.If(r15s48, 0, 0) , z3.If(r15s49, 0, 0) , z3.If(r15s50, 0, 0) , z3.If(r15s51, 90, 0) , z3.If(r16s1, 0, 0) , z3.If(r16s2, 10, 0) , z3.If(r16s3, 10, 0) , z3.If(r16s4, 90, 0) , z3.If(r16s5, 10, 0) , z3.If(r16s7, 90, 0) , z3.If(r16s8, 0, 0) , z3.If(r16s9, 0, 0) , z3.If(r16s11, 0, 0) , z3.If(r16s12, 10, 0) , z3.If(r16s13, 0, 0) , z3.If(r16s14, 0, 0) , z3.If(r16s15, 0, 0) , z3.If(r16s16, 0, 0) , z3.If(r16s17, 0, 0) , z3.If(r16s19, 0, 0) , z3.If(r16s20, 0, 0) , z3.If(r16s21, 0, 0) , z3.If(r16s22, 0, 0) , z3.If(r16s23, 0, 0) , z3.If(r16s24, 0, 0) , z3.If(r16s25, 0, 0) , z3.If(r16s26, 0, 0) , z3.If(r16s27, 0, 0) , z3.If(r16s28, 0, 0) , z3.If(r16s29, 0, 0) , z3.If(r16s30, 0, 0) , z3.If(r16s31, 90, 0) , z3.If(r16s32, 0, 0) , z3.If(r16s33, 0, 0) , z3.If(r16s34, 90, 0) , z3.If(r16s35, 0, 0) , z3.If(r16s36, 0, 0) , z3.If(r16s38, 0, 0) , z3.If(r16s39, 10, 0) , z3.If(r16s40, 0, 0) , z3.If(r16s41, 0, 0) , z3.If(r16s42, 0, 0) , z3.If(r16s43, 0, 0) , z3.If(r16s44, 10, 0) , z3.If(r16s45, 0, 0) , z3.If(r16s46, 10, 0) , z3.If(r16s48, 0, 0) , z3.If(r16s49, 0, 0) , z3.If(r16s50, 0, 0) , z3.If(r16s51, 90, 0) , z3.If(r17s0, 0, 0) , z3.If(r17s1, 0, 0) , z3.If(r17s2, 90, 0) , z3.If(r17s3, 90, 0) , z3.If(r17s4, 0, 0) , z3.If(r17s5, 0, 0) , z3.If(r17s7, 90, 0) , z3.If(r17s8, 0, 0) , z3.If(r17s11, 0, 0) , z3.If(r17s12, 0, 0) , z3.If(r17s13, 90, 0) , z3.If(r17s14, 0, 0) , z3.If(r17s15, 0, 0) , z3.If(r17s16, 10, 0) , z3.If(r17s17, 10, 0) , z3.If(r17s18, 90, 0) , z3.If(r17s19, 10, 0) , z3.If(r17s20, 0, 0) , z3.If(r17s21, 0, 0) , z3.If(r17s22, 0, 0) , z3.If(r17s23, 0, 0) , z3.If(r17s25, 0, 0) , z3.If(r17s26, 0, 0) , z3.If(r17s27, 10, 0) , z3.If(r17s28, 0, 0) , z3.If(r17s29, 10, 0) , z3.If(r17s30, 10, 0) , z3.If(r17s31, 0, 0) , z3.If(r17s32, 90, 0) , z3.If(r17s33, 0, 0) , z3.If(r17s34, 10, 0) , z3.If(r17s35, 90, 0) , z3.If(r17s36, 0, 0) , z3.If(r17s37, 10, 0) , z3.If(r17s38, 10, 0) , z3.If(r17s39, 0, 0) , z3.If(r17s40, 0, 0) , z3.If(r17s41, 10, 0) , z3.If(r17s42, 0, 0) , z3.If(r17s43, 10, 0) , z3.If(r17s44, 0, 0) , z3.If(r17s45, 0, 0) , z3.If(r17s46, 0, 0) , z3.If(r17s47, 90, 0) , z3.If(r17s48, 0, 0) , z3.If(r17s49, 0, 0) , z3.If(r17s50, 0, 0) , z3.If(r17s51, 0, 0) , z3.If(r18s0, 90, 0) , z3.If(r18s1, 0, 0) , z3.If(r18s2, 0, 0) , z3.If(r18s3, 0, 0) , z3.If(r18s4, 90, 0) , z3.If(r18s5, 0, 0) , z3.If(r18s6, 0, 0) , z3.If(r18s7, 0, 0) , z3.If(r18s8, 0, 0) , z3.If(r18s10, 0, 0) , z3.If(r18s11, 0, 0) , z3.If(r18s12, 90, 0) , z3.If(r18s13, 0, 0) , z3.If(r18s14, 0, 0) , z3.If(r18s15, 90, 0) , z3.If(r18s16, 0, 0) , z3.If(r18s17, 0, 0) , z3.If(r18s18, 90, 0) , z3.If(r18s19, 0, 0) , z3.If(r18s20, 0, 0) , z3.If(r18s21, 0, 0) , z3.If(r18s22, 0, 0) , z3.If(r18s23, 90, 0) , z3.If(r18s25, 0, 0) , z3.If(r18s26, 0, 0) , z3.If(r18s27, 0, 0) , z3.If(r18s28, 0, 0) , z3.If(r18s29, 0, 0) , z3.If(r18s30, 0, 0) , z3.If(r18s31, 0, 0) , z3.If(r18s32, 0, 0) , z3.If(r18s33, 90, 0) , z3.If(r18s34, 0, 0) , z3.If(r18s35, 0, 0) , z3.If(r18s36, 0, 0) , z3.If(r18s37, 0, 0) , z3.If(r18s38, 0, 0) , z3.If(r18s39, 0, 0) , z3.If(r18s40, 0, 0) , z3.If(r18s41, 0, 0) , z3.If(r18s42, 0, 0) , z3.If(r18s43, 0, 0) , z3.If(r18s44, 0, 0) , z3.If(r18s45, 90, 0) , z3.If(r18s46, 0, 0) , z3.If(r18s47, 0, 0) , z3.If(r18s48, 90, 0) , z3.If(r18s49, 0, 0) , z3.If(r18s50, 0, 0) , z3.If(r18s51, 90, 0) , z3.If(r19s0, 10, 0) , z3.If(r19s1, 90, 0) , z3.If(r19s3, 10, 0) , z3.If(r19s4, 90, 0) , z3.If(r19s5, 0, 0) , z3.If(r19s6, 10, 0) , z3.If(r19s7, 10, 0) , z3.If(r19s8, 0, 0) , z3.If(r19s9, 10, 0) , z3.If(r19s10, 0, 0) , z3.If(r19s11, 0, 0) , z3.If(r19s12, 0, 0) , z3.If(r19s13, 0, 0) , z3.If(r19s14, 0, 0) , z3.If(r19s15, 90, 0) , z3.If(r19s16, 90, 0) , z3.If(r19s17, 90, 0) , z3.If(r19s18, 90, 0) , z3.If(r19s19, 10, 0) , z3.If(r19s20, 0, 0) , z3.If(r19s21, 0, 0) , z3.If(r19s22, 90, 0) , z3.If(r19s23, 0, 0) , z3.If(r19s24, 10, 0) , z3.If(r19s25, 0, 0) , z3.If(r19s26, 0, 0) , z3.If(r19s27, 0, 0) , z3.If(r19s28, 0, 0) , z3.If(r19s29, 0, 0) , z3.If(r19s30, 0, 0) , z3.If(r19s31, 10, 0) , z3.If(r19s32, 0, 0) , z3.If(r19s33, 0, 0) , z3.If(r19s34, 0, 0) , z3.If(r19s35, 0, 0) , z3.If(r19s36, 0, 0) , z3.If(r19s37, 90, 0) , z3.If(r19s38, 0, 0) , z3.If(r19s39, 0, 0) , z3.If(r19s40, 0, 0) , z3.If(r19s41, 0, 0) , z3.If(r19s42, 90, 0) ,z3.If(r19s43, 0, 0) , z3.If(r19s44, 90, 0) , z3.If(r19s45, 0, 0) , z3.If(r19s46, 0, 0) , z3.If(r19s47, 0, 0) , z3.If(r19s48, 0, 0) , z3.If(r19s49, 90, 0) , z3.If(r19s50, 0, 0) , z3.If(r19s51, 90, 0) , z3.If(r20s0, 0, 0) , z3.If(r20s1, 0, 0) , z3.If(r20s2, 0, 0) , z3.If(r20s3, 0, 0) , z3.If(r20s4, 0, 0) , z3.If(r20s5, 0, 0) , z3.If(r20s6, 90, 0) , z3.If(r20s7, 0, 0) , z3.If(r20s8, 0, 0) , z3.If(r20s9, 90, 0) , z3.If(r20s10, 0, 0) , z3.If(r20s11, 0, 0) , z3.If(r20s12, 0, 0) , z3.If(r20s13, 90, 0) , z3.If(r20s15, 0, 0) , z3.If(r20s17, 90, 0) , z3.If(r20s18, 0, 0) , z3.If(r20s19, 0, 0) , z3.If(r20s20, 0, 0) , z3.If(r20s21, 0, 0) , z3.If(r20s22, 0, 0) , z3.If(r20s23, 0, 0) , z3.If(r20s24, 90, 0) , z3.If(r20s25, 0, 0) , z3.If(r20s26, 90, 0) , z3.If(r20s30, 90, 0) , z3.If(r20s31, 0, 0) , z3.If(r20s32, 90, 0) , z3.If(r20s33, 0, 0) , z3.If(r20s34, 90, 0) , z3.If(r20s35, 0, 0) , z3.If(r20s36, 90, 0) , z3.If(r20s37, 90, 0) , z3.If(r20s38, 0, 0) , z3.If(r20s39, 0, 0) , z3.If(r20s40, 0, 0) , z3.If(r20s41, 0, 0) , z3.If(r20s42, 0, 0) , z3.If(r20s43, 0, 0) , z3.If(r20s44, 0, 0) , z3.If(r20s45, 90, 0) , z3.If(r20s46, 0, 0) , z3.If(r20s47, 90, 0) , z3.If(r20s48, 0, 0) , z3.If(r20s49, 0, 0) , z3.If(r20s50, 0, 0) , z3.If(r20s51, 0, 0) , z3.If(r21s1, 0, 0) , z3.If(r21s2, 90, 0) , z3.If(r21s3, 0, 0) , z3.If(r21s4, 0, 0) , z3.If(r21s5, 90, 0) , z3.If(r21s6, 0, 0) , z3.If(r21s7, 0, 0) , z3.If(r21s8, 0, 0) , z3.If(r21s10, 0, 0) , z3.If(r21s12, 90, 0) , z3.If(r21s14, 10, 0) , z3.If(r21s15, 90, 0) , z3.If(r21s18, 90, 0) , z3.If(r21s19, 0, 0) , z3.If(r21s20, 0, 0) , z3.If(r21s22, 0, 0) , z3.If(r21s23, 10, 0) , z3.If(r21s24, 0, 0) , z3.If(r21s25, 0, 0) , z3.If(r21s26, 0, 0) , z3.If(r21s27, 0, 0) , z3.If(r21s28, 0, 0) , z3.If(r21s29, 0, 0) , z3.If(r21s30, 0, 0) , z3.If(r21s31, 0, 0) , z3.If(r21s33, 0, 0) , z3.If(r21s34, 0, 0) , z3.If(r21s35, 90, 0) , z3.If(r21s37, 10, 0) , z3.If(r21s38, 10, 0) , z3.If(r21s39, 0, 0) , z3.If(r21s40, 0, 0) , z3.If(r21s41, 0, 0) , z3.If(r21s42, 0, 0) , z3.If(r21s43, 10, 0) , z3.If(r21s44, 0, 0) , z3.If(r21s45, 0, 0) , z3.If(r21s46, 0, 0) , z3.If(r21s47, 0, 0) , z3.If(r21s48, 0, 0) , z3.If(r21s49, 0, 0) , z3.If(r21s50, 10, 0) , z3.If(r21s51, 0, 0) , z3.If(r22s0, 0, 0) , z3.If(r22s1, 0, 0) , z3.If(r22s2, 0, 0) , z3.If(r22s3, 0, 0) , z3.If(r22s4, 90, 0) , z3.If(r22s5, 90, 0) , z3.If(r22s6, 0, 0) , z3.If(r22s7, 0, 0) , z3.If(r22s8, 0, 0) , z3.If(r22s9, 10, 0) , z3.If(r22s10, 10, 0) , z3.If(r22s11, 0, 0) , z3.If(r22s12, 0, 0) , z3.If(r22s13, 10, 0) , z3.If(r22s14, 0, 0) , z3.If(r22s15, 0, 0) , z3.If(r22s16, 90, 0) , z3.If(r22s17, 0, 0) , z3.If(r22s18, 0, 0) , z3.If(r22s19, 0, 0) , z3.If(r22s20, 10, 0) , z3.If(r22s21, 0, 0) , z3.If(r22s22, 0, 0) , z3.If(r22s23, 0, 0) , z3.If(r22s24, 0, 0) , z3.If(r22s25, 0, 0) , z3.If(r22s26, 0, 0) , z3.If(r22s27, 0, 0) , z3.If(r22s28, 0, 0) , z3.If(r22s29, 0, 0) , z3.If(r22s30, 0, 0) , z3.If(r22s31, 0, 0) , z3.If(r22s32, 90, 0) , z3.If(r22s33, 10, 0) , z3.If(r22s34, 10, 0) , z3.If(r22s35, 0, 0) , z3.If(r22s36, 0, 0) , z3.If(r22s37, 0, 0) , z3.If(r22s38, 0, 0) , z3.If(r22s39, 90, 0) , z3.If(r22s40, 0, 0) , z3.If(r22s41, 0, 0) , z3.If(r22s42, 10, 0) , z3.If(r22s43, 0, 0) , z3.If(r22s44, 0, 0) , z3.If(r22s45, 90, 0) , z3.If(r22s46, 0, 0) , z3.If(r22s47, 0, 0) , z3.If(r22s48, 0, 0) , z3.If(r22s49, 0, 0) , z3.If(r22s50, 0, 0) , z3.If(r22s51, 0, 0) , z3.If(r23s0, 0, 0) , z3.If(r23s1, 0, 0) , z3.If(r23s2, 0, 0) , z3.If(r23s3, 90, 0) , z3.If(r23s4, 90, 0) , z3.If(r23s5, 0, 0) , z3.If(r23s6, 10, 0) , z3.If(r23s7, 0, 0) , z3.If(r23s8, 10, 0) , z3.If(r23s10, 90, 0) , z3.If(r23s11, 0, 0) , z3.If(r23s12, 0, 0) , z3.If(r23s13, 0, 0) , z3.If(r23s14, 0, 0) , z3.If(r23s15, 0, 0) , z3.If(r23s16, 90, 0) , z3.If(r23s17, 0, 0) , z3.If(r23s18, 0, 0) , z3.If(r23s19, 0, 0) , z3.If(r23s20, 0, 0) , z3.If(r23s21, 0, 0) , z3.If(r23s22, 0, 0) , z3.If(r23s23, 0, 0) , z3.If(r23s25, 0, 0) , z3.If(r23s26, 0, 0) , z3.If(r23s27, 0, 0) , z3.If(r23s28, 0, 0) , z3.If(r23s29, 0, 0) , z3.If(r23s30, 0, 0) , z3.If(r23s31, 10, 0) , z3.If(r23s32, 90, 0) , z3.If(r23s33, 0, 0) , z3.If(r23s34, 10, 0) , z3.If(r23s35, 0, 0) , z3.If(r23s36, 0, 0) , z3.If(r23s37, 0, 0) , z3.If(r23s38, 0, 0) , z3.If(r23s39, 90, 0) , z3.If(r23s40, 0, 0) , z3.If(r23s41, 0, 0) , z3.If(r23s42, 90, 0) , z3.If(r23s43, 0, 0) , z3.If(r23s44, 0, 0) , z3.If(r23s45, 90, 0) , z3.If(r23s46, 0, 0) , z3.If(r23s47, 0, 0) , z3.If(r23s48, 10, 0) , z3.If(r23s49, 0, 0) , z3.If(r23s50, 0, 0) , z3.If(r23s51, 0, 0)]
# 80/20
# scores = [z3.If(r0s0, 80, 0) , z3.If(r0s1, 0, 0) , z3.If(r0s2, 0, 0) , z3.If(r0s4, 0, 0) , z3.If(r0s5, 0, 0) , z3.If(r0s7, 0, 0) , z3.If(r0s8, 0, 0) , z3.If(r0s9, 0, 0) , z3.If(r0s10, 0, 0) , z3.If(r0s12, 0, 0) , z3.If(r0s13, 0, 0) , z3.If(r0s14, 20, 0) , z3.If(r0s15, 0, 0) , z3.If(r0s16, 0, 0) , z3.If(r0s18, 20, 0) , z3.If(r0s19, 0, 0) , z3.If(r0s20, 0, 0) , z3.If(r0s21, 0, 0) , z3.If(r0s22, 0, 0) , z3.If(r0s23, 80, 0) , z3.If(r0s24, 80, 0) , z3.If(r0s25, 0, 0) , z3.If(r0s26, 20, 0) , z3.If(r0s27, 0, 0) , z3.If(r0s28, 80, 0) , z3.If(r0s29, 80, 0) , z3.If(r0s30, 80, 0) , z3.If(r0s31, 0, 0) , z3.If(r0s32, 80, 0) , z3.If(r0s34, 0, 0) , z3.If(r0s35, 0, 0) , z3.If(r0s36, 0, 0) , z3.If(r0s37, 80, 0) , z3.If(r0s38, 0, 0) , z3.If(r0s39, 0, 0) , z3.If(r0s40, 20, 0) , z3.If(r0s41, 0, 0) , z3.If(r0s42, 0, 0) , z3.If(r0s43, 0, 0) , z3.If(r0s44, 80, 0) , z3.If(r0s45, 0, 0) , z3.If(r0s46, 0, 0) , z3.If(r0s47, 0, 0) , z3.If(r0s48, 0, 0) , z3.If(r0s49, 0, 0) , z3.If(r0s50, 0, 0) , z3.If(r0s51, 0, 0) , z3.If(r1s0, 80, 0) , z3.If(r1s1, 0, 0) , z3.If(r1s2, 20, 0) , z3.If(r1s3, 80, 0) , z3.If(r1s4, 0, 0) , z3.If(r1s5, 0, 0) , z3.If(r1s6, 0, 0) , z3.If(r1s7, 0, 0) , z3.If(r1s8, 0, 0) , z3.If(r1s9, 80, 0) , z3.If(r1s10, 20, 0) , z3.If(r1s11, 80, 0) , z3.If(r1s12, 0, 0) , z3.If(r1s13, 0, 0) , z3.If(r1s14, 0, 0) , z3.If(r1s15, 0, 0) , z3.If(r1s16, 0, 0) , z3.If(r1s17, 20, 0) , z3.If(r1s18, 80, 0) , z3.If(r1s19, 0, 0) , z3.If(r1s20, 0, 0) , z3.If(r1s21, 0, 0) , z3.If(r1s22, 80, 0) , z3.If(r1s23, 0, 0) , z3.If(r1s24, 0, 0) , z3.If(r1s25, 0, 0) , z3.If(r1s26, 0, 0) , z3.If(r1s27, 0, 0) , z3.If(r1s28, 80, 0) , z3.If(r1s29, 20, 0) , z3.If(r1s30, 0, 0) , z3.If(r1s31, 0, 0) , z3.If(r1s32, 0, 0) , z3.If(r1s33, 0, 0) , z3.If(r1s35, 0, 0) , z3.If(r1s36, 20, 0) , z3.If(r1s37, 0, 0) , z3.If(r1s38, 0, 0) , z3.If(r1s39, 0, 0) , z3.If(r1s40, 80, 0) , z3.If(r1s41, 0, 0) , z3.If(r1s42, 0, 0) , z3.If(r1s43, 0, 0) , z3.If(r1s44, 0, 0) , z3.If(r1s45, 0, 0) , z3.If(r1s46, 0, 0) , z3.If(r1s47, 0, 0) , z3.If(r1s48, 0, 0) , z3.If(r1s49, 0, 0) , z3.If(r1s50, 0, 0) , z3.If(r1s51, 0, 0) , z3.If(r2s0, 0, 0) , z3.If(r2s1, 0, 0) , z3.If(r2s2, 80, 0) , z3.If(r2s3, 0, 0) , z3.If(r2s4, 0, 0) , z3.If(r2s5, 0, 0) , z3.If(r2s6, 0, 0) , z3.If(r2s7, 20, 0) , z3.If(r2s8, 0, 0) , z3.If(r2s9, 0, 0) , z3.If(r2s10, 0, 0) , z3.If(r2s11, 0, 0) , z3.If(r2s12, 80, 0) , z3.If(r2s14, 20, 0) , z3.If(r2s15, 80, 0) , z3.If(r2s18, 0, 0) , z3.If(r2s19, 0, 0) , z3.If(r2s20, 0, 0) , z3.If(r2s22, 0, 0) , z3.If(r2s23, 0, 0) , z3.If(r2s24, 0, 0) , z3.If(r2s25, 0, 0) , z3.If(r2s26, 0, 0) , z3.If(r2s27, 0, 0) , z3.If(r2s28, 0, 0) , z3.If(r2s29, 0, 0) , z3.If(r2s30, 0, 0) , z3.If(r2s31, 0, 0) , z3.If(r2s33, 0, 0) , z3.If(r2s34, 0, 0) , z3.If(r2s35, 0, 0) , z3.If(r2s36, 0, 0) , z3.If(r2s37, 20, 0) , z3.If(r2s38, 0, 0) , z3.If(r2s39, 0, 0) , z3.If(r2s40, 0, 0) , z3.If(r2s41, 0, 0) , z3.If(r2s42, 0, 0) , z3.If(r2s43, 0, 0) , z3.If(r2s44, 0, 0) , z3.If(r2s45, 0, 0) , z3.If(r2s46, 20, 0) , z3.If(r2s47, 0, 0) , z3.If(r2s48, 0, 0) , z3.If(r2s49, 20, 0) , z3.If(r2s50, 0, 0) , z3.If(r2s51, 20, 0) , z3.If(r3s0, 0, 0) , z3.If(r3s2, 80, 0) , z3.If(r3s3, 80, 0) , z3.If(r3s4, 0, 0) , z3.If(r3s5, 0, 0) , z3.If(r3s6, 80, 0) , z3.If(r3s7, 0, 0) , z3.If(r3s8, 20, 0) , z3.If(r3s10, 0, 0) , z3.If(r3s11, 20, 0) , z3.If(r3s12, 0, 0) , z3.If(r3s15, 0, 0) , z3.If(r3s17, 20, 0) , z3.If(r3s18, 20, 0) , z3.If(r3s19, 0, 0) , z3.If(r3s20, 0, 0) , z3.If(r3s21, 0, 0) , z3.If(r3s22, 0, 0) , z3.If(r3s23, 0, 0) , z3.If(r3s25, 80, 0) , z3.If(r3s31, 20, 0) , z3.If(r3s32, 80, 0) , z3.If(r3s33, 0, 0) , z3.If(r3s34, 0, 0) , z3.If(r3s35, 80, 0) , z3.If(r3s36, 80, 0) , z3.If(r3s37, 80, 0) , z3.If(r3s38, 80, 0) , z3.If(r3s39, 0, 0) , z3.If(r3s40, 80, 0) , z3.If(r3s41, 20, 0) , z3.If(r3s42, 80, 0) , z3.If(r3s43, 20, 0) , z3.If(r3s44, 0, 0) , z3.If(r3s45, 0, 0) , z3.If(r3s46, 20, 0) , z3.If(r3s47, 80, 0) , z3.If(r3s48, 0, 0) , z3.If(r3s49, 20, 0) , z3.If(r3s50, 80, 0) , z3.If(r3s51, 20, 0) , z3.If(r4s0, 20, 0) , z3.If(r4s1, 0, 0) , z3.If(r4s2, 20, 0) , z3.If(r4s3, 0, 0) , z3.If(r4s4, 80, 0) , z3.If(r4s5, 80, 0) , z3.If(r4s6, 0, 0) , z3.If(r4s7, 0, 0) , z3.If(r4s8, 0, 0) , z3.If(r4s9, 0, 0) , z3.If(r4s10, 80, 0) , z3.If(r4s11, 0, 0) , z3.If(r4s12, 0, 0) , z3.If(r4s13, 0, 0) , z3.If(r4s14, 0, 0) , z3.If(r4s15, 0, 0) , z3.If(r4s16, 20, 0) , z3.If(r4s17, 0, 0) , z3.If(r4s18, 80, 0) , z3.If(r4s19, 0, 0) , z3.If(r4s20, 0, 0) , z3.If(r4s21, 0, 0) , z3.If(r4s22, 20, 0) , z3.If(r4s23, 0, 0) , z3.If(r4s24, 0, 0) , z3.If(r4s25, 0, 0) , z3.If(r4s26, 0, 0) , z3.If(r4s27, 0, 0) , z3.If(r4s28, 0, 0) , z3.If(r4s29, 0, 0) , z3.If(r4s30, 0, 0) , z3.If(r4s31, 0, 0) , z3.If(r4s32, 20, 0) , z3.If(r4s33, 0, 0) , z3.If(r4s34, 0, 0) , z3.If(r4s35, 0, 0) , z3.If(r4s36, 0, 0) , z3.If(r4s37, 0, 0) , z3.If(r4s38, 0, 0) , z3.If(r4s39, 0, 0) , z3.If(r4s40, 0, 0) , z3.If(r4s41, 0, 0) , z3.If(r4s42, 0, 0) , z3.If(r4s43, 0, 0) , z3.If(r4s44, 0, 0) , z3.If(r4s45, 0, 0) , z3.If(r4s46, 0, 0) , z3.If(r4s47, 20, 0) , z3.If(r4s48, 0, 0) , z3.If(r4s49, 0, 0) , z3.If(r4s50, 0, 0) , z3.If(r4s51, 80, 0) , z3.If(r5s0, 80, 0) , z3.If(r5s1, 0, 0) , z3.If(r5s2, 20, 0) , z3.If(r5s3, 20, 0) , z3.If(r5s4, 20, 0) , z3.If(r5s5, 0, 0) , z3.If(r5s6, 0, 0) , z3.If(r5s7, 0, 0) , z3.If(r5s8, 80, 0) , z3.If(r5s9, 80, 0) , z3.If(r5s10, 0, 0) , z3.If(r5s11, 0, 0) , z3.If(r5s12, 0, 0) , z3.If(r5s13, 0, 0) , z3.If(r5s14, 0, 0) , z3.If(r5s15, 0, 0) , z3.If(r5s16, 0, 0) , z3.If(r5s17, 80, 0) , z3.If(r5s18, 0, 0) , z3.If(r5s19, 20, 0) , z3.If(r5s20, 0, 0) , z3.If(r5s21, 0, 0) , z3.If(r5s22, 20, 0) , z3.If(r5s23, 0, 0) , z3.If(r5s24, 80, 0) , z3.If(r5s25, 0, 0) , z3.If(r5s26, 0, 0) , z3.If(r5s27, 0, 0) , z3.If(r5s28, 20, 0) , z3.If(r5s29, 0, 0) , z3.If(r5s30, 0, 0) , z3.If(r5s31, 0, 0) , z3.If(r5s32, 0, 0) , z3.If(r5s33, 0, 0) , z3.If(r5s34, 20, 0) , z3.If(r5s35, 0, 0) , z3.If(r5s36, 0, 0) , z3.If(r5s37, 80, 0) , z3.If(r5s38, 0, 0) , z3.If(r5s39, 20, 0) , z3.If(r5s40, 0, 0) , z3.If(r5s41, 0, 0) , z3.If(r5s42, 0, 0) , z3.If(r5s43, 0, 0) , z3.If(r5s44, 0, 0) , z3.If(r5s45, 0, 0) , z3.If(r5s46, 0, 0) , z3.If(r5s47, 0, 0) , z3.If(r5s48, 80, 0) , z3.If(r5s49, 20, 0) , z3.If(r5s50, 0, 0) , z3.If(r5s51, 0, 0) , z3.If(r6s0, 20, 0) , z3.If(r6s1, 0, 0) , z3.If(r6s2, 20, 0) , z3.If(r6s3, 20, 0) , z3.If(r6s4, 20, 0) , z3.If(r6s5, 0, 0) , z3.If(r6s7, 0, 0) , z3.If(r6s8, 0, 0) , z3.If(r6s9, 20, 0) , z3.If(r6s11, 0, 0) , z3.If(r6s12, 0, 0) , z3.If(r6s13, 0, 0) , z3.If(r6s14, 0, 0) , z3.If(r6s15, 0, 0) , z3.If(r6s16, 0, 0) , z3.If(r6s17, 20, 0) , z3.If(r6s18, 80, 0) , z3.If(r6s19, 80, 0) , z3.If(r6s20, 0, 0) , z3.If(r6s21, 20, 0) , z3.If(r6s22, 0, 0) , z3.If(r6s23, 0, 0) , z3.If(r6s24, 20, 0) , z3.If(r6s25, 0, 0) , z3.If(r6s26, 0, 0) , z3.If(r6s27, 0, 0) , z3.If(r6s28, 80, 0) , z3.If(r6s29, 20, 0) , z3.If(r6s30, 80, 0) , z3.If(r6s31, 20, 0) , z3.If(r6s32, 0, 0) , z3.If(r6s33, 0, 0) , z3.If(r6s34, 0, 0) , z3.If(r6s36, 0, 0) , z3.If(r6s39, 0, 0) , z3.If(r6s40, 0, 0) , z3.If(r6s41, 0, 0) , z3.If(r6s42, 20, 0) , z3.If(r6s44, 0, 0) , z3.If(r6s45, 0, 0) , z3.If(r6s47, 80, 0) , z3.If(r6s48, 0, 0) , z3.If(r6s49, 80, 0) , z3.If(r6s50, 0, 0) , z3.If(r6s51, 20, 0) , z3.If(r7s0, 80, 0) , z3.If(r7s1, 0, 0) , z3.If(r7s2, 0, 0) , z3.If(r7s3, 20, 0) , z3.If(r7s4, 0, 0) , z3.If(r7s5, 0, 0) , z3.If(r7s6, 0, 0) , z3.If(r7s7, 20, 0) , z3.If(r7s10, 0, 0) , z3.If(r7s12, 0, 0) , z3.If(r7s13, 80, 0) , z3.If(r7s14, 0, 0) , z3.If(r7s15, 0, 0) , z3.If(r7s16, 0, 0) , z3.If(r7s17, 20, 0) , z3.If(r7s18, 20, 0) , z3.If(r7s19, 0, 0) , z3.If(r7s20, 0, 0) , z3.If(r7s22, 80, 0) , z3.If(r7s23, 80, 0) , z3.If(r7s24, 20, 0) , z3.If(r7s25, 0, 0) , z3.If(r7s26, 80, 0) , z3.If(r7s27, 0, 0) , z3.If(r7s28, 80, 0) , z3.If(r7s29, 80, 0) , z3.If(r7s30, 0, 0) , z3.If(r7s31, 80, 0) , z3.If(r7s32, 0, 0) , z3.If(r7s33, 0, 0) , z3.If(r7s34, 0, 0) , z3.If(r7s35, 0, 0) , z3.If(r7s36, 20, 0) , z3.If(r7s37, 80, 0) , z3.If(r7s38, 80, 0) , z3.If(r7s39, 0, 0) , z3.If(r7s40, 80, 0) , z3.If(r7s41, 0, 0) , z3.If(r7s42, 80, 0) , z3.If(r7s43, 80, 0) , z3.If(r7s44, 20, 0) , z3.If(r7s45, 0, 0) , z3.If(r7s46, 0, 0) , z3.If(r7s47, 80, 0) , z3.If(r7s49, 80, 0) , z3.If(r7s50, 0, 0) , z3.If(r7s51, 0, 0) , z3.If(r8s0, 0, 0) , z3.If(r8s1, 0, 0) , z3.If(r8s2, 0, 0) , z3.If(r8s3, 0, 0) , z3.If(r8s4, 0, 0) , z3.If(r8s5, 0, 0) , z3.If(r8s6, 0, 0) , z3.If(r8s7, 0, 0) , z3.If(r8s8, 0, 0) , z3.If(r8s9, 80, 0) , z3.If(r8s10, 0, 0) , z3.If(r8s11, 0, 0) , z3.If(r8s12, 0, 0) , z3.If(r8s14, 0, 0) , z3.If(r8s15, 0, 0) , z3.If(r8s18, 0, 0) , z3.If(r8s19, 0, 0) , z3.If(r8s20, 0, 0) , z3.If(r8s21, 0, 0) , z3.If(r8s22, 20, 0) , z3.If(r8s23, 20, 0) , z3.If(r8s24, 20, 0) , z3.If(r8s25, 20, 0) , z3.If(r8s26, 0, 0) , z3.If(r8s27, 0, 0) , z3.If(r8s28, 0, 0) , z3.If(r8s29, 0, 0) , z3.If(r8s30, 0, 0) , z3.If(r8s31, 20, 0) , z3.If(r8s33, 80, 0) , z3.If(r8s34, 80, 0) , z3.If(r8s35, 20, 0) , z3.If(r8s36, 0, 0) , z3.If(r8s37, 80, 0) , z3.If(r8s38, 80, 0) , z3.If(r8s39, 0, 0) , z3.If(r8s40, 20, 0) , z3.If(r8s41, 0, 0) , z3.If(r8s42, 20, 0) , z3.If(r8s43, 80, 0) , z3.If(r8s44, 0, 0) , z3.If(r8s45, 0, 0) , z3.If(r8s46, 0, 0) , z3.If(r8s47, 80, 0) , z3.If(r8s48, 0, 0) , z3.If(r8s49, 20, 0) , z3.If(r8s50, 20, 0) , z3.If(r8s51, 20, 0) , z3.If(r9s0, 80, 0) , z3.If(r9s1, 0, 0) , z3.If(r9s2, 0, 0) , z3.If(r9s3, 0, 0) , z3.If(r9s4, 0, 0) , z3.If(r9s5, 0, 0) , z3.If(r9s6, 0, 0) , z3.If(r9s7, 0, 0) , z3.If(r9s8, 0, 0) , z3.If(r9s9, 0, 0) , z3.If(r9s10, 0, 0) , z3.If(r9s11, 0, 0) , z3.If(r9s12, 0, 0) , z3.If(r9s13, 80, 0) , z3.If(r9s14, 80, 0) , z3.If(r9s15, 0, 0) , z3.If(r9s16, 0, 0) , z3.If(r9s17, 80, 0) , z3.If(r9s18, 0, 0) , z3.If(r9s19, 0, 0) , z3.If(r9s20, 0, 0) , z3.If(r9s21, 0, 0) , z3.If(r9s22, 0, 0) , z3.If(r9s23, 0, 0) , z3.If(r9s24, 80, 0) , z3.If(r9s25, 0, 0) , z3.If(r9s26, 20, 0) , z3.If(r9s27, 0, 0) , z3.If(r9s28, 80, 0) , z3.If(r9s29, 80, 0) , z3.If(r9s30, 0, 0) , z3.If(r9s31, 0, 0) , z3.If(r9s32, 80, 0) , z3.If(r9s33, 20, 0) , z3.If(r9s34, 0, 0) , z3.If(r9s35, 0, 0) , z3.If(r9s36, 80, 0) , z3.If(r9s37, 80, 0) , z3.If(r9s38, 0, 0) , z3.If(r9s39, 0, 0) , z3.If(r9s40, 80, 0) , z3.If(r9s41, 0, 0) , z3.If(r9s42, 0, 0) , z3.If(r9s43, 0, 0) , z3.If(r9s44, 80, 0) , z3.If(r9s45, 0, 0) , z3.If(r9s46, 80, 0) , z3.If(r9s47, 0, 0) , z3.If(r9s48, 0, 0) , z3.If(r9s49, 80, 0) , z3.If(r9s50, 0, 0) , z3.If(r9s51, 0, 0) , z3.If(r10s0, 0, 0) , z3.If(r10s1, 0, 0) , z3.If(r10s2, 0, 0) , z3.If(r10s3, 0, 0) , z3.If(r10s4, 0, 0) , z3.If(r10s5, 0, 0) , z3.If(r10s6, 0, 0) , z3.If(r10s7, 0, 0) , z3.If(r10s9, 80, 0) , z3.If(r10s10, 0, 0) , z3.If(r10s12, 80, 0) , z3.If(r10s13, 80, 0) , z3.If(r10s14, 0, 0) , z3.If(r10s15, 0, 0) , z3.If(r10s16, 0, 0) , z3.If(r10s17, 0, 0) , z3.If(r10s18, 80, 0) , z3.If(r10s19, 0, 0) , z3.If(r10s20, 0, 0) , z3.If(r10s21, 80, 0) , z3.If(r10s22, 0, 0) , z3.If(r10s23, 0, 0) , z3.If(r10s24, 0, 0) , z3.If(r10s25, 0, 0) , z3.If(r10s26, 0, 0) , z3.If(r10s27, 0, 0) , z3.If(r10s28, 80, 0) , z3.If(r10s29, 0, 0) , z3.If(r10s30, 80, 0) , z3.If(r10s31, 80, 0) , z3.If(r10s32, 80, 0) , z3.If(r10s33, 0, 0) , z3.If(r10s34, 0, 0) , z3.If(r10s35, 80, 0) , z3.If(r10s36, 0, 0) , z3.If(r10s37, 80, 0) , z3.If(r10s38, 0, 0) , z3.If(r10s39, 0, 0) , z3.If(r10s40, 80, 0) , z3.If(r10s41, 0, 0) , z3.If(r10s42, 0, 0) , z3.If(r10s43, 0, 0) , z3.If(r10s44, 0, 0) , z3.If(r10s45, 0, 0) , z3.If(r10s46, 0, 0) , z3.If(r10s47, 0, 0) , z3.If(r10s49, 0, 0) , z3.If(r10s50, 0, 0) , z3.If(r10s51, 0, 0) , z3.If(r11s0, 0, 0) , z3.If(r11s1, 0, 0) , z3.If(r11s2, 0, 0) , z3.If(r11s3, 80, 0) , z3.If(r11s4, 80, 0) , z3.If(r11s5, 20, 0) , z3.If(r11s6, 0, 0) , z3.If(r11s7, 0, 0) , z3.If(r11s8, 0, 0) , z3.If(r11s9, 80, 0) , z3.If(r11s10, 0, 0) , z3.If(r11s11, 80, 0) , z3.If(r11s12, 0, 0) , z3.If(r11s13, 80, 0) , z3.If(r11s14, 0, 0) , z3.If(r11s15, 0, 0) , z3.If(r11s16, 0, 0) , z3.If(r11s17, 0, 0) , z3.If(r11s18, 0, 0) , z3.If(r11s19, 0, 0) , z3.If(r11s20, 0, 0) , z3.If(r11s21, 0, 0) , z3.If(r11s22, 0, 0) , z3.If(r11s23, 0, 0) , z3.If(r11s24, 0, 0) , z3.If(r11s25, 80, 0) , z3.If(r11s26, 0, 0) , z3.If(r11s27, 0, 0) , z3.If(r11s28, 0, 0) , z3.If(r11s29, 0, 0) , z3.If(r11s30, 0, 0) , z3.If(r11s31, 0, 0) , z3.If(r11s32, 0, 0) , z3.If(r11s33, 0, 0) , z3.If(r11s34, 20, 0) , z3.If(r11s35, 0, 0) , z3.If(r11s36, 0, 0) , z3.If(r11s37, 0, 0) , z3.If(r11s38, 0, 0) , z3.If(r11s39, 0, 0) , z3.If(r11s40, 0, 0) , z3.If(r11s41, 0, 0) , z3.If(r11s43, 0, 0) , z3.If(r11s44, 0, 0) , z3.If(r11s45, 0, 0) , z3.If(r11s46, 0, 0) , z3.If(r11s48, 0, 0) , z3.If(r11s50, 0, 0) , z3.If(r11s51, 0, 0) , z3.If(r12s1, 0, 0) , z3.If(r12s2, 0, 0) , z3.If(r12s3, 0, 0) , z3.If(r12s4, 80, 0) , z3.If(r12s5, 80, 0) , z3.If(r12s6, 0, 0) , z3.If(r12s7, 0, 0) , z3.If(r12s8, 20, 0) , z3.If(r12s9, 0, 0) , z3.If(r12s10, 0, 0) , z3.If(r12s12, 0, 0) , z3.If(r12s13, 20, 0) , z3.If(r12s14, 0, 0) , z3.If(r12s15, 0, 0) , z3.If(r12s16, 20, 0) , z3.If(r12s19, 0, 0) , z3.If(r12s20, 0, 0) , z3.If(r12s21, 0, 0) , z3.If(r12s22, 0, 0) , z3.If(r12s23, 0, 0) , z3.If(r12s24, 80, 0) , z3.If(r12s25, 0, 0) , z3.If(r12s26, 0, 0) , z3.If(r12s27, 20, 0) , z3.If(r12s28, 0, 0) , z3.If(r12s29, 0, 0) , z3.If(r12s30, 0, 0) , z3.If(r12s31, 80, 0) , z3.If(r12s32, 80, 0) , z3.If(r12s33, 0, 0) , z3.If(r12s34, 80, 0) , z3.If(r12s35, 0, 0) , z3.If(r12s36, 0, 0) , z3.If(r12s37, 0, 0) , z3.If(r12s38, 80, 0) , z3.If(r12s39, 0, 0) , z3.If(r12s40, 20, 0) , z3.If(r12s41, 80, 0) , z3.If(r12s42, 0, 0) , z3.If(r12s43, 20, 0) , z3.If(r12s44, 20, 0) , z3.If(r12s45, 20, 0) , z3.If(r12s46, 0, 0) , z3.If(r12s48, 0, 0) , z3.If(r12s49, 0, 0) , z3.If(r12s50, 0, 0) , z3.If(r12s51, 0, 0) , z3.If(r13s0, 80, 0) , z3.If(r13s2, 20, 0) , z3.If(r13s3, 0, 0) , z3.If(r13s4, 0, 0) , z3.If(r13s5, 0, 0) , z3.If(r13s6, 0, 0) , z3.If(r13s7, 0, 0) , z3.If(r13s10, 0, 0) , z3.If(r13s12, 0, 0) , z3.If(r13s13, 20, 0) , z3.If(r13s14, 80, 0) , z3.If(r13s15, 0, 0) , z3.If(r13s16, 0, 0) , z3.If(r13s18, 0, 0) , z3.If(r13s19, 80, 0) , z3.If(r13s20, 0, 0) , z3.If(r13s22, 0, 0) , z3.If(r13s23, 0, 0) , z3.If(r13s25, 0, 0) , z3.If(r13s26, 0, 0) , z3.If(r13s27, 0, 0) , z3.If(r13s28, 0, 0) , z3.If(r13s29, 0, 0) , z3.If(r13s30, 0, 0) , z3.If(r13s31, 0, 0) , z3.If(r13s32, 0, 0) , z3.If(r13s33, 0, 0) , z3.If(r13s34, 0, 0) , z3.If(r13s35, 20, 0) , z3.If(r13s37, 20, 0) , z3.If(r13s38, 80, 0) , z3.If(r13s39, 0, 0) , z3.If(r13s40, 20, 0) , z3.If(r13s41, 0, 0) , z3.If(r13s42, 0, 0) , z3.If(r13s43, 80, 0) , z3.If(r13s44, 0, 0) , z3.If(r13s45, 0, 0) , z3.If(r13s46, 20, 0) , z3.If(r13s47, 20, 0) , z3.If(r13s49, 20, 0) , z3.If(r13s50, 0, 0) , z3.If(r13s51, 20, 0) , z3.If(r14s0, 0, 0) , z3.If(r14s1, 0, 0) , z3.If(r14s2, 0, 0) , z3.If(r14s4, 0, 0) , z3.If(r14s5, 0, 0) , z3.If(r14s7, 80, 0) , z3.If(r14s8, 80, 0) , z3.If(r14s9, 0, 0) , z3.If(r14s10, 0, 0) , z3.If(r14s11, 0, 0) , z3.If(r14s12, 0, 0) , z3.If(r14s13, 80, 0) , z3.If(r14s14, 0, 0) , z3.If(r14s15, 0, 0) , z3.If(r14s17, 20, 0) , z3.If(r14s18, 0, 0) , z3.If(r14s19, 20, 0) , z3.If(r14s20, 0, 0) , z3.If(r14s21, 0, 0) , z3.If(r14s22, 0, 0) , z3.If(r14s23, 0, 0) , z3.If(r14s24, 80, 0) , z3.If(r14s25, 20, 0) , z3.If(r14s26, 0, 0) , z3.If(r14s27, 0, 0) , z3.If(r14s28, 0, 0) , z3.If(r14s29, 0, 0) , z3.If(r14s30, 0, 0) , z3.If(r14s34, 80, 0) , z3.If(r14s36, 0, 0) , z3.If(r14s38, 0, 0) , z3.If(r14s39, 0, 0) , z3.If(r14s40, 0, 0) , z3.If(r14s41, 0, 0) , z3.If(r14s42, 80, 0) , z3.If(r14s43, 80, 0) , z3.If(r14s44, 80, 0) , z3.If(r14s45, 0, 0) , z3.If(r14s46, 0, 0) , z3.If(r14s47, 20, 0) , z3.If(r14s48, 80, 0) , z3.If(r14s49, 80, 0) , z3.If(r14s50, 0, 0) , z3.If(r14s51, 0, 0) , z3.If(r15s1, 0, 0) , z3.If(r15s2, 0, 0) , z3.If(r15s3, 0, 0) , z3.If(r15s4, 0, 0) , z3.If(r15s5, 0, 0) , z3.If(r15s6, 0, 0) , z3.If(r15s7, 0, 0) , z3.If(r15s8, 0, 0) , z3.If(r15s9, 80, 0) , z3.If(r15s10, 0, 0) , z3.If(r15s11, 0, 0) , z3.If(r15s12, 80, 0) , z3.If(r15s13, 80, 0) , z3.If(r15s14, 0, 0) , z3.If(r15s15, 0, 0) , z3.If(r15s16, 0, 0) , z3.If(r15s17, 0, 0) , z3.If(r15s19, 0, 0) , z3.If(r15s20, 0, 0) , z3.If(r15s21, 0, 0) , z3.If(r15s22, 80, 0) , z3.If(r15s23, 80, 0) , z3.If(r15s24, 80, 0) , z3.If(r15s25, 0, 0) , z3.If(r15s26, 0, 0) , z3.If(r15s27, 0, 0) , z3.If(r15s28, 0, 0) , z3.If(r15s29, 0, 0) , z3.If(r15s30, 0, 0) , z3.If(r15s31, 0, 0) , z3.If(r15s32, 0, 0) , z3.If(r15s33, 0, 0) , z3.If(r15s34, 80, 0) , z3.If(r15s35, 0, 0) , z3.If(r15s36, 0, 0) , z3.If(r15s37, 80, 0) , z3.If(r15s38, 0, 0) , z3.If(r15s39, 0, 0) , z3.If(r15s40, 0, 0) , z3.If(r15s41, 0, 0) , z3.If(r15s42, 0, 0) , z3.If(r15s43, 0, 0) , z3.If(r15s44, 0, 0) , z3.If(r15s45, 0, 0) , z3.If(r15s46, 0, 0) , z3.If(r15s48, 0, 0) , z3.If(r15s49, 0, 0) , z3.If(r15s50, 0, 0) , z3.If(r15s51, 80, 0) , z3.If(r16s1, 0, 0) , z3.If(r16s2, 20, 0) , z3.If(r16s3, 20, 0) , z3.If(r16s4, 80, 0) , z3.If(r16s5, 20, 0) , z3.If(r16s7, 80, 0) , z3.If(r16s8, 0, 0) , z3.If(r16s9, 0, 0) , z3.If(r16s11, 0, 0) , z3.If(r16s12, 20, 0) , z3.If(r16s13, 0, 0) , z3.If(r16s14, 0, 0) , z3.If(r16s15, 0, 0) , z3.If(r16s16, 0, 0) , z3.If(r16s17, 0, 0) , z3.If(r16s19, 0, 0) , z3.If(r16s20, 0, 0) , z3.If(r16s21, 0, 0) , z3.If(r16s22, 0, 0) , z3.If(r16s23, 0, 0) , z3.If(r16s24, 0, 0) , z3.If(r16s25, 0, 0) , z3.If(r16s26, 0, 0) , z3.If(r16s27, 0, 0) , z3.If(r16s28, 0, 0) , z3.If(r16s29, 0, 0) , z3.If(r16s30, 0, 0) , z3.If(r16s31, 80, 0) , z3.If(r16s32, 0, 0) , z3.If(r16s33, 0, 0) , z3.If(r16s34, 80, 0) , z3.If(r16s35, 0, 0) , z3.If(r16s36, 0, 0) , z3.If(r16s38, 0, 0) , z3.If(r16s39, 20, 0) , z3.If(r16s40, 0, 0) , z3.If(r16s41, 0, 0) , z3.If(r16s42, 0, 0) , z3.If(r16s43, 0, 0) , z3.If(r16s44, 20, 0) , z3.If(r16s45, 0, 0) , z3.If(r16s46, 20, 0) , z3.If(r16s48, 0, 0) , z3.If(r16s49, 0, 0) , z3.If(r16s50, 0, 0) , z3.If(r16s51, 80, 0) , z3.If(r17s0, 0, 0) , z3.If(r17s1, 0, 0) , z3.If(r17s2, 80, 0) , z3.If(r17s3, 80, 0) , z3.If(r17s4, 0, 0) , z3.If(r17s5, 0, 0) , z3.If(r17s7, 80, 0) , z3.If(r17s8, 0, 0) , z3.If(r17s11, 0, 0) , z3.If(r17s12, 0, 0) , z3.If(r17s13, 80, 0) , z3.If(r17s14, 0, 0) , z3.If(r17s15, 0, 0) , z3.If(r17s16, 20, 0) , z3.If(r17s17, 20, 0) , z3.If(r17s18, 80, 0) , z3.If(r17s19, 20, 0) , z3.If(r17s20, 0, 0) , z3.If(r17s21, 0, 0) , z3.If(r17s22, 0, 0) , z3.If(r17s23, 0, 0) , z3.If(r17s25, 0, 0) , z3.If(r17s26, 0, 0) , z3.If(r17s27, 20, 0) , z3.If(r17s28, 0, 0) , z3.If(r17s29, 20, 0) , z3.If(r17s30, 20, 0) , z3.If(r17s31, 0, 0) , z3.If(r17s32, 80, 0) , z3.If(r17s33, 0, 0) , z3.If(r17s34, 20, 0) , z3.If(r17s35, 80, 0) , z3.If(r17s36, 0, 0) , z3.If(r17s37, 20, 0) , z3.If(r17s38, 20, 0) , z3.If(r17s39, 0, 0) , z3.If(r17s40, 0, 0) , z3.If(r17s41, 20, 0) , z3.If(r17s42, 0, 0) , z3.If(r17s43, 20, 0) , z3.If(r17s44, 0, 0) , z3.If(r17s45, 0, 0) , z3.If(r17s46, 0, 0) , z3.If(r17s47, 80, 0) , z3.If(r17s48, 0, 0) , z3.If(r17s49, 0, 0) , z3.If(r17s50, 0, 0) , z3.If(r17s51, 0, 0) , z3.If(r18s0, 80, 0) , z3.If(r18s1, 0, 0) , z3.If(r18s2, 0, 0) , z3.If(r18s3, 0, 0) , z3.If(r18s4, 80, 0) , z3.If(r18s5, 0, 0) , z3.If(r18s6, 0, 0) , z3.If(r18s7, 0, 0) , z3.If(r18s8, 0, 0) , z3.If(r18s10, 0, 0) , z3.If(r18s11, 0, 0) , z3.If(r18s12, 80, 0) , z3.If(r18s13, 0, 0) , z3.If(r18s14, 0, 0) , z3.If(r18s15, 80, 0) , z3.If(r18s16, 0, 0) , z3.If(r18s17, 0, 0) , z3.If(r18s18, 80, 0) , z3.If(r18s19, 0, 0) , z3.If(r18s20, 0, 0) , z3.If(r18s21, 0, 0) , z3.If(r18s22, 0, 0) , z3.If(r18s23, 80, 0) , z3.If(r18s25, 0, 0) , z3.If(r18s26, 0, 0) , z3.If(r18s27, 0, 0) , z3.If(r18s28, 0, 0) , z3.If(r18s29, 0, 0) , z3.If(r18s30, 0, 0) , z3.If(r18s31, 0, 0) , z3.If(r18s32, 0, 0) , z3.If(r18s33, 80, 0) , z3.If(r18s34, 0, 0) , z3.If(r18s35, 0, 0) , z3.If(r18s36, 0, 0) , z3.If(r18s37, 0, 0) , z3.If(r18s38, 0, 0) , z3.If(r18s39, 0, 0) , z3.If(r18s40, 0, 0) , z3.If(r18s41, 0, 0) , z3.If(r18s42, 0, 0) , z3.If(r18s43, 0, 0) , z3.If(r18s44, 0, 0) , z3.If(r18s45, 80, 0) , z3.If(r18s46, 0, 0) , z3.If(r18s47, 0, 0) , z3.If(r18s48, 80, 0) , z3.If(r18s49, 0, 0) , z3.If(r18s50, 0, 0) , z3.If(r18s51, 80, 0) , z3.If(r19s0, 20, 0) , z3.If(r19s1, 80, 0) , z3.If(r19s3, 20, 0) , z3.If(r19s4, 80, 0) , z3.If(r19s5, 0, 0) , z3.If(r19s6, 20, 0) , z3.If(r19s7, 20, 0) , z3.If(r19s8, 0, 0) , z3.If(r19s9, 20, 0) , z3.If(r19s10, 0, 0) , z3.If(r19s11, 0, 0) , z3.If(r19s12, 0, 0) , z3.If(r19s13, 0, 0) , z3.If(r19s14, 0, 0) , z3.If(r19s15, 80, 0) , z3.If(r19s16, 80, 0) , z3.If(r19s17, 80, 0) , z3.If(r19s18, 80, 0) , z3.If(r19s19, 20, 0) , z3.If(r19s20, 0, 0) , z3.If(r19s21, 0, 0) , z3.If(r19s22, 80, 0) , z3.If(r19s23, 0, 0) , z3.If(r19s24, 20, 0) , z3.If(r19s25, 0, 0) , z3.If(r19s26, 0, 0) , z3.If(r19s27, 0, 0) , z3.If(r19s28, 0, 0) , z3.If(r19s29, 0, 0) , z3.If(r19s30, 0, 0) , z3.If(r19s31, 20, 0) , z3.If(r19s32, 0, 0) , z3.If(r19s33, 0, 0) , z3.If(r19s34, 0, 0) , z3.If(r19s35, 0, 0) , z3.If(r19s36, 0, 0) , z3.If(r19s37, 80, 0) , z3.If(r19s38, 0, 0) , z3.If(r19s39, 0, 0) , z3.If(r19s40, 0, 0) , z3.If(r19s41, 0, 0) , z3.If(r19s42, 80, 0) , z3.If(r19s43, 0, 0) , z3.If(r19s44, 80, 0) , z3.If(r19s45, 0, 0) , z3.If(r19s46, 0, 0) , z3.If(r19s47, 0, 0) , z3.If(r19s48, 0, 0) , z3.If(r19s49, 80, 0) , z3.If(r19s50, 0, 0) , z3.If(r19s51, 80, 0) , z3.If(r20s0, 0, 0) , z3.If(r20s1, 0, 0) , z3.If(r20s2, 0, 0) , z3.If(r20s3, 0, 0) , z3.If(r20s4, 0, 0) , z3.If(r20s5, 0, 0) , z3.If(r20s6, 80, 0) , z3.If(r20s7, 0, 0) , z3.If(r20s8, 0, 0) , z3.If(r20s9, 80, 0) , z3.If(r20s10, 0, 0) , z3.If(r20s11, 0, 0) , z3.If(r20s12, 0, 0) , z3.If(r20s13, 80, 0) , z3.If(r20s15, 0, 0) , z3.If(r20s17, 80, 0) , z3.If(r20s18, 0, 0) , z3.If(r20s19, 0, 0) , z3.If(r20s20, 0, 0) , z3.If(r20s21, 0, 0) , z3.If(r20s22, 0, 0) , z3.If(r20s23, 0, 0) , z3.If(r20s24, 80, 0) , z3.If(r20s25, 0, 0) , z3.If(r20s26, 80, 0) , z3.If(r20s30, 80, 0) , z3.If(r20s31, 0, 0) , z3.If(r20s32, 80, 0) , z3.If(r20s33, 0, 0) , z3.If(r20s34, 80, 0) , z3.If(r20s35, 0, 0) , z3.If(r20s36, 80, 0) , z3.If(r20s37, 80, 0) , z3.If(r20s38, 0, 0) , z3.If(r20s39, 0, 0) , z3.If(r20s40, 0, 0) , z3.If(r20s41, 0, 0) , z3.If(r20s42, 0, 0) , z3.If(r20s43, 0, 0) , z3.If(r20s44, 0, 0) , z3.If(r20s45, 80, 0) , z3.If(r20s46, 0, 0) , z3.If(r20s47, 80, 0) , z3.If(r20s48, 0, 0) , z3.If(r20s49, 0, 0) , z3.If(r20s50, 0, 0) , z3.If(r20s51, 0, 0) , z3.If(r21s1, 0, 0) , z3.If(r21s2, 80, 0) , z3.If(r21s3, 0, 0) , z3.If(r21s4, 0, 0) , z3.If(r21s5, 80, 0) , z3.If(r21s6, 0, 0) , z3.If(r21s7, 0, 0) , z3.If(r21s8, 0, 0) , z3.If(r21s10, 0, 0) , z3.If(r21s12, 80, 0) , z3.If(r21s14, 20, 0) , z3.If(r21s15, 80, 0) , z3.If(r21s18, 80, 0) , z3.If(r21s19, 0, 0) , z3.If(r21s20, 0, 0) , z3.If(r21s22, 0, 0) , z3.If(r21s23, 20, 0) , z3.If(r21s24, 0, 0) , z3.If(r21s25, 0, 0) , z3.If(r21s26, 0, 0) , z3.If(r21s27, 0, 0) , z3.If(r21s28, 0, 0) , z3.If(r21s29, 0, 0) , z3.If(r21s30, 0, 0) , z3.If(r21s31, 0, 0) , z3.If(r21s33, 0, 0) , z3.If(r21s34, 0, 0) , z3.If(r21s35, 80, 0) , z3.If(r21s37, 20, 0) , z3.If(r21s38, 20, 0) , z3.If(r21s39, 0, 0) , z3.If(r21s40, 0, 0) , z3.If(r21s41, 0, 0) , z3.If(r21s42, 0, 0) , z3.If(r21s43, 20, 0) , z3.If(r21s44, 0, 0) , z3.If(r21s45, 0, 0) , z3.If(r21s46, 0, 0) , z3.If(r21s47, 0, 0) , z3.If(r21s48, 0, 0) , z3.If(r21s49, 0, 0) , z3.If(r21s50, 20, 0) , z3.If(r21s51, 0, 0) , z3.If(r22s0, 0, 0) , z3.If(r22s1, 0, 0) , z3.If(r22s2, 0, 0) , z3.If(r22s3, 0, 0) , z3.If(r22s4, 80, 0) , z3.If(r22s5, 80, 0) , z3.If(r22s6, 0, 0) , z3.If(r22s7, 0, 0) , z3.If(r22s8, 0, 0) , z3.If(r22s9, 20, 0) , z3.If(r22s10, 20, 0) , z3.If(r22s11, 0, 0) , z3.If(r22s12, 0, 0) , z3.If(r22s13, 20, 0) , z3.If(r22s14, 0, 0) , z3.If(r22s15, 0, 0) , z3.If(r22s16, 80, 0) , z3.If(r22s17, 0, 0) , z3.If(r22s18, 0, 0) , z3.If(r22s19, 0, 0) , z3.If(r22s20, 20, 0) , z3.If(r22s21, 0, 0) , z3.If(r22s22, 0, 0) , z3.If(r22s23, 0, 0) , z3.If(r22s24, 0, 0) , z3.If(r22s25, 0, 0) , z3.If(r22s26, 0, 0) , z3.If(r22s27, 0, 0) , z3.If(r22s28, 0, 0) , z3.If(r22s29, 0, 0) , z3.If(r22s30, 0, 0) , z3.If(r22s31, 0, 0) , z3.If(r22s32, 80, 0) , z3.If(r22s33, 20, 0) , z3.If(r22s34, 20, 0) , z3.If(r22s35, 0, 0) , z3.If(r22s36, 0, 0) , z3.If(r22s37, 0, 0) , z3.If(r22s38, 0, 0) , z3.If(r22s39, 80, 0) , z3.If(r22s40, 0, 0) , z3.If(r22s41, 0, 0) , z3.If(r22s42, 20, 0) , z3.If(r22s43, 0, 0) , z3.If(r22s44, 0, 0) , z3.If(r22s45, 80, 0) , z3.If(r22s46, 0, 0) , z3.If(r22s47, 0, 0) , z3.If(r22s48, 0, 0) , z3.If(r22s49, 0, 0) , z3.If(r22s50, 0, 0) , z3.If(r22s51, 0, 0) , z3.If(r23s0, 0, 0) , z3.If(r23s1, 0, 0) , z3.If(r23s2, 0, 0) , z3.If(r23s3, 80, 0) , z3.If(r23s4, 80, 0) , z3.If(r23s5, 0, 0) , z3.If(r23s6, 20, 0) , z3.If(r23s7, 0, 0) , z3.If(r23s8, 20, 0) , z3.If(r23s10, 80, 0) , z3.If(r23s11, 0, 0) , z3.If(r23s12, 0, 0) , z3.If(r23s13, 0, 0) , z3.If(r23s14, 0, 0) , z3.If(r23s15, 0, 0) , z3.If(r23s16, 80, 0) , z3.If(r23s17, 0, 0) , z3.If(r23s18, 0, 0) , z3.If(r23s19, 0, 0) , z3.If(r23s20, 0, 0) , z3.If(r23s21, 0, 0) , z3.If(r23s22, 0, 0) , z3.If(r23s23, 0, 0) , z3.If(r23s25, 0, 0) , z3.If(r23s26, 0, 0) , z3.If(r23s27, 0, 0) , z3.If(r23s28, 0, 0) , z3.If(r23s29, 0, 0) , z3.If(r23s30, 0, 0) , z3.If(r23s31, 20, 0) , z3.If(r23s32, 80, 0) , z3.If(r23s33, 0, 0) , z3.If(r23s34, 20, 0) , z3.If(r23s35, 0, 0) , z3.If(r23s36, 0, 0) , z3.If(r23s37, 0, 0) , z3.If(r23s38, 0, 0) , z3.If(r23s39, 80, 0) , z3.If(r23s40, 0, 0) , z3.If(r23s41, 0, 0) , z3.If(r23s42, 80, 0) , z3.If(r23s43, 0, 0) , z3.If(r23s44, 0, 0) , z3.If(r23s45, 80, 0) , z3.If(r23s46, 0, 0) , z3.If(r23s47, 0, 0) , z3.If(r23s48, 20, 0) , z3.If(r23s49, 0, 0) , z3.If(r23s50, 0, 0) , z3.If(r23s51, 0, 0)]
# 70/30
# scores = [z3.If(r0s0, 70, 0) , z3.If(r0s1, 0, 0) , z3.If(r0s2, 0, 0) , z3.If(r0s4, 0, 0) , z3.If(r0s5, 0, 0) , z3.If(r0s7, 0, 0) , z3.If(r0s8, 0, 0) , z3.If(r0s9, 0, 0) , z3.If(r0s10, 0, 0) , z3.If(r0s12, 0, 0) , z3.If(r0s13, 0, 0) , z3.If(r0s14, 30, 0) , z3.If(r0s15, 0, 0) , z3.If(r0s16, 0, 0) , z3.If(r0s18, 30, 0) , z3.If(r0s19, 0, 0) , z3.If(r0s20, 0, 0) , z3.If(r0s21, 0, 0) , z3.If(r0s22, 0, 0) , z3.If(r0s23, 70, 0) , z3.If(r0s24, 70, 0) , z3.If(r0s25, 0, 0) , z3.If(r0s26, 30, 0) , z3.If(r0s27, 0, 0) , z3.If(r0s28, 70, 0) , z3.If(r0s29, 70, 0) , z3.If(r0s30, 70, 0) , z3.If(r0s31, 0, 0) , z3.If(r0s32, 70, 0) , z3.If(r0s34, 0, 0) , z3.If(r0s35, 0, 0) , z3.If(r0s36, 0, 0) , z3.If(r0s37, 70, 0) , z3.If(r0s38, 0, 0) , z3.If(r0s39, 0, 0) , z3.If(r0s40, 30, 0) , z3.If(r0s41, 0, 0) , z3.If(r0s42, 0, 0) , z3.If(r0s43, 0, 0) , z3.If(r0s44, 70, 0) , z3.If(r0s45, 0, 0) , z3.If(r0s46, 0, 0) , z3.If(r0s47, 0, 0) , z3.If(r0s48, 0, 0) , z3.If(r0s49, 0, 0) , z3.If(r0s50, 0, 0) , z3.If(r0s51, 0, 0) , z3.If(r1s0, 70, 0) , z3.If(r1s1, 0, 0) , z3.If(r1s2, 30, 0) , z3.If(r1s3, 70, 0) , z3.If(r1s4, 0, 0) , z3.If(r1s5, 0, 0) , z3.If(r1s6, 0, 0) , z3.If(r1s7, 0, 0) , z3.If(r1s8, 0, 0) , z3.If(r1s9, 70, 0) , z3.If(r1s10, 30, 0) , z3.If(r1s11, 70, 0) , z3.If(r1s12, 0, 0) , z3.If(r1s13, 0, 0) , z3.If(r1s14, 0, 0) , z3.If(r1s15, 0, 0) , z3.If(r1s16, 0, 0) , z3.If(r1s17, 30, 0) , z3.If(r1s18, 70, 0) , z3.If(r1s19, 0, 0) , z3.If(r1s20, 0, 0) , z3.If(r1s21, 0, 0) , z3.If(r1s22, 70, 0) , z3.If(r1s23, 0, 0) , z3.If(r1s24, 0, 0) , z3.If(r1s25, 0, 0) , z3.If(r1s26, 0, 0) , z3.If(r1s27, 0, 0) , z3.If(r1s28, 70, 0) , z3.If(r1s29, 30, 0) , z3.If(r1s30, 0, 0) , z3.If(r1s31, 0, 0) , z3.If(r1s32, 0, 0) , z3.If(r1s33, 0, 0) , z3.If(r1s35, 0, 0) , z3.If(r1s36, 30, 0) , z3.If(r1s37, 0, 0) , z3.If(r1s38, 0, 0) , z3.If(r1s39, 0, 0) , z3.If(r1s40, 70, 0) , z3.If(r1s41, 0, 0) , z3.If(r1s42, 0, 0) , z3.If(r1s43, 0, 0) , z3.If(r1s44, 0, 0) , z3.If(r1s45, 0, 0) , z3.If(r1s46, 0, 0) , z3.If(r1s47, 0, 0) , z3.If(r1s48, 0, 0) , z3.If(r1s49, 0, 0) , z3.If(r1s50, 0, 0) , z3.If(r1s51, 0, 0) , z3.If(r2s0, 0, 0) , z3.If(r2s1, 0, 0) , z3.If(r2s2, 70, 0) , z3.If(r2s3, 0, 0) , z3.If(r2s4, 0, 0) , z3.If(r2s5, 0, 0) , z3.If(r2s6, 0, 0) , z3.If(r2s7, 30, 0) , z3.If(r2s8, 0, 0) , z3.If(r2s9, 0, 0) , z3.If(r2s10, 0, 0) , z3.If(r2s11, 0, 0) , z3.If(r2s12, 70, 0) , z3.If(r2s14, 30, 0) , z3.If(r2s15, 70, 0) , z3.If(r2s18, 0, 0) , z3.If(r2s19, 0, 0) , z3.If(r2s20, 0, 0) , z3.If(r2s22, 0, 0) , z3.If(r2s23, 0, 0) , z3.If(r2s24, 0, 0) , z3.If(r2s25, 0, 0) , z3.If(r2s26, 0, 0) , z3.If(r2s27, 0, 0) , z3.If(r2s28, 0, 0) , z3.If(r2s29, 0, 0) , z3.If(r2s30, 0, 0) , z3.If(r2s31, 0, 0) , z3.If(r2s33, 0, 0) , z3.If(r2s34, 0, 0) , z3.If(r2s35, 0, 0) , z3.If(r2s36, 0, 0) , z3.If(r2s37, 30, 0) , z3.If(r2s38, 0, 0) , z3.If(r2s39, 0, 0) , z3.If(r2s40, 0, 0) , z3.If(r2s41, 0, 0) , z3.If(r2s42, 0, 0) , z3.If(r2s43, 0, 0) , z3.If(r2s44, 0, 0) , z3.If(r2s45, 0, 0) , z3.If(r2s46, 30, 0) , z3.If(r2s47, 0, 0) , z3.If(r2s48, 0, 0) , z3.If(r2s49, 30, 0) , z3.If(r2s50, 0, 0) , z3.If(r2s51, 30, 0) , z3.If(r3s0, 0, 0) , z3.If(r3s2, 70, 0) , z3.If(r3s3, 70, 0) , z3.If(r3s4, 0, 0) , z3.If(r3s5, 0, 0) , z3.If(r3s6, 70, 0) , z3.If(r3s7, 0, 0) , z3.If(r3s8, 30, 0) , z3.If(r3s10, 0, 0) , z3.If(r3s11, 30, 0) , z3.If(r3s12, 0, 0) , z3.If(r3s15, 0, 0) , z3.If(r3s17, 30, 0) , z3.If(r3s18, 30, 0) , z3.If(r3s19, 0, 0) , z3.If(r3s20, 0, 0) , z3.If(r3s21, 0, 0) , z3.If(r3s22, 0, 0) , z3.If(r3s23, 0, 0) , z3.If(r3s25, 70, 0) , z3.If(r3s31, 30, 0) , z3.If(r3s32, 70, 0) , z3.If(r3s33, 0, 0) , z3.If(r3s34, 0, 0) , z3.If(r3s35, 70, 0) , z3.If(r3s36, 70, 0) , z3.If(r3s37, 70, 0) , z3.If(r3s38, 70, 0) , z3.If(r3s39, 0, 0) , z3.If(r3s40, 70, 0) , z3.If(r3s41, 30, 0) , z3.If(r3s42, 70, 0) , z3.If(r3s43, 30, 0) , z3.If(r3s44, 0, 0) , z3.If(r3s45, 0, 0) , z3.If(r3s46, 30, 0) , z3.If(r3s47, 70, 0) , z3.If(r3s48, 0, 0) , z3.If(r3s49, 30, 0) , z3.If(r3s50, 70, 0) , z3.If(r3s51, 30, 0) , z3.If(r4s0, 30, 0) , z3.If(r4s1, 0, 0) , z3.If(r4s2, 30, 0) , z3.If(r4s3, 0, 0) , z3.If(r4s4, 70, 0) , z3.If(r4s5, 70, 0) , z3.If(r4s6, 0, 0) , z3.If(r4s7, 0, 0) , z3.If(r4s8, 0, 0) , z3.If(r4s9, 0, 0) , z3.If(r4s10, 70, 0) , z3.If(r4s11, 0, 0) , z3.If(r4s12, 0, 0) , z3.If(r4s13, 0, 0) , z3.If(r4s14, 0, 0) , z3.If(r4s15, 0, 0) , z3.If(r4s16, 30, 0) , z3.If(r4s17, 0, 0) , z3.If(r4s18, 70, 0) , z3.If(r4s19, 0, 0) , z3.If(r4s20, 0, 0) , z3.If(r4s21, 0, 0) , z3.If(r4s22, 30, 0) , z3.If(r4s23, 0, 0) , z3.If(r4s24, 0, 0) , z3.If(r4s25, 0, 0) , z3.If(r4s26, 0, 0) , z3.If(r4s27, 0, 0) , z3.If(r4s28, 0, 0) , z3.If(r4s29, 0, 0) , z3.If(r4s30, 0, 0) , z3.If(r4s31, 0, 0) , z3.If(r4s32, 30, 0) , z3.If(r4s33, 0, 0) , z3.If(r4s34, 0, 0) , z3.If(r4s35, 0, 0) , z3.If(r4s36, 0, 0) , z3.If(r4s37, 0, 0) , z3.If(r4s38, 0, 0) , z3.If(r4s39, 0, 0) , z3.If(r4s40, 0, 0) , z3.If(r4s41, 0, 0) , z3.If(r4s42, 0, 0) , z3.If(r4s43, 0, 0) , z3.If(r4s44, 0, 0) , z3.If(r4s45, 0, 0) , z3.If(r4s46, 0, 0) , z3.If(r4s47, 30, 0) , z3.If(r4s48, 0, 0) , z3.If(r4s49, 0, 0) , z3.If(r4s50, 0, 0) , z3.If(r4s51, 70, 0) , z3.If(r5s0, 70, 0) , z3.If(r5s1, 0, 0) , z3.If(r5s2, 30, 0) , z3.If(r5s3, 30, 0) , z3.If(r5s4, 30, 0) , z3.If(r5s5, 0, 0) , z3.If(r5s6, 0, 0) , z3.If(r5s7, 0, 0) , z3.If(r5s8, 70, 0) , z3.If(r5s9, 70, 0) , z3.If(r5s10, 0, 0) , z3.If(r5s11, 0, 0) , z3.If(r5s12, 0, 0) , z3.If(r5s13, 0, 0) , z3.If(r5s14, 0, 0) , z3.If(r5s15, 0, 0) , z3.If(r5s16, 0, 0) , z3.If(r5s17, 70, 0) , z3.If(r5s18, 0, 0) , z3.If(r5s19, 30, 0) , z3.If(r5s20, 0, 0) , z3.If(r5s21, 0, 0) , z3.If(r5s22, 30, 0) , z3.If(r5s23, 0, 0) , z3.If(r5s24, 70, 0) , z3.If(r5s25, 0, 0) , z3.If(r5s26, 0, 0) , z3.If(r5s27, 0, 0) , z3.If(r5s28, 30, 0) , z3.If(r5s29, 0, 0) , z3.If(r5s30, 0, 0) , z3.If(r5s31, 0, 0) , z3.If(r5s32, 0, 0) , z3.If(r5s33, 0, 0) , z3.If(r5s34, 30, 0) , z3.If(r5s35, 0, 0) , z3.If(r5s36, 0, 0) , z3.If(r5s37, 70, 0) , z3.If(r5s38, 0, 0) , z3.If(r5s39, 30, 0) , z3.If(r5s40, 0, 0) , z3.If(r5s41, 0, 0) , z3.If(r5s42, 0, 0) , z3.If(r5s43, 0, 0) , z3.If(r5s44, 0, 0) , z3.If(r5s45, 0, 0) , z3.If(r5s46, 0, 0) , z3.If(r5s47, 0, 0) , z3.If(r5s48, 70, 0) , z3.If(r5s49, 30, 0) , z3.If(r5s50, 0, 0) , z3.If(r5s51, 0, 0) , z3.If(r6s0, 30, 0) , z3.If(r6s1, 0, 0) , z3.If(r6s2, 30, 0) , z3.If(r6s3, 30, 0) , z3.If(r6s4, 30, 0) , z3.If(r6s5, 0, 0) , z3.If(r6s7, 0, 0) , z3.If(r6s8, 0, 0) , z3.If(r6s9, 30, 0) , z3.If(r6s11, 0, 0) , z3.If(r6s12, 0, 0) , z3.If(r6s13, 0, 0) , z3.If(r6s14, 0, 0) , z3.If(r6s15, 0, 0) , z3.If(r6s16, 0, 0) , z3.If(r6s17, 30, 0) , z3.If(r6s18, 70, 0) , z3.If(r6s19, 70, 0) , z3.If(r6s20, 0, 0) , z3.If(r6s21, 30, 0) , z3.If(r6s22, 0, 0) , z3.If(r6s23, 0, 0) , z3.If(r6s24, 30, 0) , z3.If(r6s25, 0, 0) , z3.If(r6s26, 0, 0) , z3.If(r6s27, 0, 0) , z3.If(r6s28, 70, 0) , z3.If(r6s29, 30, 0) , z3.If(r6s30, 70, 0) , z3.If(r6s31, 30, 0) , z3.If(r6s32, 0, 0) , z3.If(r6s33, 0, 0) , z3.If(r6s34, 0, 0) , z3.If(r6s36, 0, 0) , z3.If(r6s39, 0, 0) , z3.If(r6s40, 0, 0) , z3.If(r6s41, 0, 0) , z3.If(r6s42, 30, 0) , z3.If(r6s44, 0, 0) , z3.If(r6s45, 0, 0) , z3.If(r6s47, 70, 0) , z3.If(r6s48, 0, 0) , z3.If(r6s49, 70, 0) , z3.If(r6s50, 0, 0) , z3.If(r6s51, 30, 0) , z3.If(r7s0, 70, 0) , z3.If(r7s1, 0, 0) , z3.If(r7s2, 0, 0) , z3.If(r7s3, 30, 0) , z3.If(r7s4, 0, 0) , z3.If(r7s5, 0, 0) , z3.If(r7s6, 0, 0) , z3.If(r7s7, 30, 0) , z3.If(r7s10, 0, 0) , z3.If(r7s12, 0, 0) , z3.If(r7s13, 70, 0) , z3.If(r7s14, 0, 0) , z3.If(r7s15, 0, 0) , z3.If(r7s16, 0, 0) , z3.If(r7s17, 30, 0) , z3.If(r7s18, 30, 0) , z3.If(r7s19, 0, 0) , z3.If(r7s20, 0, 0) , z3.If(r7s22, 70, 0) , z3.If(r7s23, 70, 0) , z3.If(r7s24, 30, 0) , z3.If(r7s25, 0, 0) , z3.If(r7s26, 70, 0) , z3.If(r7s27, 0, 0) , z3.If(r7s28, 70, 0) , z3.If(r7s29, 70, 0) , z3.If(r7s30, 0, 0) , z3.If(r7s31, 70, 0) , z3.If(r7s32, 0, 0) , z3.If(r7s33, 0, 0) , z3.If(r7s34, 0, 0) , z3.If(r7s35, 0, 0) , z3.If(r7s36, 30, 0) , z3.If(r7s37, 70, 0) , z3.If(r7s38, 70, 0) , z3.If(r7s39, 0, 0) , z3.If(r7s40, 70, 0) , z3.If(r7s41, 0, 0) , z3.If(r7s42, 70, 0) , z3.If(r7s43, 70, 0) , z3.If(r7s44, 30, 0) , z3.If(r7s45, 0, 0) , z3.If(r7s46, 0, 0) , z3.If(r7s47, 70, 0) , z3.If(r7s49, 70, 0) , z3.If(r7s50, 0, 0) , z3.If(r7s51, 0, 0) , z3.If(r8s0, 0, 0) , z3.If(r8s1, 0, 0) , z3.If(r8s2, 0, 0) , z3.If(r8s3, 0, 0) , z3.If(r8s4, 0, 0) , z3.If(r8s5, 0, 0) , z3.If(r8s6, 0, 0) , z3.If(r8s7, 0, 0) , z3.If(r8s8, 0, 0) , z3.If(r8s9, 70, 0) , z3.If(r8s10, 0, 0) , z3.If(r8s11, 0, 0) , z3.If(r8s12, 0, 0) , z3.If(r8s14, 0, 0) , z3.If(r8s15, 0, 0) , z3.If(r8s18, 0, 0) , z3.If(r8s19, 0, 0) , z3.If(r8s20, 0, 0) , z3.If(r8s21, 0, 0) , z3.If(r8s22, 30, 0) , z3.If(r8s23, 30, 0) , z3.If(r8s24, 30, 0) , z3.If(r8s25, 30, 0) , z3.If(r8s26, 0, 0) , z3.If(r8s27, 0, 0) , z3.If(r8s28, 0, 0) , z3.If(r8s29, 0, 0) , z3.If(r8s30, 0, 0) , z3.If(r8s31, 30, 0) , z3.If(r8s33, 70, 0) , z3.If(r8s34, 70, 0) , z3.If(r8s35, 30, 0) , z3.If(r8s36, 0, 0) , z3.If(r8s37, 70, 0) , z3.If(r8s38, 70, 0) , z3.If(r8s39, 0, 0) , z3.If(r8s40, 30, 0) , z3.If(r8s41, 0, 0) , z3.If(r8s42, 30, 0) , z3.If(r8s43, 70, 0) , z3.If(r8s44, 0, 0) , z3.If(r8s45, 0, 0) , z3.If(r8s46, 0, 0) , z3.If(r8s47, 70, 0) , z3.If(r8s48, 0, 0) , z3.If(r8s49, 30, 0) , z3.If(r8s50, 30, 0) , z3.If(r8s51, 30, 0) , z3.If(r9s0, 70, 0) , z3.If(r9s1, 0, 0) , z3.If(r9s2, 0, 0) , z3.If(r9s3, 0, 0) , z3.If(r9s4, 0, 0) , z3.If(r9s5, 0, 0) , z3.If(r9s6, 0, 0) , z3.If(r9s7, 0, 0) , z3.If(r9s8, 0, 0) , z3.If(r9s9, 0, 0) , z3.If(r9s10, 0, 0) , z3.If(r9s11, 0, 0) , z3.If(r9s12, 0, 0) , z3.If(r9s13, 70, 0) , z3.If(r9s14, 70, 0) , z3.If(r9s15, 0, 0) , z3.If(r9s16, 0, 0) , z3.If(r9s17, 70, 0) , z3.If(r9s18, 0, 0) , z3.If(r9s19, 0, 0) , z3.If(r9s20, 0, 0) , z3.If(r9s21, 0, 0) , z3.If(r9s22, 0, 0) , z3.If(r9s23, 0, 0) , z3.If(r9s24, 70, 0) , z3.If(r9s25, 0, 0) , z3.If(r9s26, 30, 0) , z3.If(r9s27, 0, 0) , z3.If(r9s28, 70, 0) , z3.If(r9s29, 70, 0) , z3.If(r9s30, 0, 0) , z3.If(r9s31, 0, 0) , z3.If(r9s32, 70, 0) , z3.If(r9s33, 30, 0) , z3.If(r9s34, 0, 0) , z3.If(r9s35, 0, 0) , z3.If(r9s36, 70, 0) , z3.If(r9s37, 70, 0) , z3.If(r9s38, 0, 0) , z3.If(r9s39, 0, 0) , z3.If(r9s40, 70, 0) , z3.If(r9s41, 0, 0) , z3.If(r9s42, 0, 0) , z3.If(r9s43, 0, 0) , z3.If(r9s44, 70, 0) , z3.If(r9s45, 0, 0) , z3.If(r9s46, 70, 0) , z3.If(r9s47, 0, 0) , z3.If(r9s48, 0, 0) , z3.If(r9s49, 70, 0) , z3.If(r9s50, 0, 0) , z3.If(r9s51, 0, 0) , z3.If(r10s0, 0, 0) , z3.If(r10s1, 0, 0) , z3.If(r10s2, 0, 0) , z3.If(r10s3, 0, 0) , z3.If(r10s4, 0, 0) , z3.If(r10s5, 0, 0) , z3.If(r10s6, 0, 0) , z3.If(r10s7, 0, 0) , z3.If(r10s9, 70, 0) , z3.If(r10s10, 0, 0) , z3.If(r10s12, 70, 0) , z3.If(r10s13, 70, 0) , z3.If(r10s14, 0, 0) , z3.If(r10s15, 0, 0) , z3.If(r10s16, 0, 0) , z3.If(r10s17, 0, 0) , z3.If(r10s18, 70, 0) , z3.If(r10s19, 0, 0) , z3.If(r10s20, 0, 0) , z3.If(r10s21, 70, 0) , z3.If(r10s22, 0, 0) , z3.If(r10s23, 0, 0) , z3.If(r10s24, 0, 0) , z3.If(r10s25, 0, 0) , z3.If(r10s26, 0, 0) , z3.If(r10s27, 0, 0) , z3.If(r10s28, 70, 0) , z3.If(r10s29, 0, 0) , z3.If(r10s30, 70, 0) , z3.If(r10s31, 70, 0) , z3.If(r10s32, 70, 0) , z3.If(r10s33, 0, 0) , z3.If(r10s34, 0, 0) , z3.If(r10s35, 70, 0) , z3.If(r10s36, 0, 0) , z3.If(r10s37, 70, 0) , z3.If(r10s38, 0, 0) , z3.If(r10s39, 0, 0) , z3.If(r10s40, 70, 0) , z3.If(r10s41, 0, 0) , z3.If(r10s42, 0, 0) , z3.If(r10s43, 0, 0) , z3.If(r10s44, 0, 0) , z3.If(r10s45, 0, 0) , z3.If(r10s46, 0, 0) , z3.If(r10s47, 0, 0) , z3.If(r10s49, 0, 0) , z3.If(r10s50, 0, 0) , z3.If(r10s51, 0, 0) , z3.If(r11s0, 0, 0) , z3.If(r11s1, 0, 0) , z3.If(r11s2, 0, 0) , z3.If(r11s3, 70, 0) , z3.If(r11s4, 70, 0) , z3.If(r11s5, 30, 0) , z3.If(r11s6, 0, 0) , z3.If(r11s7, 0, 0) , z3.If(r11s8, 0, 0) , z3.If(r11s9, 70, 0) , z3.If(r11s10, 0, 0) , z3.If(r11s11, 70, 0) , z3.If(r11s12, 0, 0) , z3.If(r11s13, 70, 0) , z3.If(r11s14, 0, 0) , z3.If(r11s15, 0, 0) , z3.If(r11s16, 0, 0) , z3.If(r11s17, 0, 0) , z3.If(r11s18, 0, 0) , z3.If(r11s19, 0, 0) , z3.If(r11s20, 0, 0) , z3.If(r11s21, 0, 0) , z3.If(r11s22, 0, 0) , z3.If(r11s23, 0, 0) , z3.If(r11s24, 0, 0) , z3.If(r11s25, 70, 0) , z3.If(r11s26, 0, 0) , z3.If(r11s27, 0, 0) , z3.If(r11s28, 0, 0) , z3.If(r11s29, 0, 0) , z3.If(r11s30, 0, 0) , z3.If(r11s31, 0, 0) , z3.If(r11s32, 0, 0) , z3.If(r11s33, 0, 0) , z3.If(r11s34, 30, 0) , z3.If(r11s35, 0, 0) , z3.If(r11s36, 0, 0) , z3.If(r11s37, 0, 0) , z3.If(r11s38, 0, 0) , z3.If(r11s39, 0, 0) , z3.If(r11s40, 0, 0) , z3.If(r11s41, 0, 0) , z3.If(r11s43, 0, 0) , z3.If(r11s44, 0, 0) , z3.If(r11s45, 0, 0) , z3.If(r11s46, 0, 0) , z3.If(r11s48, 0, 0) , z3.If(r11s50, 0, 0) , z3.If(r11s51, 0, 0) , z3.If(r12s1, 0, 0) , z3.If(r12s2, 0, 0) , z3.If(r12s3, 0, 0) , z3.If(r12s4, 70, 0) , z3.If(r12s5, 70, 0) , z3.If(r12s6, 0, 0) , z3.If(r12s7, 0, 0) , z3.If(r12s8, 30, 0) , z3.If(r12s9, 0, 0) , z3.If(r12s10, 0, 0) , z3.If(r12s12, 0, 0) , z3.If(r12s13, 30, 0) , z3.If(r12s14, 0, 0) , z3.If(r12s15, 0, 0) , z3.If(r12s16, 30, 0) , z3.If(r12s19, 0, 0) , z3.If(r12s20, 0, 0) , z3.If(r12s21, 0, 0) , z3.If(r12s22, 0, 0) , z3.If(r12s23, 0, 0) , z3.If(r12s24, 70, 0) , z3.If(r12s25, 0, 0) , z3.If(r12s26, 0, 0) , z3.If(r12s27, 30, 0) , z3.If(r12s28, 0, 0) , z3.If(r12s29, 0, 0) , z3.If(r12s30, 0, 0) , z3.If(r12s31, 70, 0) , z3.If(r12s32, 70, 0) , z3.If(r12s33, 0, 0) , z3.If(r12s34, 70, 0) , z3.If(r12s35, 0, 0) , z3.If(r12s36, 0, 0) , z3.If(r12s37, 0, 0) , z3.If(r12s38, 70, 0) , z3.If(r12s39, 0, 0) , z3.If(r12s40, 30, 0) , z3.If(r12s41, 70, 0) , z3.If(r12s42, 0, 0) , z3.If(r12s43, 30, 0) , z3.If(r12s44, 30, 0) , z3.If(r12s45, 30, 0) , z3.If(r12s46, 0, 0) , z3.If(r12s48, 0, 0) , z3.If(r12s49, 0, 0) , z3.If(r12s50, 0, 0) , z3.If(r12s51, 0, 0) , z3.If(r13s0, 70, 0) , z3.If(r13s2, 30, 0) , z3.If(r13s3, 0, 0) , z3.If(r13s4, 0, 0) , z3.If(r13s5, 0, 0) , z3.If(r13s6, 0, 0) , z3.If(r13s7, 0, 0) , z3.If(r13s10, 0, 0) , z3.If(r13s12, 0, 0) , z3.If(r13s13, 30, 0) , z3.If(r13s14, 70, 0) , z3.If(r13s15, 0, 0) , z3.If(r13s16, 0, 0) , z3.If(r13s18, 0, 0) , z3.If(r13s19, 70, 0) , z3.If(r13s20, 0, 0) , z3.If(r13s22, 0, 0) , z3.If(r13s23, 0, 0) , z3.If(r13s25, 0, 0) , z3.If(r13s26, 0, 0) , z3.If(r13s27, 0, 0) , z3.If(r13s28, 0, 0) , z3.If(r13s29, 0, 0) , z3.If(r13s30, 0, 0) , z3.If(r13s31, 0, 0) , z3.If(r13s32, 0, 0) , z3.If(r13s33, 0, 0) , z3.If(r13s34, 0, 0) , z3.If(r13s35, 30, 0) , z3.If(r13s37, 30, 0) , z3.If(r13s38, 70, 0) , z3.If(r13s39, 0, 0) , z3.If(r13s40, 30, 0) , z3.If(r13s41, 0, 0) , z3.If(r13s42, 0, 0) , z3.If(r13s43, 70, 0) , z3.If(r13s44, 0, 0) , z3.If(r13s45, 0, 0) , z3.If(r13s46, 30, 0) , z3.If(r13s47, 30, 0) , z3.If(r13s49, 30, 0) , z3.If(r13s50, 0, 0) , z3.If(r13s51, 30, 0) , z3.If(r14s0, 0, 0) , z3.If(r14s1, 0, 0) , z3.If(r14s2, 0, 0) , z3.If(r14s4, 0, 0) , z3.If(r14s5, 0, 0) , z3.If(r14s7, 70, 0) , z3.If(r14s8, 70, 0) , z3.If(r14s9, 0, 0) , z3.If(r14s10, 0, 0) , z3.If(r14s11, 0, 0) , z3.If(r14s12, 0, 0) , z3.If(r14s13, 70, 0) , z3.If(r14s14, 0, 0) , z3.If(r14s15, 0, 0) , z3.If(r14s17, 30, 0) , z3.If(r14s18, 0, 0) , z3.If(r14s19, 30, 0) , z3.If(r14s20, 0, 0) , z3.If(r14s21, 0, 0) , z3.If(r14s22, 0, 0) , z3.If(r14s23, 0, 0) , z3.If(r14s24, 70, 0) , z3.If(r14s25, 30, 0) , z3.If(r14s26, 0, 0) , z3.If(r14s27, 0, 0) , z3.If(r14s28, 0, 0) , z3.If(r14s29, 0, 0) , z3.If(r14s30, 0, 0) , z3.If(r14s34, 70, 0) , z3.If(r14s36, 0, 0) , z3.If(r14s38, 0, 0) , z3.If(r14s39, 0, 0) , z3.If(r14s40, 0, 0) , z3.If(r14s41, 0, 0) , z3.If(r14s42, 70, 0) , z3.If(r14s43, 70, 0) , z3.If(r14s44, 70, 0) , z3.If(r14s45, 0, 0) , z3.If(r14s46, 0, 0) , z3.If(r14s47, 30, 0) , z3.If(r14s48, 70, 0) , z3.If(r14s49, 70, 0) , z3.If(r14s50, 0, 0) , z3.If(r14s51, 0, 0) , z3.If(r15s1, 0, 0) , z3.If(r15s2, 0, 0) , z3.If(r15s3, 0, 0) , z3.If(r15s4, 0, 0) , z3.If(r15s5, 0, 0) , z3.If(r15s6, 0, 0) , z3.If(r15s7, 0, 0) , z3.If(r15s8, 0, 0) , z3.If(r15s9, 70, 0) , z3.If(r15s10, 0, 0) , z3.If(r15s11, 0, 0) , z3.If(r15s12, 70, 0) , z3.If(r15s13, 70, 0) , z3.If(r15s14, 0, 0) , z3.If(r15s15, 0, 0) , z3.If(r15s16, 0, 0) , z3.If(r15s17, 0, 0) , z3.If(r15s19, 0, 0) , z3.If(r15s20, 0, 0) , z3.If(r15s21, 0, 0) , z3.If(r15s22, 70, 0) , z3.If(r15s23, 70, 0) , z3.If(r15s24, 70, 0) , z3.If(r15s25, 0, 0) , z3.If(r15s26, 0, 0) , z3.If(r15s27, 0, 0) , z3.If(r15s28, 0, 0) , z3.If(r15s29, 0, 0) , z3.If(r15s30, 0, 0) , z3.If(r15s31, 0, 0) , z3.If(r15s32, 0, 0) , z3.If(r15s33, 0, 0) , z3.If(r15s34, 70, 0) , z3.If(r15s35, 0, 0) , z3.If(r15s36, 0, 0) , z3.If(r15s37, 70, 0) , z3.If(r15s38, 0, 0) , z3.If(r15s39, 0, 0) , z3.If(r15s40, 0, 0) , z3.If(r15s41, 0, 0) , z3.If(r15s42, 0, 0) , z3.If(r15s43, 0, 0) , z3.If(r15s44, 0, 0) , z3.If(r15s45, 0, 0) , z3.If(r15s46, 0, 0) , z3.If(r15s48, 0, 0) , z3.If(r15s49, 0, 0) , z3.If(r15s50, 0, 0) , z3.If(r15s51, 70, 0) , z3.If(r16s1, 0, 0) , z3.If(r16s2, 30, 0) , z3.If(r16s3, 30, 0) , z3.If(r16s4, 70, 0) , z3.If(r16s5, 30, 0) , z3.If(r16s7, 70, 0) , z3.If(r16s8, 0, 0) , z3.If(r16s9, 0, 0) , z3.If(r16s11, 0, 0) , z3.If(r16s12, 30, 0) , z3.If(r16s13, 0, 0) , z3.If(r16s14, 0, 0) , z3.If(r16s15, 0, 0) , z3.If(r16s16, 0, 0) , z3.If(r16s17, 0, 0) , z3.If(r16s19, 0, 0) , z3.If(r16s20, 0, 0) , z3.If(r16s21, 0, 0) , z3.If(r16s22, 0, 0) , z3.If(r16s23, 0, 0) , z3.If(r16s24, 0, 0) , z3.If(r16s25, 0, 0) , z3.If(r16s26, 0, 0) , z3.If(r16s27, 0, 0) , z3.If(r16s28, 0, 0) , z3.If(r16s29, 0, 0) , z3.If(r16s30, 0, 0) , z3.If(r16s31, 70, 0) , z3.If(r16s32, 0, 0) , z3.If(r16s33, 0, 0) , z3.If(r16s34, 70, 0) , z3.If(r16s35, 0, 0) , z3.If(r16s36, 0, 0) , z3.If(r16s38, 0, 0) , z3.If(r16s39, 30, 0) , z3.If(r16s40, 0, 0) , z3.If(r16s41, 0, 0) , z3.If(r16s42, 0, 0) , z3.If(r16s43, 0, 0) , z3.If(r16s44, 30, 0) , z3.If(r16s45, 0, 0) , z3.If(r16s46, 30, 0) , z3.If(r16s48, 0, 0) , z3.If(r16s49, 0, 0) , z3.If(r16s50, 0, 0) , z3.If(r16s51, 70, 0) , z3.If(r17s0, 0, 0) , z3.If(r17s1, 0, 0) , z3.If(r17s2, 70, 0) , z3.If(r17s3, 70, 0) , z3.If(r17s4, 0, 0) , z3.If(r17s5, 0, 0) , z3.If(r17s7, 70, 0) , z3.If(r17s8, 0, 0) , z3.If(r17s11, 0, 0) , z3.If(r17s12, 0, 0) , z3.If(r17s13, 70, 0) , z3.If(r17s14, 0, 0) , z3.If(r17s15, 0, 0) , z3.If(r17s16, 30, 0) , z3.If(r17s17, 30, 0) , z3.If(r17s18, 70, 0) , z3.If(r17s19, 30, 0) , z3.If(r17s20, 0, 0) , z3.If(r17s21, 0, 0) , z3.If(r17s22, 0, 0) , z3.If(r17s23, 0, 0) , z3.If(r17s25, 0, 0) , z3.If(r17s26, 0, 0) , z3.If(r17s27, 30, 0) , z3.If(r17s28, 0, 0) , z3.If(r17s29, 30, 0) , z3.If(r17s30, 30, 0) , z3.If(r17s31, 0, 0) , z3.If(r17s32, 70, 0) , z3.If(r17s33, 0, 0) , z3.If(r17s34, 30, 0) , z3.If(r17s35, 70, 0) , z3.If(r17s36, 0, 0) , z3.If(r17s37, 30, 0) , z3.If(r17s38, 30, 0) , z3.If(r17s39, 0, 0) , z3.If(r17s40, 0, 0) , z3.If(r17s41, 30, 0) , z3.If(r17s42, 0, 0) , z3.If(r17s43, 30, 0) , z3.If(r17s44, 0, 0) , z3.If(r17s45, 0, 0) , z3.If(r17s46, 0, 0) , z3.If(r17s47, 70, 0) , z3.If(r17s48, 0, 0) , z3.If(r17s49, 0, 0) , z3.If(r17s50, 0, 0) , z3.If(r17s51, 0, 0) , z3.If(r18s0, 70, 0) , z3.If(r18s1, 0, 0) , z3.If(r18s2, 0, 0) , z3.If(r18s3, 0, 0) , z3.If(r18s4, 70, 0) , z3.If(r18s5, 0, 0) , z3.If(r18s6, 0, 0) , z3.If(r18s7, 0, 0) , z3.If(r18s8, 0, 0) , z3.If(r18s10, 0, 0) , z3.If(r18s11, 0, 0) , z3.If(r18s12, 70, 0) , z3.If(r18s13, 0, 0) , z3.If(r18s14, 0, 0) , z3.If(r18s15, 70, 0) , z3.If(r18s16, 0, 0) , z3.If(r18s17, 0, 0) , z3.If(r18s18, 70, 0) , z3.If(r18s19, 0, 0) , z3.If(r18s20, 0, 0) , z3.If(r18s21, 0, 0) , z3.If(r18s22, 0, 0) , z3.If(r18s23, 70, 0) , z3.If(r18s25, 0, 0) , z3.If(r18s26, 0, 0) , z3.If(r18s27, 0, 0) , z3.If(r18s28, 0, 0) , z3.If(r18s29, 0, 0) , z3.If(r18s30, 0, 0) , z3.If(r18s31, 0, 0) , z3.If(r18s32, 0, 0) , z3.If(r18s33, 70, 0) , z3.If(r18s34, 0, 0) , z3.If(r18s35, 0, 0) , z3.If(r18s36, 0, 0) , z3.If(r18s37, 0, 0) , z3.If(r18s38, 0, 0) , z3.If(r18s39, 0, 0) , z3.If(r18s40, 0, 0) , z3.If(r18s41, 0, 0) , z3.If(r18s42, 0, 0) , z3.If(r18s43, 0, 0) , z3.If(r18s44, 0, 0) , z3.If(r18s45, 70, 0) , z3.If(r18s46, 0, 0) , z3.If(r18s47, 0, 0) , z3.If(r18s48, 70, 0) , z3.If(r18s49, 0, 0) , z3.If(r18s50, 0, 0) , z3.If(r18s51, 70, 0) , z3.If(r19s0, 30, 0) , z3.If(r19s1, 70, 0) , z3.If(r19s3, 30, 0) , z3.If(r19s4, 70, 0) , z3.If(r19s5, 0, 0) , z3.If(r19s6, 30, 0) , z3.If(r19s7, 30, 0) , z3.If(r19s8, 0, 0) , z3.If(r19s9, 30, 0) , z3.If(r19s10, 0, 0) , z3.If(r19s11, 0, 0) , z3.If(r19s12, 0, 0) , z3.If(r19s13, 0, 0) , z3.If(r19s14, 0, 0) , z3.If(r19s15, 70, 0) , z3.If(r19s16, 70, 0) , z3.If(r19s17, 70, 0) , z3.If(r19s18, 70, 0) , z3.If(r19s19, 30, 0) , z3.If(r19s20, 0, 0) , z3.If(r19s21, 0, 0) , z3.If(r19s22, 70, 0) , z3.If(r19s23, 0, 0) , z3.If(r19s24, 30, 0) , z3.If(r19s25, 0, 0) , z3.If(r19s26, 0, 0) , z3.If(r19s27, 0, 0) , z3.If(r19s28, 0, 0) , z3.If(r19s29, 0, 0) , z3.If(r19s30, 0, 0) , z3.If(r19s31, 30, 0) , z3.If(r19s32, 0, 0) , z3.If(r19s33, 0, 0) , z3.If(r19s34, 0, 0) , z3.If(r19s35, 0, 0) , z3.If(r19s36, 0, 0) , z3.If(r19s37, 70, 0) , z3.If(r19s38, 0, 0) , z3.If(r19s39, 0, 0) , z3.If(r19s40, 0, 0) , z3.If(r19s41, 0, 0) , z3.If(r19s42, 70, 0) , z3.If(r19s43, 0, 0) , z3.If(r19s44, 70, 0) , z3.If(r19s45, 0, 0) , z3.If(r19s46, 0, 0) , z3.If(r19s47, 0, 0) , z3.If(r19s48, 0, 0) , z3.If(r19s49, 70, 0) , z3.If(r19s50, 0, 0) , z3.If(r19s51, 70, 0) , z3.If(r20s0, 0, 0) , z3.If(r20s1, 0, 0) , z3.If(r20s2, 0, 0) , z3.If(r20s3, 0, 0) , z3.If(r20s4, 0, 0) , z3.If(r20s5, 0, 0) , z3.If(r20s6, 70, 0) , z3.If(r20s7, 0, 0) , z3.If(r20s8, 0, 0) , z3.If(r20s9, 70, 0) , z3.If(r20s10, 0, 0) , z3.If(r20s11, 0, 0) , z3.If(r20s12, 0, 0) , z3.If(r20s13, 70, 0) , z3.If(r20s15, 0, 0) , z3.If(r20s17, 70, 0) , z3.If(r20s18, 0, 0) , z3.If(r20s19, 0, 0) , z3.If(r20s20, 0, 0) , z3.If(r20s21, 0, 0) , z3.If(r20s22, 0, 0) , z3.If(r20s23, 0, 0) , z3.If(r20s24, 70, 0) , z3.If(r20s25, 0, 0) , z3.If(r20s26, 70, 0) , z3.If(r20s30, 70, 0) , z3.If(r20s31, 0, 0) , z3.If(r20s32, 70, 0) , z3.If(r20s33, 0, 0) , z3.If(r20s34, 70, 0) , z3.If(r20s35, 0, 0) , z3.If(r20s36, 70, 0) , z3.If(r20s37, 70, 0) , z3.If(r20s38, 0, 0) , z3.If(r20s39, 0, 0) , z3.If(r20s40, 0, 0) , z3.If(r20s41, 0, 0) , z3.If(r20s42, 0, 0) , z3.If(r20s43, 0, 0) , z3.If(r20s44, 0, 0) , z3.If(r20s45, 70, 0) , z3.If(r20s46, 0, 0) , z3.If(r20s47, 70, 0) , z3.If(r20s48, 0, 0) , z3.If(r20s49, 0, 0) , z3.If(r20s50, 0, 0) , z3.If(r20s51, 0, 0) , z3.If(r21s1, 0, 0) , z3.If(r21s2, 70, 0) , z3.If(r21s3, 0, 0) , z3.If(r21s4, 0, 0) , z3.If(r21s5, 70, 0) , z3.If(r21s6, 0, 0) , z3.If(r21s7, 0, 0) , z3.If(r21s8, 0, 0) , z3.If(r21s10, 0, 0) , z3.If(r21s12, 70, 0) , z3.If(r21s14, 30, 0) , z3.If(r21s15, 70, 0) , z3.If(r21s18, 70, 0) , z3.If(r21s19, 0, 0) , z3.If(r21s20, 0, 0) , z3.If(r21s22, 0, 0) , z3.If(r21s23, 30, 0) , z3.If(r21s24, 0, 0) , z3.If(r21s25, 0, 0) , z3.If(r21s26, 0, 0) , z3.If(r21s27, 0, 0) , z3.If(r21s28, 0, 0) , z3.If(r21s29, 0, 0) , z3.If(r21s30, 0, 0) , z3.If(r21s31, 0, 0) , z3.If(r21s33, 0, 0) , z3.If(r21s34, 0, 0) , z3.If(r21s35, 70, 0) , z3.If(r21s37, 30, 0) , z3.If(r21s38, 30, 0) , z3.If(r21s39, 0, 0) , z3.If(r21s40, 0, 0) , z3.If(r21s41, 0, 0) , z3.If(r21s42, 0, 0) , z3.If(r21s43, 30, 0) , z3.If(r21s44, 0, 0) , z3.If(r21s45, 0, 0) , z3.If(r21s46, 0, 0) , z3.If(r21s47, 0, 0) , z3.If(r21s48, 0, 0) , z3.If(r21s49, 0, 0) , z3.If(r21s50, 30, 0) , z3.If(r21s51, 0, 0) , z3.If(r22s0, 0, 0) , z3.If(r22s1, 0, 0) , z3.If(r22s2, 0, 0) , z3.If(r22s3, 0, 0) , z3.If(r22s4, 70, 0) , z3.If(r22s5, 70, 0) , z3.If(r22s6, 0, 0) , z3.If(r22s7, 0, 0) , z3.If(r22s8, 0, 0) , z3.If(r22s9, 30, 0) , z3.If(r22s10, 30, 0) , z3.If(r22s11, 0, 0) , z3.If(r22s12, 0, 0) , z3.If(r22s13, 30, 0) , z3.If(r22s14, 0, 0) , z3.If(r22s15, 0, 0) , z3.If(r22s16, 70, 0) , z3.If(r22s17, 0, 0) , z3.If(r22s18, 0, 0) , z3.If(r22s19, 0, 0) , z3.If(r22s20, 30, 0) , z3.If(r22s21, 0, 0) , z3.If(r22s22, 0, 0) , z3.If(r22s23, 0, 0) , z3.If(r22s24, 0, 0) , z3.If(r22s25, 0, 0) , z3.If(r22s26, 0, 0) , z3.If(r22s27, 0, 0) , z3.If(r22s28, 0, 0) , z3.If(r22s29, 0, 0) , z3.If(r22s30, 0, 0) , z3.If(r22s31, 0, 0) , z3.If(r22s32, 70, 0) , z3.If(r22s33, 30, 0) , z3.If(r22s34, 30, 0) , z3.If(r22s35, 0, 0) , z3.If(r22s36, 0, 0) , z3.If(r22s37, 0, 0) , z3.If(r22s38, 0, 0) , z3.If(r22s39, 70, 0) , z3.If(r22s40, 0, 0) , z3.If(r22s41, 0, 0) , z3.If(r22s42, 30, 0) , z3.If(r22s43, 0, 0) , z3.If(r22s44, 0, 0) , z3.If(r22s45, 70, 0) , z3.If(r22s46, 0, 0) , z3.If(r22s47, 0, 0) , z3.If(r22s48, 0, 0) , z3.If(r22s49, 0, 0) , z3.If(r22s50, 0, 0) , z3.If(r22s51, 0, 0) , z3.If(r23s0, 0, 0) , z3.If(r23s1, 0, 0) , z3.If(r23s2, 0, 0) , z3.If(r23s3, 70, 0) , z3.If(r23s4, 70, 0) , z3.If(r23s5, 0, 0) , z3.If(r23s6, 30, 0) , z3.If(r23s7, 0, 0) , z3.If(r23s8, 30, 0) , z3.If(r23s10, 70, 0) , z3.If(r23s11, 0, 0) , z3.If(r23s12, 0, 0) , z3.If(r23s13, 0, 0) , z3.If(r23s14, 0, 0) , z3.If(r23s15, 0, 0) , z3.If(r23s16, 70, 0) , z3.If(r23s17, 0, 0) , z3.If(r23s18, 0, 0) , z3.If(r23s19, 0, 0) , z3.If(r23s20, 0, 0) , z3.If(r23s21, 0, 0) , z3.If(r23s22, 0, 0) , z3.If(r23s23, 0, 0) , z3.If(r23s25, 0, 0) , z3.If(r23s26, 0, 0) , z3.If(r23s27, 0, 0) , z3.If(r23s28, 0, 0) , z3.If(r23s29, 0, 0) , z3.If(r23s30, 0, 0) , z3.If(r23s31, 30, 0) , z3.If(r23s32, 70, 0) , z3.If(r23s33, 0, 0) , z3.If(r23s34, 30, 0) , z3.If(r23s35, 0, 0) , z3.If(r23s36, 0, 0) , z3.If(r23s37, 0, 0) , z3.If(r23s38, 0, 0) , z3.If(r23s39, 70, 0) , z3.If(r23s40, 0, 0) , z3.If(r23s41, 0, 0) , z3.If(r23s42, 70, 0) , z3.If(r23s43, 0, 0) , z3.If(r23s44, 0, 0) , z3.If(r23s45, 70, 0) , z3.If(r23s46, 0, 0) , z3.If(r23s47, 0, 0) , z3.If(r23s48, 30, 0) , z3.If(r23s49, 0, 0) , z3.If(r23s50, 0, 0) , z3.If(r23s51, 0, 0)]
# 60/40
# scores = [z3.If(r0s0, 60, 0) , z3.If(r0s1, 0, 0) , z3.If(r0s2, 0, 0) , z3.If(r0s4, 0, 0) , z3.If(r0s5, 0, 0) , z3.If(r0s7, 0, 0) , z3.If(r0s8, 0, 0) , z3.If(r0s9, 0, 0) , z3.If(r0s10, 0, 0) , z3.If(r0s12, 0, 0) , z3.If(r0s13, 0, 0) , z3.If(r0s14, 40, 0) , z3.If(r0s15, 0, 0) , z3.If(r0s16, 0, 0) , z3.If(r0s18, 40, 0) , z3.If(r0s19, 0, 0) , z3.If(r0s20, 0, 0) , z3.If(r0s21, 0, 0) , z3.If(r0s22, 0, 0) , z3.If(r0s23, 60, 0) , z3.If(r0s24, 60, 0) , z3.If(r0s25, 0, 0) , z3.If(r0s26, 40, 0) , z3.If(r0s27, 0, 0) , z3.If(r0s28, 60, 0) , z3.If(r0s29, 60, 0) , z3.If(r0s30, 60, 0) , z3.If(r0s31, 0, 0) , z3.If(r0s32, 60, 0) , z3.If(r0s34, 0, 0) , z3.If(r0s35, 0, 0) , z3.If(r0s36, 0, 0) , z3.If(r0s37, 60, 0) , z3.If(r0s38, 0, 0) , z3.If(r0s39, 0, 0) , z3.If(r0s40, 40, 0) , z3.If(r0s41, 0, 0) , z3.If(r0s42, 0, 0) , z3.If(r0s43, 0, 0) , z3.If(r0s44, 60, 0) , z3.If(r0s45, 0, 0) , z3.If(r0s46, 0, 0) , z3.If(r0s47, 0, 0) , z3.If(r0s48, 0, 0) , z3.If(r0s49, 0, 0) , z3.If(r0s50, 0, 0) , z3.If(r0s51, 0, 0) , z3.If(r1s0, 60, 0) , z3.If(r1s1, 0, 0) , z3.If(r1s2, 40, 0) , z3.If(r1s3, 60, 0) , z3.If(r1s4, 0, 0) , z3.If(r1s5, 0, 0) , z3.If(r1s6, 0, 0) , z3.If(r1s7, 0, 0) , z3.If(r1s8, 0, 0) , z3.If(r1s9, 60, 0) , z3.If(r1s10, 40, 0) , z3.If(r1s11, 60, 0) , z3.If(r1s12, 0, 0) , z3.If(r1s13, 0, 0) , z3.If(r1s14, 0, 0) , z3.If(r1s15, 0, 0) , z3.If(r1s16, 0, 0) , z3.If(r1s17, 40, 0) , z3.If(r1s18, 60, 0) , z3.If(r1s19, 0, 0) , z3.If(r1s20, 0, 0) , z3.If(r1s21, 0, 0) , z3.If(r1s22, 60, 0) , z3.If(r1s23, 0, 0) , z3.If(r1s24, 0, 0) , z3.If(r1s25, 0, 0) , z3.If(r1s26, 0, 0) , z3.If(r1s27, 0, 0) , z3.If(r1s28, 60, 0) , z3.If(r1s29, 40, 0) , z3.If(r1s30, 0, 0) , z3.If(r1s31, 0, 0) , z3.If(r1s32, 0, 0) , z3.If(r1s33, 0, 0) , z3.If(r1s35, 0, 0) , z3.If(r1s36, 40, 0) , z3.If(r1s37, 0, 0) , z3.If(r1s38, 0, 0) , z3.If(r1s39, 0, 0) , z3.If(r1s40, 60, 0) , z3.If(r1s41, 0, 0) , z3.If(r1s42, 0, 0) , z3.If(r1s43, 0, 0) , z3.If(r1s44, 0, 0) , z3.If(r1s45, 0, 0) , z3.If(r1s46, 0, 0) , z3.If(r1s47, 0, 0) , z3.If(r1s48, 0, 0) , z3.If(r1s49, 0, 0) , z3.If(r1s50, 0, 0) , z3.If(r1s51, 0, 0) , z3.If(r2s0, 0, 0) , z3.If(r2s1, 0, 0) , z3.If(r2s2, 60, 0) , z3.If(r2s3, 0, 0) , z3.If(r2s4, 0, 0) , z3.If(r2s5, 0, 0) , z3.If(r2s6, 0, 0) , z3.If(r2s7, 40, 0) , z3.If(r2s8, 0, 0) , z3.If(r2s9, 0, 0) , z3.If(r2s10, 0, 0) , z3.If(r2s11, 0, 0) , z3.If(r2s12, 60, 0) , z3.If(r2s14, 40, 0) , z3.If(r2s15, 60, 0) , z3.If(r2s18, 0, 0) , z3.If(r2s19, 0, 0) , z3.If(r2s20, 0, 0) , z3.If(r2s22, 0, 0) , z3.If(r2s23, 0, 0) , z3.If(r2s24, 0, 0) , z3.If(r2s25, 0, 0) , z3.If(r2s26, 0, 0) , z3.If(r2s27, 0, 0) , z3.If(r2s28, 0, 0) , z3.If(r2s29, 0, 0) , z3.If(r2s30, 0, 0) , z3.If(r2s31, 0, 0) , z3.If(r2s33, 0, 0) , z3.If(r2s34, 0, 0) , z3.If(r2s35, 0, 0) , z3.If(r2s36, 0, 0) , z3.If(r2s37, 40, 0) , z3.If(r2s38, 0, 0) , z3.If(r2s39, 0, 0) , z3.If(r2s40, 0, 0) , z3.If(r2s41, 0, 0) , z3.If(r2s42, 0, 0) , z3.If(r2s43, 0, 0) , z3.If(r2s44, 0, 0) , z3.If(r2s45, 0, 0) , z3.If(r2s46, 40, 0) , z3.If(r2s47, 0, 0) , z3.If(r2s48, 0, 0) , z3.If(r2s49, 40, 0) , z3.If(r2s50, 0, 0) , z3.If(r2s51, 40, 0) , z3.If(r3s0, 0, 0) , z3.If(r3s2, 60, 0) , z3.If(r3s3, 60, 0) , z3.If(r3s4, 0, 0) , z3.If(r3s5, 0, 0) , z3.If(r3s6, 60, 0) , z3.If(r3s7, 0, 0) , z3.If(r3s8, 40, 0) , z3.If(r3s10, 0, 0) , z3.If(r3s11, 40, 0) , z3.If(r3s12, 0, 0) , z3.If(r3s15, 0, 0) , z3.If(r3s17, 40, 0) , z3.If(r3s18, 40, 0) , z3.If(r3s19, 0, 0) , z3.If(r3s20, 0, 0) , z3.If(r3s21, 0, 0) , z3.If(r3s22, 0, 0) , z3.If(r3s23, 0, 0) , z3.If(r3s25, 60, 0) , z3.If(r3s31, 40, 0) , z3.If(r3s32, 60, 0) , z3.If(r3s33, 0, 0) , z3.If(r3s34, 0, 0) , z3.If(r3s35, 60, 0) , z3.If(r3s36, 60, 0) , z3.If(r3s37, 60, 0) , z3.If(r3s38, 60, 0) , z3.If(r3s39, 0, 0) , z3.If(r3s40, 60, 0) , z3.If(r3s41, 40, 0) , z3.If(r3s42, 60, 0) , z3.If(r3s43, 40, 0) , z3.If(r3s44, 0, 0) , z3.If(r3s45, 0, 0) , z3.If(r3s46, 40, 0) , z3.If(r3s47, 60, 0) , z3.If(r3s48, 0, 0) , z3.If(r3s49, 40, 0) , z3.If(r3s50, 60, 0) , z3.If(r3s51, 40, 0) , z3.If(r4s0, 40, 0) , z3.If(r4s1, 0, 0) , z3.If(r4s2, 40, 0) , z3.If(r4s3, 0, 0) , z3.If(r4s4, 60, 0) , z3.If(r4s5, 60, 0) , z3.If(r4s6, 0, 0) , z3.If(r4s7, 0, 0) , z3.If(r4s8, 0, 0) , z3.If(r4s9, 0, 0) , z3.If(r4s10, 60, 0) , z3.If(r4s11, 0, 0) , z3.If(r4s12, 0, 0) , z3.If(r4s13, 0, 0) , z3.If(r4s14, 0, 0) , z3.If(r4s15, 0, 0) , z3.If(r4s16, 40, 0) , z3.If(r4s17, 0, 0) , z3.If(r4s18, 60, 0) , z3.If(r4s19, 0, 0) , z3.If(r4s20, 0, 0) , z3.If(r4s21, 0, 0) , z3.If(r4s22, 40, 0) , z3.If(r4s23, 0, 0) , z3.If(r4s24, 0, 0) , z3.If(r4s25, 0, 0) , z3.If(r4s26, 0, 0) , z3.If(r4s27, 0, 0) , z3.If(r4s28, 0, 0) , z3.If(r4s29, 0, 0) , z3.If(r4s30, 0, 0) , z3.If(r4s31, 0, 0) , z3.If(r4s32, 40, 0) , z3.If(r4s33, 0, 0) , z3.If(r4s34, 0, 0) , z3.If(r4s35, 0, 0) , z3.If(r4s36, 0, 0) , z3.If(r4s37, 0, 0) , z3.If(r4s38, 0, 0) , z3.If(r4s39, 0, 0) , z3.If(r4s40, 0, 0) , z3.If(r4s41, 0, 0) , z3.If(r4s42, 0, 0) , z3.If(r4s43, 0, 0) , z3.If(r4s44, 0, 0) , z3.If(r4s45, 0, 0) , z3.If(r4s46, 0, 0) , z3.If(r4s47, 40, 0) , z3.If(r4s48, 0, 0) , z3.If(r4s49, 0, 0) , z3.If(r4s50, 0, 0) , z3.If(r4s51, 60, 0) , z3.If(r5s0, 60, 0) , z3.If(r5s1, 0, 0) , z3.If(r5s2, 40, 0) , z3.If(r5s3, 40, 0) , z3.If(r5s4, 40, 0) , z3.If(r5s5, 0, 0) , z3.If(r5s6, 0, 0) , z3.If(r5s7, 0, 0) , z3.If(r5s8, 60, 0) , z3.If(r5s9, 60, 0) , z3.If(r5s10, 0, 0) , z3.If(r5s11, 0, 0) , z3.If(r5s12, 0, 0) , z3.If(r5s13, 0, 0) , z3.If(r5s14, 0, 0) , z3.If(r5s15, 0, 0) , z3.If(r5s16, 0, 0) , z3.If(r5s17, 60, 0) , z3.If(r5s18, 0, 0) , z3.If(r5s19, 40, 0) , z3.If(r5s20, 0, 0) , z3.If(r5s21, 0, 0) , z3.If(r5s22, 40, 0) , z3.If(r5s23, 0, 0) , z3.If(r5s24, 60, 0) , z3.If(r5s25, 0, 0) , z3.If(r5s26, 0, 0) , z3.If(r5s27, 0, 0) , z3.If(r5s28, 40, 0) , z3.If(r5s29, 0, 0) , z3.If(r5s30, 0, 0) , z3.If(r5s31, 0, 0) , z3.If(r5s32, 0, 0) , z3.If(r5s33, 0, 0) , z3.If(r5s34, 40, 0) , z3.If(r5s35, 0, 0) , z3.If(r5s36, 0, 0) , z3.If(r5s37, 60, 0) , z3.If(r5s38, 0, 0) , z3.If(r5s39, 40, 0) , z3.If(r5s40, 0, 0) , z3.If(r5s41, 0, 0) , z3.If(r5s42, 0, 0) , z3.If(r5s43, 0, 0) , z3.If(r5s44, 0, 0) , z3.If(r5s45, 0, 0) , z3.If(r5s46, 0, 0) , z3.If(r5s47, 0, 0) , z3.If(r5s48, 60, 0) , z3.If(r5s49, 40, 0) , z3.If(r5s50, 0, 0) , z3.If(r5s51, 0, 0) , z3.If(r6s0, 40, 0) , z3.If(r6s1, 0, 0) , z3.If(r6s2, 40, 0) , z3.If(r6s3, 40, 0) , z3.If(r6s4, 40, 0) , z3.If(r6s5, 0, 0) , z3.If(r6s7, 0, 0) , z3.If(r6s8, 0, 0) , z3.If(r6s9, 40, 0) , z3.If(r6s11, 0, 0) , z3.If(r6s12, 0, 0) , z3.If(r6s13, 0, 0) , z3.If(r6s14, 0, 0) , z3.If(r6s15, 0, 0) , z3.If(r6s16, 0, 0) , z3.If(r6s17, 40, 0) , z3.If(r6s18, 60, 0) , z3.If(r6s19, 60, 0) , z3.If(r6s20, 0, 0) , z3.If(r6s21, 40, 0) , z3.If(r6s22, 0, 0) , z3.If(r6s23, 0, 0) , z3.If(r6s24, 40, 0) , z3.If(r6s25, 0, 0) , z3.If(r6s26, 0, 0) , z3.If(r6s27, 0, 0) , z3.If(r6s28, 60, 0) , z3.If(r6s29, 40, 0) , z3.If(r6s30, 60, 0) , z3.If(r6s31, 40, 0) , z3.If(r6s32, 0, 0) , z3.If(r6s33, 0, 0) , z3.If(r6s34, 0, 0) , z3.If(r6s36, 0, 0) , z3.If(r6s39, 0, 0) , z3.If(r6s40, 0, 0) , z3.If(r6s41, 0, 0) , z3.If(r6s42, 40, 0) , z3.If(r6s44, 0, 0) , z3.If(r6s45, 0, 0) , z3.If(r6s47, 60, 0) , z3.If(r6s48, 0, 0) , z3.If(r6s49, 60, 0) , z3.If(r6s50, 0, 0) , z3.If(r6s51, 40, 0) , z3.If(r7s0, 60, 0) , z3.If(r7s1, 0, 0) , z3.If(r7s2, 0, 0) , z3.If(r7s3, 40, 0) , z3.If(r7s4, 0, 0) , z3.If(r7s5, 0, 0) , z3.If(r7s6, 0, 0) , z3.If(r7s7, 40, 0) , z3.If(r7s10, 0, 0) , z3.If(r7s12, 0, 0) , z3.If(r7s13, 60, 0) , z3.If(r7s14, 0, 0) , z3.If(r7s15, 0, 0) , z3.If(r7s16, 0, 0) , z3.If(r7s17, 40, 0) , z3.If(r7s18, 40, 0) , z3.If(r7s19, 0, 0) , z3.If(r7s20, 0, 0) , z3.If(r7s22, 60, 0) , z3.If(r7s23, 60, 0) , z3.If(r7s24, 40, 0) , z3.If(r7s25, 0, 0) , z3.If(r7s26, 60, 0) , z3.If(r7s27, 0, 0) , z3.If(r7s28, 60, 0) , z3.If(r7s29, 60, 0) , z3.If(r7s30, 0, 0) , z3.If(r7s31, 60, 0) , z3.If(r7s32, 0, 0) , z3.If(r7s33, 0, 0) , z3.If(r7s34, 0, 0) , z3.If(r7s35, 0, 0) , z3.If(r7s36, 40, 0) , z3.If(r7s37, 60, 0) , z3.If(r7s38, 60, 0) , z3.If(r7s39, 0, 0) , z3.If(r7s40, 60, 0) , z3.If(r7s41, 0, 0) , z3.If(r7s42, 60, 0) , z3.If(r7s43, 60, 0) , z3.If(r7s44, 40, 0) , z3.If(r7s45, 0, 0) , z3.If(r7s46, 0, 0) , z3.If(r7s47, 60, 0) , z3.If(r7s49, 60, 0) , z3.If(r7s50, 0, 0) , z3.If(r7s51, 0, 0) , z3.If(r8s0, 0, 0) , z3.If(r8s1, 0, 0) , z3.If(r8s2, 0, 0) , z3.If(r8s3, 0, 0) , z3.If(r8s4, 0, 0) , z3.If(r8s5, 0, 0) , z3.If(r8s6, 0, 0) , z3.If(r8s7, 0, 0) , z3.If(r8s8, 0, 0) , z3.If(r8s9, 60, 0) , z3.If(r8s10, 0, 0) , z3.If(r8s11, 0, 0) , z3.If(r8s12, 0, 0) , z3.If(r8s14, 0, 0) , z3.If(r8s15, 0, 0) , z3.If(r8s18, 0, 0) , z3.If(r8s19, 0, 0) , z3.If(r8s20, 0, 0) , z3.If(r8s21, 0, 0) , z3.If(r8s22, 40, 0) , z3.If(r8s23, 40, 0) , z3.If(r8s24, 40, 0) , z3.If(r8s25, 40, 0) , z3.If(r8s26, 0, 0) , z3.If(r8s27, 0, 0) , z3.If(r8s28, 0, 0) , z3.If(r8s29, 0, 0) , z3.If(r8s30, 0, 0) , z3.If(r8s31, 40, 0) , z3.If(r8s33, 60, 0) , z3.If(r8s34, 60, 0) , z3.If(r8s35, 40, 0) , z3.If(r8s36, 0, 0) , z3.If(r8s37, 60, 0) , z3.If(r8s38, 60, 0) , z3.If(r8s39, 0, 0) , z3.If(r8s40, 40, 0) , z3.If(r8s41, 0, 0) , z3.If(r8s42, 40, 0) , z3.If(r8s43, 60, 0) , z3.If(r8s44, 0, 0) , z3.If(r8s45, 0, 0) , z3.If(r8s46, 0, 0) , z3.If(r8s47, 60, 0) , z3.If(r8s48, 0, 0) , z3.If(r8s49, 40, 0) , z3.If(r8s50, 40, 0) , z3.If(r8s51, 40, 0) , z3.If(r9s0, 60, 0) , z3.If(r9s1, 0, 0) , z3.If(r9s2, 0, 0) , z3.If(r9s3, 0, 0) , z3.If(r9s4, 0, 0) , z3.If(r9s5, 0, 0) , z3.If(r9s6, 0, 0) , z3.If(r9s7, 0, 0) , z3.If(r9s8, 0, 0) , z3.If(r9s9, 0, 0) , z3.If(r9s10, 0, 0) , z3.If(r9s11, 0, 0) , z3.If(r9s12, 0, 0) , z3.If(r9s13, 60, 0) , z3.If(r9s14, 60, 0) , z3.If(r9s15, 0, 0) , z3.If(r9s16, 0, 0) , z3.If(r9s17, 60, 0) , z3.If(r9s18, 0, 0) , z3.If(r9s19, 0, 0) , z3.If(r9s20, 0, 0) , z3.If(r9s21, 0, 0) , z3.If(r9s22, 0, 0) , z3.If(r9s23, 0, 0) , z3.If(r9s24, 60, 0) , z3.If(r9s25, 0, 0) , z3.If(r9s26, 40, 0) , z3.If(r9s27, 0, 0) , z3.If(r9s28, 60, 0) , z3.If(r9s29, 60, 0) , z3.If(r9s30, 0, 0) , z3.If(r9s31, 0, 0) , z3.If(r9s32, 60, 0) , z3.If(r9s33, 40, 0) , z3.If(r9s34, 0, 0) , z3.If(r9s35, 0, 0) , z3.If(r9s36, 60, 0) , z3.If(r9s37, 60, 0) , z3.If(r9s38, 0, 0) , z3.If(r9s39, 0, 0) , z3.If(r9s40, 60, 0) , z3.If(r9s41, 0, 0) , z3.If(r9s42, 0, 0) , z3.If(r9s43, 0, 0) , z3.If(r9s44, 60, 0) , z3.If(r9s45, 0, 0) , z3.If(r9s46, 60, 0) , z3.If(r9s47, 0, 0) , z3.If(r9s48, 0, 0) , z3.If(r9s49, 60, 0) , z3.If(r9s50, 0, 0) , z3.If(r9s51, 0, 0) , z3.If(r10s0, 0, 0) , z3.If(r10s1, 0, 0) , z3.If(r10s2, 0, 0) , z3.If(r10s3, 0, 0) , z3.If(r10s4, 0, 0) , z3.If(r10s5, 0, 0) , z3.If(r10s6, 0, 0) , z3.If(r10s7, 0, 0) , z3.If(r10s9, 60, 0) , z3.If(r10s10, 0, 0) , z3.If(r10s12, 60, 0) , z3.If(r10s13, 60, 0) , z3.If(r10s14, 0, 0) , z3.If(r10s15, 0, 0) , z3.If(r10s16, 0, 0) , z3.If(r10s17, 0, 0) , z3.If(r10s18, 60, 0) , z3.If(r10s19, 0, 0) , z3.If(r10s20, 0, 0) , z3.If(r10s21, 60, 0) , z3.If(r10s22, 0, 0) , z3.If(r10s23, 0, 0) , z3.If(r10s24, 0, 0) , z3.If(r10s25, 0, 0) , z3.If(r10s26, 0, 0) , z3.If(r10s27, 0, 0) , z3.If(r10s28, 60, 0) , z3.If(r10s29, 0, 0) , z3.If(r10s30, 60, 0) , z3.If(r10s31, 60, 0) , z3.If(r10s32, 60, 0) , z3.If(r10s33, 0, 0) , z3.If(r10s34, 0, 0) , z3.If(r10s35, 60, 0) , z3.If(r10s36, 0, 0) , z3.If(r10s37, 60, 0) , z3.If(r10s38, 0, 0) , z3.If(r10s39, 0, 0) , z3.If(r10s40, 60, 0) , z3.If(r10s41, 0, 0) , z3.If(r10s42, 0, 0) , z3.If(r10s43, 0, 0) , z3.If(r10s44, 0, 0) , z3.If(r10s45, 0, 0) , z3.If(r10s46, 0, 0) , z3.If(r10s47, 0, 0) , z3.If(r10s49, 0, 0) , z3.If(r10s50, 0, 0) , z3.If(r10s51, 0, 0) , z3.If(r11s0, 0, 0) , z3.If(r11s1, 0, 0) , z3.If(r11s2, 0, 0) , z3.If(r11s3, 60, 0) , z3.If(r11s4, 60, 0) , z3.If(r11s5, 40, 0) , z3.If(r11s6, 0, 0) , z3.If(r11s7, 0, 0) , z3.If(r11s8, 0, 0) , z3.If(r11s9, 60, 0) , z3.If(r11s10, 0, 0) , z3.If(r11s11, 60, 0) , z3.If(r11s12, 0, 0) , z3.If(r11s13, 60, 0) , z3.If(r11s14, 0, 0) , z3.If(r11s15, 0, 0) , z3.If(r11s16, 0, 0) , z3.If(r11s17, 0, 0) , z3.If(r11s18, 0, 0) , z3.If(r11s19, 0, 0) , z3.If(r11s20, 0, 0) , z3.If(r11s21, 0, 0) , z3.If(r11s22, 0, 0) , z3.If(r11s23, 0, 0) , z3.If(r11s24, 0, 0) , z3.If(r11s25, 60, 0) , z3.If(r11s26, 0, 0) , z3.If(r11s27, 0, 0) , z3.If(r11s28, 0, 0) , z3.If(r11s29, 0, 0) , z3.If(r11s30, 0, 0) , z3.If(r11s31, 0, 0) , z3.If(r11s32, 0, 0) , z3.If(r11s33, 0, 0) , z3.If(r11s34, 40, 0) , z3.If(r11s35, 0, 0) , z3.If(r11s36, 0, 0) , z3.If(r11s37, 0, 0) , z3.If(r11s38, 0, 0) , z3.If(r11s39, 0, 0) , z3.If(r11s40, 0, 0) , z3.If(r11s41, 0, 0) , z3.If(r11s43, 0, 0) , z3.If(r11s44, 0, 0) , z3.If(r11s45, 0, 0) , z3.If(r11s46, 0, 0) , z3.If(r11s48, 0, 0) , z3.If(r11s50, 0, 0) , z3.If(r11s51, 0, 0) , z3.If(r12s1, 0, 0) , z3.If(r12s2, 0, 0) , z3.If(r12s3, 0, 0) , z3.If(r12s4, 60, 0) , z3.If(r12s5, 60, 0) , z3.If(r12s6, 0, 0) , z3.If(r12s7, 0, 0) , z3.If(r12s8, 40, 0) , z3.If(r12s9, 0, 0) , z3.If(r12s10, 0, 0) , z3.If(r12s12, 0, 0) , z3.If(r12s13, 40, 0) , z3.If(r12s14, 0, 0) , z3.If(r12s15, 0, 0) , z3.If(r12s16, 40, 0) , z3.If(r12s19, 0, 0) , z3.If(r12s20, 0, 0) , z3.If(r12s21, 0, 0) , z3.If(r12s22, 0, 0) , z3.If(r12s23, 0, 0) , z3.If(r12s24, 60, 0) , z3.If(r12s25, 0, 0) , z3.If(r12s26, 0, 0) , z3.If(r12s27, 40, 0) , z3.If(r12s28, 0, 0) , z3.If(r12s29, 0, 0) , z3.If(r12s30, 0, 0) , z3.If(r12s31, 60, 0) , z3.If(r12s32, 60, 0) , z3.If(r12s33, 0, 0) , z3.If(r12s34, 60, 0) , z3.If(r12s35, 0, 0) , z3.If(r12s36, 0, 0) , z3.If(r12s37, 0, 0) , z3.If(r12s38, 60, 0) , z3.If(r12s39, 0, 0) , z3.If(r12s40, 40, 0) , z3.If(r12s41, 60, 0) , z3.If(r12s42, 0, 0) , z3.If(r12s43, 40, 0) , z3.If(r12s44, 40, 0) , z3.If(r12s45, 40, 0) , z3.If(r12s46, 0, 0) , z3.If(r12s48, 0, 0) , z3.If(r12s49, 0, 0) , z3.If(r12s50, 0, 0) , z3.If(r12s51, 0, 0) , z3.If(r13s0, 60, 0) , z3.If(r13s2, 40, 0) , z3.If(r13s3, 0, 0) , z3.If(r13s4, 0, 0) , z3.If(r13s5, 0, 0) , z3.If(r13s6, 0, 0) , z3.If(r13s7, 0, 0) , z3.If(r13s10, 0, 0) , z3.If(r13s12, 0, 0) , z3.If(r13s13, 40, 0) , z3.If(r13s14, 60, 0) , z3.If(r13s15, 0, 0) , z3.If(r13s16, 0, 0) , z3.If(r13s18, 0, 0) , z3.If(r13s19, 60, 0) , z3.If(r13s20, 0, 0) , z3.If(r13s22, 0, 0) , z3.If(r13s23, 0, 0) , z3.If(r13s25, 0, 0) , z3.If(r13s26, 0, 0) , z3.If(r13s27, 0, 0) , z3.If(r13s28, 0, 0) , z3.If(r13s29, 0, 0) , z3.If(r13s30, 0, 0) , z3.If(r13s31, 0, 0) , z3.If(r13s32, 0, 0) , z3.If(r13s33, 0, 0) , z3.If(r13s34, 0, 0) , z3.If(r13s35, 40, 0) , z3.If(r13s37, 40, 0) , z3.If(r13s38, 60, 0) , z3.If(r13s39, 0, 0) , z3.If(r13s40, 40, 0) , z3.If(r13s41, 0, 0) , z3.If(r13s42, 0, 0) , z3.If(r13s43, 60, 0) , z3.If(r13s44, 0, 0) , z3.If(r13s45, 0, 0) , z3.If(r13s46, 40, 0) , z3.If(r13s47, 40, 0) , z3.If(r13s49, 40, 0) , z3.If(r13s50, 0, 0) , z3.If(r13s51, 40, 0) , z3.If(r14s0, 0, 0) , z3.If(r14s1, 0, 0) , z3.If(r14s2, 0, 0) , z3.If(r14s4, 0, 0) , z3.If(r14s5, 0, 0) , z3.If(r14s7, 60, 0) , z3.If(r14s8, 60, 0) , z3.If(r14s9, 0, 0) , z3.If(r14s10, 0, 0) , z3.If(r14s11, 0, 0) , z3.If(r14s12, 0, 0) , z3.If(r14s13, 60, 0) , z3.If(r14s14, 0, 0) , z3.If(r14s15, 0, 0) , z3.If(r14s17, 40, 0) , z3.If(r14s18, 0, 0) , z3.If(r14s19, 40, 0) , z3.If(r14s20, 0, 0) , z3.If(r14s21, 0, 0) , z3.If(r14s22, 0, 0) , z3.If(r14s23, 0, 0) , z3.If(r14s24, 60, 0) , z3.If(r14s25, 40, 0) , z3.If(r14s26, 0, 0) , z3.If(r14s27, 0, 0) , z3.If(r14s28, 0, 0) , z3.If(r14s29, 0, 0) , z3.If(r14s30, 0, 0) , z3.If(r14s34, 60, 0) , z3.If(r14s36, 0, 0) , z3.If(r14s38, 0, 0) , z3.If(r14s39, 0, 0) , z3.If(r14s40, 0, 0) , z3.If(r14s41, 0, 0) , z3.If(r14s42, 60, 0) , z3.If(r14s43, 60, 0) , z3.If(r14s44, 60, 0) , z3.If(r14s45, 0, 0) , z3.If(r14s46, 0, 0) , z3.If(r14s47, 40, 0) , z3.If(r14s48, 60, 0) , z3.If(r14s49, 60, 0) , z3.If(r14s50, 0, 0) , z3.If(r14s51, 0, 0) , z3.If(r15s1, 0, 0) , z3.If(r15s2, 0, 0) , z3.If(r15s3, 0, 0) , z3.If(r15s4, 0, 0) , z3.If(r15s5, 0, 0) , z3.If(r15s6, 0, 0) , z3.If(r15s7, 0, 0) , z3.If(r15s8, 0, 0) , z3.If(r15s9, 60, 0) , z3.If(r15s10, 0, 0) , z3.If(r15s11, 0, 0) , z3.If(r15s12, 60, 0) , z3.If(r15s13, 60, 0) , z3.If(r15s14, 0, 0) , z3.If(r15s15, 0, 0) , z3.If(r15s16, 0, 0) , z3.If(r15s17, 0, 0) , z3.If(r15s19, 0, 0) , z3.If(r15s20, 0, 0) , z3.If(r15s21, 0, 0) , z3.If(r15s22, 60, 0) , z3.If(r15s23, 60, 0) , z3.If(r15s24, 60, 0) , z3.If(r15s25, 0, 0) , z3.If(r15s26, 0, 0) , z3.If(r15s27, 0, 0) , z3.If(r15s28, 0, 0) , z3.If(r15s29, 0, 0) , z3.If(r15s30, 0, 0) , z3.If(r15s31, 0, 0) , z3.If(r15s32, 0, 0) , z3.If(r15s33, 0, 0) , z3.If(r15s34, 60, 0) , z3.If(r15s35, 0, 0) , z3.If(r15s36, 0, 0) , z3.If(r15s37, 60, 0) , z3.If(r15s38, 0, 0) , z3.If(r15s39, 0, 0) , z3.If(r15s40, 0, 0) , z3.If(r15s41, 0, 0) , z3.If(r15s42, 0, 0) , z3.If(r15s43, 0, 0) , z3.If(r15s44, 0, 0) , z3.If(r15s45, 0, 0) , z3.If(r15s46, 0, 0) , z3.If(r15s48, 0, 0) , z3.If(r15s49, 0, 0) , z3.If(r15s50, 0, 0) , z3.If(r15s51, 60, 0) , z3.If(r16s1, 0, 0) , z3.If(r16s2, 40, 0) , z3.If(r16s3, 40, 0) , z3.If(r16s4, 60, 0) , z3.If(r16s5, 40, 0) , z3.If(r16s7, 60, 0) , z3.If(r16s8, 0, 0) , z3.If(r16s9, 0, 0) , z3.If(r16s11, 0, 0) , z3.If(r16s12, 40, 0) , z3.If(r16s13, 0, 0) , z3.If(r16s14, 0, 0) , z3.If(r16s15, 0, 0) , z3.If(r16s16, 0, 0) , z3.If(r16s17, 0, 0) , z3.If(r16s19, 0, 0) , z3.If(r16s20, 0, 0) , z3.If(r16s21, 0, 0) , z3.If(r16s22, 0, 0) , z3.If(r16s23, 0, 0) , z3.If(r16s24, 0, 0) , z3.If(r16s25, 0, 0) , z3.If(r16s26, 0, 0) , z3.If(r16s27, 0, 0) , z3.If(r16s28, 0, 0) , z3.If(r16s29, 0, 0) , z3.If(r16s30, 0, 0) , z3.If(r16s31, 60, 0) , z3.If(r16s32, 0, 0) , z3.If(r16s33, 0, 0) , z3.If(r16s34, 60, 0) , z3.If(r16s35, 0, 0) , z3.If(r16s36, 0, 0) , z3.If(r16s38, 0, 0) , z3.If(r16s39, 40, 0) , z3.If(r16s40, 0, 0) , z3.If(r16s41, 0, 0) , z3.If(r16s42, 0, 0) , z3.If(r16s43, 0, 0) , z3.If(r16s44, 40, 0) , z3.If(r16s45, 0, 0) , z3.If(r16s46, 40, 0) , z3.If(r16s48, 0, 0) , z3.If(r16s49, 0, 0) , z3.If(r16s50, 0, 0) , z3.If(r16s51, 60, 0) , z3.If(r17s0, 0, 0) , z3.If(r17s1, 0, 0) , z3.If(r17s2, 60, 0) , z3.If(r17s3, 60, 0) , z3.If(r17s4, 0, 0) , z3.If(r17s5, 0, 0) , z3.If(r17s7, 60, 0) , z3.If(r17s8, 0, 0) , z3.If(r17s11, 0, 0) , z3.If(r17s12, 0, 0) , z3.If(r17s13, 60, 0) , z3.If(r17s14, 0, 0) , z3.If(r17s15, 0, 0) , z3.If(r17s16, 40, 0) , z3.If(r17s17, 40, 0) , z3.If(r17s18, 60, 0) , z3.If(r17s19, 40, 0) , z3.If(r17s20, 0, 0) , z3.If(r17s21, 0, 0) , z3.If(r17s22, 0, 0) , z3.If(r17s23, 0, 0) , z3.If(r17s25, 0, 0) , z3.If(r17s26, 0, 0) , z3.If(r17s27, 40, 0) , z3.If(r17s28, 0, 0) , z3.If(r17s29, 40, 0) , z3.If(r17s30, 40, 0) , z3.If(r17s31, 0, 0) , z3.If(r17s32, 60, 0) , z3.If(r17s33, 0, 0) , z3.If(r17s34, 40, 0) , z3.If(r17s35, 60, 0) , z3.If(r17s36, 0, 0) , z3.If(r17s37, 40, 0) , z3.If(r17s38, 40, 0) , z3.If(r17s39, 0, 0) , z3.If(r17s40, 0, 0) , z3.If(r17s41, 40, 0) , z3.If(r17s42, 0, 0) , z3.If(r17s43, 40, 0) , z3.If(r17s44, 0, 0) , z3.If(r17s45, 0, 0) , z3.If(r17s46, 0, 0) , z3.If(r17s47, 60, 0) , z3.If(r17s48, 0, 0) , z3.If(r17s49, 0, 0) , z3.If(r17s50, 0, 0) , z3.If(r17s51, 0, 0) , z3.If(r18s0, 60, 0) , z3.If(r18s1, 0, 0) , z3.If(r18s2, 0, 0) , z3.If(r18s3, 0, 0) , z3.If(r18s4, 60, 0) , z3.If(r18s5, 0, 0) , z3.If(r18s6, 0, 0) , z3.If(r18s7, 0, 0) , z3.If(r18s8, 0, 0) , z3.If(r18s10, 0, 0) , z3.If(r18s11, 0, 0) , z3.If(r18s12, 60, 0) , z3.If(r18s13, 0, 0) , z3.If(r18s14, 0, 0) , z3.If(r18s15, 60, 0) , z3.If(r18s16, 0, 0) , z3.If(r18s17, 0, 0) , z3.If(r18s18, 60, 0) , z3.If(r18s19, 0, 0) , z3.If(r18s20, 0, 0) , z3.If(r18s21, 0, 0) , z3.If(r18s22, 0, 0) , z3.If(r18s23, 60, 0) , z3.If(r18s25, 0, 0) , z3.If(r18s26, 0, 0) , z3.If(r18s27, 0, 0) , z3.If(r18s28, 0, 0) , z3.If(r18s29, 0, 0) , z3.If(r18s30, 0, 0) , z3.If(r18s31, 0, 0) , z3.If(r18s32, 0, 0) , z3.If(r18s33, 60, 0) , z3.If(r18s34, 0, 0) , z3.If(r18s35, 0, 0) , z3.If(r18s36, 0, 0) , z3.If(r18s37, 0, 0) , z3.If(r18s38, 0, 0) , z3.If(r18s39, 0, 0) , z3.If(r18s40, 0, 0) , z3.If(r18s41, 0, 0) , z3.If(r18s42, 0, 0) , z3.If(r18s43, 0, 0) , z3.If(r18s44, 0, 0) , z3.If(r18s45, 60, 0) , z3.If(r18s46, 0, 0) , z3.If(r18s47, 0, 0) , z3.If(r18s48, 60, 0) , z3.If(r18s49, 0, 0) , z3.If(r18s50, 0, 0) , z3.If(r18s51, 60, 0) , z3.If(r19s0, 40, 0) , z3.If(r19s1, 60, 0) , z3.If(r19s3, 40, 0) , z3.If(r19s4, 60, 0) , z3.If(r19s5, 0, 0) , z3.If(r19s6, 40, 0) , z3.If(r19s7, 40, 0) , z3.If(r19s8, 0, 0) , z3.If(r19s9, 40, 0) , z3.If(r19s10, 0, 0) , z3.If(r19s11, 0, 0) , z3.If(r19s12, 0, 0) , z3.If(r19s13, 0, 0) , z3.If(r19s14, 0, 0) , z3.If(r19s15, 60, 0) , z3.If(r19s16, 60, 0) , z3.If(r19s17, 60, 0) , z3.If(r19s18, 60, 0) , z3.If(r19s19, 40, 0) , z3.If(r19s20, 0, 0) , z3.If(r19s21, 0, 0) , z3.If(r19s22, 60, 0) , z3.If(r19s23, 0, 0) , z3.If(r19s24, 40, 0) , z3.If(r19s25, 0, 0) , z3.If(r19s26, 0, 0) , z3.If(r19s27, 0, 0) , z3.If(r19s28, 0, 0) , z3.If(r19s29, 0, 0) , z3.If(r19s30, 0, 0) , z3.If(r19s31, 40, 0) , z3.If(r19s32, 0, 0) , z3.If(r19s33, 0, 0) , z3.If(r19s34, 0, 0) , z3.If(r19s35, 0, 0) , z3.If(r19s36, 0, 0) , z3.If(r19s37, 60, 0) , z3.If(r19s38, 0, 0) , z3.If(r19s39, 0, 0) , z3.If(r19s40, 0, 0) , z3.If(r19s41, 0, 0) , z3.If(r19s42, 60, 0) , z3.If(r19s43, 0, 0) , z3.If(r19s44, 60, 0) , z3.If(r19s45, 0, 0) , z3.If(r19s46, 0, 0) , z3.If(r19s47, 0, 0) , z3.If(r19s48, 0, 0) , z3.If(r19s49, 60, 0) , z3.If(r19s50, 0, 0) , z3.If(r19s51, 60, 0) , z3.If(r20s0, 0, 0) , z3.If(r20s1, 0, 0) , z3.If(r20s2, 0, 0) , z3.If(r20s3, 0, 0) , z3.If(r20s4, 0, 0) , z3.If(r20s5, 0, 0) , z3.If(r20s6, 60, 0) , z3.If(r20s7, 0, 0) , z3.If(r20s8, 0, 0) , z3.If(r20s9, 60, 0) , z3.If(r20s10, 0, 0) , z3.If(r20s11, 0, 0) , z3.If(r20s12, 0, 0) , z3.If(r20s13, 60, 0) , z3.If(r20s15, 0, 0) , z3.If(r20s17, 60, 0) , z3.If(r20s18, 0, 0) , z3.If(r20s19, 0, 0) , z3.If(r20s20, 0, 0) , z3.If(r20s21, 0, 0) , z3.If(r20s22, 0, 0) , z3.If(r20s23, 0, 0) , z3.If(r20s24, 60, 0) , z3.If(r20s25, 0, 0) , z3.If(r20s26, 60, 0) , z3.If(r20s30, 60, 0) , z3.If(r20s31, 0, 0) , z3.If(r20s32, 60, 0) , z3.If(r20s33, 0, 0) , z3.If(r20s34, 60, 0) , z3.If(r20s35, 0, 0) , z3.If(r20s36, 60, 0) , z3.If(r20s37, 60, 0) , z3.If(r20s38, 0, 0) , z3.If(r20s39, 0, 0) , z3.If(r20s40, 0, 0) , z3.If(r20s41, 0, 0) , z3.If(r20s42, 0, 0) , z3.If(r20s43, 0, 0) , z3.If(r20s44, 0, 0) , z3.If(r20s45, 60, 0) , z3.If(r20s46, 0, 0) , z3.If(r20s47, 60, 0) , z3.If(r20s48, 0, 0) , z3.If(r20s49, 0, 0) , z3.If(r20s50, 0, 0) , z3.If(r20s51, 0, 0) , z3.If(r21s1, 0, 0) , z3.If(r21s2, 60, 0) , z3.If(r21s3, 0, 0) , z3.If(r21s4, 0, 0) , z3.If(r21s5, 60, 0) , z3.If(r21s6, 0, 0) , z3.If(r21s7, 0, 0) , z3.If(r21s8, 0, 0) , z3.If(r21s10, 0, 0) , z3.If(r21s12, 60, 0) , z3.If(r21s14, 40, 0) , z3.If(r21s15, 60, 0) , z3.If(r21s18, 60, 0) , z3.If(r21s19, 0, 0) , z3.If(r21s20, 0, 0) , z3.If(r21s22, 0, 0) , z3.If(r21s23, 40, 0) , z3.If(r21s24, 0, 0) , z3.If(r21s25, 0, 0) , z3.If(r21s26, 0, 0) , z3.If(r21s27, 0, 0) , z3.If(r21s28, 0, 0) , z3.If(r21s29, 0, 0) , z3.If(r21s30, 0, 0) , z3.If(r21s31, 0, 0) , z3.If(r21s33, 0, 0) , z3.If(r21s34, 0, 0) , z3.If(r21s35, 60, 0) , z3.If(r21s37, 40, 0) , z3.If(r21s38, 40, 0) , z3.If(r21s39, 0, 0) , z3.If(r21s40, 0, 0) , z3.If(r21s41, 0, 0) , z3.If(r21s42, 0, 0) , z3.If(r21s43, 40, 0) , z3.If(r21s44, 0, 0) , z3.If(r21s45, 0, 0) , z3.If(r21s46, 0, 0) , z3.If(r21s47, 0, 0) , z3.If(r21s48, 0, 0) , z3.If(r21s49, 0, 0) , z3.If(r21s50, 40, 0) , z3.If(r21s51, 0, 0) , z3.If(r22s0, 0, 0) , z3.If(r22s1, 0, 0) , z3.If(r22s2, 0, 0) , z3.If(r22s3, 0, 0) , z3.If(r22s4, 60, 0) , z3.If(r22s5, 60, 0) , z3.If(r22s6, 0, 0) , z3.If(r22s7, 0, 0) , z3.If(r22s8, 0, 0) , z3.If(r22s9, 40, 0) , z3.If(r22s10, 40, 0) , z3.If(r22s11, 0, 0) , z3.If(r22s12, 0, 0) , z3.If(r22s13, 40, 0) , z3.If(r22s14, 0, 0) , z3.If(r22s15, 0, 0) , z3.If(r22s16, 60, 0) , z3.If(r22s17, 0, 0) , z3.If(r22s18, 0, 0) , z3.If(r22s19, 0, 0) , z3.If(r22s20, 40, 0) , z3.If(r22s21, 0, 0) , z3.If(r22s22, 0, 0) , z3.If(r22s23, 0, 0) , z3.If(r22s24, 0, 0) , z3.If(r22s25, 0, 0) , z3.If(r22s26, 0, 0) , z3.If(r22s27, 0, 0) , z3.If(r22s28, 0, 0) , z3.If(r22s29, 0, 0) , z3.If(r22s30, 0, 0) , z3.If(r22s31, 0, 0) , z3.If(r22s32, 60, 0) , z3.If(r22s33, 40, 0) , z3.If(r22s34, 40, 0) , z3.If(r22s35, 0, 0) , z3.If(r22s36, 0, 0) , z3.If(r22s37, 0, 0) , z3.If(r22s38, 0, 0) , z3.If(r22s39, 60, 0) , z3.If(r22s40, 0, 0) , z3.If(r22s41, 0, 0) , z3.If(r22s42, 40, 0) , z3.If(r22s43, 0, 0) , z3.If(r22s44, 0, 0) , z3.If(r22s45, 60, 0) , z3.If(r22s46, 0, 0) , z3.If(r22s47, 0, 0) , z3.If(r22s48, 0, 0) , z3.If(r22s49, 0, 0) , z3.If(r22s50, 0, 0) , z3.If(r22s51, 0, 0) , z3.If(r23s0, 0, 0) , z3.If(r23s1, 0, 0) , z3.If(r23s2, 0, 0) , z3.If(r23s3, 60, 0) , z3.If(r23s4, 60, 0) , z3.If(r23s5, 0, 0) , z3.If(r23s6, 40, 0) , z3.If(r23s7, 0, 0) , z3.If(r23s8, 40, 0) , z3.If(r23s10, 60, 0) , z3.If(r23s11, 0, 0) , z3.If(r23s12, 0, 0) , z3.If(r23s13, 0, 0) , z3.If(r23s14, 0, 0) , z3.If(r23s15, 0, 0) , z3.If(r23s16, 60, 0) , z3.If(r23s17, 0, 0) , z3.If(r23s18, 0, 0) , z3.If(r23s19, 0, 0) , z3.If(r23s20, 0, 0) , z3.If(r23s21, 0, 0) , z3.If(r23s22, 0, 0) , z3.If(r23s23, 0, 0) , z3.If(r23s25, 0, 0) , z3.If(r23s26, 0, 0) , z3.If(r23s27, 0, 0) , z3.If(r23s28, 0, 0) , z3.If(r23s29, 0, 0) , z3.If(r23s30, 0, 0) , z3.If(r23s31, 40, 0) , z3.If(r23s32, 60, 0) , z3.If(r23s33, 0, 0) , z3.If(r23s34, 40, 0) , z3.If(r23s35, 0, 0) , z3.If(r23s36, 0, 0) , z3.If(r23s37, 0, 0) , z3.If(r23s38, 0, 0) , z3.If(r23s39, 60, 0) , z3.If(r23s40, 0, 0) , z3.If(r23s41, 0, 0) , z3.If(r23s42, 60, 0) , z3.If(r23s43, 0, 0) , z3.If(r23s44, 0, 0) , z3.If(r23s45, 60, 0) , z3.If(r23s46, 0, 0) , z3.If(r23s47, 0, 0) , z3.If(r23s48, 40, 0) , z3.If(r23s49, 0, 0) , z3.If(r23s50, 0, 0) , z3.If(r23s51, 0, 0)]
###############################################################################################################
# other settings
# rules = []

# 100/0/0
scores = [z3.If(r0s0, 100, 0) , z3.If(r0s1, 0, 0) , z3.If(r0s2, 0, 0) , z3.If(r0s4, 0, 0) , z3.If(r0s5, 0, 0) , z3.If(r0s7, 0, 0) , z3.If(r0s8, 0, 0) , z3.If(r0s9, 0, 0) , z3.If(r0s10, 0, 0) , z3.If(r0s12, 0, 0) , z3.If(r0s13, 0, 0) , z3.If(r0s14, 0, 0) , z3.If(r0s15, 0, 0) , z3.If(r0s16, 0, 0) , z3.If(r0s18, 0, 0) , z3.If(r0s19, 0, 0) , z3.If(r0s20, 0, 0) , z3.If(r0s21, 0, 0) , z3.If(r0s22, 0, 0) , z3.If(r0s23, 100, 0) , z3.If(r0s24, 100, 0) , z3.If(r0s25, 0, 0) , z3.If(r0s26, 0, 0) , z3.If(r0s27, 0, 0) , z3.If(r0s28, 100, 0) , z3.If(r0s29, 100, 0) , z3.If(r0s30, 100, 0) , z3.If(r0s31, 0, 0) , z3.If(r0s32, 100, 0) , z3.If(r0s34, 0, 0) , z3.If(r0s35, 0, 0) , z3.If(r0s36, 0, 0) , z3.If(r0s37, 100, 0) , z3.If(r0s38, 0, 0) , z3.If(r0s39, 0, 0) , z3.If(r0s40, 0, 0) , z3.If(r0s41, 0, 0) , z3.If(r0s42, 0, 0) , z3.If(r0s43, 0, 0) , z3.If(r0s44, 100, 0) , z3.If(r0s45, 0, 0) , z3.If(r0s46, 0, 0) , z3.If(r0s47, 0, 0) , z3.If(r0s48, 0, 0) , z3.If(r0s49, 0, 0) , z3.If(r0s50, 0, 0) , z3.If(r0s51, 0, 0) , z3.If(r1s0, 100, 0) , z3.If(r1s1, 0, 0) , z3.If(r1s2, 0, 0) , z3.If(r1s3, 100, 0) , z3.If(r1s4, 0, 0) , z3.If(r1s5, 0, 0) , z3.If(r1s6, 0, 0) , z3.If(r1s7, 0, 0) , z3.If(r1s8, 0, 0) , z3.If(r1s9, 100, 0) , z3.If(r1s10, 0, 0) , z3.If(r1s11, 100, 0) , z3.If(r1s12, 0, 0) , z3.If(r1s13, 0, 0) , z3.If(r1s14, 0, 0) , z3.If(r1s15, 0, 0) , z3.If(r1s16, 0, 0) , z3.If(r1s17, 0, 0) , z3.If(r1s18, 100, 0) , z3.If(r1s19, 0, 0) , z3.If(r1s20, 0, 0) , z3.If(r1s21, 0, 0) , z3.If(r1s22, 100, 0) , z3.If(r1s23, 0, 0) , z3.If(r1s24, 0, 0) , z3.If(r1s25, 0, 0) , z3.If(r1s26, 0, 0) , z3.If(r1s27, 0, 0) , z3.If(r1s28, 100, 0) , z3.If(r1s29, 0, 0) , z3.If(r1s30, 0, 0) , z3.If(r1s31, 0, 0) , z3.If(r1s32, 0, 0) , z3.If(r1s33, 0, 0) , z3.If(r1s35, 0, 0) , z3.If(r1s36, 0, 0) , z3.If(r1s37, 0, 0) , z3.If(r1s38, 0, 0) , z3.If(r1s39, 0, 0) , z3.If(r1s40, 100, 0) , z3.If(r1s41, 0, 0) , z3.If(r1s42, 0, 0) , z3.If(r1s43, 0, 0) , z3.If(r1s44, 0, 0) , z3.If(r1s45, 0, 0) , z3.If(r1s46, 0, 0) , z3.If(r1s47, 0, 0) , z3.If(r1s48, 0, 0) , z3.If(r1s49, 0, 0) , z3.If(r1s50, 0, 0) , z3.If(r1s51, 0, 0) , z3.If(r2s0, 0, 0) , z3.If(r2s1, 0, 0) , z3.If(r2s2, 100, 0) , z3.If(r2s3, 0, 0) , z3.If(r2s4, 0, 0) , z3.If(r2s5, 0, 0) , z3.If(r2s6, 0, 0) , z3.If(r2s7, 0, 0) , z3.If(r2s8, 0, 0) , z3.If(r2s9, 0, 0) , z3.If(r2s10, 0, 0) , z3.If(r2s11, 0, 0) , z3.If(r2s12, 100, 0) , z3.If(r2s14, 0, 0) , z3.If(r2s15, 100, 0) , z3.If(r2s18, 0, 0) , z3.If(r2s19, 0, 0) , z3.If(r2s20, 0, 0) , z3.If(r2s22, 0, 0) , z3.If(r2s23, 0, 0) , z3.If(r2s24, 0, 0) , z3.If(r2s25, 0, 0) , z3.If(r2s26, 0, 0) , z3.If(r2s27, 0, 0) , z3.If(r2s28, 0, 0) , z3.If(r2s29, 0, 0) , z3.If(r2s30, 0, 0) , z3.If(r2s31, 0, 0) , z3.If(r2s33, 0, 0) , z3.If(r2s34, 0, 0) , z3.If(r2s35, 0, 0) , z3.If(r2s36, 0, 0) , z3.If(r2s37, 0, 0) , z3.If(r2s38, 0, 0) , z3.If(r2s39, 0, 0) , z3.If(r2s40, 0, 0) , z3.If(r2s41, 0, 0) , z3.If(r2s42, 0, 0) , z3.If(r2s43, 0, 0) , z3.If(r2s44, 0, 0) , z3.If(r2s45, 0, 0) , z3.If(r2s46, 0, 0) , z3.If(r2s47, 0, 0) , z3.If(r2s48, 0, 0) , z3.If(r2s49, 0, 0) , z3.If(r2s50, 0, 0) , z3.If(r2s51, 0, 0) , z3.If(r3s0, 0, 0) , z3.If(r3s2, 100, 0) , z3.If(r3s3, 100, 0) , z3.If(r3s4, 0, 0) , z3.If(r3s5, 0, 0) , z3.If(r3s6, 100, 0) , z3.If(r3s7, 0, 0) , z3.If(r3s8, 0, 0) , z3.If(r3s10, 0, 0) , z3.If(r3s11, 0, 0) , z3.If(r3s12, 0, 0) , z3.If(r3s15, 0, 0) , z3.If(r3s17, 0, 0) , z3.If(r3s18, 0, 0) , z3.If(r3s19, 0, 0) , z3.If(r3s20, 0, 0) , z3.If(r3s21, 0, 0) , z3.If(r3s22, 0, 0) , z3.If(r3s23, 0, 0) , z3.If(r3s25, 100, 0) , z3.If(r3s31, 0, 0) , z3.If(r3s32, 100, 0) , z3.If(r3s33, 0, 0) , z3.If(r3s34, 0, 0) , z3.If(r3s35, 100, 0) , z3.If(r3s36, 100, 0) , z3.If(r3s37, 100, 0) , z3.If(r3s38, 100, 0) , z3.If(r3s39, 0, 0) , z3.If(r3s40, 100, 0) , z3.If(r3s41, 0, 0) , z3.If(r3s42, 100, 0) , z3.If(r3s43, 0, 0) , z3.If(r3s44, 0, 0) , z3.If(r3s45, 0, 0) , z3.If(r3s46, 0, 0) , z3.If(r3s47, 100, 0) , z3.If(r3s48, 0, 0) , z3.If(r3s49, 0, 0) , z3.If(r3s50, 100, 0) , z3.If(r3s51, 0, 0) , z3.If(r4s0, 0, 0) , z3.If(r4s1, 0, 0) , z3.If(r4s2, 0, 0) , z3.If(r4s3, 0, 0) , z3.If(r4s4, 100, 0) , z3.If(r4s5, 100, 0) , z3.If(r4s6, 0, 0) , z3.If(r4s7, 0, 0) , z3.If(r4s8, 0, 0) , z3.If(r4s9, 0, 0) , z3.If(r4s10, 100, 0) , z3.If(r4s11, 0, 0) , z3.If(r4s12, 0, 0) , z3.If(r4s13, 0, 0) , z3.If(r4s14, 0, 0) , z3.If(r4s15, 0, 0) , z3.If(r4s16, 0, 0) , z3.If(r4s17, 0, 0) , z3.If(r4s18, 100, 0) , z3.If(r4s19, 0, 0) , z3.If(r4s20, 0, 0) , z3.If(r4s21, 0, 0) , z3.If(r4s22, 0, 0) , z3.If(r4s23, 0, 0) , z3.If(r4s24, 0, 0) , z3.If(r4s25, 0, 0) , z3.If(r4s26, 0, 0) , z3.If(r4s27, 0, 0) , z3.If(r4s28, 0, 0) , z3.If(r4s29, 0, 0) , z3.If(r4s30, 0, 0) , z3.If(r4s31, 0, 0) , z3.If(r4s32, 0, 0) , z3.If(r4s33, 0, 0) , z3.If(r4s34, 0, 0) , z3.If(r4s35, 0, 0) , z3.If(r4s36, 0, 0) , z3.If(r4s37, 0, 0) , z3.If(r4s38, 0, 0) , z3.If(r4s39, 0, 0) , z3.If(r4s40, 0, 0) , z3.If(r4s41, 0, 0) , z3.If(r4s42, 0, 0) , z3.If(r4s43, 0, 0) , z3.If(r4s44, 0, 0) , z3.If(r4s45, 0, 0) , z3.If(r4s46, 0, 0) , z3.If(r4s47, 0, 0) , z3.If(r4s48, 0, 0) , z3.If(r4s49, 0, 0) , z3.If(r4s50, 0, 0) , z3.If(r4s51, 100, 0) , z3.If(r5s0, 100, 0) , z3.If(r5s1, 0, 0) , z3.If(r5s2, 0, 0) , z3.If(r5s3, 0, 0) , z3.If(r5s4, 0, 0) , z3.If(r5s5, 0, 0) , z3.If(r5s6, 0, 0) , z3.If(r5s7, 0, 0) , z3.If(r5s8, 100, 0) , z3.If(r5s9, 100, 0) , z3.If(r5s10, 0, 0) , z3.If(r5s11, 0, 0) , z3.If(r5s12, 0, 0) , z3.If(r5s13, 0, 0) , z3.If(r5s14, 0, 0) , z3.If(r5s15, 0, 0) , z3.If(r5s16, 0, 0) , z3.If(r5s17, 100, 0) , z3.If(r5s18, 0, 0) , z3.If(r5s19, 0, 0) , z3.If(r5s20, 0, 0) , z3.If(r5s21, 0, 0) , z3.If(r5s22, 0, 0) , z3.If(r5s23, 0, 0) , z3.If(r5s24, 100, 0) , z3.If(r5s25, 0, 0) , z3.If(r5s26, 0, 0) , z3.If(r5s27, 0, 0) , z3.If(r5s28, 0, 0) , z3.If(r5s29, 0, 0) , z3.If(r5s30, 0, 0) , z3.If(r5s31, 0, 0) , z3.If(r5s32, 0, 0) , z3.If(r5s33, 0, 0) , z3.If(r5s34, 0, 0) , z3.If(r5s35, 0, 0) , z3.If(r5s36, 0, 0) , z3.If(r5s37, 100, 0) , z3.If(r5s38, 0, 0) , z3.If(r5s39, 0, 0) , z3.If(r5s40, 0, 0) , z3.If(r5s41, 0, 0) , z3.If(r5s42, 0, 0) , z3.If(r5s43, 0, 0) , z3.If(r5s44, 0, 0) , z3.If(r5s45, 0, 0) , z3.If(r5s46, 0, 0) , z3.If(r5s47, 0, 0) , z3.If(r5s48, 100, 0) , z3.If(r5s49, 0, 0) , z3.If(r5s50, 0, 0) , z3.If(r5s51, 0, 0) , z3.If(r6s0, 0, 0) , z3.If(r6s1, 0, 0) , z3.If(r6s2, 0, 0) , z3.If(r6s3, 0, 0) , z3.If(r6s4, 0, 0) , z3.If(r6s5, 0, 0) , z3.If(r6s7, 0, 0) , z3.If(r6s8, 0, 0) , z3.If(r6s9, 0, 0) , z3.If(r6s11, 0, 0) , z3.If(r6s12, 0, 0) , z3.If(r6s13, 0, 0) , z3.If(r6s14, 0, 0) , z3.If(r6s15, 0, 0) , z3.If(r6s16, 0, 0) , z3.If(r6s17, 0, 0) , z3.If(r6s18, 100, 0) , z3.If(r6s19, 100, 0) , z3.If(r6s20, 0, 0) , z3.If(r6s21, 0, 0) , z3.If(r6s22, 0, 0) , z3.If(r6s23, 0, 0) , z3.If(r6s24, 0, 0) , z3.If(r6s25, 0, 0) , z3.If(r6s26, 0, 0) , z3.If(r6s27, 0, 0) , z3.If(r6s28, 100, 0) , z3.If(r6s29, 0, 0) , z3.If(r6s30, 100, 0) , z3.If(r6s31, 0, 0) , z3.If(r6s32, 0, 0) , z3.If(r6s33, 0, 0) , z3.If(r6s34, 0, 0) , z3.If(r6s36, 0, 0) , z3.If(r6s39, 0, 0) , z3.If(r6s40, 0, 0) , z3.If(r6s41, 0, 0) , z3.If(r6s42, 0, 0) , z3.If(r6s44, 0, 0) , z3.If(r6s45, 0, 0) , z3.If(r6s47, 100, 0) , z3.If(r6s48, 0, 0) , z3.If(r6s49, 100, 0) , z3.If(r6s50, 0, 0) , z3.If(r6s51, 0, 0) , z3.If(r7s0, 100, 0) , z3.If(r7s1, 0, 0) , z3.If(r7s2, 0, 0) , z3.If(r7s3, 0, 0) , z3.If(r7s4, 0, 0) , z3.If(r7s5, 0, 0) , z3.If(r7s6, 0, 0) , z3.If(r7s7, 0, 0) , z3.If(r7s10, 0, 0) , z3.If(r7s12, 0, 0) , z3.If(r7s13, 100, 0) , z3.If(r7s14, 0, 0) , z3.If(r7s15, 0, 0) , z3.If(r7s16, 0, 0) , z3.If(r7s17, 0, 0) , z3.If(r7s18, 0, 0) , z3.If(r7s19, 0, 0) , z3.If(r7s20, 0, 0) , z3.If(r7s22, 100, 0) , z3.If(r7s23, 100, 0) , z3.If(r7s24, 0, 0) , z3.If(r7s25, 0, 0) , z3.If(r7s26, 100, 0) , z3.If(r7s27, 0, 0) , z3.If(r7s28, 100, 0) , z3.If(r7s29, 100, 0) , z3.If(r7s30, 0, 0) , z3.If(r7s31, 100, 0) , z3.If(r7s32, 0, 0) , z3.If(r7s33, 0, 0) , z3.If(r7s34, 0, 0) , z3.If(r7s35, 0, 0) , z3.If(r7s36, 0, 0) , z3.If(r7s37, 100, 0) , z3.If(r7s38, 100, 0) , z3.If(r7s39, 0, 0) , z3.If(r7s40, 100, 0) , z3.If(r7s41, 0, 0) , z3.If(r7s42, 100, 0) , z3.If(r7s43, 100, 0) , z3.If(r7s44, 0, 0) , z3.If(r7s45, 0, 0) , z3.If(r7s46, 0, 0) , z3.If(r7s47, 100, 0) , z3.If(r7s49, 100, 0) , z3.If(r7s50, 0, 0) , z3.If(r7s51, 0, 0) , z3.If(r8s0, 0, 0) , z3.If(r8s1, 0, 0) , z3.If(r8s2, 0, 0) , z3.If(r8s3, 0, 0) , z3.If(r8s4, 0, 0) , z3.If(r8s5, 0, 0) , z3.If(r8s6, 0, 0) , z3.If(r8s7, 0, 0) , z3.If(r8s8, 0, 0) , z3.If(r8s9, 100, 0) , z3.If(r8s10, 0, 0) , z3.If(r8s11, 0, 0) , z3.If(r8s12, 0, 0) , z3.If(r8s14, 0, 0) , z3.If(r8s15, 0, 0) , z3.If(r8s18, 0, 0) , z3.If(r8s19, 0, 0) , z3.If(r8s20, 0, 0) , z3.If(r8s21, 0, 0) , z3.If(r8s22, 0, 0) , z3.If(r8s23, 0, 0) , z3.If(r8s24, 0, 0) , z3.If(r8s25, 0, 0) , z3.If(r8s26, 0, 0) , z3.If(r8s27, 0, 0) , z3.If(r8s28, 0, 0) , z3.If(r8s29, 0, 0) , z3.If(r8s30, 0, 0) , z3.If(r8s31, 0, 0) , z3.If(r8s33, 100, 0) , z3.If(r8s34, 100, 0) , z3.If(r8s35, 0, 0) , z3.If(r8s36, 0, 0) , z3.If(r8s37, 100, 0) , z3.If(r8s38, 100, 0) , z3.If(r8s39, 0, 0) , z3.If(r8s40, 0, 0) , z3.If(r8s41, 0, 0) , z3.If(r8s42, 0, 0) , z3.If(r8s43, 100, 0) , z3.If(r8s44, 0, 0) , z3.If(r8s45, 0, 0) , z3.If(r8s46, 0, 0) , z3.If(r8s47, 100, 0) , z3.If(r8s48, 0, 0) , z3.If(r8s49, 0, 0) , z3.If(r8s50, 0, 0) , z3.If(r8s51, 0, 0) , z3.If(r9s0, 100, 0) , z3.If(r9s1, 0, 0) , z3.If(r9s2, 0, 0) , z3.If(r9s3, 0, 0) , z3.If(r9s4, 0, 0) , z3.If(r9s5, 0, 0) , z3.If(r9s6, 0, 0) , z3.If(r9s7, 0, 0) , z3.If(r9s8, 0, 0) , z3.If(r9s9, 0, 0) , z3.If(r9s10, 0, 0) , z3.If(r9s11, 0, 0) , z3.If(r9s12, 0, 0) , z3.If(r9s13, 100, 0) , z3.If(r9s14, 100, 0) , z3.If(r9s15, 0, 0) , z3.If(r9s16, 0, 0) , z3.If(r9s17, 100, 0) , z3.If(r9s18, 0, 0) , z3.If(r9s19, 0, 0) , z3.If(r9s20, 0, 0) , z3.If(r9s21, 0, 0) , z3.If(r9s22, 0, 0) , z3.If(r9s23, 0, 0) , z3.If(r9s24, 100, 0) , z3.If(r9s25, 0, 0) , z3.If(r9s26, 0, 0) , z3.If(r9s27, 0, 0) , z3.If(r9s28, 100, 0) , z3.If(r9s29, 100, 0) , z3.If(r9s30, 0, 0) , z3.If(r9s31, 0, 0) , z3.If(r9s32, 100, 0) , z3.If(r9s33, 0, 0) , z3.If(r9s34, 0, 0) , z3.If(r9s35, 0, 0) , z3.If(r9s36, 100, 0) , z3.If(r9s37, 100, 0) , z3.If(r9s38, 0, 0) , z3.If(r9s39, 0, 0) , z3.If(r9s40, 100, 0) , z3.If(r9s41, 0, 0) , z3.If(r9s42, 0, 0) , z3.If(r9s43, 0, 0) , z3.If(r9s44, 100, 0) , z3.If(r9s45, 0, 0) , z3.If(r9s46, 100, 0) , z3.If(r9s47, 0, 0) , z3.If(r9s48, 0, 0) , z3.If(r9s49, 100, 0) , z3.If(r9s50, 0, 0) , z3.If(r9s51, 0, 0) , z3.If(r10s0, 0, 0) , z3.If(r10s1, 0, 0) , z3.If(r10s2, 0, 0) , z3.If(r10s3, 0, 0) , z3.If(r10s4, 0, 0) , z3.If(r10s5, 0, 0) , z3.If(r10s6, 0, 0) , z3.If(r10s7, 0, 0) , z3.If(r10s9, 100, 0) , z3.If(r10s10, 0, 0) , z3.If(r10s12, 100, 0) , z3.If(r10s13, 100, 0) , z3.If(r10s14, 0, 0) , z3.If(r10s15, 0, 0) , z3.If(r10s16, 0, 0) , z3.If(r10s17, 0, 0) , z3.If(r10s18, 100, 0) , z3.If(r10s19, 0, 0) , z3.If(r10s20, 0, 0) , z3.If(r10s21, 100, 0) , z3.If(r10s22, 0, 0) , z3.If(r10s23, 0, 0) , z3.If(r10s24, 0, 0) , z3.If(r10s25, 0, 0) , z3.If(r10s26, 0, 0) , z3.If(r10s27, 0, 0) , z3.If(r10s28, 100, 0) , z3.If(r10s29, 0, 0) , z3.If(r10s30, 100, 0) , z3.If(r10s31, 100, 0) , z3.If(r10s32, 100, 0) , z3.If(r10s33, 0, 0) , z3.If(r10s34, 0, 0) , z3.If(r10s35, 100, 0) , z3.If(r10s36, 0, 0) , z3.If(r10s37, 100, 0) , z3.If(r10s38, 0, 0) , z3.If(r10s39, 0, 0) , z3.If(r10s40, 100, 0) , z3.If(r10s41, 0, 0) , z3.If(r10s42, 0, 0) , z3.If(r10s43, 0, 0) , z3.If(r10s44, 0, 0) , z3.If(r10s45, 0, 0) , z3.If(r10s46, 0, 0) , z3.If(r10s47, 0, 0) , z3.If(r10s49, 0, 0) , z3.If(r10s50, 0, 0) , z3.If(r10s51, 0, 0) , z3.If(r11s0, 0, 0) , z3.If(r11s1, 0, 0) , z3.If(r11s2, 0, 0) , z3.If(r11s3, 100, 0) , z3.If(r11s4, 100, 0) , z3.If(r11s5, 0, 0) , z3.If(r11s6, 0, 0) , z3.If(r11s7, 0, 0) , z3.If(r11s8, 0, 0) , z3.If(r11s9, 100, 0) , z3.If(r11s10, 0, 0) , z3.If(r11s11, 100, 0) , z3.If(r11s12, 0, 0) , z3.If(r11s13, 100, 0) , z3.If(r11s14, 0, 0) , z3.If(r11s15, 0, 0) , z3.If(r11s16, 0, 0) , z3.If(r11s17, 0, 0) , z3.If(r11s18, 0, 0) , z3.If(r11s19, 0, 0) , z3.If(r11s20, 0, 0) , z3.If(r11s21, 0, 0) , z3.If(r11s22, 0, 0) , z3.If(r11s23, 0, 0) , z3.If(r11s24, 0, 0) , z3.If(r11s25, 100, 0) , z3.If(r11s26, 0, 0) , z3.If(r11s27, 0, 0) , z3.If(r11s28, 0, 0) , z3.If(r11s29, 0, 0) , z3.If(r11s30, 0, 0) , z3.If(r11s31, 0, 0) , z3.If(r11s32, 0, 0) , z3.If(r11s33, 0, 0) , z3.If(r11s34, 0, 0) , z3.If(r11s35, 0, 0) , z3.If(r11s36, 0, 0) , z3.If(r11s37, 0, 0) , z3.If(r11s38, 0, 0) , z3.If(r11s39, 0, 0) , z3.If(r11s40, 0, 0) , z3.If(r11s41, 0, 0) , z3.If(r11s43, 0, 0) , z3.If(r11s44, 0, 0) , z3.If(r11s45, 0, 0) , z3.If(r11s46, 0, 0) , z3.If(r11s48, 0, 0) , z3.If(r11s50, 0, 0) , z3.If(r11s51, 0, 0) , z3.If(r12s1, 0, 0) , z3.If(r12s2, 0, 0) , z3.If(r12s3, 0, 0) , z3.If(r12s4, 100, 0) , z3.If(r12s5, 100, 0) , z3.If(r12s6, 0, 0) , z3.If(r12s7, 0, 0) , z3.If(r12s8, 0, 0) , z3.If(r12s9, 0, 0) , z3.If(r12s10, 0, 0) , z3.If(r12s12, 0, 0) , z3.If(r12s13, 0, 0) , z3.If(r12s14, 0, 0) , z3.If(r12s15, 0, 0) , z3.If(r12s16, 0, 0) , z3.If(r12s19, 0, 0) , z3.If(r12s20, 0, 0) , z3.If(r12s21, 0, 0) , z3.If(r12s22, 0, 0) , z3.If(r12s23, 0, 0) , z3.If(r12s24, 100, 0) , z3.If(r12s25, 0, 0) , z3.If(r12s26, 0, 0) , z3.If(r12s27, 0, 0) , z3.If(r12s28, 0, 0) , z3.If(r12s29, 0, 0) , z3.If(r12s30, 0, 0) , z3.If(r12s31, 100, 0) , z3.If(r12s32, 100, 0) , z3.If(r12s33, 0, 0) , z3.If(r12s34, 100, 0) , z3.If(r12s35, 0, 0) , z3.If(r12s36, 0, 0) , z3.If(r12s37, 0, 0) , z3.If(r12s38, 100, 0) , z3.If(r12s39, 0, 0) , z3.If(r12s40, 0, 0) , z3.If(r12s41, 100, 0) , z3.If(r12s42, 0, 0) , z3.If(r12s43, 0, 0) , z3.If(r12s44, 0, 0) , z3.If(r12s45, 0, 0) , z3.If(r12s46, 0, 0) , z3.If(r12s48, 0, 0) , z3.If(r12s49, 0, 0) , z3.If(r12s50, 0, 0) , z3.If(r12s51, 0, 0) , z3.If(r13s0, 100, 0) , z3.If(r13s2, 0, 0) , z3.If(r13s3, 0, 0) , z3.If(r13s4, 0, 0) , z3.If(r13s5, 0, 0) , z3.If(r13s6, 0, 0) , z3.If(r13s7, 0, 0) , z3.If(r13s10, 0, 0) , z3.If(r13s12, 0, 0) , z3.If(r13s13, 0, 0) , z3.If(r13s14, 100, 0) , z3.If(r13s15, 0, 0) , z3.If(r13s16, 0, 0) , z3.If(r13s18, 0, 0) , z3.If(r13s19, 100, 0) , z3.If(r13s20, 0, 0) , z3.If(r13s22, 0, 0) , z3.If(r13s23, 0, 0) , z3.If(r13s25, 0, 0) , z3.If(r13s26, 0, 0) , z3.If(r13s27, 0, 0) , z3.If(r13s28, 0, 0) , z3.If(r13s29, 0, 0) , z3.If(r13s30, 0, 0) , z3.If(r13s31, 0, 0) , z3.If(r13s32, 0, 0) , z3.If(r13s33, 0, 0) , z3.If(r13s34, 0, 0) , z3.If(r13s35, 0, 0) , z3.If(r13s37, 0, 0) , z3.If(r13s38, 100, 0) , z3.If(r13s39, 0, 0) , z3.If(r13s40, 0, 0) , z3.If(r13s41, 0, 0) , z3.If(r13s42, 0, 0) , z3.If(r13s43, 100, 0) , z3.If(r13s44, 0, 0) , z3.If(r13s45, 0, 0) , z3.If(r13s46, 0, 0) , z3.If(r13s47, 0, 0) , z3.If(r13s49, 0, 0) , z3.If(r13s50, 0, 0) , z3.If(r13s51, 0, 0) , z3.If(r14s0, 0, 0) , z3.If(r14s1, 0, 0) , z3.If(r14s2, 0, 0) , z3.If(r14s4, 0, 0) , z3.If(r14s5, 0, 0) , z3.If(r14s7, 100, 0) , z3.If(r14s8, 100, 0) , z3.If(r14s9, 0, 0) , z3.If(r14s10, 0, 0) , z3.If(r14s11, 0, 0) , z3.If(r14s12, 0, 0) , z3.If(r14s13, 100, 0) , z3.If(r14s14, 0, 0) , z3.If(r14s15, 0, 0) , z3.If(r14s17, 0, 0) , z3.If(r14s18, 0, 0) , z3.If(r14s19, 0, 0) , z3.If(r14s20, 0, 0) , z3.If(r14s21, 0, 0) , z3.If(r14s22, 0, 0) , z3.If(r14s23, 0, 0) , z3.If(r14s24, 100, 0) , z3.If(r14s25, 0, 0) , z3.If(r14s26, 0, 0) , z3.If(r14s27, 0, 0) , z3.If(r14s28, 0, 0) , z3.If(r14s29, 0, 0) , z3.If(r14s30, 0, 0) , z3.If(r14s34, 100, 0) , z3.If(r14s36, 0, 0) , z3.If(r14s38, 0, 0) , z3.If(r14s39, 0, 0) , z3.If(r14s40, 0, 0) , z3.If(r14s41, 0, 0) , z3.If(r14s42, 100, 0) , z3.If(r14s43, 100, 0) , z3.If(r14s44, 100, 0) , z3.If(r14s45, 0, 0) , z3.If(r14s46, 0, 0) , z3.If(r14s47, 0, 0) , z3.If(r14s48, 100, 0) , z3.If(r14s49, 100, 0) , z3.If(r14s50, 0, 0) , z3.If(r14s51, 0, 0) , z3.If(r15s1, 0, 0) , z3.If(r15s2, 0, 0) , z3.If(r15s3, 0, 0) , z3.If(r15s4, 0, 0) , z3.If(r15s5, 0, 0) , z3.If(r15s6, 0, 0) , z3.If(r15s7, 0, 0) , z3.If(r15s8, 0, 0) , z3.If(r15s9, 100, 0) , z3.If(r15s10, 0, 0) , z3.If(r15s11, 0, 0) , z3.If(r15s12, 100, 0) , z3.If(r15s13, 100, 0) , z3.If(r15s14, 0, 0) , z3.If(r15s15, 0, 0) , z3.If(r15s16, 0, 0) , z3.If(r15s17, 0, 0) , z3.If(r15s19, 0, 0) , z3.If(r15s20, 0, 0) , z3.If(r15s21, 0, 0) , z3.If(r15s22, 100, 0) , z3.If(r15s23, 100, 0) , z3.If(r15s24, 100, 0) , z3.If(r15s25, 0, 0) , z3.If(r15s26, 0, 0) , z3.If(r15s27, 0, 0) , z3.If(r15s28, 0, 0) , z3.If(r15s29, 0, 0) , z3.If(r15s30, 0, 0) , z3.If(r15s31, 0, 0) , z3.If(r15s32, 0, 0) , z3.If(r15s33, 0, 0) , z3.If(r15s34, 100, 0) , z3.If(r15s35, 0, 0) , z3.If(r15s36, 0, 0) , z3.If(r15s37, 100, 0) , z3.If(r15s38, 0, 0) , z3.If(r15s39, 0, 0) , z3.If(r15s40, 0, 0) , z3.If(r15s41, 0, 0) , z3.If(r15s42, 0, 0) , z3.If(r15s43, 0, 0) , z3.If(r15s44, 0, 0) , z3.If(r15s45, 0, 0) , z3.If(r15s46, 0, 0) , z3.If(r15s48, 0, 0) , z3.If(r15s49, 0, 0) , z3.If(r15s50, 0, 0) , z3.If(r15s51, 100, 0) , z3.If(r16s1, 0, 0) , z3.If(r16s2, 0, 0) , z3.If(r16s3, 0, 0) , z3.If(r16s4, 100, 0) , z3.If(r16s5, 0, 0) , z3.If(r16s7, 100, 0) , z3.If(r16s8, 0, 0) , z3.If(r16s9, 0, 0) , z3.If(r16s11, 0, 0) , z3.If(r16s12, 0, 0) , z3.If(r16s13, 0, 0) , z3.If(r16s14, 0, 0) , z3.If(r16s15, 0, 0) , z3.If(r16s16, 0, 0) , z3.If(r16s17, 0, 0) , z3.If(r16s19, 0, 0) , z3.If(r16s20, 0, 0) , z3.If(r16s21, 0, 0) , z3.If(r16s22, 0, 0) , z3.If(r16s23, 0, 0) , z3.If(r16s24, 0, 0) , z3.If(r16s25, 0, 0) , z3.If(r16s26, 0, 0) , z3.If(r16s27, 0, 0) , z3.If(r16s28, 0, 0) , z3.If(r16s29, 0, 0) , z3.If(r16s30, 0, 0) , z3.If(r16s31, 100, 0) , z3.If(r16s32, 0, 0) , z3.If(r16s33, 0, 0) , z3.If(r16s34, 100, 0) , z3.If(r16s35, 0, 0) , z3.If(r16s36, 0, 0) , z3.If(r16s38, 0, 0) , z3.If(r16s39, 0, 0) , z3.If(r16s40, 0, 0) , z3.If(r16s41, 0, 0) , z3.If(r16s42, 0, 0) , z3.If(r16s43, 0, 0) , z3.If(r16s44, 0, 0) , z3.If(r16s45, 0, 0) , z3.If(r16s46, 0, 0) , z3.If(r16s48, 0, 0) , z3.If(r16s49, 0, 0) , z3.If(r16s50, 0, 0) , z3.If(r16s51, 100, 0) , z3.If(r17s0, 0, 0) , z3.If(r17s1, 0, 0) , z3.If(r17s2, 100, 0) , z3.If(r17s3, 100, 0) , z3.If(r17s4, 0, 0) , z3.If(r17s5, 0, 0) , z3.If(r17s7, 100, 0) , z3.If(r17s8, 0, 0) , z3.If(r17s11, 0, 0) , z3.If(r17s12, 0, 0) , z3.If(r17s13, 100, 0) , z3.If(r17s14, 0, 0) , z3.If(r17s15, 0, 0) , z3.If(r17s16, 0, 0) , z3.If(r17s17, 0, 0) , z3.If(r17s18, 100, 0) , z3.If(r17s19, 0, 0) , z3.If(r17s20, 0, 0) , z3.If(r17s21, 0, 0) , z3.If(r17s22, 0, 0) , z3.If(r17s23, 0, 0) , z3.If(r17s25, 0, 0) , z3.If(r17s26, 0, 0) , z3.If(r17s27, 0, 0) , z3.If(r17s28, 0, 0) , z3.If(r17s29, 0, 0) , z3.If(r17s30, 0, 0) , z3.If(r17s31, 0, 0) , z3.If(r17s32, 100, 0) , z3.If(r17s33, 0, 0) , z3.If(r17s34, 0, 0) , z3.If(r17s35, 100, 0) , z3.If(r17s36, 0, 0) , z3.If(r17s37, 0, 0) , z3.If(r17s38, 0, 0) , z3.If(r17s39, 0, 0) , z3.If(r17s40, 0, 0) , z3.If(r17s41, 0, 0) , z3.If(r17s42, 0, 0) , z3.If(r17s43, 0, 0) , z3.If(r17s44, 0, 0) , z3.If(r17s45, 0, 0) , z3.If(r17s46, 0, 0) , z3.If(r17s47, 100, 0) , z3.If(r17s48, 0, 0) , z3.If(r17s49, 0, 0) , z3.If(r17s50, 0, 0) , z3.If(r17s51, 0, 0) , z3.If(r18s0, 100, 0) , z3.If(r18s1, 0, 0) , z3.If(r18s2, 0, 0) , z3.If(r18s3, 0, 0) , z3.If(r18s4, 100, 0) , z3.If(r18s5, 0, 0) , z3.If(r18s6, 0, 0) , z3.If(r18s7, 0, 0) , z3.If(r18s8, 0, 0) , z3.If(r18s10, 0, 0) , z3.If(r18s11, 0, 0) , z3.If(r18s12, 100, 0) , z3.If(r18s13, 0, 0) , z3.If(r18s14, 0, 0) , z3.If(r18s15, 100, 0) , z3.If(r18s16, 0, 0) , z3.If(r18s17, 0, 0) , z3.If(r18s18, 100, 0) , z3.If(r18s19, 0, 0) , z3.If(r18s20, 0, 0) , z3.If(r18s21, 0, 0) , z3.If(r18s22, 0, 0) , z3.If(r18s23, 100, 0) , z3.If(r18s25, 0, 0) , z3.If(r18s26, 0, 0) , z3.If(r18s27, 0, 0) , z3.If(r18s28, 0, 0) , z3.If(r18s29, 0, 0) , z3.If(r18s30, 0, 0) , z3.If(r18s31, 0, 0) , z3.If(r18s32, 0, 0) , z3.If(r18s33, 100, 0) , z3.If(r18s34, 0, 0) , z3.If(r18s35, 0, 0) , z3.If(r18s36, 0, 0) , z3.If(r18s37, 0, 0) , z3.If(r18s38, 0, 0) , z3.If(r18s39, 0, 0) , z3.If(r18s40, 0, 0) , z3.If(r18s41, 0, 0) , z3.If(r18s42, 0, 0) , z3.If(r18s43, 0, 0) , z3.If(r18s44, 0, 0) , z3.If(r18s45, 100, 0) , z3.If(r18s46, 0, 0) , z3.If(r18s47, 0, 0) , z3.If(r18s48, 100, 0) , z3.If(r18s49, 0, 0) , z3.If(r18s50, 0, 0) , z3.If(r18s51, 100, 0) , z3.If(r19s0, 0, 0) , z3.If(r19s1, 100, 0) , z3.If(r19s3, 0, 0) , z3.If(r19s4, 100, 0) , z3.If(r19s5, 0, 0) , z3.If(r19s6, 0, 0) , z3.If(r19s7, 0, 0) , z3.If(r19s8, 0, 0) , z3.If(r19s9, 0, 0) , z3.If(r19s10, 0, 0) , z3.If(r19s11, 0, 0) , z3.If(r19s12, 0, 0) , z3.If(r19s13, 0, 0) , z3.If(r19s14, 0, 0) , z3.If(r19s15, 100, 0) , z3.If(r19s16, 100, 0) , z3.If(r19s17, 100, 0) , z3.If(r19s18, 100, 0) , z3.If(r19s19, 0, 0) , z3.If(r19s20, 0, 0) , z3.If(r19s21, 0, 0) , z3.If(r19s22, 100, 0) , z3.If(r19s23, 0, 0) , z3.If(r19s24, 0, 0) , z3.If(r19s25, 0, 0) , z3.If(r19s26, 0, 0) , z3.If(r19s27, 0, 0) , z3.If(r19s28, 0, 0) , z3.If(r19s29, 0, 0) , z3.If(r19s30, 0, 0) , z3.If(r19s31, 0, 0) , z3.If(r19s32, 0, 0) , z3.If(r19s33, 0, 0) , z3.If(r19s34, 0, 0) , z3.If(r19s35, 0, 0) , z3.If(r19s36, 0, 0) , z3.If(r19s37, 100, 0) , z3.If(r19s38, 0, 0) , z3.If(r19s39, 0, 0) , z3.If(r19s40, 0, 0) , z3.If(r19s41, 0, 0) , z3.If(r19s42, 100, 0) , z3.If(r19s43, 0, 0) , z3.If(r19s44, 100, 0) , z3.If(r19s45, 0, 0) , z3.If(r19s46, 0, 0) , z3.If(r19s47, 0, 0) , z3.If(r19s48, 0, 0) , z3.If(r19s49, 100, 0) , z3.If(r19s50, 0, 0) , z3.If(r19s51, 100, 0) , z3.If(r20s0, 0, 0) , z3.If(r20s1, 0, 0) , z3.If(r20s2, 0, 0) , z3.If(r20s3, 0, 0) , z3.If(r20s4, 0, 0) , z3.If(r20s5, 0, 0) , z3.If(r20s6, 100, 0) , z3.If(r20s7, 0, 0) , z3.If(r20s8, 0, 0) , z3.If(r20s9, 100, 0) , z3.If(r20s10, 0, 0) , z3.If(r20s11, 0, 0) , z3.If(r20s12, 0, 0) , z3.If(r20s13, 100, 0) , z3.If(r20s15, 0, 0) , z3.If(r20s17, 100, 0) , z3.If(r20s18, 0, 0) , z3.If(r20s19, 0, 0) , z3.If(r20s20, 0, 0) , z3.If(r20s21, 0, 0) , z3.If(r20s22, 0, 0) , z3.If(r20s23, 0, 0) , z3.If(r20s24, 100, 0) , z3.If(r20s25, 0, 0) , z3.If(r20s26, 100, 0) , z3.If(r20s30, 100, 0) , z3.If(r20s31, 0, 0) , z3.If(r20s32, 100, 0) , z3.If(r20s33, 0, 0) , z3.If(r20s34, 100, 0) , z3.If(r20s35, 0, 0) , z3.If(r20s36, 100, 0) , z3.If(r20s37, 100, 0) , z3.If(r20s38, 0, 0) , z3.If(r20s39, 0, 0) , z3.If(r20s40, 0, 0) , z3.If(r20s41, 0, 0) , z3.If(r20s42, 0, 0) , z3.If(r20s43, 0, 0) , z3.If(r20s44, 0, 0) , z3.If(r20s45, 100, 0) , z3.If(r20s46, 0, 0) , z3.If(r20s47, 100, 0) , z3.If(r20s48, 0, 0) , z3.If(r20s49, 0, 0) , z3.If(r20s50, 0, 0) , z3.If(r20s51, 0, 0) , z3.If(r21s1, 0, 0) , z3.If(r21s2, 100, 0) , z3.If(r21s3, 0, 0) , z3.If(r21s4, 0, 0) , z3.If(r21s5, 100, 0) , z3.If(r21s6, 0, 0) , z3.If(r21s7, 0, 0) , z3.If(r21s8, 0, 0) , z3.If(r21s10, 0, 0) , z3.If(r21s12, 100, 0) , z3.If(r21s14, 0, 0) , z3.If(r21s15, 100, 0) , z3.If(r21s18, 100, 0) , z3.If(r21s19, 0, 0) , z3.If(r21s20, 0, 0) , z3.If(r21s22, 0, 0) , z3.If(r21s23, 0, 0) , z3.If(r21s24, 0, 0) , z3.If(r21s25, 0, 0) , z3.If(r21s26, 0, 0) , z3.If(r21s27, 0, 0) , z3.If(r21s28, 0, 0) , z3.If(r21s29, 0, 0) , z3.If(r21s30, 0, 0) , z3.If(r21s31, 0, 0) , z3.If(r21s33, 0, 0) , z3.If(r21s34, 0, 0) , z3.If(r21s35, 100, 0) , z3.If(r21s37, 0, 0) , z3.If(r21s38, 0, 0) , z3.If(r21s39, 0, 0) , z3.If(r21s40, 0, 0) , z3.If(r21s41, 0, 0) , z3.If(r21s42, 0, 0) , z3.If(r21s43, 0, 0) , z3.If(r21s44, 0, 0) , z3.If(r21s45, 0, 0) , z3.If(r21s46, 0, 0) , z3.If(r21s47, 0, 0) , z3.If(r21s48, 0, 0) , z3.If(r21s49, 0, 0) , z3.If(r21s50, 0, 0) , z3.If(r21s51, 0, 0) , z3.If(r22s0, 0, 0) , z3.If(r22s1, 0, 0) , z3.If(r22s2, 0, 0) , z3.If(r22s3, 0, 0) , z3.If(r22s4, 100, 0) , z3.If(r22s5, 100, 0) , z3.If(r22s6, 0, 0) , z3.If(r22s7, 0, 0) , z3.If(r22s8, 0, 0) , z3.If(r22s9, 0, 0) , z3.If(r22s10, 0, 0) , z3.If(r22s11, 0, 0) , z3.If(r22s12, 0, 0) , z3.If(r22s13, 0, 0) , z3.If(r22s14, 0, 0) , z3.If(r22s15, 0, 0) , z3.If(r22s16, 100, 0) , z3.If(r22s17, 0, 0) , z3.If(r22s18, 0, 0) , z3.If(r22s19, 0, 0) , z3.If(r22s20, 0, 0) , z3.If(r22s21, 0, 0) , z3.If(r22s22, 0, 0) , z3.If(r22s23, 0, 0) , z3.If(r22s24, 0, 0) , z3.If(r22s25, 0, 0) , z3.If(r22s26, 0, 0) , z3.If(r22s27, 0, 0) , z3.If(r22s28, 0, 0) , z3.If(r22s29, 0, 0) , z3.If(r22s30, 0, 0) , z3.If(r22s31, 0, 0) , z3.If(r22s32, 100, 0) , z3.If(r22s33, 0, 0) , z3.If(r22s34, 0, 0) , z3.If(r22s35, 0, 0) , z3.If(r22s36, 0, 0) , z3.If(r22s37, 0, 0) , z3.If(r22s38, 0, 0) , z3.If(r22s39, 100, 0) , z3.If(r22s40, 0, 0) , z3.If(r22s41, 0, 0) , z3.If(r22s42, 0, 0) , z3.If(r22s43, 0, 0) , z3.If(r22s44, 0, 0) , z3.If(r22s45, 100, 0) , z3.If(r22s46, 0, 0) , z3.If(r22s47, 0, 0) , z3.If(r22s48, 0, 0) , z3.If(r22s49, 0, 0) , z3.If(r22s50, 0, 0) , z3.If(r22s51, 0, 0) , z3.If(r23s0, 0, 0) , z3.If(r23s1, 0, 0) , z3.If(r23s2, 0, 0) , z3.If(r23s3, 100, 0) , z3.If(r23s4, 100, 0) , z3.If(r23s5, 0, 0) , z3.If(r23s6, 0, 0) , z3.If(r23s7, 0, 0) , z3.If(r23s8, 0, 0) , z3.If(r23s10, 100, 0) , z3.If(r23s11, 0, 0) , z3.If(r23s12, 0, 0) , z3.If(r23s13, 0, 0) , z3.If(r23s14, 0, 0) , z3.If(r23s15, 0, 0) , z3.If(r23s16, 100, 0) , z3.If(r23s17, 0, 0) , z3.If(r23s18, 0, 0) , z3.If(r23s19, 0, 0) , z3.If(r23s20, 0, 0) , z3.If(r23s21, 0, 0) , z3.If(r23s22, 0, 0) , z3.If(r23s23, 0, 0) , z3.If(r23s25, 0, 0) , z3.If(r23s26, 0, 0) , z3.If(r23s27, 0, 0) , z3.If(r23s28, 0, 0) , z3.If(r23s29, 0, 0) , z3.If(r23s30, 0, 0) , z3.If(r23s31, 0, 0) , z3.If(r23s32, 100, 0) , z3.If(r23s33, 0, 0) , z3.If(r23s34, 0, 0) , z3.If(r23s35, 0, 0) , z3.If(r23s36, 0, 0) , z3.If(r23s37, 0, 0) , z3.If(r23s38, 0, 0) , z3.If(r23s39, 100, 0) , z3.If(r23s40, 0, 0) , z3.If(r23s41, 0, 0) , z3.If(r23s42, 100, 0) , z3.If(r23s43, 0, 0) , z3.If(r23s44, 0, 0) , z3.If(r23s45, 100, 0) , z3.If(r23s46, 0, 0) , z3.If(r23s47, 0, 0) , z3.If(r23s48, 0, 0) , z3.If(r23s49, 0, 0) , z3.If(r23s50, 0, 0) , z3.If(r23s51, 0, 0)]

# 60/30/10
# scores = [z3.If(r0s0, 60, 0) , z3.If(r0s1, 10, 0) , z3.If(r0s2, 10, 0) , z3.If(r0s4, 10, 0) , z3.If(r0s5, 10, 0) , z3.If(r0s7, 10, 0) , z3.If(r0s8, 10, 0) , z3.If(r0s9, 10, 0) , z3.If(r0s10, 10, 0) , z3.If(r0s12, 10, 0) , z3.If(r0s13, 10, 0) , z3.If(r0s14, 30, 0) , z3.If(r0s15, 10, 0) , z3.If(r0s16, 10, 0) , z3.If(r0s18, 30, 0) , z3.If(r0s19, 10, 0) , z3.If(r0s20, 10, 0) , z3.If(r0s21, 10, 0) , z3.If(r0s22, 10, 0) , z3.If(r0s23, 60, 0) , z3.If(r0s24, 60, 0) , z3.If(r0s25, 10, 0) , z3.If(r0s26, 30, 0) , z3.If(r0s27, 10, 0) , z3.If(r0s28, 60, 0) , z3.If(r0s29, 60, 0) , z3.If(r0s30, 60, 0) , z3.If(r0s31, 10, 0) , z3.If(r0s32, 60, 0) , z3.If(r0s34, 10, 0) , z3.If(r0s35, 10, 0) , z3.If(r0s36, 10, 0) , z3.If(r0s37, 60, 0) , z3.If(r0s38, 10, 0) , z3.If(r0s39, 10, 0) , z3.If(r0s40, 30, 0) , z3.If(r0s41, 10, 0) , z3.If(r0s42, 10, 0) , z3.If(r0s43, 10, 0) , z3.If(r0s44, 60, 0) , z3.If(r0s45, 10, 0) , z3.If(r0s46, 10, 0) , z3.If(r0s47, 10, 0) , z3.If(r0s48, 10, 0) , z3.If(r0s49, 10, 0) , z3.If(r0s50, 10, 0) , z3.If(r0s51, 10, 0) , z3.If(r1s0, 60, 0) , z3.If(r1s1, 10, 0) , z3.If(r1s2, 30, 0) , z3.If(r1s3, 60, 0) , z3.If(r1s4, 10, 0) , z3.If(r1s5, 10, 0) , z3.If(r1s6, 10, 0) , z3.If(r1s7, 10, 0) , z3.If(r1s8, 10, 0) , z3.If(r1s9, 60, 0) , z3.If(r1s10, 30, 0) , z3.If(r1s11, 60, 0) , z3.If(r1s12, 10, 0) , z3.If(r1s13, 10, 0) , z3.If(r1s14, 10, 0) , z3.If(r1s15, 10, 0) , z3.If(r1s16, 10, 0) , z3.If(r1s17, 30, 0) , z3.If(r1s18, 60, 0) , z3.If(r1s19, 10, 0) , z3.If(r1s20, 10, 0) , z3.If(r1s21, 10, 0) , z3.If(r1s22, 60, 0) , z3.If(r1s23, 10, 0) , z3.If(r1s24, 10, 0) , z3.If(r1s25, 10, 0) , z3.If(r1s26, 10, 0) , z3.If(r1s27, 10, 0) , z3.If(r1s28, 60, 0) , z3.If(r1s29, 30, 0) , z3.If(r1s30, 10, 0) , z3.If(r1s31, 10, 0) , z3.If(r1s32, 10, 0) , z3.If(r1s33, 10, 0) , z3.If(r1s35, 10, 0) , z3.If(r1s36, 30, 0) , z3.If(r1s37, 10, 0) , z3.If(r1s38, 10, 0) , z3.If(r1s39, 10, 0) , z3.If(r1s40, 60, 0) , z3.If(r1s41, 10, 0) , z3.If(r1s42, 10, 0) , z3.If(r1s43, 10, 0) , z3.If(r1s44, 10, 0) , z3.If(r1s45, 10, 0) , z3.If(r1s46, 10, 0) , z3.If(r1s47, 10, 0) , z3.If(r1s48, 10, 0) , z3.If(r1s49, 10, 0) , z3.If(r1s50, 10, 0) , z3.If(r1s51, 10, 0) , z3.If(r2s0, 10, 0) , z3.If(r2s1, 10, 0) , z3.If(r2s2, 60, 0) , z3.If(r2s3, 10, 0) , z3.If(r2s4, 10, 0) , z3.If(r2s5, 10, 0) , z3.If(r2s6, 10, 0) , z3.If(r2s7, 30, 0) , z3.If(r2s8, 10, 0) , z3.If(r2s9, 10, 0) , z3.If(r2s10, 10, 0) , z3.If(r2s11, 10, 0) , z3.If(r2s12, 60, 0) , z3.If(r2s14, 30, 0) , z3.If(r2s15, 60, 0) , z3.If(r2s18, 10, 0) , z3.If(r2s19, 10, 0) , z3.If(r2s20, 10, 0) , z3.If(r2s22, 10, 0) , z3.If(r2s23, 10, 0) , z3.If(r2s24, 10, 0) , z3.If(r2s25, 10, 0) , z3.If(r2s26, 10, 0) , z3.If(r2s27, 10, 0) , z3.If(r2s28, 10, 0) , z3.If(r2s29, 10, 0) , z3.If(r2s30, 10, 0) , z3.If(r2s31, 10, 0) , z3.If(r2s33, 10, 0) , z3.If(r2s34, 10, 0) , z3.If(r2s35, 10, 0) , z3.If(r2s36, 10, 0) , z3.If(r2s37, 30, 0) , z3.If(r2s38, 10, 0) , z3.If(r2s39, 10, 0) , z3.If(r2s40, 10, 0) , z3.If(r2s41, 10, 0) , z3.If(r2s42, 10, 0) , z3.If(r2s43, 10, 0) , z3.If(r2s44, 10, 0) , z3.If(r2s45, 10, 0) , z3.If(r2s46, 30, 0) , z3.If(r2s47, 10, 0) , z3.If(r2s48, 10, 0) , z3.If(r2s49, 30, 0) , z3.If(r2s50, 10, 0) , z3.If(r2s51, 30, 0) , z3.If(r3s0, 10, 0) , z3.If(r3s2, 60, 0) , z3.If(r3s3, 60, 0) , z3.If(r3s4, 10, 0) , z3.If(r3s5, 10, 0) , z3.If(r3s6, 60, 0) , z3.If(r3s7, 10, 0) , z3.If(r3s8, 30, 0) , z3.If(r3s10, 10, 0) , z3.If(r3s11, 30, 0) , z3.If(r3s12, 10, 0) , z3.If(r3s15, 10, 0) , z3.If(r3s17, 30, 0) , z3.If(r3s18, 30, 0) , z3.If(r3s19, 10, 0) , z3.If(r3s20, 10, 0) , z3.If(r3s21, 10, 0) , z3.If(r3s22, 10, 0) , z3.If(r3s23, 10, 0) , z3.If(r3s25, 60, 0) , z3.If(r3s31, 30, 0) , z3.If(r3s32, 60, 0) , z3.If(r3s33, 10, 0) , z3.If(r3s34, 10, 0) , z3.If(r3s35, 60, 0) , z3.If(r3s36, 60, 0) , z3.If(r3s37, 60, 0) , z3.If(r3s38, 60, 0) , z3.If(r3s39, 10, 0) , z3.If(r3s40, 60, 0) , z3.If(r3s41, 30, 0) , z3.If(r3s42, 60, 0) , z3.If(r3s43, 30, 0) , z3.If(r3s44, 10, 0) , z3.If(r3s45, 10, 0) , z3.If(r3s46, 30, 0) , z3.If(r3s47, 60, 0) , z3.If(r3s48, 10, 0) , z3.If(r3s49, 30, 0) , z3.If(r3s50, 60, 0) , z3.If(r3s51, 30, 0) , z3.If(r4s0, 30, 0) , z3.If(r4s1, 10, 0) , z3.If(r4s2, 30, 0) , z3.If(r4s3, 10, 0) , z3.If(r4s4, 60, 0) , z3.If(r4s5, 60, 0) , z3.If(r4s6, 10, 0) , z3.If(r4s7, 10, 0) , z3.If(r4s8, 10, 0) , z3.If(r4s9, 10, 0) , z3.If(r4s10, 60, 0) , z3.If(r4s11, 10, 0) , z3.If(r4s12, 10, 0) , z3.If(r4s13, 10, 0) , z3.If(r4s14, 10, 0) , z3.If(r4s15, 10, 0) , z3.If(r4s16, 30, 0) , z3.If(r4s17, 10, 0) , z3.If(r4s18, 60, 0) , z3.If(r4s19, 10, 0) , z3.If(r4s20, 10, 0) , z3.If(r4s21, 10, 0) , z3.If(r4s22, 30, 0) , z3.If(r4s23, 10, 0) , z3.If(r4s24, 10, 0) , z3.If(r4s25, 10, 0) , z3.If(r4s26, 10, 0) , z3.If(r4s27, 10, 0) , z3.If(r4s28, 10, 0) , z3.If(r4s29, 10, 0) , z3.If(r4s30, 10, 0) , z3.If(r4s31, 10, 0) , z3.If(r4s32, 30, 0) , z3.If(r4s33, 10, 0) , z3.If(r4s34, 10, 0) , z3.If(r4s35, 10, 0) , z3.If(r4s36, 10, 0) , z3.If(r4s37, 10, 0) , z3.If(r4s38, 10, 0) , z3.If(r4s39, 10, 0) , z3.If(r4s40, 10, 0) , z3.If(r4s41, 10, 0) , z3.If(r4s42, 10, 0) , z3.If(r4s43, 10, 0) , z3.If(r4s44, 10, 0) , z3.If(r4s45, 10, 0) , z3.If(r4s46, 10, 0) , z3.If(r4s47, 30, 0) , z3.If(r4s48, 10, 0) , z3.If(r4s49, 10, 0) , z3.If(r4s50, 10, 0) , z3.If(r4s51, 60, 0) , z3.If(r5s0, 60, 0) , z3.If(r5s1, 10, 0) , z3.If(r5s2, 30, 0) , z3.If(r5s3, 30, 0) , z3.If(r5s4, 30, 0) , z3.If(r5s5, 10, 0) , z3.If(r5s6, 10, 0) , z3.If(r5s7, 10, 0) , z3.If(r5s8, 60, 0) , z3.If(r5s9, 60, 0) , z3.If(r5s10, 10, 0) , z3.If(r5s11, 10, 0) , z3.If(r5s12, 10, 0) , z3.If(r5s13, 10, 0) , z3.If(r5s14, 10, 0) , z3.If(r5s15, 10, 0) , z3.If(r5s16, 10, 0) , z3.If(r5s17, 60, 0) , z3.If(r5s18, 10, 0) , z3.If(r5s19, 30, 0) , z3.If(r5s20, 10, 0) , z3.If(r5s21, 10, 0) , z3.If(r5s22, 30, 0) , z3.If(r5s23, 10, 0) , z3.If(r5s24, 60, 0) , z3.If(r5s25, 10, 0) , z3.If(r5s26, 10, 0) , z3.If(r5s27, 10, 0) , z3.If(r5s28, 30, 0) , z3.If(r5s29, 10, 0) , z3.If(r5s30, 10, 0) , z3.If(r5s31, 10, 0) , z3.If(r5s32, 10, 0) , z3.If(r5s33, 10, 0) , z3.If(r5s34, 30, 0) , z3.If(r5s35, 10, 0) , z3.If(r5s36, 10, 0) , z3.If(r5s37, 60, 0) , z3.If(r5s38, 10, 0) , z3.If(r5s39, 30, 0) , z3.If(r5s40, 10, 0) , z3.If(r5s41, 10, 0) , z3.If(r5s42, 10, 0) , z3.If(r5s43, 10, 0) , z3.If(r5s44, 10, 0) , z3.If(r5s45, 10, 0) , z3.If(r5s46, 10, 0) , z3.If(r5s47, 10, 0) , z3.If(r5s48, 60, 0) , z3.If(r5s49, 30, 0) , z3.If(r5s50, 10, 0) , z3.If(r5s51, 10, 0) , z3.If(r6s0, 30, 0) , z3.If(r6s1, 10, 0) , z3.If(r6s2, 30, 0) , z3.If(r6s3, 30, 0) , z3.If(r6s4, 30, 0) , z3.If(r6s5, 10, 0) , z3.If(r6s7, 10, 0) , z3.If(r6s8, 10, 0) , z3.If(r6s9, 30, 0) , z3.If(r6s11, 10, 0) , z3.If(r6s12, 10, 0) , z3.If(r6s13, 10, 0) , z3.If(r6s14, 10, 0) , z3.If(r6s15, 10, 0) , z3.If(r6s16, 10, 0) , z3.If(r6s17, 30, 0) , z3.If(r6s18, 60, 0) , z3.If(r6s19, 60, 0) , z3.If(r6s20, 10, 0) , z3.If(r6s21, 30, 0) , z3.If(r6s22, 10, 0) , z3.If(r6s23, 10, 0) , z3.If(r6s24, 30, 0) , z3.If(r6s25, 10, 0) , z3.If(r6s26, 10, 0) , z3.If(r6s27, 10, 0) , z3.If(r6s28, 60, 0) , z3.If(r6s29, 30, 0) , z3.If(r6s30, 60, 0) , z3.If(r6s31, 30, 0) , z3.If(r6s32, 10, 0) , z3.If(r6s33, 10, 0) , z3.If(r6s34, 10, 0) , z3.If(r6s36, 10, 0) , z3.If(r6s39, 10, 0) , z3.If(r6s40, 10, 0) , z3.If(r6s41, 10, 0) , z3.If(r6s42, 30, 0) , z3.If(r6s44, 10, 0) , z3.If(r6s45, 10, 0) , z3.If(r6s47, 60, 0) , z3.If(r6s48, 10, 0) , z3.If(r6s49, 60, 0) , z3.If(r6s50, 10, 0) , z3.If(r6s51, 30, 0) , z3.If(r7s0, 60, 0) , z3.If(r7s1, 10, 0) , z3.If(r7s2, 10, 0) , z3.If(r7s3, 30, 0) , z3.If(r7s4, 10, 0) , z3.If(r7s5, 10, 0) , z3.If(r7s6, 10, 0) , z3.If(r7s7, 30, 0) , z3.If(r7s10, 10, 0) , z3.If(r7s12, 10, 0) , z3.If(r7s13, 60, 0) , z3.If(r7s14, 10, 0) , z3.If(r7s15, 10, 0) , z3.If(r7s16, 10, 0) , z3.If(r7s17, 30, 0) , z3.If(r7s18, 30, 0) , z3.If(r7s19, 10, 0) , z3.If(r7s20, 10, 0) , z3.If(r7s22, 60, 0) , z3.If(r7s23, 60, 0) , z3.If(r7s24, 30, 0) , z3.If(r7s25, 10, 0) , z3.If(r7s26, 60, 0) , z3.If(r7s27, 10, 0) , z3.If(r7s28, 60, 0) , z3.If(r7s29, 60, 0) , z3.If(r7s30, 10, 0) , z3.If(r7s31, 60, 0) , z3.If(r7s32, 10, 0) , z3.If(r7s33, 10, 0) , z3.If(r7s34, 10, 0) , z3.If(r7s35, 10, 0) , z3.If(r7s36, 30, 0) , z3.If(r7s37, 60, 0) , z3.If(r7s38, 60, 0) , z3.If(r7s39, 10, 0) , z3.If(r7s40, 60, 0) , z3.If(r7s41, 10, 0) , z3.If(r7s42, 60, 0) , z3.If(r7s43, 60, 0) , z3.If(r7s44, 30, 0) , z3.If(r7s45, 10, 0) , z3.If(r7s46, 10, 0) , z3.If(r7s47, 60, 0) , z3.If(r7s49, 60, 0) , z3.If(r7s50, 10, 0) , z3.If(r7s51, 10, 0) , z3.If(r8s0, 10, 0) , z3.If(r8s1, 10, 0) , z3.If(r8s2, 10, 0) , z3.If(r8s3, 10, 0) , z3.If(r8s4, 10, 0) , z3.If(r8s5, 10, 0) , z3.If(r8s6, 10, 0) , z3.If(r8s7, 10, 0) , z3.If(r8s8, 10, 0) , z3.If(r8s9, 60, 0) , z3.If(r8s10, 10, 0) , z3.If(r8s11, 10, 0) , z3.If(r8s12, 10, 0) , z3.If(r8s14, 10, 0) , z3.If(r8s15, 10, 0) , z3.If(r8s18, 10, 0) , z3.If(r8s19, 10, 0) , z3.If(r8s20, 10, 0) , z3.If(r8s21, 10, 0) , z3.If(r8s22, 30, 0) , z3.If(r8s23, 30, 0) , z3.If(r8s24, 30, 0) , z3.If(r8s25, 30, 0) , z3.If(r8s26, 10, 0) , z3.If(r8s27, 10, 0) , z3.If(r8s28, 10, 0) , z3.If(r8s29, 10, 0) , z3.If(r8s30, 10, 0) , z3.If(r8s31, 30, 0) , z3.If(r8s33, 60, 0) , z3.If(r8s34, 60, 0) , z3.If(r8s35, 30, 0) , z3.If(r8s36, 10, 0) , z3.If(r8s37, 60, 0) , z3.If(r8s38, 60, 0) , z3.If(r8s39, 10, 0) , z3.If(r8s40, 30, 0) , z3.If(r8s41, 10, 0) , z3.If(r8s42, 30, 0) , z3.If(r8s43, 60, 0) , z3.If(r8s44, 10, 0) , z3.If(r8s45, 10, 0) , z3.If(r8s46, 10, 0) , z3.If(r8s47, 60, 0) , z3.If(r8s48, 10, 0) , z3.If(r8s49, 30, 0) , z3.If(r8s50, 30, 0) , z3.If(r8s51, 30, 0) , z3.If(r9s0, 60, 0) , z3.If(r9s1, 10, 0) , z3.If(r9s2, 10, 0) , z3.If(r9s3, 10, 0) , z3.If(r9s4, 10, 0) , z3.If(r9s5, 10, 0) , z3.If(r9s6, 10, 0) , z3.If(r9s7, 10, 0) , z3.If(r9s8, 10, 0) , z3.If(r9s9, 10, 0) , z3.If(r9s10, 10, 0) , z3.If(r9s11, 10, 0) , z3.If(r9s12, 10, 0) , z3.If(r9s13, 60, 0) , z3.If(r9s14, 60, 0) , z3.If(r9s15, 10, 0) , z3.If(r9s16, 10, 0) , z3.If(r9s17, 60, 0) , z3.If(r9s18, 10, 0) , z3.If(r9s19, 10, 0) , z3.If(r9s20, 10, 0) , z3.If(r9s21, 10, 0) , z3.If(r9s22, 10, 0) , z3.If(r9s23, 10, 0) , z3.If(r9s24, 60, 0) , z3.If(r9s25, 10, 0) , z3.If(r9s26, 30, 0) , z3.If(r9s27, 10, 0) , z3.If(r9s28, 60, 0) , z3.If(r9s29, 60, 0) , z3.If(r9s30, 10, 0) , z3.If(r9s31, 10, 0) , z3.If(r9s32, 60, 0) , z3.If(r9s33, 30, 0) , z3.If(r9s34, 10, 0) , z3.If(r9s35, 10, 0) , z3.If(r9s36, 60, 0) , z3.If(r9s37, 60, 0) , z3.If(r9s38, 10, 0) , z3.If(r9s39, 10, 0) , z3.If(r9s40, 60, 0) , z3.If(r9s41, 10, 0) , z3.If(r9s42, 10, 0) , z3.If(r9s43, 10, 0) , z3.If(r9s44, 60, 0) , z3.If(r9s45, 10, 0) , z3.If(r9s46, 60, 0) , z3.If(r9s47, 10, 0) , z3.If(r9s48, 10, 0) , z3.If(r9s49, 60, 0) , z3.If(r9s50, 10, 0) , z3.If(r9s51, 10, 0) , z3.If(r10s0, 10, 0) , z3.If(r10s1, 10, 0) , z3.If(r10s2, 10, 0) , z3.If(r10s3, 10, 0) , z3.If(r10s4, 10, 0) , z3.If(r10s5, 10, 0) , z3.If(r10s6, 10, 0) , z3.If(r10s7, 10, 0) , z3.If(r10s9, 60, 0) , z3.If(r10s10, 10, 0) , z3.If(r10s12, 60, 0) , z3.If(r10s13, 60, 0) , z3.If(r10s14, 10, 0) , z3.If(r10s15, 10, 0) , z3.If(r10s16, 10, 0) , z3.If(r10s17, 10, 0) , z3.If(r10s18, 60, 0) , z3.If(r10s19, 10, 0) , z3.If(r10s20, 10, 0) , z3.If(r10s21, 60, 0) , z3.If(r10s22, 10, 0) , z3.If(r10s23, 10, 0) , z3.If(r10s24, 10, 0) , z3.If(r10s25, 10, 0) , z3.If(r10s26, 10, 0) , z3.If(r10s27, 10, 0) , z3.If(r10s28, 60, 0) , z3.If(r10s29, 10, 0) , z3.If(r10s30, 60, 0) , z3.If(r10s31, 60, 0) , z3.If(r10s32, 60, 0) , z3.If(r10s33, 10, 0) , z3.If(r10s34, 10, 0) , z3.If(r10s35, 60, 0) , z3.If(r10s36, 10, 0) , z3.If(r10s37, 60, 0) , z3.If(r10s38, 10, 0) , z3.If(r10s39, 10, 0) , z3.If(r10s40, 60, 0) , z3.If(r10s41, 10, 0) , z3.If(r10s42, 10, 0) , z3.If(r10s43, 10, 0) , z3.If(r10s44, 10, 0) , z3.If(r10s45, 10, 0) , z3.If(r10s46, 10, 0) , z3.If(r10s47, 10, 0) , z3.If(r10s49, 10, 0) , z3.If(r10s50, 10, 0) , z3.If(r10s51, 10, 0) , z3.If(r11s0, 10, 0) , z3.If(r11s1, 10, 0) , z3.If(r11s2, 10, 0) , z3.If(r11s3, 60, 0) , z3.If(r11s4, 60, 0) , z3.If(r11s5, 30, 0) , z3.If(r11s6, 10, 0) , z3.If(r11s7, 10, 0) , z3.If(r11s8, 10, 0) , z3.If(r11s9, 60, 0) , z3.If(r11s10, 10, 0) , z3.If(r11s11, 60, 0) , z3.If(r11s12, 10, 0) , z3.If(r11s13, 60, 0) , z3.If(r11s14, 10, 0) , z3.If(r11s15, 10, 0) , z3.If(r11s16, 10, 0) , z3.If(r11s17, 10, 0) , z3.If(r11s18, 10, 0) , z3.If(r11s19, 10, 0) , z3.If(r11s20, 10, 0) , z3.If(r11s21, 10, 0) , z3.If(r11s22, 10, 0) , z3.If(r11s23, 10, 0) , z3.If(r11s24, 10, 0) , z3.If(r11s25, 60, 0) , z3.If(r11s26, 10, 0) , z3.If(r11s27, 10, 0) , z3.If(r11s28, 10, 0) , z3.If(r11s29, 10, 0) , z3.If(r11s30, 10, 0) , z3.If(r11s31, 10, 0) , z3.If(r11s32, 10, 0) , z3.If(r11s33, 10, 0) , z3.If(r11s34, 30, 0) , z3.If(r11s35, 10, 0) , z3.If(r11s36, 10, 0) , z3.If(r11s37, 10, 0) , z3.If(r11s38, 10, 0) , z3.If(r11s39, 10, 0) , z3.If(r11s40, 10, 0) , z3.If(r11s41, 10, 0) , z3.If(r11s43, 10, 0) , z3.If(r11s44, 10, 0) , z3.If(r11s45, 10, 0) , z3.If(r11s46, 10, 0) , z3.If(r11s48, 10, 0) , z3.If(r11s50, 10, 0) , z3.If(r11s51, 10, 0) , z3.If(r12s1, 10, 0) , z3.If(r12s2, 10, 0) , z3.If(r12s3, 10, 0) , z3.If(r12s4, 60, 0) , z3.If(r12s5, 60, 0) , z3.If(r12s6, 10, 0) , z3.If(r12s7, 10, 0) , z3.If(r12s8, 30, 0) , z3.If(r12s9, 10, 0) , z3.If(r12s10, 10, 0) , z3.If(r12s12, 10, 0) , z3.If(r12s13, 30, 0) , z3.If(r12s14, 10, 0) , z3.If(r12s15, 10, 0) , z3.If(r12s16, 30, 0) , z3.If(r12s19, 10, 0) , z3.If(r12s20, 10, 0) , z3.If(r12s21, 10, 0) , z3.If(r12s22, 10, 0) , z3.If(r12s23, 10, 0) , z3.If(r12s24, 60, 0) , z3.If(r12s25, 10, 0) , z3.If(r12s26, 10, 0) , z3.If(r12s27, 30, 0) , z3.If(r12s28, 10, 0) , z3.If(r12s29, 10, 0) , z3.If(r12s30, 10, 0) , z3.If(r12s31, 60, 0) , z3.If(r12s32, 60, 0) , z3.If(r12s33, 10, 0) , z3.If(r12s34, 60, 0) , z3.If(r12s35, 10, 0) , z3.If(r12s36, 10, 0) , z3.If(r12s37, 10, 0) , z3.If(r12s38, 60, 0) , z3.If(r12s39, 10, 0) , z3.If(r12s40, 30, 0) , z3.If(r12s41, 60, 0) , z3.If(r12s42, 10, 0) , z3.If(r12s43, 30, 0) , z3.If(r12s44, 30, 0) , z3.If(r12s45, 30, 0) , z3.If(r12s46, 10, 0) , z3.If(r12s48, 10, 0) , z3.If(r12s49, 10, 0) , z3.If(r12s50, 10, 0) , z3.If(r12s51, 10, 0) , z3.If(r13s0, 60, 0) , z3.If(r13s2, 30, 0) , z3.If(r13s3, 10, 0) , z3.If(r13s4, 10, 0) , z3.If(r13s5, 10, 0) , z3.If(r13s6, 10, 0) , z3.If(r13s7, 10, 0) , z3.If(r13s10, 10, 0) , z3.If(r13s12, 10, 0) , z3.If(r13s13, 30, 0) , z3.If(r13s14, 60, 0) , z3.If(r13s15, 10, 0) , z3.If(r13s16, 10, 0) , z3.If(r13s18, 10, 0) , z3.If(r13s19, 60, 0) , z3.If(r13s20, 10, 0) , z3.If(r13s22, 10, 0) , z3.If(r13s23, 10, 0) , z3.If(r13s25, 10, 0) , z3.If(r13s26, 10, 0) , z3.If(r13s27, 10, 0) , z3.If(r13s28, 10, 0) , z3.If(r13s29, 10, 0) , z3.If(r13s30, 10, 0) , z3.If(r13s31, 10, 0) , z3.If(r13s32, 10, 0) , z3.If(r13s33, 10, 0) , z3.If(r13s34, 10, 0) , z3.If(r13s35, 30, 0) , z3.If(r13s37, 30, 0) , z3.If(r13s38, 60, 0) , z3.If(r13s39, 10, 0) , z3.If(r13s40, 30, 0) , z3.If(r13s41, 10, 0) , z3.If(r13s42, 10, 0) , z3.If(r13s43, 60, 0) , z3.If(r13s44, 10, 0) , z3.If(r13s45, 10, 0) , z3.If(r13s46, 30, 0) , z3.If(r13s47, 30, 0) , z3.If(r13s49, 30, 0) , z3.If(r13s50, 10, 0) , z3.If(r13s51, 30, 0) , z3.If(r14s0, 10, 0) , z3.If(r14s1, 10, 0) , z3.If(r14s2, 10, 0) , z3.If(r14s4, 10, 0) , z3.If(r14s5, 10, 0) , z3.If(r14s7, 60, 0) , z3.If(r14s8, 60, 0) , z3.If(r14s9, 10, 0) , z3.If(r14s10, 10, 0) , z3.If(r14s11, 10, 0) , z3.If(r14s12, 10, 0) , z3.If(r14s13, 60, 0) , z3.If(r14s14, 10, 0) , z3.If(r14s15, 10, 0) , z3.If(r14s17, 30, 0) , z3.If(r14s18, 10, 0) , z3.If(r14s19, 30, 0) , z3.If(r14s20, 10, 0) , z3.If(r14s21, 10, 0) , z3.If(r14s22, 10, 0) , z3.If(r14s23, 10, 0) , z3.If(r14s24, 60, 0) , z3.If(r14s25, 30, 0) , z3.If(r14s26, 10, 0) , z3.If(r14s27, 10, 0) , z3.If(r14s28, 10, 0) , z3.If(r14s29, 10, 0) , z3.If(r14s30, 10, 0) , z3.If(r14s34, 60, 0) , z3.If(r14s36, 10, 0) , z3.If(r14s38, 10, 0) , z3.If(r14s39, 10, 0) , z3.If(r14s40, 10, 0) , z3.If(r14s41, 10, 0) , z3.If(r14s42, 60, 0) , z3.If(r14s43, 60, 0) , z3.If(r14s44, 60, 0) , z3.If(r14s45, 10, 0) , z3.If(r14s46, 10, 0) , z3.If(r14s47, 30, 0) , z3.If(r14s48, 60, 0) , z3.If(r14s49, 60, 0) , z3.If(r14s50, 10, 0) , z3.If(r14s51, 10, 0) , z3.If(r15s1, 10, 0) , z3.If(r15s2, 10, 0) , z3.If(r15s3, 10, 0) , z3.If(r15s4, 10, 0) , z3.If(r15s5, 10, 0) , z3.If(r15s6, 10, 0) , z3.If(r15s7, 10, 0) , z3.If(r15s8, 10, 0) , z3.If(r15s9, 60, 0) , z3.If(r15s10, 10, 0) , z3.If(r15s11, 10, 0) , z3.If(r15s12, 60, 0) , z3.If(r15s13, 60, 0) , z3.If(r15s14, 10, 0) , z3.If(r15s15, 10, 0) , z3.If(r15s16, 10, 0) , z3.If(r15s17, 10, 0) , z3.If(r15s19, 10, 0) , z3.If(r15s20, 10, 0) , z3.If(r15s21, 10, 0) , z3.If(r15s22, 60, 0) , z3.If(r15s23, 60, 0) , z3.If(r15s24, 60, 0) , z3.If(r15s25, 10, 0) , z3.If(r15s26, 10, 0) , z3.If(r15s27, 10, 0) , z3.If(r15s28, 10, 0) , z3.If(r15s29, 10, 0) , z3.If(r15s30, 10, 0) , z3.If(r15s31, 10, 0) , z3.If(r15s32, 10, 0) , z3.If(r15s33, 10, 0) , z3.If(r15s34, 60, 0) , z3.If(r15s35, 10, 0) , z3.If(r15s36, 10, 0) , z3.If(r15s37, 60, 0) , z3.If(r15s38, 10, 0) , z3.If(r15s39, 10, 0) , z3.If(r15s40, 10, 0) , z3.If(r15s41, 10, 0) , z3.If(r15s42, 10, 0) , z3.If(r15s43, 10, 0) , z3.If(r15s44, 10, 0) , z3.If(r15s45, 10, 0) , z3.If(r15s46, 10, 0) , z3.If(r15s48, 10, 0) , z3.If(r15s49, 10, 0) , z3.If(r15s50, 10, 0) , z3.If(r15s51, 60, 0) , z3.If(r16s1, 10, 0) , z3.If(r16s2, 30, 0) , z3.If(r16s3, 30, 0) , z3.If(r16s4, 60, 0) , z3.If(r16s5, 30, 0) , z3.If(r16s7, 60, 0) , z3.If(r16s8, 10, 0) , z3.If(r16s9, 10, 0) , z3.If(r16s11, 10, 0) , z3.If(r16s12, 30, 0) , z3.If(r16s13, 10, 0) , z3.If(r16s14, 10, 0) , z3.If(r16s15, 10, 0) , z3.If(r16s16, 10, 0) , z3.If(r16s17, 10, 0) , z3.If(r16s19, 10, 0) , z3.If(r16s20, 10, 0) , z3.If(r16s21, 10, 0) , z3.If(r16s22, 10, 0) , z3.If(r16s23, 10, 0) , z3.If(r16s24, 10, 0) , z3.If(r16s25, 10, 0) , z3.If(r16s26, 10, 0) , z3.If(r16s27, 10, 0) , z3.If(r16s28, 10, 0) , z3.If(r16s29, 10, 0) , z3.If(r16s30, 10, 0) , z3.If(r16s31, 60, 0) , z3.If(r16s32, 10, 0) , z3.If(r16s33, 10, 0) , z3.If(r16s34, 60, 0) , z3.If(r16s35, 10, 0) , z3.If(r16s36, 10, 0) , z3.If(r16s38, 10, 0) , z3.If(r16s39, 30, 0) , z3.If(r16s40, 10, 0) , z3.If(r16s41, 10, 0) , z3.If(r16s42, 10, 0) , z3.If(r16s43, 10, 0) , z3.If(r16s44, 30, 0) , z3.If(r16s45, 10, 0) , z3.If(r16s46, 30, 0) , z3.If(r16s48, 10, 0) , z3.If(r16s49, 10, 0) , z3.If(r16s50, 10, 0) , z3.If(r16s51, 60, 0) , z3.If(r17s0, 10, 0) , z3.If(r17s1, 10, 0) , z3.If(r17s2, 60, 0) , z3.If(r17s3, 60, 0) , z3.If(r17s4, 10, 0) , z3.If(r17s5, 10, 0) , z3.If(r17s7, 60, 0) , z3.If(r17s8, 10, 0) , z3.If(r17s11, 10, 0) , z3.If(r17s12, 10, 0) , z3.If(r17s13, 60, 0) , z3.If(r17s14, 10, 0) , z3.If(r17s15, 10, 0) , z3.If(r17s16, 30, 0) , z3.If(r17s17, 30, 0) , z3.If(r17s18, 60, 0) , z3.If(r17s19, 30, 0) , z3.If(r17s20, 10, 0) , z3.If(r17s21, 10, 0) , z3.If(r17s22, 10, 0) , z3.If(r17s23, 10, 0) , z3.If(r17s25, 10, 0) , z3.If(r17s26, 10, 0) , z3.If(r17s27, 30, 0) , z3.If(r17s28, 10, 0) , z3.If(r17s29, 30, 0) , z3.If(r17s30, 30, 0) , z3.If(r17s31, 10, 0) , z3.If(r17s32, 60, 0) , z3.If(r17s33, 10, 0) , z3.If(r17s34, 30, 0) , z3.If(r17s35, 60, 0) , z3.If(r17s36, 10, 0) , z3.If(r17s37, 30, 0) , z3.If(r17s38, 30, 0) , z3.If(r17s39, 10, 0) , z3.If(r17s40, 10, 0) , z3.If(r17s41, 30, 0) , z3.If(r17s42, 10, 0) , z3.If(r17s43, 30, 0) , z3.If(r17s44, 10, 0) , z3.If(r17s45, 10, 0) , z3.If(r17s46, 10, 0) , z3.If(r17s47, 60, 0) , z3.If(r17s48, 10, 0) , z3.If(r17s49, 10, 0) , z3.If(r17s50, 10, 0) , z3.If(r17s51, 10, 0) , z3.If(r18s0, 60, 0) , z3.If(r18s1, 10, 0) , z3.If(r18s2, 10, 0) , z3.If(r18s3, 10, 0) , z3.If(r18s4, 60, 0) , z3.If(r18s5, 10, 0) , z3.If(r18s6, 10, 0) , z3.If(r18s7, 10, 0) , z3.If(r18s8, 10, 0) , z3.If(r18s10, 10, 0) , z3.If(r18s11, 10, 0) , z3.If(r18s12, 60, 0) , z3.If(r18s13, 10, 0) , z3.If(r18s14, 10, 0) , z3.If(r18s15, 60, 0) , z3.If(r18s16, 10, 0) , z3.If(r18s17, 10, 0) , z3.If(r18s18, 60, 0) , z3.If(r18s19, 10, 0) , z3.If(r18s20, 10, 0) , z3.If(r18s21, 10, 0) , z3.If(r18s22, 10, 0) , z3.If(r18s23, 60, 0) , z3.If(r18s25, 10, 0) , z3.If(r18s26, 10, 0) , z3.If(r18s27, 10, 0) , z3.If(r18s28, 10, 0) , z3.If(r18s29, 10, 0) , z3.If(r18s30, 10, 0) , z3.If(r18s31, 10, 0) , z3.If(r18s32, 10, 0) , z3.If(r18s33, 60, 0) , z3.If(r18s34, 10, 0) , z3.If(r18s35, 10, 0) , z3.If(r18s36, 10, 0) , z3.If(r18s37, 10, 0) , z3.If(r18s38, 10, 0) , z3.If(r18s39, 10, 0) , z3.If(r18s40, 10, 0) , z3.If(r18s41, 10, 0) , z3.If(r18s42, 10, 0) , z3.If(r18s43, 10, 0) , z3.If(r18s44, 10, 0) , z3.If(r18s45, 60, 0) , z3.If(r18s46, 10, 0) , z3.If(r18s47, 10, 0) , z3.If(r18s48, 60, 0) , z3.If(r18s49, 10, 0) , z3.If(r18s50, 10, 0) , z3.If(r18s51, 60, 0) , z3.If(r19s0, 30, 0) , z3.If(r19s1, 60, 0) , z3.If(r19s3, 30, 0) , z3.If(r19s4, 60, 0) , z3.If(r19s5, 10, 0) , z3.If(r19s6, 30, 0) , z3.If(r19s7, 30, 0) , z3.If(r19s8, 10, 0) , z3.If(r19s9, 30, 0) , z3.If(r19s10, 10, 0) , z3.If(r19s11, 10, 0) , z3.If(r19s12, 10, 0) , z3.If(r19s13, 10, 0) , z3.If(r19s14, 10, 0) , z3.If(r19s15, 60, 0) , z3.If(r19s16, 60, 0) , z3.If(r19s17, 60, 0) , z3.If(r19s18, 60, 0) , z3.If(r19s19, 30, 0) , z3.If(r19s20, 10, 0) , z3.If(r19s21, 10, 0) , z3.If(r19s22, 60, 0) , z3.If(r19s23, 10, 0) , z3.If(r19s24, 30, 0) , z3.If(r19s25, 10, 0) , z3.If(r19s26, 10, 0) , z3.If(r19s27, 10, 0) , z3.If(r19s28, 10, 0) , z3.If(r19s29, 10, 0) , z3.If(r19s30, 10, 0) , z3.If(r19s31, 30, 0) , z3.If(r19s32, 10, 0) , z3.If(r19s33, 10, 0) , z3.If(r19s34, 10, 0) , z3.If(r19s35, 10, 0) , z3.If(r19s36, 10, 0) , z3.If(r19s37, 60, 0) , z3.If(r19s38, 10, 0) , z3.If(r19s39, 10, 0) , z3.If(r19s40, 10, 0) , z3.If(r19s41, 10, 0) , z3.If(r19s42, 60, 0) , z3.If(r19s43, 10, 0) , z3.If(r19s44, 60, 0) , z3.If(r19s45, 10, 0) , z3.If(r19s46, 10, 0) , z3.If(r19s47, 10, 0) , z3.If(r19s48, 10, 0) , z3.If(r19s49, 60, 0) , z3.If(r19s50, 10, 0) , z3.If(r19s51, 60, 0) , z3.If(r20s0, 10, 0) , z3.If(r20s1, 10, 0) , z3.If(r20s2, 10, 0) , z3.If(r20s3, 10, 0) , z3.If(r20s4, 10, 0) , z3.If(r20s5, 10, 0) , z3.If(r20s6, 60, 0) , z3.If(r20s7, 10, 0) , z3.If(r20s8, 10, 0) , z3.If(r20s9, 60, 0) , z3.If(r20s10, 10, 0) , z3.If(r20s11, 10, 0) , z3.If(r20s12, 10, 0) , z3.If(r20s13, 60, 0) , z3.If(r20s15, 10, 0) , z3.If(r20s17, 60, 0) , z3.If(r20s18, 10, 0) , z3.If(r20s19, 10, 0) , z3.If(r20s20, 10, 0) , z3.If(r20s21, 10, 0) , z3.If(r20s22, 10, 0) , z3.If(r20s23, 10, 0) , z3.If(r20s24, 60, 0) , z3.If(r20s25, 10, 0) , z3.If(r20s26, 60, 0) , z3.If(r20s30, 60, 0) , z3.If(r20s31, 10, 0) , z3.If(r20s32, 60, 0) , z3.If(r20s33, 10, 0) , z3.If(r20s34, 60, 0) , z3.If(r20s35, 10, 0) , z3.If(r20s36, 60, 0) , z3.If(r20s37, 60, 0) , z3.If(r20s38, 10, 0) , z3.If(r20s39, 10, 0) , z3.If(r20s40, 10, 0) , z3.If(r20s41, 10, 0) , z3.If(r20s42, 10, 0) , z3.If(r20s43, 10, 0) , z3.If(r20s44, 10, 0) , z3.If(r20s45, 60, 0) , z3.If(r20s46, 10, 0) , z3.If(r20s47, 60, 0) , z3.If(r20s48, 10, 0) , z3.If(r20s49, 10, 0) , z3.If(r20s50, 10, 0) , z3.If(r20s51, 10, 0) , z3.If(r21s1, 10, 0) , z3.If(r21s2, 60, 0) , z3.If(r21s3, 10, 0) , z3.If(r21s4, 10, 0) , z3.If(r21s5, 60, 0) , z3.If(r21s6, 10, 0) , z3.If(r21s7, 10, 0) , z3.If(r21s8, 10, 0) , z3.If(r21s10, 10, 0) , z3.If(r21s12, 60, 0) , z3.If(r21s14, 30, 0) , z3.If(r21s15, 60, 0) , z3.If(r21s18, 60, 0) , z3.If(r21s19, 10, 0) , z3.If(r21s20, 10, 0) , z3.If(r21s22, 10, 0) , z3.If(r21s23, 30, 0) , z3.If(r21s24, 10, 0) , z3.If(r21s25, 10, 0) , z3.If(r21s26, 10, 0) , z3.If(r21s27, 10, 0) , z3.If(r21s28, 10, 0) , z3.If(r21s29, 10, 0) , z3.If(r21s30, 10, 0) , z3.If(r21s31, 10, 0) , z3.If(r21s33, 10, 0) , z3.If(r21s34, 10, 0) , z3.If(r21s35, 60, 0) , z3.If(r21s37, 30, 0) , z3.If(r21s38, 30, 0) , z3.If(r21s39, 10, 0) , z3.If(r21s40, 10, 0) , z3.If(r21s41, 10, 0) , z3.If(r21s42, 10, 0) , z3.If(r21s43, 30, 0) , z3.If(r21s44, 10, 0) , z3.If(r21s45, 10, 0) , z3.If(r21s46, 10, 0) , z3.If(r21s47, 10, 0) , z3.If(r21s48, 10, 0) , z3.If(r21s49, 10, 0) , z3.If(r21s50, 30, 0) , z3.If(r21s51, 10, 0) , z3.If(r22s0, 10, 0) , z3.If(r22s1, 10, 0) , z3.If(r22s2, 10, 0) , z3.If(r22s3, 10, 0) , z3.If(r22s4, 60, 0) , z3.If(r22s5, 60, 0) , z3.If(r22s6, 10, 0) , z3.If(r22s7, 10, 0) , z3.If(r22s8, 10, 0) , z3.If(r22s9, 30, 0) , z3.If(r22s10, 30, 0) , z3.If(r22s11, 10, 0) , z3.If(r22s12, 10, 0) , z3.If(r22s13, 30, 0) , z3.If(r22s14, 10, 0) , z3.If(r22s15, 10, 0) , z3.If(r22s16, 60, 0) , z3.If(r22s17, 10, 0) , z3.If(r22s18, 10, 0) , z3.If(r22s19, 10, 0) , z3.If(r22s20, 30, 0) , z3.If(r22s21, 10, 0) , z3.If(r22s22, 10, 0) , z3.If(r22s23, 10, 0) , z3.If(r22s24, 10, 0) , z3.If(r22s25, 10, 0) , z3.If(r22s26, 10, 0) , z3.If(r22s27, 10, 0) , z3.If(r22s28, 10, 0) , z3.If(r22s29, 10, 0) , z3.If(r22s30, 10, 0) , z3.If(r22s31, 10, 0) , z3.If(r22s32, 60, 0) , z3.If(r22s33, 30, 0) , z3.If(r22s34, 30, 0) , z3.If(r22s35, 10, 0) , z3.If(r22s36, 10, 0) , z3.If(r22s37, 10, 0) , z3.If(r22s38, 10, 0) , z3.If(r22s39, 60, 0) , z3.If(r22s40, 10, 0) , z3.If(r22s41, 10, 0) , z3.If(r22s42, 30, 0) , z3.If(r22s43, 10, 0) , z3.If(r22s44, 10, 0) , z3.If(r22s45, 60, 0) , z3.If(r22s46, 10, 0) , z3.If(r22s47, 10, 0) , z3.If(r22s48, 10, 0) , z3.If(r22s49, 10, 0) , z3.If(r22s50, 10, 0) , z3.If(r22s51, 10, 0) , z3.If(r23s0, 10, 0) , z3.If(r23s1, 10, 0) , z3.If(r23s2, 10, 0) , z3.If(r23s3, 60, 0) , z3.If(r23s4, 60, 0) , z3.If(r23s5, 10, 0) , z3.If(r23s6, 30, 0) , z3.If(r23s7, 10, 0) , z3.If(r23s8, 30, 0) , z3.If(r23s10, 60, 0) , z3.If(r23s11, 10, 0) , z3.If(r23s12, 10, 0) , z3.If(r23s13, 10, 0) , z3.If(r23s14, 10, 0) , z3.If(r23s15, 10, 0) , z3.If(r23s16, 60, 0) , z3.If(r23s17, 10, 0) , z3.If(r23s18, 10, 0) , z3.If(r23s19, 10, 0) , z3.If(r23s20, 10, 0) , z3.If(r23s21, 10, 0) , z3.If(r23s22, 10, 0) , z3.If(r23s23, 10, 0) , z3.If(r23s25, 10, 0) , z3.If(r23s26, 10, 0) , z3.If(r23s27, 10, 0) , z3.If(r23s28, 10, 0) , z3.If(r23s29, 10, 0) , z3.If(r23s30, 10, 0) , z3.If(r23s31, 30, 0) , z3.If(r23s32, 60, 0) , z3.If(r23s33, 10, 0) , z3.If(r23s34, 30, 0) , z3.If(r23s35, 10, 0) , z3.If(r23s36, 10, 0) , z3.If(r23s37, 10, 0) , z3.If(r23s38, 10, 0) , z3.If(r23s39, 60, 0) , z3.If(r23s40, 10, 0) , z3.If(r23s41, 10, 0) , z3.If(r23s42, 60, 0) , z3.If(r23s43, 10, 0) , z3.If(r23s44, 10, 0) , z3.If(r23s45, 60, 0) , z3.If(r23s46, 10, 0) , z3.If(r23s47, 10, 0) , z3.If(r23s48, 30, 0) , z3.If(r23s49, 10, 0) , z3.If(r23s50, 10, 0) , z3.If(r23s51, 10, 0)]
# 95/4/1
# scores = [z3.If(r0s0, 95, 0) , z3.If(r0s1, 1, 0) , z3.If(r0s2, 1, 0) , z3.If(r0s4, 1, 0) , z3.If(r0s5, 1, 0) , z3.If(r0s7, 1, 0) , z3.If(r0s8, 1, 0) , z3.If(r0s9, 1, 0) , z3.If(r0s10, 1, 0) , z3.If(r0s12, 1, 0) , z3.If(r0s13, 1, 0) , z3.If(r0s14, 4, 0) , z3.If(r0s15, 1, 0) , z3.If(r0s16, 1, 0) , z3.If(r0s18, 4, 0) , z3.If(r0s19, 1, 0) , z3.If(r0s20, 1, 0) , z3.If(r0s21, 1, 0) , z3.If(r0s22, 1, 0) , z3.If(r0s23, 95, 0) , z3.If(r0s24, 95, 0) , z3.If(r0s25, 1, 0) , z3.If(r0s26, 4, 0) , z3.If(r0s27, 1, 0) , z3.If(r0s28, 95, 0) , z3.If(r0s29, 95, 0) , z3.If(r0s30, 95, 0) , z3.If(r0s31, 1, 0) , z3.If(r0s32, 95, 0) , z3.If(r0s34, 1, 0) , z3.If(r0s35, 1, 0) , z3.If(r0s36, 1, 0) , z3.If(r0s37, 95, 0) , z3.If(r0s38, 1, 0) , z3.If(r0s39, 1, 0) , z3.If(r0s40, 4, 0) , z3.If(r0s41, 1, 0) , z3.If(r0s42, 1, 0) , z3.If(r0s43, 1, 0) , z3.If(r0s44, 95, 0) , z3.If(r0s45, 1, 0) , z3.If(r0s46, 1, 0) , z3.If(r0s47, 1, 0) , z3.If(r0s48, 1, 0) , z3.If(r0s49, 1, 0) , z3.If(r0s50, 1, 0) , z3.If(r0s51, 1, 0) , z3.If(r1s0, 95, 0) , z3.If(r1s1, 1, 0) , z3.If(r1s2, 4, 0) , z3.If(r1s3, 95, 0) , z3.If(r1s4, 1, 0) , z3.If(r1s5, 1, 0) , z3.If(r1s6, 1, 0) , z3.If(r1s7, 1, 0) , z3.If(r1s8, 1, 0) , z3.If(r1s9, 95, 0) , z3.If(r1s10, 4, 0) , z3.If(r1s11, 95, 0) , z3.If(r1s12, 1, 0) , z3.If(r1s13, 1, 0) , z3.If(r1s14, 1, 0) , z3.If(r1s15, 1, 0) , z3.If(r1s16, 1, 0) , z3.If(r1s17, 4, 0) , z3.If(r1s18, 95, 0) , z3.If(r1s19, 1, 0) , z3.If(r1s20, 1, 0) , z3.If(r1s21, 1, 0) , z3.If(r1s22, 95, 0) , z3.If(r1s23, 1, 0) , z3.If(r1s24, 1, 0) , z3.If(r1s25, 1, 0) , z3.If(r1s26, 1, 0) , z3.If(r1s27, 1, 0) , z3.If(r1s28, 95, 0) , z3.If(r1s29, 4, 0) , z3.If(r1s30, 1, 0) , z3.If(r1s31, 1, 0) , z3.If(r1s32, 1, 0) , z3.If(r1s33, 1, 0) , z3.If(r1s35, 1, 0) , z3.If(r1s36, 4, 0) , z3.If(r1s37, 1, 0) , z3.If(r1s38, 1, 0) , z3.If(r1s39, 1, 0) , z3.If(r1s40, 95, 0) , z3.If(r1s41, 1, 0) , z3.If(r1s42, 1, 0) , z3.If(r1s43, 1, 0) , z3.If(r1s44, 1, 0) , z3.If(r1s45, 1, 0) , z3.If(r1s46, 1, 0) , z3.If(r1s47, 1, 0) , z3.If(r1s48, 1, 0) , z3.If(r1s49, 1, 0) , z3.If(r1s50, 1, 0) , z3.If(r1s51, 1, 0) , z3.If(r2s0, 1, 0) , z3.If(r2s1, 1, 0) , z3.If(r2s2, 95, 0) , z3.If(r2s3, 1, 0) , z3.If(r2s4, 1, 0) , z3.If(r2s5, 1, 0) , z3.If(r2s6, 1, 0) , z3.If(r2s7, 4, 0) , z3.If(r2s8, 1, 0) , z3.If(r2s9, 1, 0) , z3.If(r2s10, 1, 0) , z3.If(r2s11, 1, 0) , z3.If(r2s12, 95, 0) , z3.If(r2s14, 4, 0) , z3.If(r2s15, 95, 0) , z3.If(r2s18, 1, 0) , z3.If(r2s19, 1, 0) , z3.If(r2s20, 1, 0) , z3.If(r2s22, 1, 0) , z3.If(r2s23, 1, 0) , z3.If(r2s24, 1, 0) , z3.If(r2s25, 1, 0) , z3.If(r2s26, 1, 0) , z3.If(r2s27, 1, 0) , z3.If(r2s28, 1, 0) , z3.If(r2s29, 1, 0) , z3.If(r2s30, 1, 0) , z3.If(r2s31, 1, 0) , z3.If(r2s33, 1, 0) , z3.If(r2s34, 1, 0) , z3.If(r2s35, 1, 0) , z3.If(r2s36, 1, 0) , z3.If(r2s37, 4, 0) , z3.If(r2s38, 1, 0) , z3.If(r2s39, 1, 0) , z3.If(r2s40, 1, 0) , z3.If(r2s41, 1, 0) , z3.If(r2s42, 1, 0) , z3.If(r2s43, 1, 0) , z3.If(r2s44, 1, 0) , z3.If(r2s45, 1, 0) , z3.If(r2s46, 4, 0) , z3.If(r2s47, 1, 0) , z3.If(r2s48, 1, 0) , z3.If(r2s49, 4, 0) , z3.If(r2s50, 1, 0) , z3.If(r2s51, 4, 0) , z3.If(r3s0, 1, 0) , z3.If(r3s2, 95, 0) , z3.If(r3s3, 95, 0) , z3.If(r3s4, 1, 0) , z3.If(r3s5, 1, 0) , z3.If(r3s6, 95, 0) , z3.If(r3s7, 1, 0) , z3.If(r3s8, 4, 0) , z3.If(r3s10, 1, 0) , z3.If(r3s11, 4, 0) , z3.If(r3s12, 1, 0) , z3.If(r3s15, 1, 0) , z3.If(r3s17, 4, 0) , z3.If(r3s18, 4, 0) , z3.If(r3s19, 1, 0) , z3.If(r3s20, 1, 0) , z3.If(r3s21, 1, 0) , z3.If(r3s22, 1, 0) , z3.If(r3s23, 1, 0) , z3.If(r3s25, 95, 0) , z3.If(r3s31, 4, 0) , z3.If(r3s32, 95, 0) , z3.If(r3s33, 1, 0) , z3.If(r3s34, 1, 0) , z3.If(r3s35, 95, 0) , z3.If(r3s36, 95, 0) , z3.If(r3s37, 95, 0) , z3.If(r3s38, 95, 0) , z3.If(r3s39, 1, 0) , z3.If(r3s40, 95, 0) , z3.If(r3s41, 4, 0) , z3.If(r3s42, 95, 0) , z3.If(r3s43, 4, 0) , z3.If(r3s44, 1, 0) , z3.If(r3s45, 1, 0) , z3.If(r3s46, 4, 0) , z3.If(r3s47, 95, 0) , z3.If(r3s48, 1, 0) , z3.If(r3s49, 4, 0) , z3.If(r3s50, 95, 0) , z3.If(r3s51, 4, 0) , z3.If(r4s0, 4, 0) , z3.If(r4s1, 1, 0) , z3.If(r4s2, 4, 0) , z3.If(r4s3, 1, 0) , z3.If(r4s4, 95, 0) , z3.If(r4s5, 95, 0) , z3.If(r4s6, 1, 0) , z3.If(r4s7, 1, 0) , z3.If(r4s8, 1, 0) , z3.If(r4s9, 1, 0) , z3.If(r4s10, 95, 0) , z3.If(r4s11, 1, 0) , z3.If(r4s12, 1, 0) , z3.If(r4s13, 1, 0) , z3.If(r4s14, 1, 0) , z3.If(r4s15, 1, 0) , z3.If(r4s16, 4, 0) , z3.If(r4s17, 1, 0) , z3.If(r4s18, 95, 0) , z3.If(r4s19, 1, 0) , z3.If(r4s20, 1, 0) , z3.If(r4s21, 1, 0) , z3.If(r4s22, 4, 0) , z3.If(r4s23, 1, 0) , z3.If(r4s24, 1, 0) , z3.If(r4s25, 1, 0) , z3.If(r4s26, 1, 0) , z3.If(r4s27, 1, 0) , z3.If(r4s28, 1, 0) , z3.If(r4s29, 1, 0) , z3.If(r4s30, 1, 0) , z3.If(r4s31, 1, 0) , z3.If(r4s32, 4, 0) , z3.If(r4s33, 1, 0) , z3.If(r4s34, 1, 0) , z3.If(r4s35, 1, 0) , z3.If(r4s36, 1, 0) , z3.If(r4s37, 1, 0) , z3.If(r4s38, 1, 0) , z3.If(r4s39, 1, 0) , z3.If(r4s40, 1, 0) , z3.If(r4s41, 1, 0) , z3.If(r4s42, 1, 0) , z3.If(r4s43, 1, 0) , z3.If(r4s44, 1, 0) , z3.If(r4s45, 1, 0) , z3.If(r4s46, 1, 0) , z3.If(r4s47, 4, 0) , z3.If(r4s48, 1, 0) , z3.If(r4s49, 1, 0) , z3.If(r4s50, 1, 0) , z3.If(r4s51, 95, 0) , z3.If(r5s0, 95, 0) , z3.If(r5s1, 1, 0) , z3.If(r5s2, 4, 0) , z3.If(r5s3, 4, 0) , z3.If(r5s4, 4, 0) , z3.If(r5s5, 1, 0) , z3.If(r5s6, 1, 0) , z3.If(r5s7, 1, 0) , z3.If(r5s8, 95, 0) , z3.If(r5s9, 95, 0) , z3.If(r5s10, 1, 0) , z3.If(r5s11, 1, 0) , z3.If(r5s12, 1, 0) , z3.If(r5s13, 1, 0) , z3.If(r5s14, 1, 0) , z3.If(r5s15, 1, 0) , z3.If(r5s16, 1, 0) , z3.If(r5s17, 95, 0) , z3.If(r5s18, 1, 0) , z3.If(r5s19, 4, 0) , z3.If(r5s20, 1, 0) , z3.If(r5s21, 1, 0) , z3.If(r5s22, 4, 0) , z3.If(r5s23, 1, 0) , z3.If(r5s24, 95, 0) , z3.If(r5s25, 1, 0) , z3.If(r5s26, 1, 0) , z3.If(r5s27, 1, 0) , z3.If(r5s28, 4, 0) , z3.If(r5s29, 1, 0) , z3.If(r5s30, 1, 0) , z3.If(r5s31, 1, 0) , z3.If(r5s32, 1, 0) , z3.If(r5s33, 1, 0) , z3.If(r5s34, 4, 0) , z3.If(r5s35, 1, 0) , z3.If(r5s36, 1, 0) , z3.If(r5s37, 95, 0) , z3.If(r5s38, 1, 0) , z3.If(r5s39, 4, 0) , z3.If(r5s40, 1, 0) , z3.If(r5s41, 1, 0) , z3.If(r5s42, 1, 0) , z3.If(r5s43, 1, 0) , z3.If(r5s44, 1, 0) , z3.If(r5s45, 1, 0) , z3.If(r5s46, 1, 0) , z3.If(r5s47, 1, 0) , z3.If(r5s48, 95, 0) , z3.If(r5s49, 4, 0) , z3.If(r5s50, 1, 0) , z3.If(r5s51, 1, 0) , z3.If(r6s0, 4, 0) , z3.If(r6s1, 1, 0) , z3.If(r6s2, 4, 0) , z3.If(r6s3, 4, 0) , z3.If(r6s4, 4, 0) , z3.If(r6s5, 1, 0) , z3.If(r6s7, 1, 0) , z3.If(r6s8, 1, 0) , z3.If(r6s9, 4, 0) , z3.If(r6s11, 1, 0) , z3.If(r6s12, 1, 0) , z3.If(r6s13, 1, 0) , z3.If(r6s14, 1, 0) , z3.If(r6s15, 1, 0) , z3.If(r6s16, 1, 0) , z3.If(r6s17, 4, 0) , z3.If(r6s18, 95, 0) , z3.If(r6s19, 95, 0) , z3.If(r6s20, 1, 0) , z3.If(r6s21, 4, 0) , z3.If(r6s22, 1, 0) , z3.If(r6s23, 1, 0) , z3.If(r6s24, 4, 0) , z3.If(r6s25, 1, 0) , z3.If(r6s26, 1, 0) , z3.If(r6s27, 1, 0) , z3.If(r6s28, 95, 0) , z3.If(r6s29, 4, 0) , z3.If(r6s30, 95, 0) , z3.If(r6s31, 4, 0) , z3.If(r6s32, 1, 0) , z3.If(r6s33, 1, 0) , z3.If(r6s34, 1, 0) , z3.If(r6s36, 1, 0) , z3.If(r6s39, 1, 0) , z3.If(r6s40, 1, 0) , z3.If(r6s41, 1, 0) , z3.If(r6s42, 4, 0) , z3.If(r6s44, 1, 0) , z3.If(r6s45, 1, 0) , z3.If(r6s47, 95, 0) , z3.If(r6s48, 1, 0) , z3.If(r6s49, 95, 0) , z3.If(r6s50, 1, 0) , z3.If(r6s51, 4, 0) , z3.If(r7s0, 95, 0) , z3.If(r7s1, 1, 0) , z3.If(r7s2, 1, 0) , z3.If(r7s3, 4, 0) , z3.If(r7s4, 1, 0) , z3.If(r7s5, 1, 0) , z3.If(r7s6, 1, 0) , z3.If(r7s7, 4, 0) , z3.If(r7s10, 1, 0) , z3.If(r7s12, 1, 0) , z3.If(r7s13, 95, 0) , z3.If(r7s14, 1, 0) , z3.If(r7s15, 1, 0) , z3.If(r7s16, 1, 0) , z3.If(r7s17, 4, 0) , z3.If(r7s18, 4, 0) , z3.If(r7s19, 1, 0) , z3.If(r7s20, 1, 0) , z3.If(r7s22, 95, 0) , z3.If(r7s23, 95, 0) , z3.If(r7s24, 4, 0) , z3.If(r7s25, 1, 0) , z3.If(r7s26, 95, 0) , z3.If(r7s27, 1, 0) , z3.If(r7s28, 95, 0) , z3.If(r7s29, 95, 0) , z3.If(r7s30, 1, 0) , z3.If(r7s31, 95, 0) , z3.If(r7s32, 1, 0) , z3.If(r7s33, 1, 0) , z3.If(r7s34, 1, 0) , z3.If(r7s35, 1, 0) , z3.If(r7s36, 4, 0) , z3.If(r7s37, 95, 0) , z3.If(r7s38, 95, 0) , z3.If(r7s39, 1, 0) , z3.If(r7s40, 95, 0) , z3.If(r7s41, 1, 0) , z3.If(r7s42, 95, 0) , z3.If(r7s43, 95, 0) , z3.If(r7s44, 4, 0) , z3.If(r7s45, 1, 0) , z3.If(r7s46, 1, 0) , z3.If(r7s47, 95, 0) , z3.If(r7s49, 95, 0) , z3.If(r7s50, 1, 0) , z3.If(r7s51, 1, 0) , z3.If(r8s0, 1, 0) , z3.If(r8s1, 1, 0) , z3.If(r8s2, 1, 0) , z3.If(r8s3, 1, 0) , z3.If(r8s4, 1, 0) , z3.If(r8s5, 1, 0) , z3.If(r8s6, 1, 0) , z3.If(r8s7, 1, 0) , z3.If(r8s8, 1, 0) , z3.If(r8s9, 95, 0) , z3.If(r8s10, 1, 0) , z3.If(r8s11, 1, 0) , z3.If(r8s12, 1, 0) , z3.If(r8s14, 1, 0) , z3.If(r8s15, 1, 0) , z3.If(r8s18, 1, 0) , z3.If(r8s19, 1, 0) , z3.If(r8s20, 1, 0) , z3.If(r8s21, 1, 0) , z3.If(r8s22, 4, 0) , z3.If(r8s23, 4, 0) , z3.If(r8s24, 4, 0) , z3.If(r8s25, 4, 0) , z3.If(r8s26, 1, 0) , z3.If(r8s27, 1, 0) , z3.If(r8s28, 1, 0) , z3.If(r8s29, 1, 0) , z3.If(r8s30, 1, 0) , z3.If(r8s31, 4, 0) , z3.If(r8s33, 95, 0) , z3.If(r8s34, 95, 0) , z3.If(r8s35, 4, 0) , z3.If(r8s36, 1, 0) , z3.If(r8s37, 95, 0) , z3.If(r8s38, 95, 0) , z3.If(r8s39, 1, 0) , z3.If(r8s40, 4, 0) , z3.If(r8s41, 1, 0) , z3.If(r8s42, 4, 0) , z3.If(r8s43, 95, 0) , z3.If(r8s44, 1, 0) , z3.If(r8s45, 1, 0) , z3.If(r8s46, 1, 0) , z3.If(r8s47, 95, 0) , z3.If(r8s48, 1, 0) , z3.If(r8s49, 4, 0) , z3.If(r8s50, 4, 0) , z3.If(r8s51, 4, 0) , z3.If(r9s0, 95, 0) , z3.If(r9s1, 1, 0) , z3.If(r9s2, 1, 0) , z3.If(r9s3, 1, 0) , z3.If(r9s4, 1, 0) , z3.If(r9s5, 1, 0) , z3.If(r9s6, 1, 0) , z3.If(r9s7, 1, 0) , z3.If(r9s8, 1, 0) , z3.If(r9s9, 1, 0) , z3.If(r9s10, 1, 0) , z3.If(r9s11, 1, 0) , z3.If(r9s12, 1, 0) , z3.If(r9s13, 95, 0) , z3.If(r9s14, 95, 0) , z3.If(r9s15, 1, 0) , z3.If(r9s16, 1, 0) , z3.If(r9s17, 95, 0) , z3.If(r9s18, 1, 0) , z3.If(r9s19, 1, 0) , z3.If(r9s20, 1, 0) , z3.If(r9s21, 1, 0) , z3.If(r9s22, 1, 0) , z3.If(r9s23, 1, 0) , z3.If(r9s24, 95, 0) , z3.If(r9s25, 1, 0) , z3.If(r9s26, 4, 0) , z3.If(r9s27, 1, 0) , z3.If(r9s28, 95, 0) , z3.If(r9s29, 95, 0) , z3.If(r9s30, 1, 0) , z3.If(r9s31, 1, 0) , z3.If(r9s32, 95, 0) , z3.If(r9s33, 4, 0) , z3.If(r9s34, 1, 0) , z3.If(r9s35, 1, 0) , z3.If(r9s36, 95, 0) , z3.If(r9s37, 95, 0) , z3.If(r9s38, 1, 0) , z3.If(r9s39, 1, 0) , z3.If(r9s40, 95, 0) , z3.If(r9s41, 1, 0) , z3.If(r9s42, 1, 0) , z3.If(r9s43, 1, 0) , z3.If(r9s44, 95, 0) , z3.If(r9s45, 1, 0) , z3.If(r9s46, 95, 0) , z3.If(r9s47, 1, 0) , z3.If(r9s48, 1, 0) , z3.If(r9s49, 95, 0) , z3.If(r9s50, 1, 0) , z3.If(r9s51, 1, 0) , z3.If(r10s0, 1, 0) , z3.If(r10s1, 1, 0) , z3.If(r10s2, 1, 0) , z3.If(r10s3, 1, 0) , z3.If(r10s4, 1, 0) , z3.If(r10s5, 1, 0) , z3.If(r10s6, 1, 0) , z3.If(r10s7, 1, 0) , z3.If(r10s9, 95, 0) , z3.If(r10s10, 1, 0) , z3.If(r10s12, 95, 0) , z3.If(r10s13, 95, 0) , z3.If(r10s14, 1, 0) , z3.If(r10s15, 1, 0) , z3.If(r10s16, 1, 0) , z3.If(r10s17, 1, 0) , z3.If(r10s18, 95, 0) , z3.If(r10s19, 1, 0) , z3.If(r10s20, 1, 0) , z3.If(r10s21, 95, 0) , z3.If(r10s22, 1, 0) , z3.If(r10s23, 1, 0) , z3.If(r10s24, 1, 0) , z3.If(r10s25, 1, 0) , z3.If(r10s26, 1, 0) , z3.If(r10s27, 1, 0) , z3.If(r10s28, 95, 0) , z3.If(r10s29, 1, 0) , z3.If(r10s30, 95, 0) , z3.If(r10s31, 95, 0) , z3.If(r10s32, 95, 0) , z3.If(r10s33, 1, 0) , z3.If(r10s34, 1, 0) , z3.If(r10s35, 95, 0) , z3.If(r10s36, 1, 0) , z3.If(r10s37, 95, 0) , z3.If(r10s38, 1, 0) , z3.If(r10s39, 1, 0) , z3.If(r10s40, 95, 0) , z3.If(r10s41, 1, 0) , z3.If(r10s42, 1, 0) , z3.If(r10s43, 1, 0) , z3.If(r10s44, 1, 0) , z3.If(r10s45, 1, 0) , z3.If(r10s46, 1, 0) , z3.If(r10s47, 1, 0) , z3.If(r10s49, 1, 0) , z3.If(r10s50, 1, 0) , z3.If(r10s51, 1, 0) , z3.If(r11s0, 1, 0) , z3.If(r11s1, 1, 0) , z3.If(r11s2, 1, 0) , z3.If(r11s3, 95, 0) , z3.If(r11s4, 95, 0) , z3.If(r11s5, 4, 0) , z3.If(r11s6, 1, 0) , z3.If(r11s7, 1, 0) , z3.If(r11s8, 1, 0) , z3.If(r11s9, 95, 0) , z3.If(r11s10, 1, 0) , z3.If(r11s11, 95, 0) , z3.If(r11s12, 1, 0) , z3.If(r11s13, 95, 0) , z3.If(r11s14, 1, 0) , z3.If(r11s15, 1, 0) , z3.If(r11s16, 1, 0) , z3.If(r11s17, 1, 0) , z3.If(r11s18, 1, 0) , z3.If(r11s19, 1, 0) , z3.If(r11s20, 1, 0) , z3.If(r11s21, 1, 0) , z3.If(r11s22, 1, 0) , z3.If(r11s23, 1, 0) , z3.If(r11s24, 1, 0) , z3.If(r11s25, 95, 0) , z3.If(r11s26, 1, 0) , z3.If(r11s27, 1, 0) , z3.If(r11s28, 1, 0) , z3.If(r11s29, 1, 0) , z3.If(r11s30, 1, 0) , z3.If(r11s31, 1, 0) , z3.If(r11s32, 1, 0) , z3.If(r11s33, 1, 0) , z3.If(r11s34, 4, 0) , z3.If(r11s35, 1, 0) , z3.If(r11s36, 1, 0) , z3.If(r11s37, 1, 0) , z3.If(r11s38, 1, 0) , z3.If(r11s39, 1, 0) , z3.If(r11s40, 1, 0) , z3.If(r11s41, 1, 0) , z3.If(r11s43, 1, 0) , z3.If(r11s44, 1, 0) , z3.If(r11s45, 1, 0) , z3.If(r11s46, 1, 0) , z3.If(r11s48, 1, 0) , z3.If(r11s50, 1, 0) , z3.If(r11s51, 1, 0) , z3.If(r12s1, 1, 0) , z3.If(r12s2, 1, 0) , z3.If(r12s3, 1, 0) , z3.If(r12s4, 95, 0) , z3.If(r12s5, 95, 0) , z3.If(r12s6, 1, 0) , z3.If(r12s7, 1, 0) , z3.If(r12s8, 4, 0) , z3.If(r12s9, 1, 0) , z3.If(r12s10, 1, 0) , z3.If(r12s12, 1, 0) , z3.If(r12s13, 4, 0) , z3.If(r12s14, 1, 0) , z3.If(r12s15, 1, 0) , z3.If(r12s16, 4, 0) , z3.If(r12s19, 1, 0) , z3.If(r12s20, 1, 0) , z3.If(r12s21, 1, 0) , z3.If(r12s22, 1, 0) , z3.If(r12s23, 1, 0) , z3.If(r12s24, 95, 0) , z3.If(r12s25, 1, 0) , z3.If(r12s26, 1, 0) , z3.If(r12s27, 4, 0) , z3.If(r12s28, 1, 0) , z3.If(r12s29, 1, 0) , z3.If(r12s30, 1, 0) , z3.If(r12s31, 95, 0) , z3.If(r12s32, 95, 0) , z3.If(r12s33, 1, 0) , z3.If(r12s34, 95, 0) , z3.If(r12s35, 1, 0) , z3.If(r12s36, 1, 0) , z3.If(r12s37, 1, 0) , z3.If(r12s38, 95, 0) , z3.If(r12s39, 1, 0) , z3.If(r12s40, 4, 0) , z3.If(r12s41, 95, 0) , z3.If(r12s42, 1, 0) , z3.If(r12s43, 4, 0) , z3.If(r12s44, 4, 0) , z3.If(r12s45, 4, 0) , z3.If(r12s46, 1, 0) , z3.If(r12s48, 1, 0) , z3.If(r12s49, 1, 0) , z3.If(r12s50, 1, 0) , z3.If(r12s51, 1, 0) , z3.If(r13s0, 95, 0) , z3.If(r13s2, 4, 0) , z3.If(r13s3, 1, 0) , z3.If(r13s4, 1, 0) , z3.If(r13s5, 1, 0) , z3.If(r13s6, 1, 0) , z3.If(r13s7, 1, 0) , z3.If(r13s10, 1, 0) , z3.If(r13s12, 1, 0) , z3.If(r13s13, 4, 0) , z3.If(r13s14, 95, 0) , z3.If(r13s15, 1, 0) , z3.If(r13s16, 1, 0) , z3.If(r13s18, 1, 0) , z3.If(r13s19, 95, 0) , z3.If(r13s20, 1, 0) , z3.If(r13s22, 1, 0) , z3.If(r13s23, 1, 0) , z3.If(r13s25, 1, 0) , z3.If(r13s26, 1, 0) , z3.If(r13s27, 1, 0) , z3.If(r13s28, 1, 0) , z3.If(r13s29, 1, 0) , z3.If(r13s30, 1, 0) , z3.If(r13s31, 1, 0) , z3.If(r13s32, 1, 0) , z3.If(r13s33, 1, 0) , z3.If(r13s34, 1, 0) , z3.If(r13s35, 4, 0) , z3.If(r13s37, 4, 0) , z3.If(r13s38, 95, 0) , z3.If(r13s39, 1, 0) , z3.If(r13s40, 4, 0) , z3.If(r13s41, 1, 0) , z3.If(r13s42, 1, 0) , z3.If(r13s43, 95, 0) , z3.If(r13s44, 1, 0) , z3.If(r13s45, 1, 0) , z3.If(r13s46, 4, 0) , z3.If(r13s47, 4, 0) , z3.If(r13s49, 4, 0) , z3.If(r13s50, 1, 0) , z3.If(r13s51, 4, 0) , z3.If(r14s0, 1, 0) , z3.If(r14s1, 1, 0) , z3.If(r14s2, 1, 0) , z3.If(r14s4, 1, 0) , z3.If(r14s5, 1, 0) , z3.If(r14s7, 95, 0) , z3.If(r14s8, 95, 0) , z3.If(r14s9, 1, 0) , z3.If(r14s10, 1, 0) , z3.If(r14s11, 1, 0) , z3.If(r14s12, 1, 0) , z3.If(r14s13, 95, 0) , z3.If(r14s14, 1, 0) , z3.If(r14s15, 1, 0) , z3.If(r14s17, 4, 0) , z3.If(r14s18, 1, 0) , z3.If(r14s19, 4, 0) , z3.If(r14s20, 1, 0) , z3.If(r14s21, 1, 0) , z3.If(r14s22, 1, 0) , z3.If(r14s23, 1, 0) , z3.If(r14s24, 95, 0) , z3.If(r14s25, 4, 0) , z3.If(r14s26, 1, 0) , z3.If(r14s27, 1, 0) , z3.If(r14s28, 1, 0) , z3.If(r14s29, 1, 0) , z3.If(r14s30, 1, 0) , z3.If(r14s34, 95, 0) , z3.If(r14s36, 1, 0) , z3.If(r14s38, 1, 0) , z3.If(r14s39, 1, 0) , z3.If(r14s40, 1, 0) , z3.If(r14s41, 1, 0) , z3.If(r14s42, 95, 0) , z3.If(r14s43, 95, 0) , z3.If(r14s44, 95, 0) , z3.If(r14s45, 1, 0) , z3.If(r14s46, 1, 0) , z3.If(r14s47, 4, 0) , z3.If(r14s48, 95, 0) , z3.If(r14s49, 95, 0) , z3.If(r14s50, 1, 0) , z3.If(r14s51, 1, 0) , z3.If(r15s1, 1, 0) , z3.If(r15s2, 1, 0) , z3.If(r15s3, 1, 0) , z3.If(r15s4, 1, 0) , z3.If(r15s5, 1, 0) , z3.If(r15s6, 1, 0) , z3.If(r15s7, 1, 0) , z3.If(r15s8, 1, 0) , z3.If(r15s9, 95, 0) , z3.If(r15s10, 1, 0) , z3.If(r15s11, 1, 0) , z3.If(r15s12, 95, 0) , z3.If(r15s13, 95, 0) , z3.If(r15s14, 1, 0) , z3.If(r15s15, 1, 0) , z3.If(r15s16, 1, 0) , z3.If(r15s17, 1, 0) , z3.If(r15s19, 1, 0) , z3.If(r15s20, 1, 0) , z3.If(r15s21, 1, 0) , z3.If(r15s22, 95, 0) , z3.If(r15s23, 95, 0) , z3.If(r15s24, 95, 0) , z3.If(r15s25, 1, 0) , z3.If(r15s26, 1, 0) , z3.If(r15s27, 1, 0) , z3.If(r15s28, 1, 0) , z3.If(r15s29, 1, 0) , z3.If(r15s30, 1, 0) , z3.If(r15s31, 1, 0) , z3.If(r15s32, 1, 0) , z3.If(r15s33, 1, 0) , z3.If(r15s34, 95, 0) , z3.If(r15s35, 1, 0) , z3.If(r15s36, 1, 0) , z3.If(r15s37, 95, 0) , z3.If(r15s38, 1, 0) , z3.If(r15s39, 1, 0) , z3.If(r15s40, 1, 0) , z3.If(r15s41, 1, 0) , z3.If(r15s42, 1, 0) , z3.If(r15s43, 1, 0) , z3.If(r15s44, 1, 0) , z3.If(r15s45, 1, 0) , z3.If(r15s46, 1, 0) , z3.If(r15s48, 1, 0) , z3.If(r15s49, 1, 0) , z3.If(r15s50, 1, 0) , z3.If(r15s51, 95, 0) , z3.If(r16s1, 1, 0) , z3.If(r16s2, 4, 0) , z3.If(r16s3, 4, 0) , z3.If(r16s4, 95, 0) , z3.If(r16s5, 4, 0) , z3.If(r16s7, 95, 0) , z3.If(r16s8, 1, 0) , z3.If(r16s9, 1, 0) , z3.If(r16s11, 1, 0) , z3.If(r16s12, 4, 0) , z3.If(r16s13, 1, 0) , z3.If(r16s14, 1, 0) , z3.If(r16s15, 1, 0) , z3.If(r16s16, 1, 0) , z3.If(r16s17, 1, 0) , z3.If(r16s19, 1, 0) , z3.If(r16s20, 1, 0) , z3.If(r16s21, 1, 0) , z3.If(r16s22, 1, 0) , z3.If(r16s23, 1, 0) , z3.If(r16s24, 1, 0) , z3.If(r16s25, 1, 0) , z3.If(r16s26, 1, 0) , z3.If(r16s27, 1, 0) , z3.If(r16s28, 1, 0) , z3.If(r16s29, 1, 0) , z3.If(r16s30, 1, 0) , z3.If(r16s31, 95, 0) , z3.If(r16s32, 1, 0) , z3.If(r16s33, 1, 0) , z3.If(r16s34, 95, 0) , z3.If(r16s35, 1, 0) , z3.If(r16s36, 1, 0) , z3.If(r16s38, 1, 0) , z3.If(r16s39, 4, 0) , z3.If(r16s40, 1, 0) , z3.If(r16s41, 1, 0) , z3.If(r16s42, 1, 0) , z3.If(r16s43, 1, 0) , z3.If(r16s44, 4, 0) , z3.If(r16s45, 1, 0) , z3.If(r16s46, 4, 0) , z3.If(r16s48, 1, 0) , z3.If(r16s49, 1, 0) , z3.If(r16s50, 1, 0) , z3.If(r16s51, 95, 0) , z3.If(r17s0, 1, 0) , z3.If(r17s1, 1, 0) , z3.If(r17s2, 95, 0) , z3.If(r17s3, 95, 0) , z3.If(r17s4, 1, 0) , z3.If(r17s5, 1, 0) , z3.If(r17s7, 95, 0) , z3.If(r17s8, 1, 0) , z3.If(r17s11, 1, 0) , z3.If(r17s12, 1, 0) , z3.If(r17s13, 95, 0) , z3.If(r17s14, 1, 0) , z3.If(r17s15, 1, 0) , z3.If(r17s16, 4, 0) , z3.If(r17s17, 4, 0) , z3.If(r17s18, 95, 0) , z3.If(r17s19, 4, 0) , z3.If(r17s20, 1, 0) , z3.If(r17s21, 1, 0) , z3.If(r17s22, 1, 0) , z3.If(r17s23, 1, 0) , z3.If(r17s25, 1, 0) , z3.If(r17s26, 1, 0) , z3.If(r17s27, 4, 0) , z3.If(r17s28, 1, 0) , z3.If(r17s29, 4, 0) , z3.If(r17s30, 4, 0) , z3.If(r17s31, 1, 0) , z3.If(r17s32, 95, 0) , z3.If(r17s33, 1, 0) , z3.If(r17s34, 4, 0) , z3.If(r17s35, 95, 0) , z3.If(r17s36, 1, 0) , z3.If(r17s37, 4, 0) , z3.If(r17s38, 4, 0) , z3.If(r17s39, 1, 0) , z3.If(r17s40, 1, 0) , z3.If(r17s41, 4, 0) , z3.If(r17s42, 1, 0) , z3.If(r17s43, 4, 0) , z3.If(r17s44, 1, 0) , z3.If(r17s45, 1, 0) , z3.If(r17s46, 1, 0) , z3.If(r17s47, 95, 0) , z3.If(r17s48, 1, 0) , z3.If(r17s49, 1, 0) , z3.If(r17s50, 1, 0) , z3.If(r17s51, 1, 0) , z3.If(r18s0, 95, 0) , z3.If(r18s1, 1, 0) , z3.If(r18s2, 1, 0) , z3.If(r18s3, 1, 0) , z3.If(r18s4, 95, 0) , z3.If(r18s5, 1, 0) ,z3.If(r18s6, 1, 0) , z3.If(r18s7, 1, 0) , z3.If(r18s8, 1, 0) , z3.If(r18s10, 1, 0) , z3.If(r18s11, 1, 0) , z3.If(r18s12, 95, 0) , z3.If(r18s13, 1, 0) , z3.If(r18s14, 1, 0) , z3.If(r18s15, 95, 0) , z3.If(r18s16, 1, 0) , z3.If(r18s17, 1, 0) , z3.If(r18s18, 95, 0) , z3.If(r18s19, 1, 0) , z3.If(r18s20, 1, 0) , z3.If(r18s21, 1, 0) , z3.If(r18s22, 1, 0) , z3.If(r18s23, 95, 0) , z3.If(r18s25, 1, 0) , z3.If(r18s26, 1, 0) , z3.If(r18s27, 1, 0) , z3.If(r18s28, 1, 0) , z3.If(r18s29, 1, 0) , z3.If(r18s30, 1, 0) , z3.If(r18s31, 1, 0) , z3.If(r18s32, 1, 0) , z3.If(r18s33, 95, 0) , z3.If(r18s34, 1, 0) , z3.If(r18s35, 1, 0) , z3.If(r18s36, 1, 0) , z3.If(r18s37, 1, 0) , z3.If(r18s38, 1, 0) , z3.If(r18s39, 1, 0) , z3.If(r18s40, 1, 0) , z3.If(r18s41, 1, 0) , z3.If(r18s42, 1, 0) , z3.If(r18s43, 1, 0) , z3.If(r18s44, 1, 0) , z3.If(r18s45, 95, 0) , z3.If(r18s46, 1, 0) , z3.If(r18s47, 1, 0) , z3.If(r18s48, 95, 0) , z3.If(r18s49, 1, 0) , z3.If(r18s50, 1, 0) , z3.If(r18s51, 95, 0) , z3.If(r19s0, 4, 0) , z3.If(r19s1, 95, 0) , z3.If(r19s3, 4, 0) , z3.If(r19s4, 95, 0) , z3.If(r19s5, 1, 0) , z3.If(r19s6, 4, 0) , z3.If(r19s7, 4, 0) , z3.If(r19s8, 1, 0) , z3.If(r19s9, 4, 0) , z3.If(r19s10, 1, 0) , z3.If(r19s11, 1, 0) , z3.If(r19s12, 1, 0) , z3.If(r19s13, 1, 0) , z3.If(r19s14, 1, 0) , z3.If(r19s15, 95, 0) , z3.If(r19s16, 95, 0) , z3.If(r19s17, 95, 0) , z3.If(r19s18, 95, 0) , z3.If(r19s19, 4, 0) , z3.If(r19s20, 1, 0) , z3.If(r19s21, 1, 0) , z3.If(r19s22, 95, 0) , z3.If(r19s23, 1, 0) , z3.If(r19s24, 4, 0) , z3.If(r19s25, 1, 0) , z3.If(r19s26, 1, 0) , z3.If(r19s27, 1, 0) , z3.If(r19s28, 1, 0) , z3.If(r19s29, 1, 0) , z3.If(r19s30, 1, 0) , z3.If(r19s31, 4, 0) , z3.If(r19s32, 1, 0) , z3.If(r19s33, 1, 0) , z3.If(r19s34, 1, 0) , z3.If(r19s35, 1, 0) , z3.If(r19s36, 1, 0) , z3.If(r19s37, 95, 0) , z3.If(r19s38, 1, 0) , z3.If(r19s39, 1, 0) , z3.If(r19s40, 1, 0) , z3.If(r19s41, 1, 0) , z3.If(r19s42, 95, 0) , z3.If(r19s43, 1, 0) , z3.If(r19s44, 95, 0) , z3.If(r19s45, 1, 0) , z3.If(r19s46, 1, 0) , z3.If(r19s47, 1, 0) , z3.If(r19s48, 1, 0) , z3.If(r19s49, 95, 0) , z3.If(r19s50, 1, 0) , z3.If(r19s51, 95, 0) , z3.If(r20s0, 1, 0) , z3.If(r20s1, 1, 0) , z3.If(r20s2, 1, 0) , z3.If(r20s3, 1, 0) , z3.If(r20s4, 1, 0) , z3.If(r20s5, 1, 0) , z3.If(r20s6, 95, 0) , z3.If(r20s7, 1, 0) , z3.If(r20s8, 1, 0) , z3.If(r20s9, 95, 0) , z3.If(r20s10, 1, 0) , z3.If(r20s11, 1, 0) , z3.If(r20s12, 1, 0) , z3.If(r20s13, 95, 0) , z3.If(r20s15, 1, 0) , z3.If(r20s17, 95, 0) , z3.If(r20s18, 1, 0) , z3.If(r20s19, 1, 0) , z3.If(r20s20, 1, 0) , z3.If(r20s21, 1, 0) , z3.If(r20s22, 1, 0) , z3.If(r20s23, 1, 0) , z3.If(r20s24, 95, 0) , z3.If(r20s25, 1, 0) , z3.If(r20s26, 95, 0) , z3.If(r20s30, 95, 0) , z3.If(r20s31, 1, 0) , z3.If(r20s32, 95, 0) , z3.If(r20s33, 1, 0) , z3.If(r20s34, 95, 0) , z3.If(r20s35, 1, 0) , z3.If(r20s36, 95, 0) , z3.If(r20s37, 95, 0) , z3.If(r20s38, 1, 0) , z3.If(r20s39, 1, 0) , z3.If(r20s40, 1, 0) , z3.If(r20s41, 1, 0) , z3.If(r20s42, 1, 0) , z3.If(r20s43, 1, 0) , z3.If(r20s44, 1, 0) , z3.If(r20s45, 95, 0) , z3.If(r20s46, 1, 0) , z3.If(r20s47, 95, 0) , z3.If(r20s48, 1, 0) , z3.If(r20s49, 1, 0) , z3.If(r20s50, 1, 0) , z3.If(r20s51, 1, 0) , z3.If(r21s1, 1, 0) , z3.If(r21s2, 95, 0) , z3.If(r21s3, 1, 0) , z3.If(r21s4, 1, 0) , z3.If(r21s5, 95, 0) , z3.If(r21s6, 1, 0) , z3.If(r21s7, 1, 0) , z3.If(r21s8, 1, 0) , z3.If(r21s10, 1, 0) , z3.If(r21s12, 95, 0) , z3.If(r21s14, 4, 0) , z3.If(r21s15, 95, 0) , z3.If(r21s18, 95, 0) , z3.If(r21s19, 1, 0) , z3.If(r21s20, 1, 0) , z3.If(r21s22, 1, 0) , z3.If(r21s23, 4, 0) , z3.If(r21s24, 1, 0) , z3.If(r21s25, 1, 0) , z3.If(r21s26, 1, 0) , z3.If(r21s27, 1, 0) , z3.If(r21s28, 1, 0) , z3.If(r21s29, 1, 0) , z3.If(r21s30, 1, 0) , z3.If(r21s31, 1, 0) , z3.If(r21s33, 1, 0) , z3.If(r21s34, 1, 0) , z3.If(r21s35, 95, 0) , z3.If(r21s37, 4, 0) , z3.If(r21s38, 4, 0) , z3.If(r21s39, 1, 0) , z3.If(r21s40, 1, 0) , z3.If(r21s41, 1, 0) , z3.If(r21s42, 1, 0) , z3.If(r21s43, 4, 0) , z3.If(r21s44, 1, 0) , z3.If(r21s45, 1, 0) , z3.If(r21s46, 1, 0) , z3.If(r21s47, 1, 0) , z3.If(r21s48, 1, 0) , z3.If(r21s49, 1, 0) , z3.If(r21s50, 4, 0) , z3.If(r21s51, 1, 0) , z3.If(r22s0, 1, 0) , z3.If(r22s1, 1, 0) , z3.If(r22s2, 1, 0) , z3.If(r22s3, 1, 0) , z3.If(r22s4, 95, 0) , z3.If(r22s5, 95, 0) , z3.If(r22s6, 1, 0) , z3.If(r22s7, 1, 0) , z3.If(r22s8, 1, 0) , z3.If(r22s9, 4, 0) , z3.If(r22s10, 4, 0) , z3.If(r22s11, 1, 0) , z3.If(r22s12, 1, 0) , z3.If(r22s13, 4, 0) , z3.If(r22s14, 1, 0) , z3.If(r22s15, 1, 0) , z3.If(r22s16, 95, 0) , z3.If(r22s17, 1, 0) , z3.If(r22s18, 1, 0) , z3.If(r22s19, 1, 0) , z3.If(r22s20, 4, 0) , z3.If(r22s21, 1, 0) , z3.If(r22s22, 1, 0) , z3.If(r22s23, 1, 0) , z3.If(r22s24, 1, 0) , z3.If(r22s25, 1, 0) , z3.If(r22s26, 1, 0) , z3.If(r22s27, 1, 0) , z3.If(r22s28, 1, 0) , z3.If(r22s29, 1, 0) , z3.If(r22s30, 1, 0) , z3.If(r22s31, 1, 0) , z3.If(r22s32, 95, 0) , z3.If(r22s33, 4, 0) , z3.If(r22s34, 4, 0) , z3.If(r22s35, 1, 0) , z3.If(r22s36, 1, 0) , z3.If(r22s37, 1, 0) , z3.If(r22s38, 1, 0) , z3.If(r22s39, 95, 0) , z3.If(r22s40, 1, 0) , z3.If(r22s41, 1, 0) , z3.If(r22s42, 4, 0) , z3.If(r22s43, 1, 0) , z3.If(r22s44, 1, 0) , z3.If(r22s45, 95, 0) , z3.If(r22s46, 1, 0) , z3.If(r22s47, 1, 0) , z3.If(r22s48, 1, 0) , z3.If(r22s49, 1, 0) , z3.If(r22s50, 1, 0) , z3.If(r22s51, 1, 0) , z3.If(r23s0, 1, 0) , z3.If(r23s1, 1, 0) , z3.If(r23s2, 1, 0) , z3.If(r23s3, 95, 0) , z3.If(r23s4, 95, 0) , z3.If(r23s5, 1, 0) , z3.If(r23s6, 4, 0) , z3.If(r23s7, 1, 0) , z3.If(r23s8, 4, 0) , z3.If(r23s10, 95, 0) , z3.If(r23s11, 1, 0) , z3.If(r23s12, 1, 0) , z3.If(r23s13, 1, 0) , z3.If(r23s14, 1, 0) , z3.If(r23s15, 1, 0) , z3.If(r23s16, 95, 0) , z3.If(r23s17, 1, 0) , z3.If(r23s18, 1, 0) , z3.If(r23s19, 1, 0) , z3.If(r23s20, 1, 0) , z3.If(r23s21, 1, 0) , z3.If(r23s22, 1, 0) , z3.If(r23s23, 1, 0) , z3.If(r23s25, 1, 0) , z3.If(r23s26, 1, 0) , z3.If(r23s27, 1, 0) , z3.If(r23s28, 1, 0) , z3.If(r23s29, 1, 0) , z3.If(r23s30, 1, 0) , z3.If(r23s31, 4, 0) , z3.If(r23s32, 95, 0) , z3.If(r23s33, 1, 0) , z3.If(r23s34, 4, 0) , z3.If(r23s35, 1, 0) , z3.If(r23s36, 1, 0) , z3.If(r23s37, 1, 0) , z3.If(r23s38, 1, 0) , z3.If(r23s39, 95, 0) , z3.If(r23s40, 1, 0) , z3.If(r23s41, 1, 0) , z3.If(r23s42, 95, 0) , z3.If(r23s43, 1, 0) , z3.If(r23s44, 1, 0) , z3.If(r23s45, 95, 0) , z3.If(r23s46, 1, 0) , z3.If(r23s47, 1, 0) , z3.If(r23s48, 4, 0) , z3.If(r23s49, 1, 0) , z3.If(r23s50, 1, 0) , z3.If(r23s51, 1, 0)]
# 90/9/1
# scores = [z3.If(r0s0, 90, 0) , z3.If(r0s1, 1, 0) , z3.If(r0s2, 1, 0) , z3.If(r0s4, 1, 0) , z3.If(r0s5, 1, 0) , z3.If(r0s7, 1, 0) , z3.If(r0s8, 1, 0) , z3.If(r0s9, 1, 0) , z3.If(r0s10, 1, 0) , z3.If(r0s12, 1, 0) , z3.If(r0s13, 1, 0) , z3.If(r0s14, 9, 0) , z3.If(r0s15, 1, 0) , z3.If(r0s16, 1, 0) , z3.If(r0s18, 9, 0) , z3.If(r0s19, 1, 0) , z3.If(r0s20, 1, 0) , z3.If(r0s21, 1, 0) , z3.If(r0s22, 1, 0) , z3.If(r0s23, 90, 0) , z3.If(r0s24, 90, 0) , z3.If(r0s25, 1, 0) , z3.If(r0s26, 9, 0) , z3.If(r0s27, 1, 0) , z3.If(r0s28, 90, 0) , z3.If(r0s29, 90, 0) , z3.If(r0s30, 90, 0) , z3.If(r0s31, 1, 0) , z3.If(r0s32, 90, 0) , z3.If(r0s34, 1, 0) , z3.If(r0s35, 1, 0) , z3.If(r0s36, 1, 0) , z3.If(r0s37, 90, 0) , z3.If(r0s38, 1, 0) , z3.If(r0s39, 1, 0) , z3.If(r0s40, 9, 0) , z3.If(r0s41, 1, 0) , z3.If(r0s42, 1, 0) , z3.If(r0s43, 1, 0) , z3.If(r0s44, 90, 0) , z3.If(r0s45, 1, 0) , z3.If(r0s46, 1, 0) , z3.If(r0s47, 1, 0) , z3.If(r0s48, 1, 0) , z3.If(r0s49, 1, 0) , z3.If(r0s50, 1, 0) , z3.If(r0s51, 1, 0) , z3.If(r1s0, 90, 0) , z3.If(r1s1, 1, 0) , z3.If(r1s2, 9, 0) , z3.If(r1s3, 90, 0) , z3.If(r1s4, 1, 0) , z3.If(r1s5, 1, 0) , z3.If(r1s6, 1, 0) , z3.If(r1s7, 1, 0) , z3.If(r1s8, 1, 0) , z3.If(r1s9, 90, 0) , z3.If(r1s10, 9, 0) , z3.If(r1s11, 90, 0) , z3.If(r1s12, 1, 0) , z3.If(r1s13, 1, 0) , z3.If(r1s14, 1, 0) , z3.If(r1s15, 1, 0) , z3.If(r1s16, 1, 0) , z3.If(r1s17, 9, 0) , z3.If(r1s18, 90, 0) , z3.If(r1s19, 1, 0) , z3.If(r1s20, 1, 0) , z3.If(r1s21, 1, 0) , z3.If(r1s22, 90, 0) , z3.If(r1s23, 1, 0) , z3.If(r1s24, 1, 0) , z3.If(r1s25, 1, 0) , z3.If(r1s26, 1, 0) , z3.If(r1s27, 1, 0) , z3.If(r1s28, 90, 0) , z3.If(r1s29, 9, 0) , z3.If(r1s30, 1, 0) , z3.If(r1s31, 1, 0) , z3.If(r1s32, 1, 0) , z3.If(r1s33, 1, 0) , z3.If(r1s35, 1, 0) , z3.If(r1s36, 9, 0) , z3.If(r1s37, 1, 0) , z3.If(r1s38, 1, 0) , z3.If(r1s39, 1, 0) , z3.If(r1s40, 90, 0) , z3.If(r1s41, 1, 0) , z3.If(r1s42, 1, 0) , z3.If(r1s43, 1, 0) , z3.If(r1s44, 1, 0) , z3.If(r1s45, 1, 0) , z3.If(r1s46, 1, 0) , z3.If(r1s47, 1, 0) , z3.If(r1s48, 1, 0) , z3.If(r1s49, 1, 0) , z3.If(r1s50, 1, 0) , z3.If(r1s51, 1, 0) , z3.If(r2s0, 1, 0) , z3.If(r2s1, 1, 0) , z3.If(r2s2, 90, 0) , z3.If(r2s3, 1, 0) , z3.If(r2s4, 1, 0) , z3.If(r2s5, 1, 0) , z3.If(r2s6, 1, 0) , z3.If(r2s7, 9, 0) , z3.If(r2s8, 1, 0) , z3.If(r2s9, 1, 0) , z3.If(r2s10, 1, 0) , z3.If(r2s11, 1, 0) , z3.If(r2s12, 90, 0) , z3.If(r2s14, 9, 0) , z3.If(r2s15, 90, 0) , z3.If(r2s18, 1, 0) , z3.If(r2s19, 1, 0) , z3.If(r2s20, 1, 0) , z3.If(r2s22, 1, 0) , z3.If(r2s23, 1, 0) , z3.If(r2s24, 1, 0) , z3.If(r2s25, 1, 0) , z3.If(r2s26, 1, 0) , z3.If(r2s27, 1, 0) , z3.If(r2s28, 1, 0) , z3.If(r2s29, 1, 0) , z3.If(r2s30, 1, 0) , z3.If(r2s31, 1, 0) , z3.If(r2s33, 1, 0) , z3.If(r2s34, 1, 0) , z3.If(r2s35, 1, 0) , z3.If(r2s36, 1, 0) , z3.If(r2s37, 9, 0) , z3.If(r2s38, 1, 0) , z3.If(r2s39, 1, 0) , z3.If(r2s40, 1, 0) , z3.If(r2s41, 1, 0) , z3.If(r2s42, 1, 0) , z3.If(r2s43, 1, 0) , z3.If(r2s44, 1, 0) , z3.If(r2s45, 1, 0) , z3.If(r2s46, 9, 0) , z3.If(r2s47, 1, 0) , z3.If(r2s48, 1, 0) , z3.If(r2s49, 9, 0) , z3.If(r2s50, 1, 0) , z3.If(r2s51, 9, 0) , z3.If(r3s0, 1, 0) , z3.If(r3s2, 90, 0) , z3.If(r3s3, 90, 0) , z3.If(r3s4, 1, 0) , z3.If(r3s5, 1, 0) , z3.If(r3s6, 90, 0) , z3.If(r3s7, 1, 0) , z3.If(r3s8, 9, 0) , z3.If(r3s10, 1, 0) , z3.If(r3s11, 9, 0) , z3.If(r3s12, 1, 0) , z3.If(r3s15, 1, 0) , z3.If(r3s17, 9, 0) , z3.If(r3s18, 9, 0) , z3.If(r3s19, 1, 0) , z3.If(r3s20, 1, 0) , z3.If(r3s21, 1, 0) , z3.If(r3s22, 1, 0) , z3.If(r3s23, 1, 0) , z3.If(r3s25, 90, 0) , z3.If(r3s31, 9, 0) , z3.If(r3s32, 90, 0) , z3.If(r3s33, 1, 0) , z3.If(r3s34, 1, 0) , z3.If(r3s35, 90, 0) , z3.If(r3s36, 90, 0) , z3.If(r3s37, 90, 0) , z3.If(r3s38, 90, 0) , z3.If(r3s39, 1, 0) , z3.If(r3s40, 90, 0) , z3.If(r3s41, 9, 0) , z3.If(r3s42, 90, 0) , z3.If(r3s43, 9, 0) , z3.If(r3s44, 1, 0) , z3.If(r3s45, 1, 0) , z3.If(r3s46, 9, 0) , z3.If(r3s47, 90, 0) , z3.If(r3s48, 1, 0) , z3.If(r3s49, 9, 0) , z3.If(r3s50, 90, 0) , z3.If(r3s51, 9, 0) , z3.If(r4s0, 9, 0) , z3.If(r4s1, 1, 0) , z3.If(r4s2, 9, 0) , z3.If(r4s3, 1, 0) , z3.If(r4s4, 90, 0) , z3.If(r4s5, 90, 0) , z3.If(r4s6, 1, 0) , z3.If(r4s7, 1, 0) , z3.If(r4s8, 1, 0) , z3.If(r4s9, 1, 0) , z3.If(r4s10, 90, 0) , z3.If(r4s11, 1, 0) , z3.If(r4s12, 1, 0) , z3.If(r4s13, 1, 0) , z3.If(r4s14, 1, 0) , z3.If(r4s15, 1, 0) , z3.If(r4s16, 9, 0) , z3.If(r4s17, 1, 0) , z3.If(r4s18, 90, 0) , z3.If(r4s19, 1, 0) , z3.If(r4s20, 1, 0) , z3.If(r4s21, 1, 0) , z3.If(r4s22, 9, 0) , z3.If(r4s23, 1, 0) , z3.If(r4s24, 1, 0) , z3.If(r4s25, 1, 0) , z3.If(r4s26, 1, 0) , z3.If(r4s27, 1, 0) , z3.If(r4s28, 1, 0) , z3.If(r4s29, 1, 0) , z3.If(r4s30, 1, 0) , z3.If(r4s31, 1, 0) , z3.If(r4s32, 9, 0) , z3.If(r4s33, 1, 0) , z3.If(r4s34, 1, 0) , z3.If(r4s35, 1, 0) , z3.If(r4s36, 1, 0) , z3.If(r4s37, 1, 0) , z3.If(r4s38, 1, 0) , z3.If(r4s39, 1, 0) , z3.If(r4s40, 1, 0) , z3.If(r4s41, 1, 0) , z3.If(r4s42, 1, 0) , z3.If(r4s43, 1, 0) , z3.If(r4s44, 1, 0) , z3.If(r4s45, 1, 0) , z3.If(r4s46, 1, 0) , z3.If(r4s47, 9, 0) , z3.If(r4s48, 1, 0) , z3.If(r4s49, 1, 0) , z3.If(r4s50, 1, 0) , z3.If(r4s51, 90, 0) , z3.If(r5s0, 90, 0) , z3.If(r5s1, 1, 0) , z3.If(r5s2, 9, 0) , z3.If(r5s3, 9, 0) , z3.If(r5s4, 9, 0) , z3.If(r5s5, 1, 0) , z3.If(r5s6, 1, 0) , z3.If(r5s7, 1, 0) , z3.If(r5s8, 90, 0) , z3.If(r5s9, 90, 0) , z3.If(r5s10, 1, 0) , z3.If(r5s11, 1, 0) , z3.If(r5s12, 1, 0) , z3.If(r5s13, 1, 0) , z3.If(r5s14, 1, 0) , z3.If(r5s15, 1, 0) , z3.If(r5s16, 1, 0) , z3.If(r5s17, 90, 0) , z3.If(r5s18, 1, 0) , z3.If(r5s19, 9, 0) , z3.If(r5s20, 1, 0) , z3.If(r5s21, 1, 0) , z3.If(r5s22, 9, 0) , z3.If(r5s23, 1, 0) , z3.If(r5s24, 90, 0) , z3.If(r5s25, 1, 0) , z3.If(r5s26, 1, 0) , z3.If(r5s27, 1, 0) , z3.If(r5s28, 9, 0) , z3.If(r5s29, 1, 0) , z3.If(r5s30, 1, 0) , z3.If(r5s31, 1, 0) , z3.If(r5s32, 1, 0) , z3.If(r5s33, 1, 0) , z3.If(r5s34, 9, 0) , z3.If(r5s35, 1, 0) , z3.If(r5s36, 1, 0) , z3.If(r5s37, 90, 0) , z3.If(r5s38, 1, 0) , z3.If(r5s39, 9, 0) , z3.If(r5s40, 1, 0) , z3.If(r5s41, 1, 0) , z3.If(r5s42, 1, 0) , z3.If(r5s43, 1, 0) , z3.If(r5s44, 1, 0) , z3.If(r5s45, 1, 0) , z3.If(r5s46, 1, 0) , z3.If(r5s47, 1, 0) , z3.If(r5s48, 90, 0) , z3.If(r5s49, 9, 0) , z3.If(r5s50, 1, 0) , z3.If(r5s51, 1, 0) , z3.If(r6s0, 9, 0) , z3.If(r6s1, 1, 0) , z3.If(r6s2, 9, 0) , z3.If(r6s3, 9, 0) , z3.If(r6s4, 9, 0) , z3.If(r6s5, 1, 0) , z3.If(r6s7, 1, 0) , z3.If(r6s8, 1, 0) , z3.If(r6s9, 9, 0) , z3.If(r6s11, 1, 0) , z3.If(r6s12, 1, 0) , z3.If(r6s13, 1, 0) , z3.If(r6s14, 1, 0) , z3.If(r6s15, 1, 0) , z3.If(r6s16, 1, 0) , z3.If(r6s17, 9, 0) , z3.If(r6s18, 90, 0) , z3.If(r6s19, 90, 0) , z3.If(r6s20, 1, 0) , z3.If(r6s21, 9, 0) , z3.If(r6s22, 1, 0) , z3.If(r6s23, 1, 0) , z3.If(r6s24, 9, 0) , z3.If(r6s25, 1, 0) , z3.If(r6s26, 1, 0) , z3.If(r6s27, 1, 0) , z3.If(r6s28, 90, 0) , z3.If(r6s29, 9, 0) , z3.If(r6s30, 90, 0) , z3.If(r6s31, 9, 0) , z3.If(r6s32, 1, 0) , z3.If(r6s33, 1, 0) , z3.If(r6s34, 1, 0) , z3.If(r6s36, 1, 0) , z3.If(r6s39, 1, 0) , z3.If(r6s40, 1, 0) , z3.If(r6s41, 1, 0) , z3.If(r6s42, 9, 0) , z3.If(r6s44, 1, 0) , z3.If(r6s45, 1, 0) , z3.If(r6s47, 90, 0) , z3.If(r6s48, 1, 0) , z3.If(r6s49, 90, 0) , z3.If(r6s50, 1, 0) , z3.If(r6s51, 9, 0) , z3.If(r7s0, 90, 0) , z3.If(r7s1, 1, 0) , z3.If(r7s2, 1, 0) , z3.If(r7s3, 9, 0) , z3.If(r7s4, 1, 0) , z3.If(r7s5, 1, 0) , z3.If(r7s6, 1, 0) , z3.If(r7s7, 9, 0) , z3.If(r7s10, 1, 0) , z3.If(r7s12, 1, 0) , z3.If(r7s13, 90, 0) , z3.If(r7s14, 1, 0) , z3.If(r7s15, 1, 0) , z3.If(r7s16, 1, 0) , z3.If(r7s17, 9, 0) , z3.If(r7s18, 9, 0) , z3.If(r7s19, 1, 0) , z3.If(r7s20, 1, 0) , z3.If(r7s22, 90, 0) , z3.If(r7s23, 90, 0) , z3.If(r7s24, 9, 0) , z3.If(r7s25, 1, 0) , z3.If(r7s26, 90, 0) , z3.If(r7s27, 1, 0) , z3.If(r7s28, 90, 0) , z3.If(r7s29, 90, 0) , z3.If(r7s30, 1, 0) , z3.If(r7s31, 90, 0) , z3.If(r7s32, 1, 0) , z3.If(r7s33, 1, 0) , z3.If(r7s34, 1, 0) , z3.If(r7s35, 1, 0) , z3.If(r7s36, 9, 0) , z3.If(r7s37, 90, 0) , z3.If(r7s38, 90, 0) , z3.If(r7s39, 1, 0) , z3.If(r7s40, 90, 0) , z3.If(r7s41, 1, 0) , z3.If(r7s42, 90, 0) , z3.If(r7s43, 90, 0) , z3.If(r7s44, 9, 0) , z3.If(r7s45, 1, 0) , z3.If(r7s46, 1, 0) , z3.If(r7s47, 90, 0) , z3.If(r7s49, 90, 0) , z3.If(r7s50, 1, 0) , z3.If(r7s51, 1, 0) , z3.If(r8s0, 1, 0) , z3.If(r8s1, 1, 0) , z3.If(r8s2, 1, 0) , z3.If(r8s3, 1, 0) , z3.If(r8s4, 1, 0) , z3.If(r8s5, 1, 0) , z3.If(r8s6, 1, 0) , z3.If(r8s7, 1, 0) , z3.If(r8s8, 1, 0) , z3.If(r8s9, 90, 0) , z3.If(r8s10, 1, 0) , z3.If(r8s11, 1, 0) , z3.If(r8s12, 1, 0) , z3.If(r8s14, 1, 0) , z3.If(r8s15, 1, 0) , z3.If(r8s18, 1, 0) , z3.If(r8s19, 1, 0) , z3.If(r8s20, 1, 0) , z3.If(r8s21, 1, 0) , z3.If(r8s22, 9, 0) , z3.If(r8s23, 9, 0) , z3.If(r8s24, 9, 0) , z3.If(r8s25, 9, 0) , z3.If(r8s26, 1, 0) , z3.If(r8s27, 1, 0) , z3.If(r8s28, 1, 0) , z3.If(r8s29, 1, 0) , z3.If(r8s30, 1, 0) , z3.If(r8s31, 9, 0) , z3.If(r8s33, 90, 0) , z3.If(r8s34, 90, 0) , z3.If(r8s35, 9, 0) , z3.If(r8s36, 1, 0) , z3.If(r8s37, 90, 0) , z3.If(r8s38, 90, 0) , z3.If(r8s39, 1, 0) , z3.If(r8s40, 9, 0) , z3.If(r8s41, 1, 0) , z3.If(r8s42, 9, 0) , z3.If(r8s43, 90, 0) , z3.If(r8s44, 1, 0) , z3.If(r8s45, 1, 0) , z3.If(r8s46, 1, 0) , z3.If(r8s47, 90, 0) , z3.If(r8s48, 1, 0) , z3.If(r8s49, 9, 0) , z3.If(r8s50, 9, 0) , z3.If(r8s51, 9, 0) , z3.If(r9s0, 90, 0) , z3.If(r9s1, 1, 0) , z3.If(r9s2, 1, 0) , z3.If(r9s3, 1, 0) , z3.If(r9s4, 1, 0) , z3.If(r9s5, 1, 0) , z3.If(r9s6, 1, 0) , z3.If(r9s7, 1, 0) , z3.If(r9s8, 1, 0) , z3.If(r9s9, 1, 0) , z3.If(r9s10, 1, 0) , z3.If(r9s11, 1, 0) , z3.If(r9s12, 1, 0) , z3.If(r9s13, 90, 0) , z3.If(r9s14, 90, 0) , z3.If(r9s15, 1, 0) , z3.If(r9s16, 1, 0) , z3.If(r9s17, 90, 0) , z3.If(r9s18, 1, 0) , z3.If(r9s19, 1, 0) , z3.If(r9s20, 1, 0) , z3.If(r9s21, 1, 0) , z3.If(r9s22, 1, 0) , z3.If(r9s23, 1, 0) , z3.If(r9s24, 90, 0) , z3.If(r9s25, 1, 0) , z3.If(r9s26, 9, 0) , z3.If(r9s27, 1, 0) , z3.If(r9s28, 90, 0) , z3.If(r9s29, 90, 0) , z3.If(r9s30, 1, 0) , z3.If(r9s31, 1, 0) , z3.If(r9s32, 90, 0) , z3.If(r9s33, 9, 0) , z3.If(r9s34, 1, 0) , z3.If(r9s35, 1, 0) , z3.If(r9s36, 90, 0) , z3.If(r9s37, 90, 0) , z3.If(r9s38, 1, 0) , z3.If(r9s39, 1, 0) , z3.If(r9s40, 90, 0) , z3.If(r9s41, 1, 0) , z3.If(r9s42, 1, 0) , z3.If(r9s43, 1, 0) , z3.If(r9s44, 90, 0) , z3.If(r9s45, 1, 0) , z3.If(r9s46, 90, 0) , z3.If(r9s47, 1, 0) , z3.If(r9s48, 1, 0) , z3.If(r9s49, 90, 0) , z3.If(r9s50, 1, 0) , z3.If(r9s51, 1, 0) , z3.If(r10s0, 1, 0) , z3.If(r10s1, 1, 0) , z3.If(r10s2, 1, 0) , z3.If(r10s3, 1, 0) , z3.If(r10s4, 1, 0) , z3.If(r10s5, 1, 0) , z3.If(r10s6, 1, 0) , z3.If(r10s7, 1, 0) , z3.If(r10s9, 90, 0) , z3.If(r10s10, 1, 0) , z3.If(r10s12, 90, 0) , z3.If(r10s13, 90, 0) , z3.If(r10s14, 1, 0) , z3.If(r10s15, 1, 0) , z3.If(r10s16, 1, 0) , z3.If(r10s17, 1, 0) , z3.If(r10s18, 90, 0) , z3.If(r10s19, 1, 0) , z3.If(r10s20, 1, 0) , z3.If(r10s21, 90, 0) , z3.If(r10s22, 1, 0) , z3.If(r10s23, 1, 0) , z3.If(r10s24, 1, 0) , z3.If(r10s25, 1, 0) , z3.If(r10s26, 1, 0) , z3.If(r10s27, 1, 0) , z3.If(r10s28, 90, 0) , z3.If(r10s29, 1, 0) , z3.If(r10s30, 90, 0) , z3.If(r10s31, 90, 0) , z3.If(r10s32, 90, 0) , z3.If(r10s33, 1, 0) , z3.If(r10s34, 1, 0) , z3.If(r10s35, 90, 0) , z3.If(r10s36, 1, 0) , z3.If(r10s37, 90, 0) , z3.If(r10s38, 1, 0) , z3.If(r10s39, 1, 0) , z3.If(r10s40, 90, 0) , z3.If(r10s41, 1, 0) , z3.If(r10s42, 1, 0) , z3.If(r10s43, 1, 0) , z3.If(r10s44, 1, 0) , z3.If(r10s45, 1, 0) , z3.If(r10s46, 1, 0) , z3.If(r10s47, 1, 0) , z3.If(r10s49, 1, 0) , z3.If(r10s50, 1, 0) , z3.If(r10s51, 1, 0) , z3.If(r11s0, 1, 0) , z3.If(r11s1, 1, 0) , z3.If(r11s2, 1, 0) , z3.If(r11s3, 90, 0) , z3.If(r11s4, 90, 0) , z3.If(r11s5, 9, 0) , z3.If(r11s6, 1, 0) , z3.If(r11s7, 1, 0) , z3.If(r11s8, 1, 0) , z3.If(r11s9, 90, 0) , z3.If(r11s10, 1, 0) , z3.If(r11s11, 90, 0) , z3.If(r11s12, 1, 0) , z3.If(r11s13, 90, 0) , z3.If(r11s14, 1, 0) , z3.If(r11s15, 1, 0) , z3.If(r11s16, 1, 0) , z3.If(r11s17, 1, 0) , z3.If(r11s18, 1, 0) , z3.If(r11s19, 1, 0) , z3.If(r11s20, 1, 0) , z3.If(r11s21, 1, 0) , z3.If(r11s22, 1, 0) , z3.If(r11s23, 1, 0) , z3.If(r11s24, 1, 0) , z3.If(r11s25, 90, 0) , z3.If(r11s26, 1, 0) , z3.If(r11s27, 1, 0) , z3.If(r11s28, 1, 0) , z3.If(r11s29, 1, 0) , z3.If(r11s30, 1, 0) , z3.If(r11s31, 1, 0) , z3.If(r11s32, 1, 0) , z3.If(r11s33, 1, 0) , z3.If(r11s34, 9, 0) , z3.If(r11s35, 1, 0) , z3.If(r11s36, 1, 0) , z3.If(r11s37, 1, 0) , z3.If(r11s38, 1, 0) , z3.If(r11s39, 1, 0) , z3.If(r11s40, 1, 0) , z3.If(r11s41, 1, 0) , z3.If(r11s43, 1, 0) , z3.If(r11s44, 1, 0) , z3.If(r11s45, 1, 0) , z3.If(r11s46, 1, 0) , z3.If(r11s48, 1, 0) , z3.If(r11s50, 1, 0) , z3.If(r11s51, 1, 0) , z3.If(r12s1, 1, 0) , z3.If(r12s2, 1, 0) , z3.If(r12s3, 1, 0) , z3.If(r12s4, 90, 0) , z3.If(r12s5, 90, 0) , z3.If(r12s6, 1, 0) , z3.If(r12s7, 1, 0) , z3.If(r12s8, 9, 0) , z3.If(r12s9, 1, 0) , z3.If(r12s10, 1, 0) , z3.If(r12s12, 1, 0) , z3.If(r12s13, 9, 0) , z3.If(r12s14, 1, 0) , z3.If(r12s15, 1, 0) , z3.If(r12s16, 9, 0) , z3.If(r12s19, 1, 0) , z3.If(r12s20, 1, 0) , z3.If(r12s21, 1, 0) , z3.If(r12s22, 1, 0) , z3.If(r12s23, 1, 0) , z3.If(r12s24, 90, 0) , z3.If(r12s25, 1, 0) , z3.If(r12s26, 1, 0) , z3.If(r12s27, 9, 0) , z3.If(r12s28, 1, 0) , z3.If(r12s29, 1, 0) , z3.If(r12s30, 1, 0) , z3.If(r12s31, 90, 0) , z3.If(r12s32, 90, 0) , z3.If(r12s33, 1, 0) , z3.If(r12s34, 90, 0) , z3.If(r12s35, 1, 0) , z3.If(r12s36, 1, 0) , z3.If(r12s37, 1, 0) , z3.If(r12s38, 90, 0) , z3.If(r12s39, 1, 0) , z3.If(r12s40, 9, 0) , z3.If(r12s41, 90, 0) , z3.If(r12s42, 1, 0) , z3.If(r12s43, 9, 0) , z3.If(r12s44, 9, 0) , z3.If(r12s45, 9, 0) , z3.If(r12s46, 1, 0) , z3.If(r12s48, 1, 0) , z3.If(r12s49, 1, 0) , z3.If(r12s50, 1, 0) , z3.If(r12s51, 1, 0) , z3.If(r13s0, 90, 0) , z3.If(r13s2, 9, 0) , z3.If(r13s3, 1, 0) , z3.If(r13s4, 1, 0) , z3.If(r13s5, 1, 0) , z3.If(r13s6, 1, 0) , z3.If(r13s7, 1, 0) , z3.If(r13s10, 1, 0) , z3.If(r13s12, 1, 0) , z3.If(r13s13, 9, 0) , z3.If(r13s14, 90, 0) , z3.If(r13s15, 1, 0) , z3.If(r13s16, 1, 0) , z3.If(r13s18, 1, 0) , z3.If(r13s19, 90, 0) , z3.If(r13s20, 1, 0) , z3.If(r13s22, 1, 0) , z3.If(r13s23, 1, 0) , z3.If(r13s25, 1, 0) , z3.If(r13s26, 1, 0) , z3.If(r13s27, 1, 0) , z3.If(r13s28, 1, 0) , z3.If(r13s29, 1, 0) , z3.If(r13s30, 1, 0) , z3.If(r13s31, 1, 0) , z3.If(r13s32, 1, 0) , z3.If(r13s33, 1, 0) , z3.If(r13s34, 1, 0) , z3.If(r13s35, 9, 0) , z3.If(r13s37, 9, 0) , z3.If(r13s38, 90, 0) , z3.If(r13s39, 1, 0) , z3.If(r13s40, 9, 0) , z3.If(r13s41, 1, 0) , z3.If(r13s42, 1, 0) , z3.If(r13s43, 90, 0) , z3.If(r13s44, 1, 0) , z3.If(r13s45, 1, 0) , z3.If(r13s46, 9, 0) , z3.If(r13s47, 9, 0) , z3.If(r13s49, 9, 0) , z3.If(r13s50, 1, 0) , z3.If(r13s51, 9, 0) , z3.If(r14s0, 1, 0) , z3.If(r14s1, 1, 0) , z3.If(r14s2, 1, 0) , z3.If(r14s4, 1, 0) , z3.If(r14s5, 1, 0) , z3.If(r14s7, 90, 0) , z3.If(r14s8, 90, 0) , z3.If(r14s9, 1, 0) , z3.If(r14s10, 1, 0) , z3.If(r14s11, 1, 0) , z3.If(r14s12, 1, 0) , z3.If(r14s13, 90, 0) , z3.If(r14s14, 1, 0) , z3.If(r14s15, 1, 0) , z3.If(r14s17, 9, 0) , z3.If(r14s18, 1, 0) , z3.If(r14s19, 9, 0) , z3.If(r14s20, 1, 0) , z3.If(r14s21, 1, 0) , z3.If(r14s22, 1, 0) , z3.If(r14s23, 1, 0) , z3.If(r14s24, 90, 0) , z3.If(r14s25, 9, 0) , z3.If(r14s26, 1, 0) , z3.If(r14s27, 1, 0) , z3.If(r14s28, 1, 0) , z3.If(r14s29, 1, 0) , z3.If(r14s30, 1, 0) , z3.If(r14s34, 90, 0) , z3.If(r14s36, 1, 0) , z3.If(r14s38, 1, 0) , z3.If(r14s39, 1, 0) , z3.If(r14s40, 1, 0) , z3.If(r14s41, 1, 0) , z3.If(r14s42, 90, 0) , z3.If(r14s43, 90, 0) , z3.If(r14s44, 90, 0) , z3.If(r14s45, 1, 0) , z3.If(r14s46, 1, 0) , z3.If(r14s47, 9, 0) , z3.If(r14s48, 90, 0) , z3.If(r14s49, 90, 0) , z3.If(r14s50, 1, 0) , z3.If(r14s51, 1, 0) , z3.If(r15s1, 1, 0) , z3.If(r15s2, 1, 0) , z3.If(r15s3, 1, 0) , z3.If(r15s4, 1, 0) , z3.If(r15s5, 1, 0) , z3.If(r15s6, 1, 0) , z3.If(r15s7, 1, 0) , z3.If(r15s8, 1, 0) , z3.If(r15s9, 90, 0) , z3.If(r15s10, 1, 0) , z3.If(r15s11, 1, 0) , z3.If(r15s12, 90, 0) , z3.If(r15s13, 90, 0) , z3.If(r15s14, 1, 0) , z3.If(r15s15, 1, 0) , z3.If(r15s16, 1, 0) , z3.If(r15s17, 1, 0) , z3.If(r15s19, 1, 0) , z3.If(r15s20, 1, 0) , z3.If(r15s21, 1, 0) , z3.If(r15s22, 90, 0) , z3.If(r15s23, 90, 0) , z3.If(r15s24, 90, 0) , z3.If(r15s25, 1, 0) , z3.If(r15s26, 1, 0) , z3.If(r15s27, 1, 0) , z3.If(r15s28, 1, 0) , z3.If(r15s29, 1, 0) , z3.If(r15s30, 1, 0) , z3.If(r15s31, 1, 0) , z3.If(r15s32, 1, 0) , z3.If(r15s33, 1, 0) , z3.If(r15s34, 90, 0) , z3.If(r15s35, 1, 0) , z3.If(r15s36, 1, 0) , z3.If(r15s37, 90, 0) , z3.If(r15s38, 1, 0) , z3.If(r15s39, 1, 0) , z3.If(r15s40, 1, 0) , z3.If(r15s41, 1, 0) , z3.If(r15s42, 1, 0) , z3.If(r15s43, 1, 0) , z3.If(r15s44, 1, 0) , z3.If(r15s45, 1, 0) , z3.If(r15s46, 1, 0) , z3.If(r15s48, 1, 0) , z3.If(r15s49, 1, 0) , z3.If(r15s50, 1, 0) , z3.If(r15s51, 90, 0) , z3.If(r16s1, 1, 0) , z3.If(r16s2, 9, 0) , z3.If(r16s3, 9, 0) , z3.If(r16s4, 90, 0) , z3.If(r16s5, 9, 0) , z3.If(r16s7, 90, 0) , z3.If(r16s8, 1, 0) , z3.If(r16s9, 1, 0) , z3.If(r16s11, 1, 0) , z3.If(r16s12, 9, 0) , z3.If(r16s13, 1, 0) , z3.If(r16s14, 1, 0) , z3.If(r16s15, 1, 0) , z3.If(r16s16, 1, 0) , z3.If(r16s17, 1, 0) , z3.If(r16s19, 1, 0) , z3.If(r16s20, 1, 0) , z3.If(r16s21, 1, 0) , z3.If(r16s22, 1, 0) , z3.If(r16s23, 1, 0) , z3.If(r16s24, 1, 0) , z3.If(r16s25, 1, 0) , z3.If(r16s26, 1, 0) , z3.If(r16s27, 1, 0) , z3.If(r16s28, 1, 0) , z3.If(r16s29, 1, 0) , z3.If(r16s30, 1, 0) , z3.If(r16s31, 90, 0) , z3.If(r16s32, 1, 0) , z3.If(r16s33, 1, 0) , z3.If(r16s34, 90, 0) , z3.If(r16s35, 1, 0) , z3.If(r16s36, 1, 0) , z3.If(r16s38, 1, 0) , z3.If(r16s39, 9, 0) , z3.If(r16s40, 1, 0) , z3.If(r16s41, 1, 0) , z3.If(r16s42, 1, 0) , z3.If(r16s43, 1, 0) , z3.If(r16s44, 9, 0) , z3.If(r16s45, 1, 0) , z3.If(r16s46, 9, 0) , z3.If(r16s48, 1, 0) , z3.If(r16s49, 1, 0) , z3.If(r16s50, 1, 0) , z3.If(r16s51, 90, 0) , z3.If(r17s0, 1, 0) , z3.If(r17s1, 1, 0) , z3.If(r17s2, 90, 0) , z3.If(r17s3, 90, 0) , z3.If(r17s4, 1, 0) , z3.If(r17s5, 1, 0) , z3.If(r17s7, 90, 0) , z3.If(r17s8, 1, 0) , z3.If(r17s11, 1, 0) , z3.If(r17s12, 1, 0) , z3.If(r17s13, 90, 0) , z3.If(r17s14, 1, 0) , z3.If(r17s15, 1, 0) , z3.If(r17s16, 9, 0) , z3.If(r17s17, 9, 0) , z3.If(r17s18, 90, 0) , z3.If(r17s19, 9, 0) , z3.If(r17s20, 1, 0) , z3.If(r17s21, 1, 0) , z3.If(r17s22, 1, 0) , z3.If(r17s23, 1, 0) , z3.If(r17s25, 1, 0) , z3.If(r17s26, 1, 0) , z3.If(r17s27, 9, 0) , z3.If(r17s28, 1, 0) , z3.If(r17s29, 9, 0) , z3.If(r17s30, 9, 0) , z3.If(r17s31, 1, 0) , z3.If(r17s32, 90, 0) , z3.If(r17s33, 1, 0) , z3.If(r17s34, 9, 0) , z3.If(r17s35, 90, 0) , z3.If(r17s36, 1, 0) , z3.If(r17s37, 9, 0) , z3.If(r17s38, 9, 0) , z3.If(r17s39, 1, 0) , z3.If(r17s40, 1, 0) , z3.If(r17s41, 9, 0) , z3.If(r17s42, 1, 0) , z3.If(r17s43, 9, 0) , z3.If(r17s44, 1, 0) , z3.If(r17s45, 1, 0) , z3.If(r17s46, 1, 0) , z3.If(r17s47, 90, 0) , z3.If(r17s48, 1, 0) , z3.If(r17s49, 1, 0) , z3.If(r17s50, 1, 0) , z3.If(r17s51, 1, 0) , z3.If(r18s0, 90, 0) , z3.If(r18s1, 1, 0) , z3.If(r18s2, 1, 0) , z3.If(r18s3, 1, 0) , z3.If(r18s4, 90, 0) , z3.If(r18s5, 1, 0) , z3.If(r18s6, 1, 0) , z3.If(r18s7, 1, 0) , z3.If(r18s8, 1, 0) , z3.If(r18s10, 1, 0) , z3.If(r18s11, 1, 0) , z3.If(r18s12, 90, 0) , z3.If(r18s13, 1, 0) , z3.If(r18s14, 1, 0) , z3.If(r18s15, 90, 0) , z3.If(r18s16, 1, 0) , z3.If(r18s17, 1, 0) , z3.If(r18s18, 90, 0) , z3.If(r18s19, 1, 0) , z3.If(r18s20, 1, 0) , z3.If(r18s21, 1, 0) , z3.If(r18s22, 1, 0) , z3.If(r18s23, 90, 0) , z3.If(r18s25, 1, 0) , z3.If(r18s26, 1, 0) , z3.If(r18s27, 1, 0) , z3.If(r18s28, 1, 0) , z3.If(r18s29, 1, 0) , z3.If(r18s30, 1, 0) , z3.If(r18s31, 1, 0) , z3.If(r18s32, 1, 0) , z3.If(r18s33, 90, 0) , z3.If(r18s34, 1, 0) , z3.If(r18s35, 1, 0) , z3.If(r18s36, 1, 0) , z3.If(r18s37, 1, 0) , z3.If(r18s38, 1, 0) , z3.If(r18s39, 1, 0) , z3.If(r18s40, 1, 0) , z3.If(r18s41, 1, 0) , z3.If(r18s42, 1, 0) , z3.If(r18s43, 1, 0) , z3.If(r18s44, 1, 0) , z3.If(r18s45, 90, 0) , z3.If(r18s46, 1, 0) , z3.If(r18s47, 1, 0) , z3.If(r18s48, 90, 0) , z3.If(r18s49, 1, 0) , z3.If(r18s50, 1, 0) , z3.If(r18s51, 90, 0) , z3.If(r19s0, 9, 0) , z3.If(r19s1, 90, 0) , z3.If(r19s3, 9, 0) , z3.If(r19s4, 90, 0) , z3.If(r19s5, 1, 0) , z3.If(r19s6, 9, 0) , z3.If(r19s7, 9, 0) , z3.If(r19s8, 1, 0) , z3.If(r19s9, 9, 0) , z3.If(r19s10, 1, 0) , z3.If(r19s11, 1, 0) , z3.If(r19s12, 1, 0) , z3.If(r19s13, 1, 0) , z3.If(r19s14, 1, 0) , z3.If(r19s15, 90, 0) , z3.If(r19s16, 90, 0) , z3.If(r19s17, 90, 0) , z3.If(r19s18, 90, 0) , z3.If(r19s19, 9, 0) , z3.If(r19s20, 1, 0) , z3.If(r19s21, 1, 0) , z3.If(r19s22, 90, 0) , z3.If(r19s23, 1, 0) , z3.If(r19s24, 9, 0) , z3.If(r19s25, 1, 0) , z3.If(r19s26, 1, 0) , z3.If(r19s27, 1, 0) , z3.If(r19s28, 1, 0) , z3.If(r19s29, 1, 0) , z3.If(r19s30, 1, 0) , z3.If(r19s31, 9, 0) , z3.If(r19s32, 1, 0) , z3.If(r19s33, 1, 0) , z3.If(r19s34, 1, 0) , z3.If(r19s35, 1, 0) , z3.If(r19s36, 1, 0) , z3.If(r19s37, 90, 0) , z3.If(r19s38, 1, 0) , z3.If(r19s39, 1, 0) , z3.If(r19s40, 1, 0) , z3.If(r19s41, 1, 0) , z3.If(r19s42, 90, 0) , z3.If(r19s43, 1, 0) , z3.If(r19s44, 90, 0) , z3.If(r19s45, 1, 0) , z3.If(r19s46, 1, 0) , z3.If(r19s47, 1, 0) , z3.If(r19s48, 1, 0) , z3.If(r19s49, 90, 0) , z3.If(r19s50, 1, 0) , z3.If(r19s51, 90, 0) , z3.If(r20s0, 1, 0) , z3.If(r20s1, 1, 0) , z3.If(r20s2, 1, 0) , z3.If(r20s3, 1, 0) , z3.If(r20s4, 1, 0) , z3.If(r20s5, 1, 0) , z3.If(r20s6, 90, 0) , z3.If(r20s7, 1, 0) , z3.If(r20s8, 1, 0) , z3.If(r20s9, 90, 0) , z3.If(r20s10, 1, 0) , z3.If(r20s11, 1, 0) , z3.If(r20s12, 1, 0) , z3.If(r20s13, 90, 0) , z3.If(r20s15, 1, 0) , z3.If(r20s17, 90, 0) , z3.If(r20s18, 1, 0) , z3.If(r20s19, 1, 0) , z3.If(r20s20, 1, 0) , z3.If(r20s21, 1, 0) , z3.If(r20s22, 1, 0) , z3.If(r20s23, 1, 0) , z3.If(r20s24, 90, 0) , z3.If(r20s25, 1, 0) , z3.If(r20s26, 90, 0) , z3.If(r20s30, 90, 0) , z3.If(r20s31, 1, 0) , z3.If(r20s32, 90, 0) , z3.If(r20s33, 1, 0) , z3.If(r20s34, 90, 0) , z3.If(r20s35, 1, 0) , z3.If(r20s36, 90, 0) , z3.If(r20s37, 90, 0) , z3.If(r20s38, 1, 0) , z3.If(r20s39, 1, 0) , z3.If(r20s40, 1, 0) , z3.If(r20s41, 1, 0) , z3.If(r20s42, 1, 0) , z3.If(r20s43, 1, 0) , z3.If(r20s44, 1, 0) , z3.If(r20s45, 90, 0) , z3.If(r20s46, 1, 0) , z3.If(r20s47, 90, 0) , z3.If(r20s48, 1, 0) , z3.If(r20s49, 1, 0) , z3.If(r20s50, 1, 0) , z3.If(r20s51, 1, 0) , z3.If(r21s1, 1, 0) , z3.If(r21s2, 90, 0) , z3.If(r21s3, 1, 0) , z3.If(r21s4, 1, 0) , z3.If(r21s5, 90, 0) , z3.If(r21s6, 1, 0) , z3.If(r21s7, 1, 0) , z3.If(r21s8, 1, 0) , z3.If(r21s10, 1, 0) , z3.If(r21s12, 90, 0) , z3.If(r21s14, 9, 0) , z3.If(r21s15, 90, 0) , z3.If(r21s18, 90, 0) , z3.If(r21s19, 1, 0) , z3.If(r21s20, 1, 0) , z3.If(r21s22, 1, 0) , z3.If(r21s23, 9, 0) , z3.If(r21s24, 1, 0) , z3.If(r21s25, 1, 0) , z3.If(r21s26, 1, 0) , z3.If(r21s27, 1, 0) , z3.If(r21s28, 1, 0) , z3.If(r21s29, 1, 0) , z3.If(r21s30, 1, 0) , z3.If(r21s31, 1, 0) , z3.If(r21s33, 1, 0) , z3.If(r21s34, 1, 0) , z3.If(r21s35, 90, 0) , z3.If(r21s37, 9, 0) , z3.If(r21s38, 9, 0) , z3.If(r21s39, 1, 0) , z3.If(r21s40, 1, 0) , z3.If(r21s41, 1, 0) , z3.If(r21s42, 1, 0) , z3.If(r21s43, 9, 0) , z3.If(r21s44, 1, 0) , z3.If(r21s45, 1, 0) , z3.If(r21s46, 1, 0) , z3.If(r21s47, 1, 0) , z3.If(r21s48, 1, 0) , z3.If(r21s49, 1, 0) , z3.If(r21s50, 9, 0) , z3.If(r21s51, 1, 0) , z3.If(r22s0, 1, 0) , z3.If(r22s1, 1, 0) , z3.If(r22s2, 1, 0) , z3.If(r22s3, 1, 0) , z3.If(r22s4, 90, 0) , z3.If(r22s5, 90, 0) , z3.If(r22s6, 1, 0) , z3.If(r22s7, 1, 0) , z3.If(r22s8, 1, 0) , z3.If(r22s9, 9, 0) , z3.If(r22s10, 9, 0) , z3.If(r22s11, 1, 0) , z3.If(r22s12, 1, 0) , z3.If(r22s13, 9, 0) , z3.If(r22s14, 1, 0) , z3.If(r22s15, 1, 0) , z3.If(r22s16, 90, 0) , z3.If(r22s17, 1, 0) , z3.If(r22s18, 1, 0) , z3.If(r22s19, 1, 0) , z3.If(r22s20, 9, 0) , z3.If(r22s21, 1, 0) , z3.If(r22s22, 1, 0) , z3.If(r22s23, 1, 0) , z3.If(r22s24, 1, 0) , z3.If(r22s25, 1, 0) , z3.If(r22s26, 1, 0) , z3.If(r22s27, 1, 0) , z3.If(r22s28, 1, 0) , z3.If(r22s29, 1, 0) , z3.If(r22s30, 1, 0) , z3.If(r22s31, 1, 0) , z3.If(r22s32, 90, 0) , z3.If(r22s33, 9, 0) , z3.If(r22s34, 9, 0) , z3.If(r22s35, 1, 0) , z3.If(r22s36, 1, 0) , z3.If(r22s37, 1, 0) , z3.If(r22s38, 1, 0) , z3.If(r22s39, 90, 0) , z3.If(r22s40, 1, 0) , z3.If(r22s41, 1, 0) , z3.If(r22s42, 9, 0) , z3.If(r22s43, 1, 0) , z3.If(r22s44, 1, 0) , z3.If(r22s45, 90, 0) , z3.If(r22s46, 1, 0) , z3.If(r22s47, 1, 0) , z3.If(r22s48, 1, 0) , z3.If(r22s49, 1, 0) , z3.If(r22s50, 1, 0) , z3.If(r22s51, 1, 0) , z3.If(r23s0, 1, 0) , z3.If(r23s1, 1, 0) , z3.If(r23s2, 1, 0) , z3.If(r23s3, 90, 0) , z3.If(r23s4, 90, 0) , z3.If(r23s5, 1, 0) , z3.If(r23s6, 9, 0) , z3.If(r23s7, 1, 0) , z3.If(r23s8, 9, 0) , z3.If(r23s10, 90, 0) , z3.If(r23s11, 1, 0) , z3.If(r23s12, 1, 0) , z3.If(r23s13, 1, 0) , z3.If(r23s14, 1, 0) , z3.If(r23s15, 1, 0) , z3.If(r23s16, 90, 0) , z3.If(r23s17, 1, 0) , z3.If(r23s18, 1, 0) , z3.If(r23s19, 1, 0) , z3.If(r23s20, 1, 0) , z3.If(r23s21, 1, 0) , z3.If(r23s22, 1, 0) , z3.If(r23s23, 1, 0) , z3.If(r23s25, 1, 0) , z3.If(r23s26, 1, 0) , z3.If(r23s27, 1, 0) , z3.If(r23s28, 1, 0) , z3.If(r23s29, 1, 0) , z3.If(r23s30, 1, 0) , z3.If(r23s31, 9, 0) , z3.If(r23s32, 90, 0) , z3.If(r23s33, 1, 0) , z3.If(r23s34, 9, 0) , z3.If(r23s35, 1, 0) , z3.If(r23s36, 1, 0) , z3.If(r23s37, 1, 0) , z3.If(r23s38, 1, 0) , z3.If(r23s39, 90, 0) , z3.If(r23s40, 1, 0) , z3.If(r23s41, 1, 0) , z3.If(r23s42, 90, 0) , z3.If(r23s43, 1, 0) , z3.If(r23s44, 1, 0) , z3.If(r23s45, 90, 0) , z3.If(r23s46, 1, 0) , z3.If(r23s47, 1, 0) , z3.If(r23s48, 9, 0) , z3.If(r23s49, 1, 0) , z3.If(r23s50, 1, 0) , z3.If(r23s51, 1, 0) ]

#### Rules (PbLe)

In [16]:
# rev_rules = []
# for l in pb_rev_split:
#     rev_rules.append("z3.PbLe(({0}), n)".format(l))
# # print(rev_rules)
# for i in rev_rules:
#     print(i.replace("'", ""), ',', end=' ')

In [17]:
# sub_rules = []
# for l in pb_sub_split: 
#     sub_rules.append("z3.PbLe(({0}), m)".format(l))
# # print(sub_rules)
# for i in sub_rules:
#     print(i.replace("'", ""), ',', end=' ')

### Setting variables and running the algorithm

In [18]:
# These values decide the maximum cardinality:
# n is the maximum number of submission per reviewer and m is the maximum number of reviewers per submission
###############################################################################################################
n = 5
m = 10

###############################################################################################################

# Run the algorithm
o.add([
z3.PbLe(([(r0s0,1), (r0s1,1), (r0s2,1), (r0s4,1), (r0s5,1), (r0s7,1), (r0s8,1), (r0s9,1), (r0s10,1), (r0s12,1), (r0s13,1), (r0s14,1), (r0s15,1), (r0s16,1), (r0s18,1), (r0s19,1), (r0s20,1), (r0s21,1), (r0s22,1), (r0s23,1), (r0s24,1), (r0s25,1), (r0s26,1), (r0s27,1), (r0s28,1), (r0s29,1), (r0s30,1), (r0s31,1), (r0s32,1), (r0s34,1), (r0s35,1), (r0s36,1), (r0s37,1), (r0s38,1), (r0s39,1), (r0s40,1), (r0s41,1), (r0s42,1), (r0s43,1), (r0s44,1), (r0s45,1), (r0s46,1), (r0s47,1), (r0s48,1), (r0s49,1), (r0s50,1), (r0s51,1), (r1s0,1), (r1s1,1), (r1s2,1), (r1s3,1), (r1s4,1)]), n) , z3.PbLe(([(r1s5,1), (r1s6,1), (r1s7,1), (r1s8,1), (r1s9,1), (r1s10,1), (r1s11,1), (r1s12,1), (r1s13,1), (r1s14,1), (r1s15,1), (r1s16,1), (r1s17,1), (r1s18,1), (r1s19,1), (r1s20,1), (r1s21,1), (r1s22,1), (r1s23,1), (r1s24,1), (r1s25,1), (r1s26,1), (r1s27,1), (r1s28,1), (r1s29,1), (r1s30,1), (r1s31,1), (r1s32,1), (r1s33,1), (r1s35,1), (r1s36,1), (r1s37,1), (r1s38,1), (r1s39,1), (r1s40,1), (r1s41,1), (r1s42,1), (r1s43,1), (r1s44,1), (r1s45,1), (r1s46,1), (r1s47,1), (r1s48,1), (r1s49,1), (r1s50,1), (r1s51,1), (r2s0,1), (r2s1,1), (r2s2,1), (r2s3,1), (r2s4,1), (r2s5,1)]), n) , z3.PbLe(([(r2s6,1), (r2s7,1), (r2s8,1), (r2s9,1), (r2s10,1), (r2s11,1), (r2s12,1), (r2s14,1), (r2s15,1), (r2s18,1), (r2s19,1), (r2s20,1), (r2s22,1), (r2s23,1), (r2s24,1), (r2s25,1), (r2s26,1), (r2s27,1), (r2s28,1), (r2s29,1), (r2s30,1), (r2s31,1), (r2s33,1), (r2s34,1), (r2s35,1), (r2s36,1), (r2s37,1), (r2s38,1), (r2s39,1), (r2s40,1), (r2s41,1), (r2s42,1), (r2s43,1), (r2s44,1), (r2s45,1), (r2s46,1), (r2s47,1), (r2s48,1), (r2s49,1), (r2s50,1), (r2s51,1), (r3s0,1), (r3s2,1), (r3s3,1), (r3s4,1), (r3s5,1), (r3s6,1), (r3s7,1), (r3s8,1), (r3s10,1), (r3s11,1), (r3s12,1)]), n) , z3.PbLe(([(r3s15,1), (r3s17,1), (r3s18,1), (r3s19,1), (r3s20,1), (r3s21,1), (r3s22,1), (r3s23,1), (r3s25,1), (r3s31,1), (r3s32,1), (r3s33,1), (r3s34,1), (r3s35,1), (r3s36,1), (r3s37,1), (r3s38,1), (r3s39,1), (r3s40,1), (r3s41,1), (r3s42,1), (r3s43,1), (r3s44,1), (r3s45,1), (r3s46,1), (r3s47,1), (r3s48,1), (r3s49,1), (r3s50,1), (r3s51,1), (r4s0,1), (r4s1,1), (r4s2,1), (r4s3,1), (r4s4,1), (r4s5,1), (r4s6,1), (r4s7,1), (r4s8,1), (r4s9,1), (r4s10,1), (r4s11,1), (r4s12,1), (r4s13,1), (r4s14,1), (r4s15,1), (r4s16,1), (r4s17,1), (r4s18,1), (r4s19,1), (r4s20,1), (r4s21,1)]), n) , z3.PbLe(([(r4s22,1), (r4s23,1), (r4s24,1), (r4s25,1), (r4s26,1), (r4s27,1), (r4s28,1), (r4s29,1), (r4s30,1), (r4s31,1), (r4s32,1), (r4s33,1), (r4s34,1), (r4s35,1), (r4s36,1), (r4s37,1), (r4s38,1), (r4s39,1), (r4s40,1), (r4s41,1), (r4s42,1), (r4s43,1), (r4s44,1), (r4s45,1), (r4s46,1), (r4s47,1), (r4s48,1), (r4s49,1), (r4s50,1), (r4s51,1), (r5s0,1), (r5s1,1), (r5s2,1), (r5s3,1), (r5s4,1), (r5s5,1), (r5s6,1), (r5s7,1), (r5s8,1), (r5s9,1), (r5s10,1), (r5s11,1), (r5s12,1), (r5s13,1), (r5s14,1), (r5s15,1), (r5s16,1), (r5s17,1), (r5s18,1), (r5s19,1), (r5s20,1), (r5s21,1)]), n) , z3.PbLe(([(r5s22,1), (r5s23,1), (r5s24,1), (r5s25,1), (r5s26,1), (r5s27,1), (r5s28,1), (r5s29,1), (r5s30,1), (r5s31,1), (r5s32,1), (r5s33,1), (r5s34,1), (r5s35,1), (r5s36,1), (r5s37,1), (r5s38,1), (r5s39,1), (r5s40,1), (r5s41,1), (r5s42,1), (r5s43,1), (r5s44,1), (r5s45,1), (r5s46,1), (r5s47,1), (r5s48,1), (r5s49,1), (r5s50,1), (r5s51,1), (r6s0,1), (r6s1,1), (r6s2,1), (r6s3,1), (r6s4,1), (r6s5,1), (r6s7,1), (r6s8,1), (r6s9,1), (r6s11,1), (r6s12,1), (r6s13,1), (r6s14,1), (r6s15,1), (r6s16,1), (r6s17,1), (r6s18,1), (r6s19,1), (r6s20,1), (r6s21,1), (r6s22,1), (r6s23,1)]), n) , z3.PbLe(([(r6s24,1), (r6s25,1), (r6s26,1), (r6s27,1), (r6s28,1), (r6s29,1), (r6s30,1), (r6s31,1), (r6s32,1), (r6s33,1), (r6s34,1), (r6s36,1), (r6s39,1), (r6s40,1), (r6s41,1), (r6s42,1), (r6s44,1), (r6s45,1), (r6s47,1), (r6s48,1), (r6s49,1), (r6s50,1), (r6s51,1), (r7s0,1), (r7s1,1), (r7s2,1), (r7s3,1), (r7s4,1), (r7s5,1), (r7s6,1), (r7s7,1), (r7s10,1), (r7s12,1), (r7s13,1), (r7s14,1), (r7s15,1), (r7s16,1), (r7s17,1), (r7s18,1), (r7s19,1), (r7s20,1), (r7s22,1), (r7s23,1), (r7s24,1), (r7s25,1), (r7s26,1), (r7s27,1), (r7s28,1), (r7s29,1), (r7s30,1), (r7s31,1), (r7s32,1)]), n) , z3.PbLe(([(r7s33,1), (r7s34,1), (r7s35,1), (r7s36,1), (r7s37,1), (r7s38,1), (r7s39,1), (r7s40,1), (r7s41,1), (r7s42,1), (r7s43,1), (r7s44,1), (r7s45,1), (r7s46,1), (r7s47,1), (r7s49,1), (r7s50,1), (r7s51,1), (r8s0,1), (r8s1,1), (r8s2,1), (r8s3,1), (r8s4,1), (r8s5,1), (r8s6,1), (r8s7,1), (r8s8,1), (r8s9,1), (r8s10,1), (r8s11,1), (r8s12,1), (r8s14,1), (r8s15,1), (r8s18,1), (r8s19,1), (r8s20,1), (r8s21,1), (r8s22,1), (r8s23,1), (r8s24,1), (r8s25,1), (r8s26,1), (r8s27,1), (r8s28,1), (r8s29,1), (r8s30,1), (r8s31,1), (r8s33,1), (r8s34,1), (r8s35,1), (r8s36,1), (r8s37,1)]), n) , z3.PbLe(([(r8s38,1), (r8s39,1), (r8s40,1), (r8s41,1), (r8s42,1), (r8s43,1), (r8s44,1), (r8s45,1), (r8s46,1), (r8s47,1), (r8s48,1), (r8s49,1), (r8s50,1), (r8s51,1), (r9s0,1), (r9s1,1), (r9s2,1), (r9s3,1), (r9s4,1), (r9s5,1), (r9s6,1), (r9s7,1), (r9s8,1), (r9s9,1), (r9s10,1), (r9s11,1), (r9s12,1), (r9s13,1), (r9s14,1), (r9s15,1), (r9s16,1), (r9s17,1), (r9s18,1), (r9s19,1), (r9s20,1), (r9s21,1), (r9s22,1), (r9s23,1), (r9s24,1), (r9s25,1), (r9s26,1), (r9s27,1), (r9s28,1), (r9s29,1), (r9s30,1), (r9s31,1), (r9s32,1), (r9s33,1), (r9s34,1), (r9s35,1), (r9s36,1), (r9s37,1)]), n) , z3.PbLe(([(r9s38,1), (r9s39,1), (r9s40,1), (r9s41,1), (r9s42,1), (r9s43,1), (r9s44,1), (r9s45,1), (r9s46,1), (r9s47,1), (r9s48,1), (r9s49,1), (r9s50,1), (r9s51,1), (r10s0,1), (r10s1,1), (r10s2,1), (r10s3,1), (r10s4,1), (r10s5,1), (r10s6,1), (r10s7,1), (r10s9,1), (r10s10,1), (r10s12,1), (r10s13,1), (r10s14,1), (r10s15,1), (r10s16,1), (r10s17,1), (r10s18,1), (r10s19,1), (r10s20,1), (r10s21,1), (r10s22,1), (r10s23,1), (r10s24,1), (r10s25,1), (r10s26,1), (r10s27,1), (r10s28,1), (r10s29,1), (r10s30,1), (r10s31,1), (r10s32,1), (r10s33,1), (r10s34,1), (r10s35,1), (r10s36,1), (r10s37,1), (r10s38,1), (r10s39,1)]), n) , z3.PbLe(([(r10s40,1), (r10s41,1), (r10s42,1), (r10s43,1), (r10s44,1), (r10s45,1), (r10s46,1), (r10s47,1), (r10s49,1), (r10s50,1), (r10s51,1), (r11s0,1), (r11s1,1), (r11s2,1), (r11s3,1), (r11s4,1), (r11s5,1), (r11s6,1), (r11s7,1), (r11s8,1), (r11s9,1), (r11s10,1), (r11s11,1), (r11s12,1), (r11s13,1), (r11s14,1), (r11s15,1), (r11s16,1), (r11s17,1), (r11s18,1), (r11s19,1), (r11s20,1), (r11s21,1), (r11s22,1), (r11s23,1), (r11s24,1), (r11s25,1), (r11s26,1), (r11s27,1), (r11s28,1), (r11s29,1), (r11s30,1), (r11s31,1), (r11s32,1), (r11s33,1), (r11s34,1), (r11s35,1), (r11s36,1), (r11s37,1), (r11s38,1), (r11s39,1), (r11s40,1)]), n) , z3.PbLe(([(r11s41,1), (r11s43,1), (r11s44,1), (r11s45,1), (r11s46,1), (r11s48,1), (r11s50,1), (r11s51,1), (r12s1,1), (r12s2,1), (r12s3,1), (r12s4,1), (r12s5,1), (r12s6,1), (r12s7,1), (r12s8,1), (r12s9,1), (r12s10,1), (r12s12,1), (r12s13,1), (r12s14,1), (r12s15,1), (r12s16,1), (r12s19,1), (r12s20,1), (r12s21,1), (r12s22,1), (r12s23,1), (r12s24,1), (r12s25,1), (r12s26,1), (r12s27,1), (r12s28,1), (r12s29,1), (r12s30,1), (r12s31,1), (r12s32,1), (r12s33,1), (r12s34,1), (r12s35,1), (r12s36,1), (r12s37,1), (r12s38,1), (r12s39,1), (r12s40,1), (r12s41,1), (r12s42,1), (r12s43,1), (r12s44,1), (r12s45,1), (r12s46,1), (r12s48,1)]), n) , z3.PbLe(([(r12s49,1), (r12s50,1), (r12s51,1), (r13s0,1), (r13s2,1), (r13s3,1), (r13s4,1), (r13s5,1), (r13s6,1), (r13s7,1), (r13s10,1), (r13s12,1), (r13s13,1), (r13s14,1), (r13s15,1), (r13s16,1), (r13s18,1), (r13s19,1), (r13s20,1), (r13s22,1), (r13s23,1), (r13s25,1), (r13s26,1), (r13s27,1), (r13s28,1), (r13s29,1), (r13s30,1), (r13s31,1), (r13s32,1), (r13s33,1), (r13s34,1), (r13s35,1), (r13s37,1), (r13s38,1), (r13s39,1), (r13s40,1), (r13s41,1), (r13s42,1), (r13s43,1), (r13s44,1), (r13s45,1), (r13s46,1), (r13s47,1), (r13s49,1), (r13s50,1), (r13s51,1), (r14s0,1), (r14s1,1), (r14s2,1), (r14s4,1), (r14s5,1), (r14s7,1)]), n) , z3.PbLe(([(r14s8,1), (r14s9,1), (r14s10,1), (r14s11,1), (r14s12,1), (r14s13,1), (r14s14,1), (r14s15,1), (r14s17,1), (r14s18,1), (r14s19,1), (r14s20,1), (r14s21,1), (r14s22,1), (r14s23,1), (r14s24,1), (r14s25,1), (r14s26,1), (r14s27,1), (r14s28,1), (r14s29,1), (r14s30,1), (r14s34,1), (r14s36,1), (r14s38,1), (r14s39,1), (r14s40,1), (r14s41,1), (r14s42,1), (r14s43,1), (r14s44,1), (r14s45,1), (r14s46,1), (r14s47,1), (r14s48,1), (r14s49,1), (r14s50,1), (r14s51,1), (r15s1,1), (r15s2,1), (r15s3,1), (r15s4,1), (r15s5,1), (r15s6,1), (r15s7,1), (r15s8,1), (r15s9,1), (r15s10,1), (r15s11,1), (r15s12,1), (r15s13,1), (r15s14,1)]), n) , z3.PbLe(([(r15s15,1), (r15s16,1), (r15s17,1), (r15s19,1), (r15s20,1), (r15s21,1), (r15s22,1), (r15s23,1), (r15s24,1), (r15s25,1), (r15s26,1), (r15s27,1), (r15s28,1), (r15s29,1), (r15s30,1), (r15s31,1), (r15s32,1), (r15s33,1), (r15s34,1), (r15s35,1), (r15s36,1), (r15s37,1), (r15s38,1), (r15s39,1), (r15s40,1), (r15s41,1), (r15s42,1), (r15s43,1), (r15s44,1), (r15s45,1), (r15s46,1), (r15s48,1), (r15s49,1), (r15s50,1), (r15s51,1), (r16s1,1), (r16s2,1), (r16s3,1), (r16s4,1), (r16s5,1), (r16s7,1), (r16s8,1), (r16s9,1), (r16s11,1), (r16s12,1), (r16s13,1), (r16s14,1), (r16s15,1), (r16s16,1), (r16s17,1), (r16s19,1), (r16s20,1)]), n) , z3.PbLe(([(r16s21,1), (r16s22,1), (r16s23,1), (r16s24,1), (r16s25,1), (r16s26,1), (r16s27,1), (r16s28,1), (r16s29,1), (r16s30,1), (r16s31,1), (r16s32,1), (r16s33,1), (r16s34,1), (r16s35,1), (r16s36,1), (r16s38,1), (r16s39,1), (r16s40,1), (r16s41,1), (r16s42,1), (r16s43,1), (r16s44,1), (r16s45,1), (r16s46,1), (r16s48,1), (r16s49,1), (r16s50,1), (r16s51,1), (r17s0,1), (r17s1,1), (r17s2,1), (r17s3,1), (r17s4,1), (r17s5,1), (r17s7,1), (r17s8,1), (r17s11,1), (r17s12,1), (r17s13,1), (r17s14,1), (r17s15,1), (r17s16,1), (r17s17,1), (r17s18,1), (r17s19,1), (r17s20,1), (r17s21,1), (r17s22,1), (r17s23,1), (r17s25,1), (r17s26,1)]), n) , z3.PbLe(([(r17s27,1), (r17s28,1), (r17s29,1), (r17s30,1), (r17s31,1), (r17s32,1), (r17s33,1), (r17s34,1), (r17s35,1), (r17s36,1), (r17s37,1), (r17s38,1), (r17s39,1), (r17s40,1), (r17s41,1), (r17s42,1), (r17s43,1), (r17s44,1), (r17s45,1), (r17s46,1), (r17s47,1), (r17s48,1), (r17s49,1), (r17s50,1), (r17s51,1), (r18s0,1), (r18s1,1), (r18s2,1), (r18s3,1), (r18s4,1), (r18s5,1), (r18s6,1), (r18s7,1), (r18s8,1), (r18s10,1), (r18s11,1), (r18s12,1), (r18s13,1), (r18s14,1), (r18s15,1), (r18s16,1), (r18s17,1), (r18s18,1), (r18s19,1), (r18s20,1), (r18s21,1), (r18s22,1), (r18s23,1), (r18s25,1), (r18s26,1), (r18s27,1), (r18s28,1)]), n) , z3.PbLe(([(r18s29,1), (r18s30,1), (r18s31,1), (r18s32,1), (r18s33,1), (r18s34,1), (r18s35,1), (r18s36,1), (r18s37,1), (r18s38,1), (r18s39,1), (r18s40,1), (r18s41,1), (r18s42,1), (r18s43,1), (r18s44,1), (r18s45,1), (r18s46,1), (r18s47,1), (r18s48,1), (r18s49,1), (r18s50,1), (r18s51,1), (r19s0,1), (r19s1,1), (r19s3,1), (r19s4,1), (r19s5,1), (r19s6,1), (r19s7,1), (r19s8,1), (r19s9,1), (r19s10,1), (r19s11,1), (r19s12,1), (r19s13,1), (r19s14,1), (r19s15,1), (r19s16,1), (r19s17,1), (r19s18,1), (r19s19,1), (r19s20,1), (r19s21,1), (r19s22,1), (r19s23,1), (r19s24,1), (r19s25,1), (r19s26,1), (r19s27,1), (r19s28,1), (r19s29,1)]), n) , z3.PbLe(([(r19s30,1), (r19s31,1), (r19s32,1), (r19s33,1), (r19s34,1), (r19s35,1), (r19s36,1), (r19s37,1), (r19s38,1), (r19s39,1), (r19s40,1), (r19s41,1), (r19s42,1), (r19s43,1), (r19s44,1), (r19s45,1), (r19s46,1), (r19s47,1), (r19s48,1), (r19s49,1), (r19s50,1), (r19s51,1), (r20s0,1), (r20s1,1), (r20s2,1), (r20s3,1), (r20s4,1), (r20s5,1), (r20s6,1), (r20s7,1), (r20s8,1), (r20s9,1), (r20s10,1), (r20s11,1), (r20s12,1), (r20s13,1), (r20s15,1), (r20s17,1), (r20s18,1), (r20s19,1), (r20s20,1), (r20s21,1), (r20s22,1), (r20s23,1), (r20s24,1), (r20s25,1), (r20s26,1), (r20s30,1), (r20s31,1), (r20s32,1), (r20s33,1), (r20s34,1)]), n) , z3.PbLe(([(r20s35,1), (r20s36,1), (r20s37,1), (r20s38,1), (r20s39,1), (r20s40,1), (r20s41,1), (r20s42,1), (r20s43,1), (r20s44,1), (r20s45,1), (r20s46,1), (r20s47,1), (r20s48,1), (r20s49,1), (r20s50,1), (r20s51,1), (r21s1,1), (r21s2,1), (r21s3,1), (r21s4,1), (r21s5,1), (r21s6,1), (r21s7,1), (r21s8,1), (r21s10,1), (r21s12,1), (r21s14,1), (r21s15,1), (r21s18,1), (r21s19,1), (r21s20,1), (r21s22,1), (r21s23,1), (r21s24,1), (r21s25,1), (r21s26,1), (r21s27,1), (r21s28,1), (r21s29,1), (r21s30,1), (r21s31,1), (r21s33,1), (r21s34,1), (r21s35,1), (r21s37,1), (r21s38,1), (r21s39,1), (r21s40,1), (r21s41,1), (r21s42,1), (r21s43,1)]), n) , z3.PbLe(([(r21s44,1), (r21s45,1), (r21s46,1), (r21s47,1), (r21s48,1), (r21s49,1), (r21s50,1), (r21s51,1), (r22s0,1), (r22s1,1), (r22s2,1), (r22s3,1), (r22s4,1), (r22s5,1), (r22s6,1), (r22s7,1), (r22s8,1), (r22s9,1), (r22s10,1), (r22s11,1), (r22s12,1), (r22s13,1), (r22s14,1), (r22s15,1), (r22s16,1), (r22s17,1), (r22s18,1), (r22s19,1), (r22s20,1), (r22s21,1), (r22s22,1), (r22s23,1), (r22s24,1), (r22s25,1), (r22s26,1), (r22s27,1), (r22s28,1), (r22s29,1), (r22s30,1), (r22s31,1), (r22s32,1), (r22s33,1), (r22s34,1), (r22s35,1), (r22s36,1), (r22s37,1), (r22s38,1), (r22s39,1), (r22s40,1), (r22s41,1), (r22s42,1), (r22s43,1)]), n) , z3.PbLe(([(r22s44,1), (r22s45,1), (r22s46,1), (r22s47,1), (r22s48,1), (r22s49,1), (r22s50,1), (r22s51,1), (r23s0,1), (r23s1,1), (r23s2,1), (r23s3,1), (r23s4,1), (r23s5,1), (r23s6,1), (r23s7,1), (r23s8,1), (r23s10,1), (r23s11,1), (r23s12,1), (r23s13,1), (r23s14,1), (r23s15,1), (r23s16,1), (r23s17,1), (r23s18,1), (r23s19,1), (r23s20,1), (r23s21,1), (r23s22,1), (r23s23,1), (r23s25,1), (r23s26,1), (r23s27,1), (r23s28,1), (r23s29,1), (r23s30,1), (r23s31,1), (r23s32,1), (r23s33,1), (r23s34,1), (r23s35,1), (r23s36,1), (r23s37,1), (r23s38,1), (r23s39,1), (r23s40,1), (r23s41,1), (r23s42,1), (r23s43,1), (r23s44,1), (r23s45,1)]), n) , z3.PbLe(([(r23s46,1), (r23s47,1), (r23s48,1), (r23s49,1), (r23s50,1), (r23s51,1)]), n),
z3.PbLe(([(r0s0,1), (r1s0,1), (r2s0,1), (r3s0,1), (r4s0,1), (r5s0,1), (r6s0,1), (r7s0,1), (r8s0,1), (r9s0,1), (r10s0,1), (r11s0,1), (r13s0,1), (r14s0,1), (r17s0,1), (r18s0,1), (r19s0,1), (r20s0,1), (r22s0,1), (r23s0,1), (r0s1,1), (r1s1,1), (r2s1,1), (r4s1,1)]), m) , z3.PbLe(([(r5s1,1), (r6s1,1), (r7s1,1), (r8s1,1), (r9s1,1), (r10s1,1), (r11s1,1), (r12s1,1), (r14s1,1), (r15s1,1), (r16s1,1), (r17s1,1), (r18s1,1), (r19s1,1), (r20s1,1), (r21s1,1), (r22s1,1), (r23s1,1), (r0s2,1), (r1s2,1), (r2s2,1), (r3s2,1), (r4s2,1), (r5s2,1)]), m) , z3.PbLe(([(r6s2,1), (r7s2,1), (r8s2,1), (r9s2,1), (r10s2,1), (r11s2,1), (r12s2,1), (r13s2,1), (r14s2,1), (r15s2,1), (r16s2,1), (r17s2,1), (r18s2,1), (r20s2,1), (r21s2,1), (r22s2,1), (r23s2,1), (r1s3,1), (r2s3,1), (r3s3,1), (r4s3,1), (r5s3,1), (r6s3,1), (r7s3,1)]), m) , z3.PbLe(([(r8s3,1), (r9s3,1), (r10s3,1), (r11s3,1), (r12s3,1), (r13s3,1), (r15s3,1), (r16s3,1), (r17s3,1), (r18s3,1), (r19s3,1), (r20s3,1), (r21s3,1), (r22s3,1), (r23s3,1), (r0s4,1), (r1s4,1), (r2s4,1), (r3s4,1), (r4s4,1), (r5s4,1), (r6s4,1), (r7s4,1), (r8s4,1)]), m) , z3.PbLe(([(r9s4,1), (r10s4,1), (r11s4,1), (r12s4,1), (r13s4,1), (r14s4,1), (r15s4,1), (r16s4,1), (r17s4,1), (r18s4,1), (r19s4,1), (r20s4,1), (r21s4,1), (r22s4,1), (r23s4,1), (r0s5,1), (r1s5,1), (r2s5,1), (r3s5,1), (r4s5,1), (r5s5,1), (r6s5,1), (r7s5,1), (r8s5,1)]), m) , z3.PbLe(([(r9s5,1), (r10s5,1), (r11s5,1), (r12s5,1), (r13s5,1), (r14s5,1), (r15s5,1), (r16s5,1), (r17s5,1), (r18s5,1), (r19s5,1), (r20s5,1), (r21s5,1), (r22s5,1), (r23s5,1), (r1s6,1), (r2s6,1), (r3s6,1), (r4s6,1), (r5s6,1), (r7s6,1), (r8s6,1), (r9s6,1), (r10s6,1)]), m) , z3.PbLe(([(r11s6,1), (r12s6,1), (r13s6,1), (r15s6,1), (r18s6,1), (r19s6,1), (r20s6,1), (r21s6,1), (r22s6,1), (r23s6,1), (r0s7,1), (r1s7,1), (r2s7,1), (r3s7,1), (r4s7,1), (r5s7,1), (r6s7,1), (r7s7,1), (r8s7,1), (r9s7,1), (r10s7,1), (r11s7,1), (r12s7,1), (r13s7,1)]), m) , z3.PbLe(([(r14s7,1), (r15s7,1), (r16s7,1), (r17s7,1), (r18s7,1), (r19s7,1), (r20s7,1), (r21s7,1), (r22s7,1), (r23s7,1), (r0s8,1), (r1s8,1), (r2s8,1), (r3s8,1), (r4s8,1), (r5s8,1), (r6s8,1), (r8s8,1), (r9s8,1), (r11s8,1), (r12s8,1), (r14s8,1), (r15s8,1), (r16s8,1)]), m) , z3.PbLe(([(r17s8,1), (r18s8,1), (r19s8,1), (r20s8,1), (r21s8,1), (r22s8,1), (r23s8,1), (r0s9,1), (r1s9,1), (r2s9,1), (r4s9,1), (r5s9,1), (r6s9,1), (r8s9,1), (r9s9,1), (r10s9,1), (r11s9,1), (r12s9,1), (r14s9,1), (r15s9,1), (r16s9,1), (r19s9,1), (r20s9,1), (r22s9,1)]), m) , z3.PbLe(([(r0s10,1), (r1s10,1), (r2s10,1), (r3s10,1), (r4s10,1), (r5s10,1), (r7s10,1), (r8s10,1), (r9s10,1), (r10s10,1), (r11s10,1), (r12s10,1), (r13s10,1), (r14s10,1), (r15s10,1), (r18s10,1), (r19s10,1), (r20s10,1), (r21s10,1), (r22s10,1), (r23s10,1), (r1s11,1), (r2s11,1), (r3s11,1)]), m) , z3.PbLe(([(r4s11,1), (r5s11,1), (r6s11,1), (r8s11,1), (r9s11,1), (r11s11,1), (r14s11,1), (r15s11,1), (r16s11,1), (r17s11,1), (r18s11,1), (r19s11,1), (r20s11,1), (r22s11,1), (r23s11,1), (r0s12,1), (r1s12,1), (r2s12,1), (r3s12,1), (r4s12,1), (r5s12,1), (r6s12,1), (r7s12,1), (r8s12,1)]), m) , z3.PbLe(([(r9s12,1), (r10s12,1), (r11s12,1), (r12s12,1), (r13s12,1), (r14s12,1), (r15s12,1), (r16s12,1), (r17s12,1), (r18s12,1), (r19s12,1), (r20s12,1), (r21s12,1), (r22s12,1), (r23s12,1), (r0s13,1), (r1s13,1), (r4s13,1), (r5s13,1), (r6s13,1), (r7s13,1), (r9s13,1), (r10s13,1), (r11s13,1)]), m) , z3.PbLe(([(r12s13,1), (r13s13,1), (r14s13,1), (r15s13,1), (r16s13,1), (r17s13,1), (r18s13,1), (r19s13,1), (r20s13,1), (r22s13,1), (r23s13,1), (r0s14,1), (r1s14,1), (r2s14,1), (r4s14,1), (r5s14,1), (r6s14,1), (r7s14,1), (r8s14,1), (r9s14,1), (r10s14,1), (r11s14,1), (r12s14,1), (r13s14,1)]), m) , z3.PbLe(([(r14s14,1), (r15s14,1), (r16s14,1), (r17s14,1), (r18s14,1), (r19s14,1), (r21s14,1), (r22s14,1), (r23s14,1), (r0s15,1), (r1s15,1), (r2s15,1), (r3s15,1), (r4s15,1), (r5s15,1), (r6s15,1), (r7s15,1), (r8s15,1), (r9s15,1), (r10s15,1), (r11s15,1), (r12s15,1), (r13s15,1), (r14s15,1)]), m) , z3.PbLe(([(r15s15,1), (r16s15,1), (r17s15,1), (r18s15,1), (r19s15,1), (r20s15,1), (r21s15,1), (r22s15,1), (r23s15,1), (r0s16,1), (r1s16,1), (r4s16,1), (r5s16,1), (r6s16,1), (r7s16,1), (r9s16,1), (r10s16,1), (r11s16,1), (r12s16,1), (r13s16,1), (r15s16,1), (r16s16,1), (r17s16,1), (r18s16,1)]), m) , z3.PbLe(([(r19s16,1), (r22s16,1), (r23s16,1), (r1s17,1), (r3s17,1), (r4s17,1), (r5s17,1), (r6s17,1), (r7s17,1), (r9s17,1), (r10s17,1), (r11s17,1), (r14s17,1), (r15s17,1), (r16s17,1), (r17s17,1), (r18s17,1), (r19s17,1), (r20s17,1), (r22s17,1), (r23s17,1), (r0s18,1), (r1s18,1), (r2s18,1)]), m) , z3.PbLe(([(r3s18,1), (r4s18,1), (r5s18,1), (r6s18,1), (r7s18,1), (r8s18,1), (r9s18,1), (r10s18,1), (r11s18,1), (r13s18,1), (r14s18,1), (r17s18,1), (r18s18,1), (r19s18,1), (r20s18,1), (r21s18,1), (r22s18,1), (r23s18,1), (r0s19,1), (r1s19,1), (r2s19,1), (r3s19,1), (r4s19,1), (r5s19,1)]), m) , z3.PbLe(([(r6s19,1), (r7s19,1), (r8s19,1), (r9s19,1), (r10s19,1), (r11s19,1), (r12s19,1), (r13s19,1), (r14s19,1), (r15s19,1), (r16s19,1), (r17s19,1), (r18s19,1), (r19s19,1), (r20s19,1), (r21s19,1), (r22s19,1), (r23s19,1), (r0s20,1), (r1s20,1), (r2s20,1), (r3s20,1), (r4s20,1), (r5s20,1)]), m) , z3.PbLe(([(r6s20,1), (r7s20,1), (r8s20,1), (r9s20,1), (r10s20,1), (r11s20,1), (r12s20,1), (r13s20,1), (r14s20,1), (r15s20,1), (r16s20,1), (r17s20,1), (r18s20,1), (r19s20,1), (r20s20,1), (r21s20,1), (r22s20,1), (r23s20,1), (r0s21,1), (r1s21,1), (r3s21,1), (r4s21,1), (r5s21,1), (r6s21,1)]), m) , z3.PbLe(([(r8s21,1), (r9s21,1), (r10s21,1), (r11s21,1), (r12s21,1), (r14s21,1), (r15s21,1), (r16s21,1), (r17s21,1), (r18s21,1), (r19s21,1), (r20s21,1), (r22s21,1), (r23s21,1), (r0s22,1), (r1s22,1), (r2s22,1), (r3s22,1), (r4s22,1), (r5s22,1), (r6s22,1), (r7s22,1), (r8s22,1), (r9s22,1)]), m) , z3.PbLe(([(r10s22,1), (r11s22,1), (r12s22,1), (r13s22,1), (r14s22,1), (r15s22,1), (r16s22,1), (r17s22,1), (r18s22,1), (r19s22,1), (r20s22,1), (r21s22,1), (r22s22,1), (r23s22,1), (r0s23,1), (r1s23,1), (r2s23,1), (r3s23,1), (r4s23,1), (r5s23,1), (r6s23,1), (r7s23,1), (r8s23,1), (r9s23,1)]), m) , z3.PbLe(([(r10s23,1), (r11s23,1), (r12s23,1), (r13s23,1), (r14s23,1), (r15s23,1), (r16s23,1), (r17s23,1), (r18s23,1), (r19s23,1), (r20s23,1), (r21s23,1), (r22s23,1), (r23s23,1), (r0s24,1), (r1s24,1), (r2s24,1), (r4s24,1), (r5s24,1), (r6s24,1), (r7s24,1), (r8s24,1), (r9s24,1), (r10s24,1)]), m) , z3.PbLe(([(r11s24,1), (r12s24,1), (r14s24,1), (r15s24,1), (r16s24,1), (r19s24,1), (r20s24,1), (r21s24,1), (r22s24,1), (r0s25,1), (r1s25,1), (r2s25,1), (r3s25,1), (r4s25,1), (r5s25,1), (r6s25,1), (r7s25,1), (r8s25,1), (r9s25,1), (r10s25,1), (r11s25,1), (r12s25,1), (r13s25,1), (r14s25,1)]), m) , z3.PbLe(([(r15s25,1), (r16s25,1), (r17s25,1), (r18s25,1), (r19s25,1), (r20s25,1), (r21s25,1), (r22s25,1), (r23s25,1), (r0s26,1), (r1s26,1), (r2s26,1), (r4s26,1), (r5s26,1), (r6s26,1), (r7s26,1), (r8s26,1), (r9s26,1), (r10s26,1), (r11s26,1), (r12s26,1), (r13s26,1), (r14s26,1), (r15s26,1)]), m) , z3.PbLe(([(r16s26,1), (r17s26,1), (r18s26,1), (r19s26,1), (r20s26,1), (r21s26,1), (r22s26,1), (r23s26,1), (r0s27,1), (r1s27,1), (r2s27,1), (r4s27,1), (r5s27,1), (r6s27,1), (r7s27,1), (r8s27,1), (r9s27,1), (r10s27,1), (r11s27,1), (r12s27,1), (r13s27,1), (r14s27,1), (r15s27,1), (r16s27,1)]), m) , z3.PbLe(([(r17s27,1), (r18s27,1), (r19s27,1), (r21s27,1), (r22s27,1), (r23s27,1), (r0s28,1), (r1s28,1), (r2s28,1), (r4s28,1), (r5s28,1), (r6s28,1), (r7s28,1), (r8s28,1), (r9s28,1), (r10s28,1), (r11s28,1), (r12s28,1), (r13s28,1), (r14s28,1), (r15s28,1), (r16s28,1), (r17s28,1), (r18s28,1)]), m) , z3.PbLe(([(r19s28,1), (r21s28,1), (r22s28,1), (r23s28,1), (r0s29,1), (r1s29,1), (r2s29,1), (r4s29,1), (r5s29,1), (r6s29,1), (r7s29,1), (r8s29,1), (r9s29,1), (r10s29,1), (r11s29,1), (r12s29,1), (r13s29,1), (r14s29,1), (r15s29,1), (r16s29,1), (r17s29,1), (r18s29,1), (r19s29,1), (r21s29,1)]), m) , z3.PbLe(([(r22s29,1), (r23s29,1), (r0s30,1), (r1s30,1), (r2s30,1), (r4s30,1), (r5s30,1), (r6s30,1), (r7s30,1), (r8s30,1), (r9s30,1), (r10s30,1), (r11s30,1), (r12s30,1), (r13s30,1), (r14s30,1), (r15s30,1), (r16s30,1), (r17s30,1), (r18s30,1), (r19s30,1), (r20s30,1), (r21s30,1), (r22s30,1)]), m) , z3.PbLe(([(r23s30,1), (r0s31,1), (r1s31,1), (r2s31,1), (r3s31,1), (r4s31,1), (r5s31,1), (r6s31,1), (r7s31,1), (r8s31,1), (r9s31,1), (r10s31,1), (r11s31,1), (r12s31,1), (r13s31,1), (r15s31,1), (r16s31,1), (r17s31,1), (r18s31,1), (r19s31,1), (r20s31,1), (r21s31,1), (r22s31,1), (r23s31,1)]), m) , z3.PbLe(([(r0s32,1), (r1s32,1), (r3s32,1), (r4s32,1), (r5s32,1), (r6s32,1), (r7s32,1), (r9s32,1), (r10s32,1), (r11s32,1), (r12s32,1), (r13s32,1), (r15s32,1), (r16s32,1), (r17s32,1), (r18s32,1), (r19s32,1), (r20s32,1), (r22s32,1), (r23s32,1), (r1s33,1), (r2s33,1), (r3s33,1), (r4s33,1)]), m) , z3.PbLe(([(r5s33,1), (r6s33,1), (r7s33,1), (r8s33,1), (r9s33,1), (r10s33,1), (r11s33,1), (r12s33,1), (r13s33,1), (r15s33,1), (r16s33,1), (r17s33,1), (r18s33,1), (r19s33,1), (r20s33,1), (r21s33,1), (r22s33,1), (r23s33,1), (r0s34,1), (r2s34,1), (r3s34,1), (r4s34,1), (r5s34,1), (r6s34,1)]), m) , z3.PbLe(([(r7s34,1), (r8s34,1), (r9s34,1), (r10s34,1), (r11s34,1), (r12s34,1), (r13s34,1), (r14s34,1), (r15s34,1), (r16s34,1), (r17s34,1), (r18s34,1), (r19s34,1), (r20s34,1), (r21s34,1), (r22s34,1), (r23s34,1), (r0s35,1), (r1s35,1), (r2s35,1), (r3s35,1), (r4s35,1), (r5s35,1), (r7s35,1)]), m) , z3.PbLe(([(r8s35,1), (r9s35,1), (r10s35,1), (r11s35,1), (r12s35,1), (r13s35,1), (r15s35,1), (r16s35,1), (r17s35,1), (r18s35,1), (r19s35,1), (r20s35,1), (r21s35,1), (r22s35,1), (r23s35,1), (r0s36,1), (r1s36,1), (r2s36,1), (r3s36,1), (r4s36,1), (r5s36,1), (r6s36,1), (r7s36,1), (r8s36,1)]), m) , z3.PbLe(([(r9s36,1), (r10s36,1), (r11s36,1), (r12s36,1), (r14s36,1), (r15s36,1), (r16s36,1), (r17s36,1), (r18s36,1), (r19s36,1), (r20s36,1), (r22s36,1), (r23s36,1), (r0s37,1), (r1s37,1), (r2s37,1), (r3s37,1), (r4s37,1), (r5s37,1), (r7s37,1), (r8s37,1), (r9s37,1), (r10s37,1), (r11s37,1)]), m) , z3.PbLe(([(r12s37,1), (r13s37,1), (r15s37,1), (r17s37,1), (r18s37,1), (r19s37,1), (r20s37,1), (r21s37,1), (r22s37,1), (r23s37,1), (r0s38,1), (r1s38,1), (r2s38,1), (r3s38,1), (r4s38,1), (r5s38,1), (r7s38,1), (r8s38,1), (r9s38,1), (r10s38,1), (r11s38,1), (r12s38,1), (r13s38,1), (r14s38,1)]), m) , z3.PbLe(([(r15s38,1), (r16s38,1), (r17s38,1), (r18s38,1), (r19s38,1), (r20s38,1), (r21s38,1), (r22s38,1), (r23s38,1), (r0s39,1), (r1s39,1), (r2s39,1), (r3s39,1), (r4s39,1), (r5s39,1), (r6s39,1), (r7s39,1), (r8s39,1), (r9s39,1), (r10s39,1), (r11s39,1), (r12s39,1), (r13s39,1), (r14s39,1)]), m) , z3.PbLe(([(r15s39,1), (r16s39,1), (r17s39,1), (r18s39,1), (r19s39,1), (r20s39,1), (r21s39,1), (r22s39,1), (r23s39,1), (r0s40,1), (r1s40,1), (r2s40,1), (r3s40,1), (r4s40,1), (r5s40,1), (r6s40,1), (r7s40,1), (r8s40,1), (r9s40,1), (r10s40,1), (r11s40,1), (r12s40,1), (r13s40,1), (r14s40,1)]), m) , z3.PbLe(([(r15s40,1), (r16s40,1), (r17s40,1), (r18s40,1), (r19s40,1), (r20s40,1), (r21s40,1), (r22s40,1), (r23s40,1), (r0s41,1), (r1s41,1), (r2s41,1), (r3s41,1), (r4s41,1), (r5s41,1), (r6s41,1), (r7s41,1), (r8s41,1), (r9s41,1), (r10s41,1), (r11s41,1), (r12s41,1), (r13s41,1), (r14s41,1)]), m) , z3.PbLe(([(r15s41,1), (r16s41,1), (r17s41,1), (r18s41,1), (r19s41,1), (r20s41,1), (r21s41,1), (r22s41,1), (r23s41,1), (r0s42,1), (r1s42,1), (r2s42,1), (r3s42,1), (r4s42,1), (r5s42,1), (r6s42,1), (r7s42,1), (r8s42,1), (r9s42,1), (r10s42,1), (r12s42,1), (r13s42,1), (r14s42,1), (r15s42,1)]), m) , z3.PbLe(([(r16s42,1), (r17s42,1), (r18s42,1), (r19s42,1), (r20s42,1), (r21s42,1), (r22s42,1), (r23s42,1), (r0s43,1), (r1s43,1), (r2s43,1), (r3s43,1), (r4s43,1), (r5s43,1), (r7s43,1), (r8s43,1), (r9s43,1), (r10s43,1), (r11s43,1), (r12s43,1), (r13s43,1), (r14s43,1), (r15s43,1), (r16s43,1)]), m) , z3.PbLe(([(r17s43,1), (r18s43,1), (r19s43,1), (r20s43,1), (r21s43,1), (r22s43,1), (r23s43,1), (r0s44,1), (r1s44,1), (r2s44,1), (r3s44,1), (r4s44,1), (r5s44,1), (r6s44,1), (r7s44,1), (r8s44,1), (r9s44,1), (r10s44,1), (r11s44,1), (r12s44,1), (r13s44,1), (r14s44,1), (r15s44,1), (r16s44,1)]), m) , z3.PbLe(([(r17s44,1), (r18s44,1), (r19s44,1), (r20s44,1), (r21s44,1), (r22s44,1), (r23s44,1), (r0s45,1), (r1s45,1), (r2s45,1), (r3s45,1), (r4s45,1), (r5s45,1), (r6s45,1), (r7s45,1), (r8s45,1), (r9s45,1), (r10s45,1), (r11s45,1), (r12s45,1), (r13s45,1), (r14s45,1), (r15s45,1), (r16s45,1)]), m) , z3.PbLe(([(r17s45,1), (r18s45,1), (r19s45,1), (r20s45,1), (r21s45,1), (r22s45,1), (r23s45,1), (r0s46,1), (r1s46,1), (r2s46,1), (r3s46,1), (r4s46,1), (r5s46,1), (r7s46,1), (r8s46,1), (r9s46,1), (r10s46,1), (r11s46,1), (r12s46,1), (r13s46,1), (r14s46,1), (r15s46,1), (r16s46,1), (r17s46,1)]), m) , z3.PbLe(([(r18s46,1), (r19s46,1), (r20s46,1), (r21s46,1), (r22s46,1), (r23s46,1), (r0s47,1), (r1s47,1), (r2s47,1), (r3s47,1), (r4s47,1), (r5s47,1), (r6s47,1), (r7s47,1), (r8s47,1), (r9s47,1), (r10s47,1), (r13s47,1), (r14s47,1), (r17s47,1), (r18s47,1), (r19s47,1), (r20s47,1), (r21s47,1)]), m) , z3.PbLe(([(r22s47,1), (r23s47,1), (r0s48,1), (r1s48,1), (r2s48,1), (r3s48,1), (r4s48,1), (r5s48,1), (r6s48,1), (r8s48,1), (r9s48,1), (r11s48,1), (r12s48,1), (r14s48,1), (r15s48,1), (r16s48,1), (r17s48,1), (r18s48,1), (r19s48,1), (r20s48,1), (r21s48,1), (r22s48,1), (r23s48,1), (r0s49,1)]), m) , z3.PbLe(([(r1s49,1), (r2s49,1), (r3s49,1), (r4s49,1), (r5s49,1), (r6s49,1), (r7s49,1), (r8s49,1), (r9s49,1), (r10s49,1), (r12s49,1), (r13s49,1), (r14s49,1), (r15s49,1), (r16s49,1), (r17s49,1), (r18s49,1), (r19s49,1), (r20s49,1), (r21s49,1), (r22s49,1), (r23s49,1), (r0s50,1), (r1s50,1)]), m) , z3.PbLe(([(r2s50,1), (r3s50,1), (r4s50,1), (r5s50,1), (r6s50,1), (r7s50,1), (r8s50,1), (r9s50,1), (r10s50,1), (r11s50,1), (r12s50,1), (r13s50,1), (r14s50,1), (r15s50,1), (r16s50,1), (r17s50,1), (r18s50,1), (r19s50,1), (r20s50,1), (r21s50,1), (r22s50,1), (r23s50,1), (r0s51,1), (r1s51,1)]), m) , z3.PbLe(([(r2s51,1), (r3s51,1), (r4s51,1), (r5s51,1), (r6s51,1), (r7s51,1), (r8s51,1), (r9s51,1), (r10s51,1), (r11s51,1), (r12s51,1), (r13s51,1), (r14s51,1), (r15s51,1), (r16s51,1), (r17s51,1), (r18s51,1), (r19s51,1), (r20s51,1), (r21s51,1), (r22s51,1), (r23s51,1)]), m),
score == z3.Sum(scores),
m_set == 0,
goal == score + m_set])
# rules = [rev_rules, sub_rules, others]

start = timeit.default_timer()
o.maximize(score)
if o.check() == z3.sat:
    model = o.model()
    stop = timeit.default_timer()
    print('Time: ', stop - start, 'sec.')  
    print([(d, model[d]) for d in model])

#     print(sorted ([(d, m[d]) for d in m], key = lambda x: str(x[0])))
    matches = [(d, model[d]) for d in model if model[d] == True]
#     print(matches)
else:
    print("not satisfiable")
performance = o.statistics()
# print(performance)

n_yes = 0 
n_maybe = 0
n_no = 0
for match in matches:
    if rs_score[str(match[0])] > maybe:
        n_yes += 1
    if no < rs_score[str(match[0])] < yes:
        n_maybe += 1
    if rs_score[str(match[0])] < maybe:
        n_no += 1
print('total: {0}, yes:{1}, maybe:{2}, no: {3}'.format(len(matches), n_yes, n_maybe, n_no))

Time:  0.6941224000000004 sec.
[(r0s0, False), (r0s1, False), (r0s2, False), (r0s4, False), (r0s5, False), (r0s7, False), (r0s8, False), (r0s9, False), (r0s10, False), (r0s12, False), (r0s13, False), (r0s14, False), (r0s15, False), (r0s16, False), (r0s18, False), (r0s19, False), (r0s20, False), (r0s21, False), (r0s22, False), (r0s23, False), (r0s24, True), (r0s25, False), (r0s26, False), (r0s27, False), (r0s28, False), (r0s29, True), (r0s30, False), (r0s31, False), (r0s32, False), (r0s34, False), (r0s35, False), (r0s36, False), (r0s37, True), (r0s38, False), (r0s39, False), (r0s40, False), (r0s41, False), (r0s42, False), (r0s43, False), (r0s44, True), (r0s45, False), (r0s46, False), (r0s47, False), (r0s48, False), (r0s49, False), (r0s50, False), (r0s51, False), (r1s0, True), (r1s1, False), (r1s2, False), (r1s3, False), (r1s4, False), (r1s5, False), (r1s6, False), (r1s7, False), (r1s8, False), (r1s9, True), (r1s10, False), (r1s11, True), (r1s12, False), (r1s13, False), (r1s14, False), (